<a href="https://colab.research.google.com/github/daarisameen/Ml-DL-projects/blob/main/Project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://medium.com/@sthanikamsanthosh1994/context-aware-sentence-matching-using-ml-techniques-f93a28c4d1a1
https://datascience.stackexchange.com/questions/75299/how-to-figure-out-if-two-sentences-have-the-same-meaning-with-ai
https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb
https://colab.research.google.com/github/mepky/movie-recommendation-system/blob/master/Movie_recommendation_engine.ipynb#scrollTo=H1Ta9vMPcsGW

In [9]:
# End to End ML IITM: https://colab.research.google.com/drive/1leWpYtJrBhLf426Q2Emj1o3nNylPaJ0O?usp=sharing
# heart - https://media.licdn.com/dms/document/media/D4D1FAQFRlrkzgq8vMg/feedshare-document-pdf-analyzed/0/1696704666094?e=1698883200&v=beta&t=u-Ut36lewfXkDpX04b6MTvxFsTzVb4qkLu5AxKN4GcM
# taxi - https://media.licdn.com/dms/document/media/C4D1FAQEzUwZsEB0JLA/feedshare-document-pdf-analyzed/0/1679913090300?e=1698883200&v=beta&t=cuW5AsFBS9cZ-Va3hKcNItzeuI9Xq1xx5tx8ROQKkWY

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Perceptron, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, VotingRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor

from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.feature_selection import SelectKBest, f_regression

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import GridSearchCV

import pickle

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))


In [10]:
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

# Example sentences
sentence1 = "The quick brown fox jumps over the lazy dog."
sentence2 = "The lazy dog is jumped over by the quick brown fox."

# Tokenize and obtain embeddings for the sentences
inputs1 = tokenizer(sentence1, return_tensors="pt")
inputs2 = tokenizer(sentence2, return_tensors="pt")
with torch.no_grad():
    embeddings1 = model(**inputs1).last_hidden_state.mean(dim=1)
    embeddings2 = model(**inputs2).last_hidden_state.mean(dim=1)

# Calculate cosine similarity
similarity = cosine_similarity(embeddings1, embeddings2)[0][0]

# Convert cosine similarity to a percentage
percentage_match = (similarity + 1) * 50  # Scale to 0-100

print(f"Percentage of match in meaning: {percentage_match:.2f}%")


Percentage of match in meaning: 94.94%


I have a columns called
User_ratings, Movie_name, movie_description, genre, popularity_rank
write a ML code to train the model based on its description, user_rating, genre, and popularity_rank


just get the movie name from user and
and it is given that movie name will definitely present in the data set
Output should contain list of recommended movie_names

In [28]:
# @title
data = [{"m_title": "Demon Slayer: Kimetsu no Yaiba", "r": "8.7", "year": "2019", "votes": "117,630", "genre": "Action, Adventure", "desp": "A family is attacked by demons and only two members survive - Tanjiro and his sister Nezuko, who is turning into a demon slowly. Tanjiro sets out to become a demon slayer to avenge his family and cure his sister."}, {"m_title": "Attack on Titan", "r": "9.1", "year": "2013", "votes": "408,892", "genre": "Action, Adventure", "desp": "After his hometown is destroyed and his mother is killed, young Eren Jaeger vows to cleanse the earth of the giant humanoid Titans that have brought humanity to the brink of extinction."}, {"m_title": "Vinland Saga", "r": "8.8", "year": "2019", "votes": "49,162", "genre": "Action, Adventure", "desp": "Thorfinn pursues a journey with his fathers killer in order to take revenge and end his life in a duel as an honorable warrior and pay his father a homage."}, {"m_title": "One Piece", "r": "8.9", "year": "1999", "votes": "138,438", "genre": "Action, Adventure", "desp": "Follows the adventures of Monkey D. Luffy and his pirate crew in order to find the greatest treasure ever left by the legendary Pirate, Gold Roger. The famous mystery treasure named One Piece."}, {"m_title": "Hells Paradise: Jigokuraku", "r": "8.5", "year": "2023", "votes": "2,007", "genre": "Action, Adventure", "desp": "A squad of prisoners and their guards are sent to investigate a mysterious island. They get stranded there and must rely on each other to survive the islands mysterious and monstrous residents."}, {"m_title": "Oshi No Ko", "r": "8.9", "year": "2023", "votes": "2,406", "genre": "Drama, Fantasy", "desp": "A doctor and his recently-deceased patient are reborn as twins to a famous Japanese musical idol and navigate the highs and lows of the Japanese entertainment industry as they grow up together through their lives."}, {"m_title": "My Hero Academia", "r": "8.3", "year": "2016", "votes": "69,195", "genre": "Action, Adventure", "desp": "A superhero-loving boy without any powers is determined to enroll in a prestigious hero academy and learn what it really means to be a hero."}, {"m_title": "Naruto", "r": "8.4", "year": "2002", "votes": "110,371", "genre": "Action, Adventure", "desp": "Naruto Uzumaki, a mischievous adolescent ninja, struggles as he searches for recognition and dreams of becoming the Hokage, the villages leader and strongest ninja."}, {"m_title": "Naruto: Shippuden", "r": "8.7", "year": "2007", "votes": "136,293", "genre": "Action, Adventure", "desp": "Naruto Uzumaki, is a loud, hyperactive, adolescent ninja who constantly searches for approval and recognition, as well as to become Hokage, who is acknowledged as the leader and strongest of all ninja in the village."}, {"m_title": "Jujutsu Kaisen", "r": "8.5", "year": "2020", "votes": "71,511", "genre": "Action, Adventure", "desp": "A boy swallows a cursed talisman - the finger of a demon - and becomes cursed himself. He enters a shamans school to be able to locate the demons other body parts and thus exorcise himself."}, {"m_title": "Hunter x Hunter", "r": "9.0", "year": "2011", "votes": "110,481", "genre": "Action, Adventure", "desp": "Gon Freecss aspires to become a Hunter, an exceptional being capable of greatness. With his friends and his potential, he seeks out his father, who left him when he was younger."}, {"m_title": "Death Note", "r": "9.0", "year": "2006", "votes": "341,668", "genre": "Crime, Drama", "desp": "An intelligent high school student goes on a secret crusade to eliminate criminals from the world after discovering a notebook capable of killing anyone whose name is written into it."}, {"m_title": "Chainsaw Man", "r": "8.5", "year": "2022", "votes": "36,183", "genre": "Action, Adventure", "desp": "Following a betrayal, a young man left for dead is reborn as a powerful devil-human hybrid after merging with his pet devil and is soon enlisted into an organization dedicated to hunting devils."}, {"m_title": "Black Clover", "r": "8.3", "year": "2017", "votes": "27,014", "genre": "Action, Adventure", "desp": "Asta and Yuno were abandoned together at the same church and have been inseparable since. As children, they promised that they would compete against each other to see who would become the next Emperor Magus."}, {"m_title": "JoJos Bizarre Adventure", "r": "8.5", "year": "2012", "votes": "28,176", "genre": "Action, Adventure", "desp": "The story of the Joestar family, who are possessed with intense psychic strength, and the adventures each member encounters throughout their lives."}, {"m_title": "Fullmetal Alchemist: Brotherhood", "r": "9.1", "year": "2009", "votes": "177,885", "genre": "Action, Adventure", "desp": "Two brothers search for a Philosophers Stone after an attempt to revive their deceased mother goes awry and leaves them in damaged physical forms."}, {"m_title": "Pokémon", "r": "7.5", "year": "1997", "votes": "44,943", "genre": "Action, Adventure", "desp": "Ash Ketchum, his yellow pet Pikachu, and his human friends explore a world of powerful creatures."}, {"m_title": "Bleach", "r": "8.2", "year": "2004", "votes": "60,296", "genre": "Action, Adventure", "desp": "High school student Ichigo Kurosaki, who has the ability to see ghosts, gains soul reaper powers from Rukia Kuchiki and sets out to save the world from Hollows."}, {"m_title": "One Punch Man", "r": "8.7", "year": "2015", "votes": "169,368", "genre": "Action, Comedy", "desp": "The story of Saitama, a hero that does it just for fun & can defeat his enemies with a single punch."}, {"m_title": "Dragon Ball Z", "r": "8.8", "year": "1989", "votes": "136,464", "genre": "Action, Adventure", "desp": "With the help of the powerful Dragonballs, a team of fighters led by the saiyan warrior Goku defend the planet earth from extraterrestrial enemies."}, {"m_title": "Cyberpunk: Edgerunners", "r": "8.3", "year": "2022", "votes": "58,692", "genre": "Action, Adventure", "desp": "A Street Kid trying to survive in a technology and body modification-obsessed city of the future. Having everything to lose, he chooses to stay alive by becoming an Edgerunner, a Mercenary outlaw also known as a Cyberpunk."}, {"m_title": "Monster", "r": "8.7", "year": "2004", "votes": "35,291", "genre": "Crime, Drama", "desp": "A brilliant neurosurgeon finds his life in utter turmoil after getting involved with a psychopathic former patient."}, {"m_title": "Tokyo Revengers", "r": "8.0", "year": "2021", "votes": "20,161", "genre": "Action, Drama", "desp": "Hanagaki Takemichi lives an unsatisfying life right up until his death. Waking up 12 years in the past, he reckons with the eventual fate of his friends and tries to prevent an unfortunate future."}, {"m_title": "Mashle: Magic and Muscles", "r": "7.8", "year": "2023", "votes": "602", "genre": "Action, Adventure", "desp": "Into a world of magicians is born Mash, a young boy who lacks magic but is the strongest person around. His existence and destiny will change his world."}, {"m_title": "Heavenly Delusion", "r": "8.2", "year": "2023", "votes": "693", "genre": "Adventure, Mystery", "desp": "In the outside world, 15 years have passed since an unprecedented disaster, that completely destroyed modern civilization. A group of children live in a facility isolated from the outside. watch the dude, no spoilers in summary!"}, {"m_title": "Dr. Stone", "r": "8.1", "year": "2019", "votes": "22,914", "genre": "Action, Adventure", "desp": "Awakened into a world where humanity has been petrified, scientific genius Senku and his brawny friend Taiju use their skills to rebuild civilization."}, {"m_title": "Berserk", "r": "8.7", "year": "1997", "votes": "43,835", "genre": "Action, Adventure", "desp": "Guts is a skilled swordsman who joins forces with a mercenary group named The Band of the Hawk, lead by the charismatic Griffith, and fights with them as they battle their way into the royal court."}, {"m_title": "Spy x Family", "r": "8.5", "year": "2022", "votes": "29,091", "genre": "Action, Comedy", "desp": "A spy on an undercover mission gets married and adopts a child as part of his cover. His wife and daughter have secrets of their own, and all three must strive to keep together."}, {"m_title": "Bleach: Thousand-Year Blood War", "r": "9.1", "year": "2022", "votes": "18,796", "genre": "Action, Adventure", "desp": "The peace is suddenly broken when warning sirens blare through the Soul Society. Residents are disappearing without a trace and nobody knows whos behind it. Meanwhile, a darkness is approaching Ichigo and his friends in Karakura Town."}, {"m_title": "Cowboy Bebop", "r": "8.9", "year": "1998", "votes": "126,306", "genre": "Action, Adventure", "desp": "The futuristic misadventures and tragedies of an easygoing bounty hunter and his partners."}, {"m_title": "Boruto: Naruto Next Generations", "r": "6.6", "year": "2017", "votes": "23,927", "genre": "Action, Adventure", "desp": "Son of Naruto Uzumaki, Boruto, follows his fathers footsteps along with his friends to become great ninja. Throughout all their adventures, Boruto is determined to make his mark in the ninja world and live outside of his fathers shadow."}, {"m_title": "Neon Genesis Evangelion", "r": "8.5", "year": "1995", "votes": "75,312", "genre": "Action, Drama", "desp": "A teenage boy finds himself recruited as a member of an elite team of pilots by his father."}, {"m_title": "Blue Lock", "r": "8.3", "year": "2022", "votes": "8,265", "genre": "Sport, Thriller", "desp": "High school soccer players from across Japan gather for a controversial project designed to create the best and most egoistic striker in the world."}, {"m_title": "Haikyu!!", "r": "8.7", "year": "2014", "votes": "28,876", "genre": "Comedy, Drama", "desp": "Determined to be like the volleyball championships star player nicknamed the small giant, Shoyo joins his schools volleyball club."}, {"m_title": "Mob Psycho 100", "r": "8.6", "year": "2016", "votes": "37,460", "genre": "Action, Comedy", "desp": "A psychic middle school boy tries to live a normal life and keep his growing powers under control, even though he constantly gets into trouble."}, {"m_title": "Dragon Ball Super", "r": "8.3", "year": "2015", "votes": "32,119", "genre": "Action, Adventure", "desp": "Six months after the defeat of Majin Buu, The mighty Saiyan Son Goku continues his quest on becoming stronger."}, {"m_title": "Gintama", "r": "8.7", "year": "2005", "votes": "12,742", "genre": "Action, Comedy", "desp": "In an era where aliens have invaded and taken over feudal Tokyo, an unemployed samurai finds work however he can."}, {"m_title": "Code Geass", "r": "8.7", "year": "2006", "votes": "72,404", "genre": "Action, Drama", "desp": "After being given a mysterious power to control others, an outcast prince becomes the masked leader of the rebellion against an all-powerful empire."}, {"m_title": "Dragon Ball", "r": "8.6", "year": "1986", "votes": "60,672", "genre": "Action, Adventure", "desp": "Son Gokû, a fighter with a monkey tail, goes on a quest with an assortment of odd characters in search of the Dragon Balls, a set of crystals that can give its bearer anything they desire."}, {"m_title": "Mushoku Tensei: Jobless Reincarnation", "r": "8.4", "year": "2021", "votes": "13,595", "genre": "Action, Adventure", "desp": "A 34-year-old Japanese NEET is run over by a speeding truck and dies. Before he knows it, he is reborn as Rudeus Greyrat, and begins a new life full of adventure."}, {"m_title": "Made in Abyss", "r": "8.4", "year": "2017", "votes": "12,888", "genre": "Action, Adventure", "desp": "A girl and her robot companion search for her mother, whos lost within a vast chasm."}, {"m_title": "Steins;Gate", "r": "8.8", "year": "2011", "votes": "66,236", "genre": "Comedy, Drama", "desp": "After discovering time travel, a university student and his colleagues must use their knowledge of it to stop an evil organization and their diabolical plans."}, {"m_title": "Your Lie in April", "r": "8.6", "year": "2014", "votes": "32,893", "genre": "Comedy, Drama", "desp": "A piano prodigy who lost his ability to play after suffering a traumatic event in his childhood is forced back into the spotlight by an eccentric girl with a secret of her own."}, {"m_title": "Kaguya-sama: Love is War", "r": "8.5", "year": "2019", "votes": "16,356", "genre": "Comedy, Romance", "desp": "The proudly privileged top two students of an elite school each makes it their mission to be the first to extract a confession of love from the other."}, {"m_title": "Sword Art Online", "r": "7.5", "year": "2012", "votes": "48,400", "genre": "Action, Adventure", "desp": "In the year 2022, thousands of people get trapped in a new virtual MMORPG and the lone wolf player, Kirito, works to escape."}, {"m_title": "Fairy Tail", "r": "7.9", "year": "2009", "votes": "25,964", "genre": "Action, Adventure", "desp": "Lucy, an aspiring Celestial Wizard, becomes a friend and ally to powerful wizards Natsu, Gray, and Erza, who are part of the (in)famous wizard guild, Fairy Tail."}, {"m_title": "The Promised Neverland", "r": "8.3", "year": "2019", "votes": "41,452", "genre": "Action, Adventure", "desp": "When three gifted kids at an isolated idyllic orphanage discover the secret and sinister purpose they were raised for, they look for a way to escape from their evil caretaker and lead the other children in a risky escape plan."}, {"m_title": "Nanatsu no taizai", "r": "7.8", "year": "2014", "votes": "34,410", "genre": "Action, Adventure", "desp": "The story of the Seven Deadly Sins, a group of warriors who were wrongly accused of a crime they didnt commit and went on a quest to vindicate themselves."}, {"m_title": "Re: Zero, Starting Life in Another World", "r": "8.1", "year": "2016", "votes": "20,680", "genre": "Adventure, Drama", "desp": "After being suddenly transported to another world, Subaru Natsuki and his new female partner are brutally murdered. However, Subaru awakens to a familiar scene, meeting the same girl again. The day begins to mysteriously repeat itself."}, {"m_title": "Knights of the Zodiac", "r": "8.4", "year": "1986", "votes": "10,523", "genre": "Action, Adventure", "desp": "A group a young warriors known as Saints, each in possession of a cloth guarded by a different constellation, must protect the reincarnation of the goddess Athena as she attempts to keep the Earth from being destroyed by evil forces."}, {"m_title": "Berserk", "r": "6.7", "year": "2016", "votes": "6,655", "genre": "Action, Adventure", "desp": "Guts, The Black Swordsman, is pursued by demons who are attracted to him, due to a demonic brand on his neck. His goal is to free himself, and his lover Casca, of this inescapable curse.. watch the dude, no spoilers in summary!"}, {"m_title": "The Eminence in Shadow", "r": "7.9", "year": "2022", "votes": "2,557", "genre": "Action, Adventure", "desp": "Cid Kagenou creates an elaborate plot to fight the Cult of Diabolos and builds his organization, the Shadow Garden."}, {"m_title": "High School DxD", "r": "7.6", "year": "2012", "votes": "11,920", "genre": "Action, Comedy", "desp": "After being killed on his first date, idiotic and perverted Issei Hyodo is resurrected as a demon by Rias Gremory only to be recruited into her club of high-class devils."}, {"m_title": "Bungo Stray Dogs", "r": "7.8", "year": "2016", "votes": "7,584", "genre": "Action, Comedy", "desp": "Nakajima Atsushi was kicked out of his orphanage, and now he has no place to go and no food. While he is standing by a river, on the brink of starvation, he rescues a man whimsically. watch the dude, no spoilers in summary!"}, {"m_title": "That Time I Got Reincarnated as a Slime", "r": "8.1", "year": "2018", "votes": "12,431", "genre": "Action, Adventure", "desp": "Average 37-year-old Minami Satoru dies and is reincarnated as the most unremarkable creature imaginable-a slime."}, {"m_title": "Ranking of Kings", "r": "8.5", "year": "2021", "votes": "10,231", "genre": "Action, Adventure", "desp": "The people of the kingdom look down on the young Prince Bojji, who can neither hear nor speak. They call him The Useless Prince while jeering at his supposed foolishness."}, {"m_title": "Fruits Basket", "r": "8.6", "year": "2019", "votes": "7,354", "genre": "Comedy, Drama", "desp": "After Tohru is taken in by the Soma family, she learns that twelve family members transform involuntarily into animals of the Chinese zodiac and helps them deal with the emotional pain caused by the transformations."}, {"m_title": "Parasyte: The Maxim", "r": "8.3", "year": "2014", "votes": "41,979", "genre": "Action, Drama", "desp": "17-year-old Shinichi Izumi is partially infected by a Parasyte, monsters that butcher and consume humans. He must learn to co-exist with the creature if he is to survive both the life of a Parasyte and a human as part monster, part person."}, {"m_title": "Record of Ragnarok", "r": "6.4", "year": "2021", "votes": "7,026", "genre": "Action, Adventure", "desp": "Every 1000 years, the Gods Council assemble to decide the fate of humanity. After 7 million years of human history, the gods come to the decision that humans are irredeemable and must be extinct."}, {"m_title": "Inuyasha", "r": "7.9", "year": "2000", "votes": "16,987", "genre": "Action, Adventure", "desp": "A teenage girl periodically travels back in time to feudal Japan to help a young half-demon recover the shards of a jewel of great power."}, {"m_title": "Summer Time Rendering", "r": "8.2", "year": "2022", "votes": "3,626", "genre": "Drama, Fantasy", "desp": "Upon hearing of Ushios death, Shinpei returns to his hometown of Wakayama City on Hitogashima and reunites with his childhood friends family. The funeral goes smoothly, but under the. watch the dude, no spoilers in summary!"}, {"m_title": "Detective Conan", "r": "8.5", "year": "1996", "votes": "13,974", "genre": "Action, Comedy", "desp": "The cases of a detective whose physical age was chemically reversed to that of a prepubescent boy but must hide his true mental development."}, {"m_title": "Fire Force", "r": "7.6", "year": "2019", "votes": "8,806", "genre": "Action, Drama", "desp": "A superhuman firefighter force is formed to deal with supernatural fire incidents."}, {"m_title": "Classroom of the Elite", "r": "7.7", "year": "2017", "votes": "9,421", "genre": "Drama, Thriller", "desp": "When Kiyotaka enters government-sponsored elite high school, he finds out just how merit-based this education system is."}, {"m_title": "I Got a Cheat Skill in Another World and Became Unrivaled in the Real World, Too", "r": "6.9", "year": "2023", "votes": "341", "genre": "Action, Adventure", "desp": "A door to another world stretches out before a boy whos been brutally bullied all his life, which grants him access to get cheat skills and a portal which lets him travel between his old and new worlds."}, {"m_title": "Fullmetal Alchemist", "r": "8.5", "year": "2003", "votes": "73,346", "genre": "Action, Adventure", "desp": "When a failed alchemical ritual leaves brothers Edward and Alphonse Elric with severely damaged bodies, they begin searching for the one thing that can save them: the fabled philosophers stone."}, {"m_title": "Baki", "r": "6.7", "year": "2018", "votes": "7,184", "genre": "Action, Drama", "desp": "The protagonist, Baki Hanma, trains with an intense focus to become strong enough to surpass his father, Yujiro Hanma, the strongest fighter in the world."}, {"m_title": "Tomo-chan Is a Girl!", "r": "7.6", "year": "2023", "votes": "1,254", "genre": "Comedy, Drama", "desp": "Tomo is a tomboy who has been in love with her childhood friend Junichiro, and strives to make him see her as a romantic candidate."}, {"m_title": "86", "r": "8.2", "year": "2021", "votes": "5,812", "genre": "Action, Drama", "desp": "The Republic of San Magnolia is at war with its neighboring country, the Empire of Giad. Both sides use unmanned drones to conduct a war without casualties. The story follows Lena as she commands a squad of drones called the 86."}, {"m_title": "Goblin Slayer", "r": "7.5", "year": "2018", "votes": "8,631", "genre": "Action, Adventure", "desp": "In a fantasy world, a lone hero makes his living by exterminating all goblins he encounters. But one day he meets a friend, and his life begins to become more intense."}, {"m_title": "Pokémon Horizons", "r": "8.4", "year": "2023", "votes": "119", "genre": "Action, Adventure", "desp": "Liko, whose partner Pokémon is Sprigatito, and Roy will encounter many characters during their journey, including a group called the Rising Volt Tacklers."}, {"m_title": "My Love Story with Yamada-kun at Lv999", "r": "7.8", "year": "2023", "votes": "265", "genre": "Comedy, Romance", "desp": "After her boyfriend breaks up with her for another girl, Akane Kinoshita decides she is going to make him regret his decision. Unexpectedly, however, she meets a socially awkward gamer who might just solve all of her problems."}, {"m_title": "Is It Wrong to Try to Pick Up Girls in a Dungeon?", "r": "7.4", "year": "2015", "votes": "9,148", "genre": "Action, Adventure", "desp": "In the city of Orario, where adventurers hunt monsters in a place called Dungeon, Bell Cranels life was saved by a swordswoman, who is then become the source of Bells determination to get stronger."}, {"m_title": "Akame ga Kill!", "r": "7.8", "year": "2014", "votes": "26,332", "genre": "Action, Adventure", "desp": "A countryside boy named Tatsumi sets out on a journey to The Capital to make a name for himself and met a seemingly dangerous group of Assassins known as Night Raid. Their journey begins."}, {"m_title": "Overlord", "r": "7.7", "year": "2015", "votes": "11,656", "genre": "Action, Adventure", "desp": "An office worker in a dystopian world logs onto a video game for the last time only to find out that he, along with his entire guild, has been transported to another reality."}, {"m_title": "Konosuba: Gods Blessing on This Wonderful World!", "r": "7.8", "year": "2016", "votes": "12,629", "genre": "Adventure, Comedy", "desp": "It was a happy day for Kazuma - right up to the moment he died. A goddess intervenes and offers him a second chance in a magical land."}, {"m_title": "KamiKatsu: Working for God in a Godless World", "r": "6.6", "year": "2023", "votes": "114", "genre": "Fantasy", "desp": "Yukitos parents are the leaders of a cult. After he gets sacrificed, he gets reincarnated into another world where religion doesnt exist and porn books are akin to a childs doodles."}, {"m_title": "Assassination Classroom", "r": "8.0", "year": "2013", "votes": "18,028", "genre": "Action, Comedy", "desp": "A powerful creature claims that within a year, Earth will be destroyed by him, but he offers mankind a chance by becoming a homeroom teacher where he teaches his students about how to kill him. An assassination classroom begins."}, {"m_title": "Rascal Does Not Dream of Bunny Girl Senpai", "r": "8.1", "year": "2018", "votes": "13,869", "genre": "Drama, Fantasy", "desp": "The seemingly normal life of Sakuta Azusagawa, a high school student changes drastically when he meets a wild bunny girl that seems to be invisible to others."}, {"m_title": "Highschool of the Dead", "r": "7.2", "year": "2010", "votes": "14,213", "genre": "Action, Drama", "desp": "High school students are overwhelmed with the start of the zombie apocalypse."}, {"m_title": "To Your Eternity", "r": "8.4", "year": "2021", "votes": "8,634", "genre": "Adventure, Drama", "desp": "A lonely boy wandering the Arctic regions of North America meets a wolf, and the two become fast friends, depending on each other to survive the harsh environment. But the boy has a history, and the wolf is more than meets the eye as well."}, {"m_title": "Prison School", "r": "7.6", "year": "2015", "votes": "6,509", "genre": "Comedy", "desp": "Hachimitsu Academy, once an all-girls school, has become co-ed, and teen Kiyoshi is one of five boys to enroll. When he is caught peeping, Kiyoshi is sent to the schools prison, where his punishment is carried out."}, {"m_title": "Psycho-Pass", "r": "8.2", "year": "2012", "votes": "20,431", "genre": "Action, Crime", "desp": "Believing in humanity and order, policewoman Akane Tsunemori obeys the ruling, computerized, precognitive Sibyl System. But when she faces a criminal mastermind who can elude this perfect system, she questions both Sibyl and herself."}, {"m_title": "Ultraman", "r": "6.8", "year": "2019", "votes": "2,281", "genre": "Action, Adventure", "desp": "Shinjiro Hayata learns that his father passed on the Ultraman Factor to him and fights against evil."}, {"m_title": "Dragon Ball", "r": "8.6", "year": "1995", "votes": "61,824", "genre": "Action, Adventure", "desp": "Dragon Ball tells the tale of a young warrior by the name of Son Goku, a young peculiar boy with a tail who embarks on a quest to become stronger and learns of the Dragon Balls, when, once all 7 are gathered, grant any wish of choice."}, {"m_title": "Trigun Stampede", "r": "7.5", "year": "2023", "votes": "1,080", "genre": "Action, Adventure", "desp": "Follow gunman Vash the Stampede as he struggles to maintain his pacifist ways while avoiding the immense bounty on his head."}, {"m_title": "Fighting Spirit", "r": "8.8", "year": "2000", "votes": "7,891", "genre": "Action, Comedy", "desp": "Ippo, a teenage boy with a pure heart and unrelenting determination, discovers a passion for boxing after veteran fighter Takamura saves him from bullies."}, {"m_title": "Mobile Suit Gundam: The Witch from Mercury", "r": "7.7", "year": "2022", "votes": "645", "genre": "Action, Drama", "desp": "In an era when a multitude of corporations have entered space and built a huge economic system, Suletta Mercury, from the remote planet Mercury, transfers to the Asticassia School of. watch the dude, no spoilers in summary!"}, {"m_title": "Samurai Champloo", "r": "8.6", "year": "2004", "votes": "51,374", "genre": "Action, Adventure", "desp": "Fuu, a waitress who works in a teahouse, rescues two master swordsmen, Mugen and Jin, from their execution to help her find the samurai who smells of sunflowers."}, {"m_title": "Horimiya", "r": "8.1", "year": "I) (2021", "votes": "11,344", "genre": "Comedy, Drama", "desp": "Two very different people - an academically successful schoolgirl and a quiet loser schoolboy - meet and develop a friendship."}, {"m_title": "Dororo", "r": "8.3", "year": "2019", "votes": "16,066", "genre": "Action, Adventure", "desp": "A father in pursuit of power gives parts of his new-born son to a host of demons. Years later, the son battles those demons to regain what is rightfully his."}, {"m_title": "Slam Dunk", "r": "8.8", "year": "1993", "votes": "6,399", "genre": "Comedy, Drama", "desp": "About Sakuragi Hanamichi, a freshman of Shohoku High School who joins the basketball team because of the girl he has a crush on, Haruko. Although he is newbie in this sport, he is no ordinary basketball player."}, {"m_title": "The Ancient Magus Bride", "r": "7.7", "year": "2017", "votes": "3,384", "genre": "Drama, Fantasy", "desp": "Hatori Chise, 15 years old. Lost, without hope and without family - she sells herself to a non-human mage known as Elias Ainsworth. Hesitant, she starts a new life with him as his new apprentice."}, {"m_title": "Food Wars", "r": "8.1", "year": "2015", "votes": "11,072", "genre": "Comedy, Drama", "desp": "Soma Yukihira enrolls in an elite culinary school to become a full-time chef and surpass his fathers culinary skills."}, {"m_title": "Black Lagoon", "r": "7.9", "year": "2006", "votes": "16,228", "genre": "Action, Adventure", "desp": "A Japanese businessman, captured by modern-day pirates, is written off and left for dead by his company. Tired of the corporate life, he opts to stick with the mercenaries that kidnapped him, becoming part of their gang."}, {"m_title": "My Dress-Up Darling", "r": "8.0", "year": "2022", "votes": "10,435", "genre": "Comedy, Romance", "desp": "Traumatized by a childhood incident with a friend who took exception to his love of traditional dolls, doll-artisan hopeful Wakana Gojou passes his days as a loner, finding solace in a room at his high school."}, {"m_title": "Darling in the Franxx", "r": "7.3", "year": "2018", "votes": "11,787", "genre": "Action, Drama", "desp": "In a future world where humanity has been driven to endangerment by giant beasts, a strike force is assembled to destroy the monsters and save the world."}, {"m_title": "The Rising of the Shield Hero", "r": "7.9", "year": "2019", "votes": "15,022", "genre": "Action, Adventure", "desp": "A gamer is magically summoned into a parallel universe, where he is chosen as one of four heroes destined to save the world from its prophesied doom."}, {"m_title": "Trigun", "r": "8.2", "year": "1998", "votes": "22,872", "genre": "Action, Adventure", "desp": "Vash the Stampede is the most infamous outlaw on the planet Gunsmoke and with a 60 billion double dollar price on his head the most sought after!"}, {"m_title": "Dragon Ball Z Kai", "r": "8.3", "year": "2009", "votes": "23,475", "genre": "Action, Adventure", "desp": "An HD and enhanced remaster of Dragon Ball Z."}, {"m_title": "Sailor Moon", "r": "7.7", "year": "1992", "votes": "6,870", "genre": "Action, Adventure", "desp": "A group of schoolgirls discover they are incarnations of super-powered alien princesses, and use their abilities to defend the earth."}, {"m_title": "Rent-a-Girlfriend", "r": "7.1", "year": "2020", "votes": "6,763", "genre": "Comedy, Romance", "desp": "After being dumped by his girlfriend, a college student enlists a dating service to rent a girlfriend who, unbeknownst to him, happens to be both a fellow student and his neighbor."}, {"m_title": "Junji Ito Maniac: Japanese Tales of the Macabre", "r": "6.0", "year": "2023", "votes": "2,602", "genre": "Drama, Horror", "desp": "From the mind of horror manga maestro Junji Ito comes a spine-tingling selection of some of his most bizarre, disturbing and terrifying tales."}, {"m_title": "Another", "r": "7.5", "year": "2012", "votes": "16,470", "genre": "Drama, Fantasy", "desp": "A young man named, Koichi Sakakibara, transfers to a new school where he finds himself drawn into a mystery involving a mysterious girl and a series of gruesome deaths."}, {"m_title": "Kurokos Basketball", "r": "8.3", "year": "2012", "votes": "9,893", "genre": "Comedy, Sport", "desp": "Kagami meets Kuroko, the 6th member of the Miracle Generation, and they make a pact to defeat the other members of the team. A battle of light (Kagami) and shadow (Kuroko) begins."}, {"m_title": "Yu Yu Hakusho: Ghost Files", "r": "8.5", "year": "1992", "votes": "14,413", "genre": "Action, Adventure", "desp": "One day, 14-year-old Yusuke Urameshi suddenly finds himself dead, having died pushing a child out of the way of oncoming traffic. Since he has such a bad personality, even the Spirit World. watch the dude, no spoilers in summary!"}, {"m_title": "Komi Cant Communicate", "r": "7.8", "year": "2021", "votes": "7,560", "genre": "Comedy, Drama", "desp": "An elite and aloof schoolgirl is in fact severely withdrawn and anxious about communicating with others. An ordinary schoolboy befriends her and helps her to open up and talk with people."}, {"m_title": "Gurren Lagann", "r": "8.3", "year": "2007", "votes": "19,428", "genre": "Action, Adventure", "desp": "Two friends, Simon and Kamina, become the symbols of rebellion against the powerful Spiral King, who forced mankind into subterranean villages."}, {"m_title": "Nier: Automata Ver1.1a", "r": "7.4", "year": "2023", "votes": "762", "genre": "Action, Adventure", "desp": "Animated adaptation of NieR: Automata (2017), where a long-running war on post-apocalyptic Earth between androids and machines soon unveils the truth of the world."}, {"m_title": "Initial D: First Stage", "r": "8.4", "year": "1998", "votes": "3,549", "genre": "Action, Comedy", "desp": "The story of a young delivery boy who becomes a drift-racing legend."}, {"m_title": "Berserk: The Golden Age Arc - Memorial Edition", "r": "8.5", "year": "2022", "votes": "834", "genre": "Action, Adventure", "desp": "A lone sellsword named Guts gets recruited into a mercenary band and attempts to help the Bands leader, Griffith, on his rise to power."}, {"m_title": "Dragon Ball Z", "r": "8.8", "year": "1989", "votes": "82,587", "genre": "Action, Adventure", "desp": "The adventures of Earths martial arts defender, Son Goku, continue with a new family and the revelation of his alien origins. Now Goku and his allies must defend the planet from an onslaught of new extraterrestrial enemies."}, {"m_title": "Bocchi the Rock!", "r": "8.3", "year": "2022", "votes": "2,589", "genre": "Comedy, Music", "desp": "Hitori Bocchi-chan Goto is a lonely high school girl whose heart lies in her guitar; she meets Nijika Ijichi and joins Kessoku Band."}, {"m_title": "Legend of the Galactic Heroes", "r": "9.0", "year": "1988", "votes": "3,960", "genre": "Action, Drama", "desp": "Various people, especially two rising commanders, cope with a massive continual space conflict between two interstellar nations."}, {"m_title": "Resident Evil: Infinite Darkness", "r": "5.7", "year": "2021", "votes": "11,501", "genre": "Action, Horror", "desp": "Federal agent Leon S. Kennedy teams up with TerraSave staff member Claire Redfield to investigate a zombie outbreak. Based on the popular video game series of the same name by Capcom."}, {"m_title": "Fate/Zero", "r": "8.2", "year": "2011", "votes": "13,811", "genre": "Action, Adventure", "desp": "Seven chosen mages and their summoned heroic spirits fight against each other to try and win the Holy Grail: a magical device that can grant any wish."}, {"m_title": "Dead Mount Death Play", "r": "7.4", "year": "2023", "votes": "121", "genre": "Action, Fantasy", "desp": "A necromancer in a distant era incarnates himself into a young boy in modern Tokyo. But he gets caught in a mystery about the boy and his own past."}, {"m_title": "Why Raeliana Ended Up at the Dukes Mansion", "r": "8.5", "year": "2023", "votes": "72", "genre": "Fantasy, Romance", "desp": "Living inside a fairy tale may sound like a dream, but for this heroine, its more of a nightmare. After her mysterious death, Rinko is reborn as Raeliana - a loved and wealthy character in. watch the anime dude!"}, {"m_title": "My Teen Romantic Comedy SNAFU", "r": "7.9", "year": "2013", "votes": "7,610", "genre": "Comedy, Drama", "desp": "About an antisocial high school student named Hikigaya Hachiman with a distorted view on life and no friends or girlfriend. His life change when he was forced to enter the Volunteer Service Club by his teacher."}, {"m_title": "Bastard!! Heavy Metal, Dark Fantasy", "r": "6.5", "year": "2022", "votes": "1,952", "genre": "Action, Adventure", "desp": "When evil forces threaten to resurrect Anthrasax, the God of Destruction, the Kingdom of Meta-llicana calls on a volatile dark wizard for help."}, {"m_title": "Banana Fish", "r": "8.2", "year": "2018", "votes": "6,940", "genre": "Action, Adventure", "desp": "The story captures the war between a mafia boss named Dino and a young gang leader searching for redemption."}, {"m_title": "Blade Runner: Black Lotus", "r": "6.2", "year": "2021", "votes": "3,005", "genre": "Action, Drama", "desp": "Animated series that takes place 17 years before Blade Runner 2049."}, {"m_title": "Redo of Healer", "r": "6.4", "year": "2021", "votes": "3,892", "genre": "Action, Adventure", "desp": "Keyaru is a healing magician who is sexually abused repeatedly by others. Using his healing magic, he goes back four years into the past to get revenge on the ones who abused him."}, {"m_title": "How NOT to Summon a Demon Lord", "r": "7.0", "year": "2018", "votes": "3,977", "genre": "Adventure, Comedy", "desp": "An elite player is summoned by two maidens to save a fantasy game."}, {"m_title": "TONIKAWA: Over the Moon for You", "r": "7.7", "year": "2020", "votes": "3,972", "genre": "Comedy, Romance", "desp": "A Young Prodigy accepts a Sudden Proposal from a Mysterious Woman who he falls in love with after she saves him from a Near-Death Experience."}, {"m_title": "Knights of the Zodiac: Saint Seiya", "r": "4.6", "year": "2019", "votes": "2,059", "genre": "Action, Adventure", "desp": "A digital remake from the Japanese anime classic Knights of the Zodiac (1986)."}, {"m_title": "Rurouni Kenshin", "r": "8.5", "year": "1996", "votes": "19,822", "genre": "Action, Adventure", "desp": "The adventures of a young wandering swordsman who stumbles upon a struggling martial arts school in Meiji era Japan."}, {"m_title": "The Quintessential Quintuplets", "r": "7.7", "year": "2019", "votes": "5,806", "genre": "Comedy, Drama", "desp": "A poor, straight A student is hired to tutor some rich quintuplets. The story is told as a flashback from a future that shows that he ends up getting married to one of them."}, {"m_title": "KILL la KILL", "r": "7.9", "year": "2013", "votes": "15,407", "genre": "Action, Comedy", "desp": "A young girl arrives at a school of superhumans to find out the truth behind her fathers murder."}, {"m_title": "Hunter x Hunter", "r": "8.6", "year": "1999", "votes": "8,762", "genre": "Action, Adventure", "desp": "The history of Gon Freecs, a boy who left home alone to become a Hunter. In the hard test, Gon meets Leorio, Kurapika and Killua."}, {"m_title": "Skip and Loafer", "r": "8.1", "year": "2023", "votes": "175", "genre": "Comedy, Drama", "desp": "Excellent student Iwakura Mitsumi has always dreamed about leaving her small town, going to a prestigious university, and making positive change in the world. But shes so focused on. watch the dude, no spoilers in summary!"}, {"m_title": "The Dangers in My Heart", "r": "7.3", "year": "2023", "votes": "180", "genre": "Comedy, Drama", "desp": "Ichikawa Kyoutarou, a boy barely clinging to the bottom rung of his schools social ladder, secretly believes hes the tortured lead in some psychological thriller. He spends his days. watch the dude, no spoilers in summary!"}, {"m_title": "Uncle from Another World", "r": "7.4", "year": "2022", "votes": "1,551", "genre": "Adventure, Comedy", "desp": "Takafumis uncle wakes after a 17-year coma. He had actually gone to another world, and has returned with magical powers. But now that hes back, Takafumi must help him to survive in his home world."}, {"m_title": "Kakegurui", "r": "7.1", "year": "2017", "votes": "11,078", "genre": "Drama, Mystery", "desp": "A gambling prodigy comes to an elite school run by games and turns the order upside down."}, {"m_title": "Beastars", "r": "7.7", "year": "2019", "votes": "11,055", "genre": "Crime, Drama", "desp": "In a world of anthropomorphic animals, a reclusive wolfs complicated relationship with a kindly rabbit is tested by a classmates murder, a charismatic deers influence, and his own burgeoning predatory instincts."}, {"m_title": "Call of the Night", "r": "7.4", "year": "2022", "votes": "2,128", "genre": "Comedy, Fantasy", "desp": "Kou Yamori is an average middle school student who struggles with grasping the complex concept of love. Because he sees little sense in surrendering to the norm, he soon stops going to. watch the dude, no spoilers in summary!"}, {"m_title": "Super Dragon Ball Heroes", "r": "6.7", "year": "2018", "votes": "2,826", "genre": "Action, Adventure", "desp": "The Z-Fighters set out to rescue Trunks from a prison planet."}, {"m_title": "The Misfit of Demon King Academy", "r": "7.3", "year": "2020", "votes": "4,137", "genre": "Adventure, Comedy", "desp": "A demon mage reincarnates into the body of his descendant hoping to find an era where he can have some peace and quiet. He ends up going to a school run by his significantly weaker descendants."}, {"m_title": "My Home Hero", "r": "7.2", "year": "2023", "votes": "195", "genre": "Crime, Drama", "desp": "Tetsuo discovers that Matori, his daughters boyfriend, has been physically abusive towards her. Tetsuo uncovers that Matori belongs to a criminal gang and that he plans to extort money from Reikas wealthy grandparents."}, {"m_title": "Nana", "r": "8.5", "year": "2006", "votes": "4,644", "genre": "Comedy, Drama", "desp": "Two ostensibly opposite women, both named Nana, become roommates in Tokyo and grow inseparable. However, their relationship is jeopardized as the harsh realities of adult life take shape."}, {"m_title": "Edens Zero", "r": "7.1", "year": "2021", "votes": "1,679", "genre": "Action, Adventure", "desp": "A young boy cut off from the world meets his friends and sets off on a hard, long journey to be reborn."}, {"m_title": "Kingdom", "r": "8.5", "year": "2012", "votes": "2,517", "genre": "Action, Drama", "desp": "In ancient China, a young boy dreams of becoming the greatest general of the Qin Kingdom."}, {"m_title": "Ouran High School Host Club", "r": "8.2", "year": "2006", "votes": "10,804", "genre": "Comedy, Romance", "desp": "Youll fall for the Ouran Host Club: Tamakis truly romantic. Kaoru and Hikaru display brotherly love, Kyoyas brainy, Honeys innocent, and Moris manly. Oh, and dont forget Haruhi. He knows what girls want, because hes a girl too."}, {"m_title": "Arifureta: From Commonplace to Worlds Strongest", "r": "7.1", "year": "2019", "votes": "3,442", "genre": "Action, Adventure", "desp": "Seventeen-year-old Hajime Nagumo is your average, everyday otaku. However, his simple life of pulling all-nighters and sleeping in school is suddenly turned upside down when he, along with. watch the dude, no spoilers in summary!"}, {"m_title": "Buddy Daddies", "r": "8.0", "year": "2023", "votes": "989", "genre": "Action, Comedy", "desp": "Assassins Kazuki Kurusu and Rei Suwa meet Miri, a girl looking for her father on Christmas Day. Kazuki, Rei, and Miri unexpectedly end up living together."}, {"m_title": "Ergo Proxy", "r": "7.9", "year": "2006", "votes": "15,122", "genre": "Action, Adventure", "desp": "In a post-apocalyptic future humans live in peace with androids in a domed city. However, a strange series of murders has intruded into bored inspector Re-l Mayers life."}, {"m_title": "The Future Diary", "r": "7.5", "year": "2011", "votes": "18,509", "genre": "Action, Drama", "desp": "A young man competes with people around the world for a chance to become the succesor of God, with a diary that is able to tell the future."}, {"m_title": "Konosuba: An Explosion on This Wonderful World!", "r": "7.2", "year": "2023", "votes": "179", "genre": "Adventure, Comedy", "desp": "One year before a certain useless goddess and NEET extraordinaire hit the scene, Megumin, the Greatest Genius of the Crimson Magic Clan, is hard at work. Ever since a life-changing. watch the dude, no spoilers in summary!"}, {"m_title": "The Legendary Hero Is Dead!", "r": "6.5", "year": "2023", "votes": "115", "genre": "Action, Adventure", "desp": "Sion Bladen sealed away the Demon King and his army at Hells Gate. Now with the Demon King ready to rise again, Sion jumps back into battle to face his sworn enemy. Unfortunately, he ends. watch the dude, no spoilers in summary!"}, {"m_title": "Black Butler", "r": "7.7", "year": "2008", "votes": "8,561", "genre": "Action, Comedy", "desp": "Ciel Phantomhive takes care of the many unsettling events in Victorian England. Aided by Sebastian Michaelis, his loyal butler with seemingly inhuman abilities. But is there more to this black-clad butler than meets the eye?"}, {"m_title": "Darker Than Black", "r": "7.7", "year": "2007", "votes": "12,118", "genre": "Action, Drama", "desp": "In Tokyo, an impenetrable field known as Hells Gate appeared ten years ago. At the same time, psychics who wield paranormal powers at the cost of their conscience also emerged. Hei is. watch the dude, no spoilers in summary!"}, {"m_title": "Noragami", "r": "7.9", "year": "2014", "votes": "17,098", "genre": "Action, Adventure", "desp": "A minor god seeking to gain widespread worship teams up with a human girl he saved to gain fame, recognition and at least one shrine dedicated to him."}, {"m_title": "Digimon: Digital Monsters", "r": "7.3", "year": "1999", "votes": "22,657", "genre": "Action, Adventure", "desp": "A group of young teens is unexpectedly sent to the mysterious Digital World and paired up with their own powerful, morphing monster called the Digimon. Together the entire group set out on an adventure to fight evil and save the world."}, {"m_title": "Monster Musume: Everyday Life with Monster Girls", "r": "6.6", "year": "2015", "votes": "2,885", "genre": "Comedy, Fantasy", "desp": "In a world where humans co-exist with fantasy beings, a young man must share a flat with a bevy of beastly beauties."}, {"m_title": "The Disastrous Life of Saiki K.", "r": "8.3", "year": "2016", "votes": "8,821", "genre": "Comedy, Fantasy", "desp": "Saiki Kusuo is a powerful psychic who hates attracting attention, yet he is surrounded by colorful characters who always find a way to remove him from his everyday life."}, {"m_title": "Ghost in the Shell: Stand Alone Complex", "r": "8.5", "year": "2002", "votes": "20,231", "genre": "Action, Crime", "desp": "The futuristic adventures of a female cyborg counter intelligence agent and her support team."}, {"m_title": "High-Rise Invasion", "r": "6.5", "year": "2021", "votes": "3,580", "genre": "Action, Horror", "desp": "High schooler Yuri finds herself atop a skyscraper in a strange world, where she must survive against masked assailants bent on killing their prey."}, {"m_title": "Tokyo Ghoul: re", "r": "6.6", "year": "2018", "votes": "10,780", "genre": "Action, Drama", "desp": "Two years have passed since the CCGs raid on Anteiku. Although the atmosphere in Tokyo has changed drastically due to the increased influence of the CCG, ghouls continue to pose a problem. watch the dude, no spoilers in summary!"}, {"m_title": "Mushi-Shi", "r": "8.5", "year": "2005", "votes": "13,981", "genre": "Drama, Fantasy", "desp": "Ginko is an expert travelling around to investigate a primeval life-form, the Mushi, and helping people with Mushi-related supernatural problems."}, {"m_title": "Baki the Grappler", "r": "7.5", "year": "2001", "votes": "1,794", "genre": "Action, Drama", "desp": "Baki Hanma competes in an underground fighting tournament organized by Tokugawa. Masters of various fighting styles come from all over the world in order to determine who is the strongest fighter on Earth."}, {"m_title": "Interspecies Reviewers", "r": "7.4", "year": "2020", "votes": "1,757", "genre": "Comedy, Fantasy", "desp": "From elves to succubi to fairies and more, Our heroes: Stunk, a human, Zel, an elf and a hermaphrodite angel named Crimvael are here to rate the red-light delights of all manner of monster girls."}, {"m_title": "Dorohedoro", "r": "8.1", "year": "2020", "votes": "9,813", "genre": "Action, Comedy", "desp": "In a sad town a clan of sorcerers use its people to experiment with the dark arts."}, {"m_title": "Insomniacs After School", "r": "7.8", "year": "2023", "votes": "154", "genre": "Comedy, Drama", "desp": "Two insomniac students meet and start to spend their nights in an observatory together."}, {"m_title": "Deadman Wonderland", "r": "7.1", "year": "2011", "votes": "11,263", "genre": "Action, Drama", "desp": "After being falsely convicted for the murder of his entire class, a young man must learn to survive in a mysterious prison with a perverted take on incarceration while also looking to clear his name."}, {"m_title": "Tomodachi Game", "r": "7.6", "year": "2022", "votes": "2,312", "genre": "Drama, Mystery", "desp": "Yuuichi Katagiri knows the hardships of not having enough funds. He works hard to save up in order to go on the high school trip. However, after the class money is all collected; its stolen. Suspicion falls on two of Yuuichis friends."}, {"m_title": "Campfire Cooking in Another World with My Absurd Skill", "r": "7.6", "year": "2023", "votes": "754", "genre": "Adventure, Comedy", "desp": "Tsuyoshi Mukouda is transported to a fantasy world and discovers his only skill is Food Menu."}, {"m_title": "Baki Hanma", "r": "6.7", "year": "2021", "votes": "2,316", "genre": "Action, Drama", "desp": "To gain the skills he needs to surpass his powerful father, Baki enters Arizona State Prison to take on the notorious inmate known as Mr. Unchained."}, {"m_title": "More Than a Married Couple, But Not Lovers.", "r": "7.4", "year": "2022", "votes": "1,249", "genre": "Comedy, Drama", "desp": "High school student Jiro Yakuin paired with Akari Watanabe, a gal who finds Jiro repulsive at first, but they agree to attain enough points so that they can switch partners to the people they really like."}, {"m_title": "Welcome to Demon-School, Iruma-kun", "r": "7.7", "year": "2019", "votes": "1,944", "genre": "Comedy, Drama", "desp": "A human child is adopted by a demon and sent to a demon school. He must find a way to survive in the dangerous but fun world he has entered."}, {"m_title": "Seraph of the End", "r": "7.4", "year": "2015", "votes": "5,983", "genre": "Action, Adventure", "desp": "In 2012, the world allegedly comes to an end at the hands of a human-made virus, ravaging the global populace and leaving only children untouched. It is at this time that vampires emerge from the recesses of the earth."}, {"m_title": "Toradora!", "r": "8.0", "year": "2008", "votes": "16,940", "genre": "Comedy, Drama", "desp": "Toradora tells the tale of Ryuji (dragon) and Taiga (tiger) helping each other confess to their crushes."}, {"m_title": "Moriarty the Patriot", "r": "8.0", "year": "2020", "votes": "3,111", "genre": "Crime, Mystery", "desp": "William James Moriarty lives as a regular noble while also being a consultant for the common folk to solve their problems. However, deep inside him lies a desire to destroy the current structure that dominates British society."}, {"m_title": "Soul Eater", "r": "7.8", "year": "2008", "votes": "13,606", "genre": "Action, Adventure", "desp": "Set in the Shinigami technical school for weapon meisters, the series revolves around 3 duos. These pairs are a partnership between a weapon meister and a human weapon. Trying to reach a. watch the dude, no spoilers in summary!"}, {"m_title": "Bikini Warriors", "r": "4.9", "year": "2015", "votes": "419", "genre": "Comedy, Fantasy", "desp": "When darkness threatens the world, four heroines hold the only hope for salvation - if they can even manage to get out of the first town, that is."}, {"m_title": "Claymore", "r": "8.0", "year": "2007", "votes": "14,349", "genre": "Action, Adventure", "desp": "In a world rife with deadly creatures called youma, a young silver eyed woman, Clare, works on behalf of an organization that trains female youma halfbreeds into warriors with the ability. watch the anime dude!"}, {"m_title": "Domestic Girlfriend", "r": "6.8", "year": "2019", "votes": "3,701", "genre": "Drama, Romance", "desp": "The daily life of high-school student Natsuo is heavily complicated after his single father marries the mother of his teacher crush Hina Tachibana and her sister Rui - who he had shared his first experience with with at a preceding mixer."}, {"m_title": "Gantz", "r": "7.2", "year": "2004", "votes": "8,233", "genre": "Action, Drama", "desp": "A group of people are resurrected from their deaths by a mysterious black ball called GANTZ to combat alien criminals hiding on Earth."}, {"m_title": "Kimi ni Todoke: From Me to You", "r": "7.8", "year": "2009", "votes": "4,468", "genre": "Comedy, Drama", "desp": "Sawako Kuronuma is misunderstood due to her resemblance to the ghost girl from The Ring. But one day the nicest boy in the class, Kazehaya befriends her and everything changed after that and also everyone perspective of Sawako but theres going to be struggle await for her up in the future."}, {"m_title": "Ranking of Kings: The Treasure Chest of Courage", "r": "8.3", "year": "2023", "votes": "80", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Harem in the Labyrinth of Another World", "r": "6.2", "year": "2022", "votes": "1,004", "genre": "Action, Adventure", "desp": "A high school student is transported to a fantasy world with distinct cheating advantages. He sets out on a quest to gain a harem of beautiful girls."}, {"m_title": "Dragon Ball GT", "r": "6.8", "year": "1996", "votes": "27,660", "genre": "Action, Adventure", "desp": "After Goku is made a kid again by the Black Star Dragon Balls, he goes on a journey to get back to his old self."}, {"m_title": "Aoashi", "r": "8.1", "year": "2022", "votes": "2,284", "genre": "Comedy, Drama", "desp": "Ashito Aoi, who lives in Ehime prefecture. Ashito has a strong talent in soccer, but he tries to hide it. Due to his very straightforward personality, he causes a disaster that serves as a huge setback for him"}, {"m_title": "Why the Hell are You Here, Teacher!?", "r": "6.3", "year": "2019", "votes": "1,706", "genre": "Short, Comedy", "desp": "A comedy about female teachers who cant help developing romantic interests in their male students."}, {"m_title": "Super Crooks", "r": "7.1", "year": "2021", "votes": "2,837", "genre": "Action, Adventure", "desp": "Johnny Bolt recruits a group of ragtag supervillains for one last heist. Their target: A ruthless super-powered crime boss. What can go wrong?"}, {"m_title": "Worlds End Harem", "r": "5.9", "year": "2021", "votes": "1,293", "genre": "Drama, Romance", "desp": "A man wakes from suspended animation to find himself in a world where only a million men survived an epidemic. He is selected for breeding as part of a repopulation program."}, {"m_title": "Aggretsuko", "r": "8.0", "year": "2018", "votes": "6,958", "genre": "Short, Comedy", "desp": "Frustrated with her thankless office job, the 25-year-old red panda copes with her daily struggles by belting out heavy metal karaoke after work."}, {"m_title": "Steins;Gate 0", "r": "8.4", "year": "2018", "votes": "10,091", "genre": "Drama, Mystery", "desp": "Mad scientist Okabe struggles to recover from past failure and tests a new artificial intelligence"}, {"m_title": "Great Teacher Onizuka", "r": "8.5", "year": "1999", "votes": "9,987", "genre": "Comedy, Drama", "desp": "About Eikichi Onizuka, a 22-year-old ex-gangster member and a virgin. He has one ambition that no one ever expected from him. His solely life purpose is to become the greatest high school teacher ever."}, {"m_title": "To LOVE-Ru", "r": "6.4", "year": "2008", "votes": "1,458", "genre": "Action, Comedy", "desp": "When an alien from the planet Deviluke runs away and goes into hiding on earth,she meets a human named Yuuki Rito. Watch as she wants to get married to him while he wants no part of it"}, {"m_title": "Supernatural: The Animation", "r": "7.3", "year": "2011", "votes": "2,129", "genre": "Action, Drama", "desp": "After losing their mother to a demon, two brothers grow up fighting supernatural beings."}, {"m_title": "Romantic Killer", "r": "7.6", "year": "2022", "votes": "1,751", "genre": "Comedy, Fantasy", "desp": "High schooler Anzu Hoshino has a great life. Every day she plays video games, pigs out on snacks, and pets her beloved cat. But this blissful existence is turned into a confusing mess when. watch the dude, no spoilers in summary!"}, {"m_title": "Scums Wish", "r": "6.8", "year": "I) (2017", "votes": "2,136", "genre": "Drama, Romance", "desp": "A perfect couple struggles with a secret longing each has for someone else."}, {"m_title": "The Testament of Sister New Devil", "r": "6.6", "year": "2015", "votes": "2,913", "genre": "Action, Comedy", "desp": "First-year high school student, Toujo Basara, was suddenly have two beautiful step-sisters adopted by his father. But Mio and Marias true forms are actually the newbie Demon Lord and a succubus!?"}, {"m_title": "My Life as Inukai-sans Dog", "r": "4.3", "year": "2023", "votes": "242", "genre": "Comedy", "desp": "A boy is transformed into a dog and becomes the pet to a pretty schoolgirl."}, {"m_title": "When They Cry", "r": "7.9", "year": "2006", "votes": "6,059", "genre": "Drama, Horror", "desp": "The story of a group of young friends and the mysterious events that occur in the rural village of Hinamizawa."}, {"m_title": "Fate/stay night", "r": "7.3", "year": "2006", "votes": "6,106", "genre": "Action, Adventure", "desp": "Amateur mage Shirou Emiya gets involved in a fight between mages using mythological heroes."}, {"m_title": "Maken-Ki! Battling Venus", "r": "5.8", "year": "2011", "votes": "518", "genre": "Action, Comedy", "desp": "Takeru Ohyama is a typical perverted teenage boy. His new school doesnt require entrance exams, and it just turned co-ed! Unfortunately, his dreams of a happy high school life are dashed. watch the dude, no spoilers in summary!"}, {"m_title": "The Way of the Househusband", "r": "7.3", "year": "2021", "votes": "5,339", "genre": "Action, Comedy", "desp": "A retired gangster spends his time as a househusband carrying out home chores."}, {"m_title": "Dont Toy with Me, Miss Nagatoro", "r": "7.2", "year": "2021", "votes": "4,083", "genre": "Comedy, Romance", "desp": "High schooler Hayase Nagatoro loves to spend her free time doing one thing, and that is to bully her Senpai. After Nagatoro and her friends stumble upon the aspiring artists drawings, they. watch the anime dude!"}, {"m_title": "The Case Study of Vanitas", "r": "7.6", "year": "2021", "votes": "2,120", "genre": "Fantasy, Horror", "desp": "A human wields a magic book that can cure vampires of their bloodlust, and sets out to find vampires to cure with the book."}, {"m_title": "Welcome to the N.H.K.", "r": "8.3", "year": "2006", "votes": "7,298", "genre": "Comedy, Drama", "desp": "This surreal dramedy follows Satou Tatsuhiro as he attempts to escape the evil machinations of the NHK."}, {"m_title": "Farming Life in Another World", "r": "7.1", "year": "2023", "votes": "422", "genre": "Fantasy", "desp": "Reborn in another world, a young man chooses to live a quiet life as a farmer."}, {"m_title": "Rosario + Vampire", "r": "6.9", "year": "2008", "votes": "3,391", "genre": "Comedy, Fantasy", "desp": "Tskune accidentally gets on a bus to a school of monsters."}, {"m_title": "Maid Sama!", "r": "7.9", "year": "2010", "votes": "7,969", "genre": "Comedy, Romance", "desp": "Ayuzawa Misaki serves as the student council president at Seika High. However, unbeknownst to her classmates, she works part-time as an employee at a Maid Cafe. Usui Takumi, a boy from her school, discovers this secret."}, {"m_title": "Ghost Stories", "r": "8.0", "year": "2000", "votes": "2,339", "genre": "Comedy, Fantasy", "desp": "A group of school kids and a possessed cat fight ghosts in their neighborhood."}, {"m_title": "Miss Kobayashis Dragon Maid", "r": "7.8", "year": "2017", "votes": "4,746", "genre": "Comedy, Fantasy", "desp": "An office worker employs a kindhearted dragon to serve as her maid, and enters a world of comic misadventures."}, {"m_title": "Fate/stay night [Unlimited Blade Works]", "r": "8.0", "year": "2014", "votes": "8,349", "genre": "Action, Adventure", "desp": "A group of seven mages gets chosen to become masters of seven classes of heroic spirits, in order to fight and win the Holy Grail."}, {"m_title": "The Qwaser of Stigmata", "r": "5.5", "year": "2010", "votes": "412", "genre": "Action, Comedy", "desp": "When tomo and her adopted sister Mafuyu Oribe rescue a strange wounded man, they have no idea what theyre getting involved with or what the consequences will be. Alexander Nikolaevith Hell. watch the anime dude!"}, {"m_title": "The Angel Next Door Spoils Me Rotten", "r": "7.4", "year": "2023", "votes": "814", "genre": "Comedy, Romance", "desp": "Mahiru is so beautiful that they call her angel, Amane is an average boy who has never thought much about beauty, but everything changes when he sees Mahiru during a storm, and an incredible and unlikely relationship begins."}, {"m_title": "The Worlds Finest Assassin Gets Reincarnated in Another World as an Aristocrat", "r": "7.3", "year": "2021", "votes": "2,505", "genre": "Action, Fantasy", "desp": "When the worlds greatest assassin reaches the end of his life, he is sent by a goddess to be reborn in a magical world and use his skills to save that world."}, {"m_title": "Gudetama: An Eggcellent Adventure", "r": "7.3", "year": "2022", "votes": "1,290", "genre": "Short, Adventure", "desp": "A lethargic, empathetic road movie about finding ones parents - for everyone who just wants to laze about. Having resigned itself to the fact that it will just end up on someones plate, Gudetama just wants to be lazy all the time."}, {"m_title": "Golden Kamuy", "r": "8.1", "year": "2018", "votes": "2,057", "genre": "Action, Adventure", "desp": "In the early 20th century in Hokkaido, Japan, a treasure hunt between various parties takes place for a hidden stash of gold."}, {"m_title": "ReLIFE", "r": "7.8", "year": "2016", "votes": "5,513", "genre": "Comedy, Drama", "desp": "A young man, failing at life, is offered a new start as a 10 years younger high-school student."}, {"m_title": "Bakemonogatari", "r": "8.0", "year": "2009", "votes": "6,082", "genre": "Action, Comedy", "desp": "Third-year high school student Koyomi Araragi is human again. Cured of his vampirism, he seeks to help other supernaturals with their problems. Koyomi becomes involved in their lives, revealing secrets in people he once knew."}, {"m_title": "Clannad", "r": "7.8", "year": "2007", "votes": "9,855", "genre": "Comedy, Drama", "desp": "A high school student who cares little about school or others meets a lonely girl who had to repeat a year while all her friends finished high school. He decides to hang out with her and soon meets more friendly students."}, {"m_title": "Shaman King", "r": "7.0", "year": "2021", "votes": "2,144", "genre": "Action, Adventure", "desp": "Shaman King follows the adventures of a 13-year-old shaman and his teammate a samurai warrior spirit, who traverse the world fighting evil spirits and misguided shamans on their journey to be the next Shaman King."}, {"m_title": "Blue Spring Ride", "r": "7.5", "year": "2014", "votes": "5,547", "genre": "Comedy, Drama", "desp": "Revolves around Futaba, a girl who was in love with a boy named Ko Tanaka in middle school. However, it did not work because he transferred but in high school, her world is turned around once again when she meets him again."}, {"m_title": "Ikki tousen", "r": "5.9", "year": "2003", "votes": "1,226", "genre": "Action, Adventure", "desp": "Seven rival high schools battle each other for victory and total supremacy. The students will do what ever it takes to come out on top and see their school leading the way."}, {"m_title": "Hyouka", "r": "7.8", "year": "2012", "votes": "5,493", "genre": "Comedy, Drama", "desp": "Hyouka centers on Oreki Hotarou and his life in the Classic Literature Club. His mundane world is flipped upside down as they begin to solve various mysteries."}, {"m_title": "Golden Time", "r": "7.6", "year": "2013", "votes": "4,093", "genre": "Comedy, Drama", "desp": "Banri Tada is a freshman at a Tokyo law school. After an accident, he suffers severe memory loss. Despite the incident, he befriends fellow freshman, Mitsuo Yanagisawa, which leads him to the beautiful, yet obsessive, Kouko Kaga."}, {"m_title": "Yasuke", "r": "6.2", "year": "2021", "votes": "5,660", "genre": "Action, Adventure", "desp": "He came from Africa and fought alongside a mighty feudal lord in brutal 16th century Japan. They called him the Black Samurai, and he became a legend."}, {"m_title": "Grand Blue", "r": "7.8", "year": "2018", "votes": "3,586", "genre": "Adventure, Comedy", "desp": "A college student spends his year at the seaside town of Izu, having fun on the beach with his school friends."}, {"m_title": "Great Pretender", "r": "7.9", "year": "2020", "votes": "5,872", "genre": "Action, Adventure", "desp": "A Japanese conman crosses paths with a French rival, and gets entangled in conspiracies and scams beyond his wildest dreams."}, {"m_title": "Blue Exorcist", "r": "7.4", "year": "2011", "votes": "14,040", "genre": "Action, Drama", "desp": "After discovering that hes the son of Satan, a young man must join the True Cross Academy in order to master his abilities and defeat Satan himself."}, {"m_title": "Heavens Lost Property", "r": "7.2", "year": "2009", "votes": "1,953", "genre": "Action, Comedy", "desp": "Girl-crazy Tomokis quiet life gets turned upside down when beautiful, winged Ikaros falls from the sky -- and starts calling him master."}, {"m_title": "Queens Blade: The Exiled Virgin", "r": "6.1", "year": "2009", "votes": "455", "genre": "Action, Adventure", "desp": "Multiple women from around the continent come together to prepare for the Queens Blade tournament, a tournament that tests the skills of the warriors."}, {"m_title": "Lycoris Recoil", "r": "7.6", "year": "2022", "votes": "2,051", "genre": "Action, Comedy", "desp": "In a downtown Tokyo café, a group of girls will help sort out any problems to help the local customers."}, {"m_title": "Goddess Cafe Terrace", "r": "7.5", "year": "2023", "votes": "69", "genre": "Comedy, Romance", "desp": "When Hayatos grandmother passed, he planned to sell her rundown cafe on the shore-not realizing that it was also home to five young women. but even their best intentions might not be enough to make it work."}, {"m_title": "Pokémon Journeys: The Series", "r": "7.1", "year": "2019", "votes": "851", "genre": "Adventure, Comedy", "desp": "With his partner Go and his Pokémon Scorbunny in the Galar region, Ash with his Pikachu tries to capture Mew and all the Pokémon. Meanwhile, the three members of Team Rocket Jessie, James. watch the dude, no spoilers in summary!"}, {"m_title": "Magi: The Labyrinth of Magic", "r": "7.8", "year": "2012", "votes": "4,380", "genre": "Action, Adventure", "desp": "Aladdin, Alibaba, and Morgiana go their separate ways after spending time together in Sindria to develop their own strengths, make new friends and prepare for what is coming."}, {"m_title": "Tower of God", "r": "7.6", "year": "2020", "votes": "6,349", "genre": "Action, Adventure", "desp": "Reach the top, and everything will be yours. At the top of the tower exists everything in this world, and all of it can be yours."}, {"m_title": "Bofuri: I Dont Want to Get Hurt, So Ill Max Out My Defense", "r": "7.4", "year": "2020", "votes": "2,041", "genre": "Action, Adventure", "desp": "Honjou Kaede is invited by her friend Shiramine Risa to play a virtual reality MMO game with her. While Kaede doesnt dislike games, what she really, truly dislikes is being in pain."}, {"m_title": "Kamisama hajimemashita", "r": "8.1", "year": "2012", "votes": "4,342", "genre": "Comedy, Fantasy", "desp": "Nanami is left homeless after her father runs away due to debts. When she saves a man named Mikage from dogs, he gives her his house which turns out to be a shrine and she becomes the new deity."}, {"m_title": "The God of High School", "r": "7.2", "year": "2020", "votes": "7,103", "genre": "Action, Adventure", "desp": "A tournament takes place to determine the best fighter among all high school students in Korea."}, {"m_title": "Odd Taxi", "r": "8.4", "year": "2021", "votes": "4,772", "genre": "Comedy, Crime", "desp": "In a city which should be familiar, there is the feeling of something a little different."}, {"m_title": "Saekano: How to Raise a Boring Girlfriend", "r": "7.0", "year": "2015", "votes": "1,403", "genre": "Comedy, Drama", "desp": "Aki,a highschool otaku work to support his game making. With remarkable luck, he bumps into Kato, the most beautiful girl he has ever seen. Naturally, the meeting twists his life into a complicated relationships."}, {"m_title": "Saikyou Onmyouji no Isekai Tenseiki", "r": "7.1", "year": "2023", "votes": "529", "genre": "Action, Adventure", "desp": "Haruyoshi Kuga uses a reincarnation spell to bring Seika Lamprogue happiness, but his onmyo art exceeds the worlds magic."}, {"m_title": "Fist of the North Star", "r": "8.2", "year": "1984", "votes": "3,345", "genre": "Action, Adventure", "desp": "After a nuclear war turns Earth into a lawless wasteland, Kenshiro, a practitioner of the deadly master art Hokuto Shinken, fights a succession of tyrannical warriors to restore order."}, {"m_title": "Sonny Boy", "r": "7.6", "year": "2021", "votes": "1,404", "genre": "Drama, Mystery", "desp": "A group of students on summer vacation find themselves transported to another dimension and granted superpowers to survive there."}, {"m_title": "Clannad: After Story", "r": "8.6", "year": "2008", "votes": "11,727", "genre": "Comedy, Drama", "desp": "A sequel to the recently wrapped anime based on the Key Clannad franchise. After Story is a path that opens in the original visual novel after all the other paths have been opened. Life. watch the dude, no spoilers in summary!"}, {"m_title": "School Days", "r": "6.0", "year": "2007", "votes": "2,683", "genre": "Drama, Romance", "desp": "Makoto Itou has a crush on Kotonoha Katsura, who rides the same train as him daily. Following urban legend, he sets her photo as his phone wallpaper - a love charm. When classmate Sekai Saionji notices, it seems his luck really does change."}, {"m_title": "Vivy: Fluorite Eyes Song", "r": "8.0", "year": "2021", "votes": "3,577", "genre": "Action, Adventure", "desp": "The female looking and first autonomous AI named Diva is created for one mission to make everyone happy by singing. Shortly Diva is confronted by an unknown program from the future with objective to make Diva join him and destroy AIs."}, {"m_title": "The Outcast", "r": "7.0", "year": "2016", "votes": "252", "genre": "Action, Adventure", "desp": "The protagonist, Cho Soran, leads a very common college students life until he finds himself caught up in a terrible incident that happened in a small village. As he was walking through a. watch the dude, no spoilers in summary!"}, {"m_title": "Wonder Egg Priority", "r": "7.4", "year": "2021", "votes": "3,510", "genre": "Action, Drama", "desp": "While on a midnight stroll, a girl named Ai picks up an egg. A voice coaxes her: If you want to change the future, you only need to choose now. Now, believe in yourself and break the egg. What awaits Ai after the breaking of the egg?"}, {"m_title": "Mobile Suit Gundam: Iron-Blooded Orphans", "r": "7.9", "year": "2015", "votes": "1,928", "genre": "Action, Drama", "desp": "300 years after the Calamity War, a ragtag group of freedom fighters and an ancient Mobile Suit rise to the call for justice, once and for all."}, {"m_title": "Yashahime: Princess Half-Demon", "r": "7.1", "year": "2020", "votes": "1,123", "genre": "Action, Adventure", "desp": "An adventure following Sesshomarus half-demon twin daughters, Towa and Setsuna, as they discover the mysteries of their past along with Moroha, the daughter of Inuyasha and Kagome."}, {"m_title": "The Aristocrats Otherworldly Adventure: Serving Gods Who Go Too Far", "r": "6.7", "year": "2023", "votes": "125", "genre": "Action, Fantasy", "desp": "Teenager Shiinya Kazuya died while saving young girls from an attacker, only to find himself reincarnated into a world of swords and magic. It was a dream come true. Though he remembers his. watch the anime dude!"}, {"m_title": "Devils Line", "r": "6.8", "year": "2018", "votes": "1,362", "genre": "Action, Crime", "desp": "Vampires seem to be living among humans. Of course the government does not know of their existence, because their appearance does not differ from humans. They also do not need to drink. watch the dude, no spoilers in summary!"}, {"m_title": "Nisekoi", "r": "7.2", "year": "2014", "votes": "3,790", "genre": "Comedy, Romance", "desp": "The heir to a Yakuza Family is forced to be in a relationship with the daughter of the head of a gangster family to avoid a gang war, much to their chagrin."}, {"m_title": "Ghost in the Shell SAC_2045", "r": "6.4", "year": "2020", "votes": "2,746", "genre": "Action, Crime", "desp": "Hired as a mercenary unit, the former members of Japans elite Section 9 are faced with the sudden appearance of Post-Human, a being with tremendous intelligence and physical capabilities."}, {"m_title": "The Irregular at Magic High School", "r": "7.0", "year": "2014", "votes": "3,735", "genre": "Action, Drama", "desp": "Siblings Miyuki and Tatsuya Shiba are turning the National Magic University Affiliated First High School campus upside down."}, {"m_title": "Junji Itô: Korekushon", "r": "6.6", "year": "2018", "votes": "1,394", "genre": "Drama, Horror", "desp": "A collection of animated horror stories based on the works of Japanese artist Junji Ito."}, {"m_title": "Yona of the Dawn", "r": "8.0", "year": "2014", "votes": "4,405", "genre": "Action, Adventure", "desp": "After being betrayed and run out of her home, the red-haired Princess Yona searches for four legendary dragons to take back her stolen kingdom."}, {"m_title": "Tsukimichi: Moonlit Fantasy", "r": "7.6", "year": "2021", "votes": "1,791", "genre": "Action, Adventure", "desp": "The series tells the story of normal kid Makoto Misumi who, after being transported to a fantasy world to serve as its hero, is cut loose when the worlds unnamed goddess decides he doesnt measure up to her standards of beauty."}, {"m_title": "Darwins Game", "r": "7.2", "year": "2020", "votes": "2,940", "genre": "Action, Drama", "desp": "An unknowing Sudou Kaname is invited to try out a new mysterious mobile app game called Darwins Game, but later realizes that hes in for more than hes bargained for when he finds out that theres no way to quit the game."}, {"m_title": "Hensuki: Are you willing to fall in love with a pervert, as long as shes a cutie?", "r": "6.3", "year": "2019", "votes": "877", "genre": "Comedy, Drama", "desp": "A mysterious love letter winds up in Keikis locker with a pair of white panties! Will he be able to find the panty-leaving-Cinderella?"}, {"m_title": "Angels of Death", "r": "6.5", "year": "2018", "votes": "2,448", "genre": "Adventure, Horror", "desp": "When Rachel wakes up with no memories in the basement of an unfamiliar building, she runs into Zack, a scythe-carrying serial killer."}, {"m_title": "Blood Blockade Battlefront", "r": "7.2", "year": "2015", "votes": "2,372", "genre": "Action, Adventure", "desp": "In a city where the paranormal and ordinary co-exist, the Libra organization battles to ensure peace and order."}, {"m_title": "Vermeil in Gold: A Desperate Magician Barges Into the Magical World Alongside the Strongest Calamity", "r": "6.5", "year": "2022", "votes": "1,131", "genre": "Comedy, Fantasy", "desp": "Alto Goldfield, a student at a magical academy inscribes a magic circle, ends up summoning a powerful demon named Vermeil, makes her his familiar."}, {"m_title": "Yu-Gi-Oh! Duel Monsters", "r": "8.2", "year": "II) (2000", "votes": "3,366", "genre": "Action, Adventure", "desp": "Yu-Gi-Oh! follows the adventures of high school student Yugi, who has a magical secret that comes to life when he plays his favorite card game: Duel Monsters."}, {"m_title": "Log Horizon", "r": "7.6", "year": "2013", "votes": "4,468", "genre": "Action, Adventure", "desp": "Elder Tale is an online fantasy RPG that has become popular worldwide. But when its twelfth expansion package, Novasphere Pioneers is installed, thirty thousand players in Japan are. watch the dude, no spoilers in summary!"}, {"m_title": "Fighting Spirit: New Challenger", "r": "8.8", "year": "2009", "votes": "2,819", "genre": "Action, Comedy", "desp": "Ippo continues to defend his title and training to reach his goal, Ichiro Miyata. While Takamura goes for the world, and faces the world champion, Brian Hawk. In what is to become his toughest fight so far."}, {"m_title": "Saga of Tanya the Evil", "r": "7.6", "year": "2017", "votes": "3,725", "genre": "Action, Adventure", "desp": "A girl with blond hair, blue eyes, and porcelain skin fights on the front lines of a brutal war and climbs the ranks of the imperial army."}, {"m_title": "Cardcaptor Sakura", "r": "8.0", "year": "1998", "votes": "5,296", "genre": "Action, Adventure", "desp": "Sakura stumbled upon the book of Clow Cards in a library. Accidentally setting the magical cards loose, its now up to Sakura to catch them all with her best friend Tomoyo, and Kerberos, the guardian of the cards."}, {"m_title": "The Greatest Demon Lord Is Reborn as a Typical Nobody", "r": "6.2", "year": "2022", "votes": "974", "genre": "Action, Adventure", "desp": "As the most powerful entity of all time, the Demon Lord Varvatos thinks life is a big fat snore. When he takes matters into his own hands and decides to reincarnate, he calibrates his. watch the dude, no spoilers in summary!"}, {"m_title": "Shiki", "r": "7.6", "year": "2010", "votes": "3,649", "genre": "Drama, Horror", "desp": "When many deaths occur in the quiet village of Sotoba, Doctor Toshio Ozaki initially suspects an epidemic but soon becomes convinced that something else is causing them."}, {"m_title": "A Galaxy Next Door", "r": "8.0", "year": "I) (2023", "votes": "86", "genre": "Comedy, Fantasy", "desp": "A struggling manga artist starts a romance with his new assistant, who turns out to be an extraterrestrial being in hiding."}, {"m_title": "Shikimoris Not Just a Cutie", "r": "6.6", "year": "2022", "votes": "2,066", "genre": "Comedy, Drama", "desp": "Shikimori seems like the perfect girlfriend. But she has a cool dark side that comes out under the right circumstances. And her boyfriend Izumi loves to be around when that happens."}, {"m_title": "Cautious Hero: The Hero Is Overpowered but Overly Cautious", "r": "7.4", "year": "2019", "votes": "3,051", "genre": "Action, Adventure", "desp": "The goddess Rista summons a hero to help her hard mode video game-like world. The hero, Seiya, is exceptional in every way, but he is incredibly cautious. He does things like buy three sets. watch the anime dude!"}, {"m_title": "Trinity Seven", "r": "6.7", "year": "2014", "votes": "2,547", "genre": "Action, Comedy", "desp": "In this romantic comedy but sometimes serious magical school story, life as Arata Kasuga knows it is wiped out by a bizarre incident known as Collapse Phenomenon, which causes worldwide. watch the anime dude!"}, {"m_title": "Citrus", "r": "6.5", "year": "2018", "votes": "2,430", "genre": "Drama, Romance", "desp": "Fashionable Yuzu’s life goes from sweet to sour when the beautiful but stern Mei falls into her life."}, {"m_title": "Drifters", "r": "7.6", "year": "2016", "votes": "3,549", "genre": "Action, Adventure", "desp": "Various historical fighters are summoned from across time to wage war on each other, with the fate of the world in the balance."}, {"m_title": "Saint Seiya: The Hades Chapter - Sanctuary", "r": "8.5", "year": "2002", "votes": "3,274", "genre": "Action, Adventure", "desp": "Hades is planning to take over the world, to achieve that goal, he sends out deceased Gold Saints to take Athenas head. Seiya and the other Bronze Saints come to help but their help isnt. watch the dude, no spoilers in summary!"}, {"m_title": "Danganronpa: The Animation", "r": "6.8", "year": "2013", "votes": "2,852", "genre": "Action, Crime", "desp": "A group of teenagers enroll in a prestigious high school, which turns out to be a devious trap designed to tempt the pupils to murder one another in order to graduate."}, {"m_title": "Masamune-kuns Revenge", "r": "6.4", "year": "2017", "votes": "2,750", "genre": "Comedy, Drama", "desp": "As a child, Masamune Makabe once suffered greatly at the hands of a wealthy and beautiful girl named Aki Adagaki, who nicknamed him Piggy due to his chubby appearance. Seeking revenge. watch the dude, no spoilers in summary!"}, {"m_title": "Inazuma Eleven", "r": "8.1", "year": "2008", "votes": "2,157", "genre": "Adventure, Comedy", "desp": "A talented young goalkeeper tries to revive his schools lackluster team in this anime series based on the hit manga and video game."}, {"m_title": "Spriggan", "r": "6.8", "year": "2022", "votes": "1,291", "genre": "Action, Adventure", "desp": "An ancient alien civilizations relics on Earth hold dangerous powers. The ARCAM corporations Spriggan agents must keep them out of the wrong hands."}, {"m_title": "Girls Bravo", "r": "6.5", "year": "2004", "votes": "959", "genre": "Comedy, Fantasy", "desp": "Small for his age, Yukinari has been bullied and abused by girls all his life. Now in high school, he has developed a rare condition: whenever girls touch him, or even come close, he breaks. watch the anime dude!"}, {"m_title": "Angel Beats!", "r": "7.6", "year": "2010", "votes": "15,235", "genre": "Action, Comedy", "desp": "Rebellious teens fight in armed combat against one dispassionate girls supernatural powers in an afterlife high school."}, {"m_title": "Fighting Spirit: Rising", "r": "8.7", "year": "2013", "votes": "2,562", "genre": "Action, Comedy", "desp": "Ippo the Japanese Featherweight champion will defend his title against amazing challengers. Takamura fights for the second WBC title in the middleweight class, and Aoki fights for the Lightweight title."}, {"m_title": "Sailor Moon Crystal", "r": "7.8", "year": "2014", "votes": "2,281", "genre": "Action, Adventure", "desp": "Usagi Tsukino is chosen to be a guardian of justice and is sent on a quest to locate a Silver Crystal before the Dark Kingdom invades the Earth."}, {"m_title": "Natsumes Book of Friends", "r": "8.1", "year": "2008", "votes": "2,247", "genre": "Comedy, Drama", "desp": "When Natsume Takashi inherits a book that belonged to his late grandmother he realizes the book is filled with the names of spirits she defeated and bound to her will. He then decides to return their names so they can be free once again."}, {"m_title": "Ranma ½", "r": "7.9", "year": "1989", "votes": "5,363", "genre": "Action, Comedy", "desp": "A girl is involuntarily engaged to a boy who turns female when hit with cold water and male when hit with hot."}, {"m_title": "The Hidden Dungeon Only I Can Enter", "r": "6.4", "year": "2021", "votes": "1,746", "genre": "Action, Adventure", "desp": "The Hidden Dungeon is a place of legend where rare treasures and items are hidden. Nor, the third son of an impoverished noble family whos lost the one job offer he had, was lucky enough to hear about this dungeon."}, {"m_title": "Space Dandy", "r": "8.1", "year": "2014", "votes": "4,336", "genre": "Adventure, Comedy", "desp": "The cosmic and comic adventures of Space Dandy, a handsome space explorer tasked to tour the universe and discover new alien beings."}, {"m_title": "Trapped in a Dating Sim: The World of Otome Games is Tough for Mobs", "r": "7.2", "year": "2022", "votes": "1,110", "genre": "Comedy, Fantasy", "desp": "A young man is transported into a video game world dominated by females."}, {"m_title": "Magical Destroyers", "r": "6.8", "year": "2023", "votes": "61", "genre": "Action, Adventure", "desp": "Four otakus who will fight for freedom, when a mysterious force suddenly starts eliminating the otaku culture in the year 2008."}, {"m_title": "Snow White with the Red Hair", "r": "7.7", "year": "2015", "votes": "3,244", "genre": "Drama, Fantasy", "desp": "Shirayuki was a young girl born with unique apple-red hair. She meets a famous but foolish Prince Raji, who falls in love with her at first sight and orders her to become his concubine.. watch the dude, no spoilers in summary!"}, {"m_title": "Blood-C", "r": "6.2", "year": "2011", "votes": "1,823", "genre": "Action, Fantasy", "desp": "Saya is a seemingly normal girl living with her father in a shrine. During the day she goes to school and spends time with her friends at a local cafe. But when night falls Saya is called upon to protect her village from strange monsters."}, {"m_title": "Durarara!!", "r": "7.8", "year": "2010", "votes": "7,530", "genre": "Action, Comedy", "desp": "Tired of his mundane life, Mikado Ryugamine decides to move to Ikebukuro, a district in Tokyo, when a friend invites him. With everything from invisible gangs to rumored beings, Ikebukuro is full of connected mysteries where peoples pasts intertwine with the present."}, {"m_title": "March Comes in Like a Lion", "r": "8.3", "year": "2016", "votes": "3,056", "genre": "Comedy, Drama", "desp": "A 17 year old socially awkward orphaned shogi player, dealing with adult problems like financial difficulties, loneliness, and depression."}, {"m_title": "Seirei Gensouki: Spirit Chronicles", "r": "7.2", "year": "2021", "votes": "1,430", "genre": "Action, Adventure", "desp": "Amakawa Haruto is a young man who died before reuniting with his childhood friend who disappeared five years ago. Rio is a boy living in the slums who wants revenge for his mother who was. watch the dude, no spoilers in summary!"}, {"m_title": "Kemeko Deluxe!", "r": "6.3", "year": "2008", "votes": "75", "genre": "Action, Comedy", "desp": "6-year old Sanpeita Kobayashi got engaged to his childhood friend, a mysterious pink-haired girl who promised they would meet again 10 years later, and she would become his bride. The series begins then."}, {"m_title": "Vampire Knight", "r": "7.3", "year": "2008", "votes": "5,523", "genre": "Action, Drama", "desp": "Yuki Cross, along with her best friend Zero, attempts to keep the peace between humans and vampires at Cross Academy, but personal issues soon threaten the situation."}, {"m_title": "From the New World", "r": "7.9", "year": "2012", "votes": "3,415", "genre": "Action, Adventure", "desp": "In a post-apocalyptic world set a thousand years after our era, the remaining humans, now with telekinesis, live in a seemingly peaceful society, but dark secrets of the past will soon be discovered by a small group of friends."}, {"m_title": "Sasaki and Miyano", "r": "7.7", "year": "2022", "votes": "1,218", "genre": "Comedy, Drama", "desp": "Shy and easily flustered Miyano harbors an embarrassing secret - he is a fudanshi, a boy who likes boys love manga. Intrigued, the clueless delinquent Sasaki requests to borrow one, marking a shift in their strange dynamic."}, {"m_title": "Akudama Drive", "r": "7.3", "year": "2020", "votes": "2,279", "genre": "Action, Crime", "desp": "Long ago, a war broke out in Japan that pitted Kantou against Kansai and divided the country. Kansai became dependent on Kantou, but slowly the police and government began losing control of Kansai to criminals known as Akudama."}, {"m_title": "Exception", "r": "6.5", "year": "2022", "votes": "959", "genre": "Drama, Horror", "desp": "In the distant future, human beings have been forced to leave earth and migrate to another galaxy. An advanced team of spacecraft arrives at a planet that is to be terraformed. Each member of the team is output by a biological 3D printer."}, {"m_title": "Monogatari Series: Second Season", "r": "8.6", "year": "2013", "votes": "2,728", "genre": "Adventure, Comedy", "desp": "Walking to school one day Tsubasa Hanekawa encounters a huge white tiger apparition at a crossroads that talks to her. The next day her house burns down. Homeless, and not wanting to stay. watch the dude, no spoilers in summary!"}, {"m_title": "How a Realist Hero Rebuilt the Kingdom", "r": "7.2", "year": "2021", "votes": "1,681", "genre": "Action, Comedy", "desp": "O, Hero! When Kazuya Souma is unexpectedly transported to another world, he knows the people expect a hero. But Soumas idea of heroism is more practical than most, he wants to rebuild the. watch the dude, no spoilers in summary!"}, {"m_title": "Stitch!", "r": "4.5", "year": "2008", "votes": "315", "genre": "Comedy, Family", "desp": "Follows a girl named Yuuna and her adventures with a mysterious space alien on an island off the coast of Japan."}, {"m_title": "Kotaro Lives Alone", "r": "8.4", "year": "2022", "votes": "3,502", "genre": "Comedy, Drama", "desp": "An unpopular manga artist has to care for his new neighbor, a 5-year-old child living alone."}, {"m_title": "Karakai Jouzu no Takagi-san", "r": "7.7", "year": "2018", "votes": "2,504", "genre": "Comedy, Romance", "desp": "If you blush, you lose. Nishikata has been teased mercilessly by Takagi and always winds up blushing! But he vows to one day get back at her."}, {"m_title": "Reincarnated as a Sword", "r": "7.3", "year": "2022", "votes": "728", "genre": "Action, Adventure", "desp": "The hero of Tensei shitara Ken deshita differs from your standard otherworldly protagonist in that he is reincarnated as a sword! Beginning his quest by spawning in the middle of a. watch the dude, no spoilers in summary!"}, {"m_title": "Gangsta.", "r": "7.2", "year": "2015", "votes": "3,031", "genre": "Action, Drama", "desp": "Two handymen take on jobs for both the mob and the police that no one else can handle. Worick and Nicolas work in a town which is full of mafia, hoodlums and prostitutes. They face a challenge after taking on a job of crushing a gang"}, {"m_title": "B: The Beginning", "r": "7.1", "year": "2018", "votes": "5,227", "genre": "Action, Crime", "desp": "In this anime series, scientists hope new humans will create universal peace, but theyre kidnapped by an evil group with very different plans."}, {"m_title": "Hip Whip Girl: Keijo!!!!!!", "r": "6.8", "year": "2016", "votes": "944", "genre": "Action, Comedy", "desp": "Young girls participate in a gambling sport known as Keijo, in which contestants stand on floating platforms and must push each other off using their breasts and buttocks."}, {"m_title": "Summoned to Another World for a Second Time", "r": "7.2", "year": "2023", "votes": "81", "genre": "Action, Adventure", "desp": "There was once a man who was summoned to another world, and saved it. Of course, he became too popular there, and turned into an isekai-normie. However, that man fell into a trap and. watch the dude, no spoilers in summary!"}, {"m_title": "Beyond the Boundary", "r": "7.2", "year": "2013", "votes": "2,657", "genre": "Action, Comedy", "desp": "High school student Mirai finds that his schoolmate Akihito is an immortal being and half a spirit. When Akihito learns that Mirai is a spirit world warrior their lives become intertwined."}, {"m_title": "Witchblade", "r": "7.0", "year": "2006", "votes": "1,523", "genre": "Action, Adventure", "desp": "In order to protect her daughter, a young single mother is forced to fight after acquiring a powerful ancient weapon."}, {"m_title": "Ajin", "r": "7.6", "year": "2016", "votes": "6,671", "genre": "Action, Adventure", "desp": "A high school student discovers he is an Ajin, a demi-human being. He ends up a fugitive on the run because of his powers."}, {"m_title": "The Iceblade Sorcerer Shall Rule the World", "r": "6.3", "year": "2023", "votes": "380", "genre": "Action, Adventure", "desp": "Everyone knows of the most powerful soldier, the Iceblade Magician, who led the country to victory three years ago, but few know his true name: Ray White. After weathering through a bloody. watch the dude, no spoilers in summary!"}, {"m_title": "Beyblade Burst", "r": "6.8", "year": "2016", "votes": "674", "genre": "Action, Adventure", "desp": "Valt Aoi is a hot-blooded kid who loves to attack and wields a Beyblade named Valkyrie. His close friend Shu Kurenai is an elite Blader who is a genius but still puts in a lot of effort, and wields the Beyblade named Spriggan."}, {"m_title": "Engage Kiss", "r": "6.4", "year": "2022", "votes": "569", "genre": "Action, Comedy", "desp": "Bayron City is a mega-float type of city in the Pacific Ocean, which doesnt belong to any particular nation. As the mine for Orgonium, a new energy resource, this city is the most noticed. watch the dude, no spoilers in summary!"}, {"m_title": "Blue Period", "r": "7.7", "year": "2021", "votes": "2,862", "genre": "Drama", "desp": "Bored with life, popular high schooler Yatora Yaguchi jumps into the beautiful yet unrelenting world of art after finding inspiration in a painting."}, {"m_title": "Beast Tamer", "r": "6.7", "year": "2022", "votes": "607", "genre": "Action, Comedy", "desp": "Rin, an animal tamer, is kicked out of the group of heroes because others think her skills are weak. A kind person, he now lives freely as an adventurer. During his journey, he meets superior species, tames them, and his life changes"}, {"m_title": "Gate", "r": "7.5", "year": "2015", "votes": "3,773", "genre": "Action, Adventure", "desp": "A gate appears in modern-day Tokyo which allows access to a fantasy world. A Japanese military squad is formed to establish ties between both worlds."}, {"m_title": "Higehiro: After Being Rejected, I Shaved and Took in a High School Runaway", "r": "7.4", "year": "2021", "votes": "2,951", "genre": "Comedy, Drama", "desp": "An office worker takes in a runaway school girl. Despite the difference in age and personalities, they form a bond."}, {"m_title": "Attack on Titan: Junior High", "r": "7.1", "year": "2015", "votes": "2,552", "genre": "Action, Adventure", "desp": "After his lunch is eaten by a Titan on the first day of junior high, young Eren Jaeger vows to destroy the human food-stealing creatures with the help of his classmates and hilarity ensues."}, {"m_title": "In/Spectre", "r": "6.5", "year": "2020", "votes": "1,147", "genre": "Action, Comedy", "desp": "Meet the cute, smug, cane-wielding 17-year old Iwanaga Kotoko as she (very) aggressively tries to woo the older Sakuragawa Kurou and act as a mediator/problem-solver for the supernatural world."}, {"m_title": "Digimon Adventure: 2020", "r": "6.8", "year": "2020", "votes": "643", "genre": "Action, Adventure", "desp": "Taichi is preparing for a summer camping trip until he encounters a strange phenomenon that sweeps him up into the Digital World along with the other DigiDestined."}, {"m_title": "Love, Chunibyo & Other Delusions", "r": "7.4", "year": "2012", "votes": "4,419", "genre": "Comedy, Drama", "desp": "Yuuta suffered from chuunibyou while he was in middle school. When he graduated, he try to forgot about it but he accidentally got into a contract with Rikka and disrupts his desperately ordinary life."}, {"m_title": "Gungrave", "r": "7.9", "year": "2003", "votes": "5,145", "genre": "Action, Crime", "desp": "Gungrave follows the story of best friends Brandon Heat and Harry MacDowell as they join and rise in the ranks of Big Daddys Millenion crime syndicate. The story begins in the future. watch the dude, no spoilers in summary!"}, {"m_title": "D.Gray-man", "r": "7.7", "year": "2006", "votes": "4,372", "genre": "Action, Adventure", "desp": "Young Allen Walker, an exorcist, fights Akuma to save the world."}, {"m_title": "Plunderer", "r": "6.4", "year": "2020", "votes": "1,337", "genre": "Action, Fantasy", "desp": "In a post-apocalyptic world, each citizen has their identity branded with their own Count, which defines their livelihood. But a mysterious figure has an agenda to steal away these Counts."}, {"m_title": "Mobile Suit Gundam Wing", "r": "8.0", "year": "1995", "votes": "4,693", "genre": "Action, Adventure", "desp": "A squadron of giant robot pilots of a beleaguered Earth colony bring the war to their oppressive home planet."}, {"m_title": "Orange", "r": "7.6", "year": "2016", "votes": "4,092", "genre": "Drama, Romance", "desp": "A high school student receives a letter from her future self who wants to fix her past mistakes."}, {"m_title": "The Ice Guy and His Cool Female Colleague", "r": "6.8", "year": "2023", "votes": "453", "genre": "Comedy, Fantasy", "desp": "Himuro is an office worker who belongs to a race of supernatural beings with strange powers. Whenever he is deep in concentration, he subconsciously plunges his poor coworkers into a close. watch the dude, no spoilers in summary!"}, {"m_title": "Little Witch Academia", "r": "7.8", "year": "2017", "votes": "3,645", "genre": "Comedy, Family", "desp": "Akko enrolls at the Luna Nova Witchcraft Academy. Shes not the best student, but her bright attitude is the key to her and her friends success."}, {"m_title": "Free!", "r": "7.4", "year": "2013", "votes": "3,353", "genre": "Comedy, Drama", "desp": "Free! revolves around members of a high school boys swimming team. Four boys-Haruka, Makoto, Nagisa and Rin-all participated in a swimming tournament shortly before graduating elementary. watch the dude, no spoilers in summary!"}, {"m_title": "Kirby: Right Back at Ya!", "r": "7.0", "year": "2001", "votes": "2,261", "genre": "Action, Adventure", "desp": "Kirby crashes onto Planet Popstar and befriends the villagers that live In Cappytown. Unfortunately, King Dedede, the greedy ruler and Escargon, his assistant, arent happy with that and do whatever they can to drive him out."}, {"m_title": "Tsugumomo", "r": "6.6", "year": "2017", "votes": "414", "genre": "Action, Mystery", "desp": "A normal high school student treasures a gift his mother gave him. One day, he finds out the true power of this gift and is drawn into a whole new world, complete with a harem of beautiful women."}, {"m_title": "RWBY: Ice Queendom", "r": "6.0", "year": "2022", "votes": "530", "genre": "Action, Adventure", "desp": "In the world of Remnant, a place where science and fairy tales coexist, human civilization is plagued by deadly monsters known as the Grimm. For a time, victory by the Grimm seemed all but. watch the dude, no spoilers in summary!"}, {"m_title": "Mushi-Shi: Next Passage", "r": "8.8", "year": "2014", "votes": "3,021", "genre": "Drama, Fantasy", "desp": "They existed long before anyone can remember. They are simple and strange in nature, not resembling any other plant or animal in this world. In ancient times, people revered these bizarre. watch the dude, no spoilers in summary!"}, {"m_title": "Black Summoner", "r": "7.1", "year": "2022", "votes": "857", "genre": "Fantasy", "desp": "Waking up in a strange new place with no memory of his past life Kelvin learns that hes bartered away those very memories in exchange for powerful new abilities during his recent. watch the dude, no spoilers in summary!"}, {"m_title": "Urusei Yatsura", "r": "7.0", "year": "2022", "votes": "403", "genre": "Action, Adventure", "desp": "A young man cursed with misfortune finds himself tied to an alien wife."}, {"m_title": "Tsuki ga kirei", "r": "7.8", "year": "2017", "votes": "2,189", "genre": "Drama, Romance", "desp": "A romance forms between two shy third year middle school students."}, {"m_title": "Sankarea", "r": "6.7", "year": "2012", "votes": "1,291", "genre": "Comedy, Drama", "desp": "Furuya has a fascination for zombies and collects trinkets and watches any movie and plays any game with them in it. When his cat dies however this hobby becomes somewhat of an obsession as. watch the anime dude!"}, {"m_title": "Revolutionary Girl Utena", "r": "8.1", "year": "1997", "votes": "1,844", "genre": "Comedy, Drama", "desp": "A tomboyish schoolgirl finds herself forced into repeated duels for another girl who has a role in a world revolution."}, {"m_title": "Banished from the Heros Party, I Decided to Live a Quiet Life in the Countryside", "r": "6.7", "year": "2021", "votes": "1,165", "genre": "Adventure, Fantasy", "desp": "Red was once a member of the Heros party, a powerful group destined to save the world from the evil forces of Taraxon, the Raging Demon Lord. That is, until one of his comrades kicked him. watch the dude, no spoilers in summary!"}, {"m_title": "The Melancholy of Haruhi Suzumiya", "r": "7.8", "year": "2006", "votes": "8,763", "genre": "Comedy, Fantasy", "desp": "The crazy adventures of the SOS Brigade, led by the insane but charismatic Haruhi."}, {"m_title": "Land of the Lustrous", "r": "7.9", "year": "2017", "votes": "1,789", "genre": "Action, Fantasy", "desp": "The Houseki fight against the Moon Dwellers, beings who attack them and use their shattered bodies as decorations. Each Houseki is assigned a role, and Phos, the youngest, is assigned to create an encyclopedia of natural history."}, {"m_title": "Megalo Box", "r": "7.8", "year": "2018", "votes": "3,518", "genre": "Action, Drama", "desp": "Junk Dog, an underground fighter with the alias of Gearless Joe sets out to join the worlds most prestigious mecha boxing championship, Megalonia."}, {"m_title": "Strike the Blood", "r": "6.9", "year": "2013", "votes": "1,763", "genre": "Action, Comedy", "desp": "The Fourth Primogenitor-Thats the worlds strongest vampire that should only exist in legends. Accompanied by twelve Kenjuu and spreading calamity, this phantom vampire appeared in Japan.. watch the dude, no spoilers in summary!"}, {"m_title": "Revenger", "r": "6.6", "year": "2023", "votes": "282", "genre": "Action, Adventure", "desp": "A vengeful samurai works with a mysterious organization to take revenge on behalf of those who have no power."}, {"m_title": "Jibaku Shounen Hanako-kun", "r": "7.5", "year": "2020", "votes": "1,954", "genre": "Comedy, Fantasy", "desp": "Hanako is an urban legend said to be the spirit of a young girl that haunts school toilets, but as Nene finds out, her academys Hanako is actually a boy, and he is only one of many spirits inhabiting the place."}, {"m_title": "Onimai: Im Now Your Sister!", "r": "6.9", "year": "2023", "votes": "281", "genre": "Comedy", "desp": "When a lazy, erotic-game loving Otaku is transformed into a girl by his younger scientist sisters experiments, he becomes determined to change back, only to recognize that with his new opportunity, he may want to turn his life around."}, {"m_title": "Make My Day", "r": "5.4", "year": "2023", "votes": "532", "genre": "Adventure, Drama", "desp": "On an icy mining planet (aka a prison world) mysterious disappearances point to deadly secrets hidden within the mines."}, {"m_title": "Mugen no jûnin: Immortal", "r": "7.5", "year": "2019", "votes": "1,065", "genre": "Action, Fantasy", "desp": "A young girl named Rin seeks to avenge her parents, who were ruthlessly killed by a group of master swordsmen, with the help of cursed samurai Manji, who is seemingly immortal."}, {"m_title": "Takt Op. Destiny", "r": "6.9", "year": "2021", "votes": "1,832", "genre": "Action, Adventure", "desp": "In a world where aliens have invaded Earth and banned music, a group of fighter musicians use music as a weapon to maintain a resistance."}, {"m_title": "Dragon Quest: The Adventure of Dai", "r": "8.0", "year": "2020", "votes": "1,089", "genre": "Action, Adventure", "desp": "Dai is the only human living on the island and his dream is to grow up to be a hero."}, {"m_title": "Fate/Apocrypha", "r": "6.6", "year": "2017", "votes": "2,508", "genre": "Action, Drama", "desp": "In an alternate universe, the Holy Grail War was won by the Yggdamillenia family. Now two factions with 7 Master-Servant pairs each and 1 extra Servant as the Ruler are fighting to win the Greater Holy Grail."}, {"m_title": "Mobile Suit Gundam 00", "r": "8.1", "year": "2007", "votes": "2,537", "genre": "Action, Drama", "desp": "In the distant future, mankind has used up all of its fossil fuels, forcing them to turn to Solar Power as an alternate energy source. As a result, this causes a rift to form between richer. watch the anime dude!"}, {"m_title": "Grimgar, Ashes and Illusions", "r": "7.4", "year": "2016", "votes": "2,917", "genre": "Adventure, Drama", "desp": "Haruhiro suddenly wakes up, with no memory but his name. There are a lot of people around, but noone seems to know anything. Haruhiro forms a party of some of the people and start exploring."}, {"m_title": "Moomin", "r": "8.3", "year": "1990", "votes": "4,364", "genre": "Adventure, Comedy", "desp": "The eerie and charming animated adventures of a creature called Moomin and his friends and family."}, {"m_title": "Triage X", "r": "6.0", "year": "2015", "votes": "267", "genre": "Action", "desp": "A vigilante organization known as Black Label task themselves with killing undesirable people."}, {"m_title": "Outlaw Star", "r": "7.9", "year": "1998", "votes": "3,945", "genre": "Action, Adventure", "desp": "The adventures of an outlaw crew of an advanced starship."}, {"m_title": "Samurai 7", "r": "7.6", "year": "2004", "votes": "4,262", "genre": "Action, Adventure", "desp": "Set in an alternate feudal Japan where mechas and giant airships are a common thing for humans to see. With in this time period is a small village that gets raided by bandits during every. watch the dude, no spoilers in summary!"}, {"m_title": "Gyakkyô burai Kaiji", "r": "8.2", "year": "2007", "votes": "3,425", "genre": "Action, Thriller", "desp": "Kaiji Itou is a good-for-nothing loiterer who spends his days drinking beer and stealing hubcaps-that is, until he ends up being tricked by his former co-worker. Unable to suddenly repay. watch the dude, no spoilers in summary!"}, {"m_title": "Tenchi Muyo!", "r": "7.6", "year": "1992", "votes": "2,163", "genre": "Adventure, Comedy", "desp": "Every man needs a good woman. Tenchi Masaki has six."}, {"m_title": "Persona 5: The Animation", "r": "6.7", "year": "2018", "votes": "693", "genre": "Action, Adventure", "desp": "Follows Ren Amamiya and a rebellious group of teenagers in Tokyo who form the Phantom Thieves of Hearts to rebel against the corruption and slavery in society."}, {"m_title": "Mieruko-chan", "r": "7.2", "year": "2021", "votes": "2,009", "genre": "Comedy, Fantasy", "desp": "She can see dead people... she just chooses to ignore them. Thats Mikos plan anyway, with horrifying (and sometimes hilarious) results."}, {"m_title": "Noblesse", "r": "7.1", "year": "2020", "votes": "1,636", "genre": "Action, Adventure", "desp": "Raizel awakens from his 820-year slumber. He holds the special title of Noblesse which is a pure-blooded Noble and protector of all other Nobles. In an attempt to protect Raizel, his. watch the dude, no spoilers in summary!"}, {"m_title": "Basilisk: The Kouga Ninja Scrolls", "r": "7.8", "year": "2005", "votes": "4,147", "genre": "Action, Adventure", "desp": "The year is 1614 AD. Two warring ninja clans, each supporting a son of Hidetada Tokugawa as the next shogun, send ten representatives each to fight to the death for the possession Tokugawa. watch the dude, no spoilers in summary!"}, {"m_title": "Mother of the Goddess Dormitory", "r": "6.1", "year": "2021", "votes": "660", "genre": "Comedy, Romance", "desp": "Nagumo Koushi. His house burned down on his birthday. His father has never had a job and a single dollar behind him. At the age of 12, he becomes homeless. Having wondered around the city. watch the dude, no spoilers in summary!"}, {"m_title": "My One-Hit Kill Sister", "r": "6.8", "year": "2023", "votes": "50", "genre": "Action, Comedy", "desp": "As a result of an accident in the real world, Asahi Ikusaba is transported to another world. He tries to enjoy the different world he imagined, but his abilities prove to be the weakest.. watch the dude, no spoilers in summary!"}, {"m_title": "Benriya Saitou-san, Isekai ni Iku", "r": "7.0", "year": "2023", "votes": "414", "genre": "Adventure, Comedy", "desp": "Handyman Saitou has never felt special in his life. When hes dropped into a medieval fantasy world, he gathers a party of unique beings to survive. Surrounded by a heavy warrior, a. watch the dude, no spoilers in summary!"}, {"m_title": "Guilty Crown", "r": "7.0", "year": "2011", "votes": "5,350", "genre": "Action, Romance", "desp": "After the outbreak of the unidentified virus Lost Christmas in 2029, Japan has been under the control of a multi-nation organization called GHQ. Ohma Shu is a 17 year old boy who has a. watch the dude, no spoilers in summary!"}, {"m_title": "Seinto Seiya: The Lost Canvas - Meio Shinwa", "r": "8.1", "year": "2009", "votes": "2,099", "genre": "Action, Adventure", "desp": "Story of the Holy War taking place in the 18th century, 250 years before the original Saint Seiya series. It centers on the fight between Tenma, one of the 88 Saints following Athena, and Alone, the reincarnation of the God Hades."}, {"m_title": "Higurashi: When They Cry - GOU", "r": "6.9", "year": "2020", "votes": "669", "genre": "Drama, Horror", "desp": "Keiichi Maebara is in his new home in the peaceful Hinamizawa village, making friends with girls from school. Hes here for a big festival. But something about this town seems off, with that fear what dark secrets can this small town hide?"}, {"m_title": "Rage of Bahamut: Genesis", "r": "7.3", "year": "2014", "votes": "1,585", "genre": "Action, Adventure", "desp": "The story first starts off around Favaro Leone who is a bounty hunter, collecting money off of bounties he collects (demons). Favaro Leone soon runs into Amira who has no clue what is. watch the dude, no spoilers in summary!"}, {"m_title": "Yami shibai", "r": "7.1", "year": "2013", "votes": "822", "genre": "Short, Horror", "desp": "Every week at 5 p.m. an old man in a yellow mask shows up at a childrens playground and tells them ghost stories based on myths and urban legends of Japanese origin. The man tells the. watch the dude, no spoilers in summary!"}, {"m_title": "The Heroic Legend of Arslan", "r": "7.5", "year": "2015", "votes": "2,115", "genre": "Action, Adventure", "desp": "In the prosperous kingdom of Pars lies the Royal Capital of Ecbatana, a city of splendor and wonder, ruled by the undefeated and fearsome King Andragoras. Arslan is the young and curious. watch the dude, no spoilers in summary!"}, {"m_title": "Platinum End", "r": "6.1", "year": "2021", "votes": "1,767", "genre": "Action, Drama", "desp": "A young man is recruited by an angel into a battle to become the next God."}, {"m_title": "ID:Invaded", "r": "7.6", "year": "2020", "votes": "2,916", "genre": "Crime, Drama", "desp": "A detective is called upon to solve a murder in a virtual reality world. But how can he manage a case when its rules can and are constantly being changed?"}, {"m_title": "Bloom Into You", "r": "7.8", "year": "2018", "votes": "1,624", "genre": "Drama, Romance", "desp": "Yuu has always loved shoujo manga and awaits the day she gets a love confession that sends her heart aflutter with bubbles and blushes, and yet when a junior high classmate confesses his. watch the dude, no spoilers in summary!"}, {"m_title": "Silver Fang", "r": "8.9", "year": "1986", "votes": "2,998", "genre": "Action, Adventure", "desp": "An Akita-Inu puppy is destined to defend his people against a monstrous bear."}, {"m_title": "Captain Tsubasa", "r": "7.9", "year": "1983", "votes": "7,000", "genre": "Comedy, Drama", "desp": "Tsubasa Oozora is an 11-year-old elementary school student who is deeply in love with football and dreams of one day winning the FIFA World Cup for Japan."}, {"m_title": "By the Grace of the Gods", "r": "7.0", "year": "2020", "votes": "1,079", "genre": "Adventure, Fantasy", "desp": "Ryoma didnt have good luck in his former life. But three deities decide to give him a second chance in a fantasy world of magic."}, {"m_title": "K", "r": "7.0", "year": "2012", "votes": "2,488", "genre": "Action, Drama", "desp": "A young boy is wanted for a crime he has no recollection of committing and must go on the run, hunted by two kings and their forces."}, {"m_title": "Hundred", "r": "6.3", "year": "2016", "votes": "1,169", "genre": "Action, Romance", "desp": "Hundred is the only weapon that can counter the mysterious Savage life form attacking Earth. To become a Slayer who wields this Hundred, Hayato Kisaragi successfully enrolls in the marine. watch the dude, no spoilers in summary!"}, {"m_title": "Shaman King", "r": "8.0", "year": "2001", "votes": "6,199", "genre": "Action, Adventure", "desp": "SHAMAN KING follows the adventures of a 13-year-old shaman and his teammate a samurai warrior spirit, who traverse the world fighting evil spirits and misguided shamans on their journey to be the next Shaman King."}, {"m_title": "Full Metal Panic!", "r": "7.6", "year": "2002", "votes": "5,702", "genre": "Action, Adventure", "desp": "Sousuke Sagara, a member of a covert anti-terrorist private military organization known as Mithril, tasked with protecting Kaname Chidori, a spirited Japanese high school girl."}, {"m_title": "Immoral Guild", "r": "6.1", "year": "2022", "votes": "228", "genre": "Comedy, Fantasy", "desp": "The story follows the skilled hunter Kikuru Madan, who has decided to retire out of fear of wasting his youth. One day, a guild staff member suggests that he go on a quest with a new female martial artist named Hitamu Kyan."}, {"m_title": "Rokudos Bad Girls", "r": "6.5", "year": "2023", "votes": "32", "genre": "Comedy, Fantasy", "desp": "Rokudou Tousuke is a loser, and everyone knows it. He gets picked on at school, gets snide looks from passersby, and can only muster up the courage to complain in secret with his equally. watch the dude, no spoilers in summary!"}, {"m_title": "Say I love you", "r": "7.3", "year": "2012", "votes": "3,801", "genre": "Drama, Romance", "desp": "Mei is a quiet girl, who has had no boyfriend and couldnt even make friends in 16 years. One day she accidentally injures Yamato, the most popular boy in school, and for some reason Yamato takes interest in her."}, {"m_title": "A Place Further Than the Universe", "r": "8.2", "year": "2018", "votes": "2,484", "genre": "Adventure, Comedy", "desp": "A group of high school girls join an expedition headed towards the Antarctic."}, {"m_title": "Knights of Sidonia", "r": "7.7", "year": "2014", "votes": "4,495", "genre": "Action, Adventure", "desp": "A thousand years after the destruction of Earth by gigantic aliens called Gauna, mankind is living on massive spaceships. One of these space ships is Sidonia, which is protected by large pilot controlled exosuits called Gardes."}, {"m_title": "A Couple of Cuckoos", "r": "6.7", "year": "2022", "votes": "861", "genre": "Comedy, Romance", "desp": "Two children who got swapped at birth grow up to become engaged to each other. Its an arrangement neither of them like, at first."}, {"m_title": "Death March to the Parallel World Rhapsody", "r": "6.6", "year": "2018", "votes": "2,103", "genre": "Adventure, Comedy", "desp": "A computer programmer is transported to a fantasy world where life is an adventure thats fun and full of girls."}, {"m_title": "Planetes", "r": "8.1", "year": "2003", "votes": "3,761", "genre": "Drama, Sci-Fi", "desp": "Ai Tanabe joins the Debris Section of the Technora Corporation as they work to remove the debris left around Earth. As Ai tries to accommodate to space life, she learns more about her crew on the dilapidated Toy Box."}, {"m_title": "Amagi Brilliant Park", "r": "7.0", "year": "2014", "votes": "1,240", "genre": "Comedy, Fantasy", "desp": "The film is set in an amusement park in the Tokyo suburbs of Japan, with a princess at the center of the story."}, {"m_title": "Spice and Wolf", "r": "7.8", "year": "2008", "votes": "4,703", "genre": "Adventure, Drama", "desp": "Kraft Lawrence goes from town to town to make profits as a travelling merchant, with the help of a wolf deity by the name of Holo."}, {"m_title": "Kenichi: The Mightiest Disciple", "r": "8.1", "year": "2006", "votes": "2,270", "genre": "Action, Comedy", "desp": "Weak Legs Kenichi Shirahama would rather spend his time reading self improvement books than fighting."}, {"m_title": "Peter Grill to Kenja no Jikan", "r": "5.4", "year": "2020", "votes": "541", "genre": "Comedy, Fantasy", "desp": "Peter Grill is the strongest man in the world, but that means hes got a host of women who are after him and steal him away from the woman he loves."}, {"m_title": "Wise Mans Grandchild", "r": "6.8", "year": "2019", "votes": "2,615", "genre": "Action, Adventure", "desp": "A recently deceased young man is sent to another world and is taught magic by master Merlin. He quickly masters everything he is taught. However he has one fatal weakness, common sense."}, {"m_title": "Fruits Basket", "r": "7.9", "year": "2001", "votes": "6,003", "genre": "Comedy, Drama", "desp": "After her mothers death, Tohru Honda finds herself living with the Sohma family consisting of three cousins: Yuki, the prince charming of their high school, Kyo the hot headed, short. watch the dude, no spoilers in summary!"}, {"m_title": "Charlotte", "r": "7.5", "year": "2015", "votes": "7,176", "genre": "Drama, Family", "desp": "The story takes place in an alternate reality where a small percentage of children manifest superhuman abilities upon reaching puberty. A focus is placed on Yuu Otosaka, a high school boy who awakens the ability to temporarily possess others, which brings him to the attention of Nao Tomori, the student council president of a school founded as a haven for children with such abilities."}, {"m_title": "Birdie Wing: Golf Girls Story", "r": "7.0", "year": "2022", "votes": "151", "genre": "Sport", "desp": "Eve and Aoi Amawashi, two golfers who come from completely different backgrounds, and have the completely opposite play styles, will shake the world of golf."}, {"m_title": "Tonari no kaibutsu-kun", "r": "7.2", "year": "2012", "votes": "4,506", "genre": "Comedy, Romance", "desp": "Shizuku, a hardworking student who isnt interested in socializing, finds an unlikely friend, Haru, an emotionally volatile boy at her school."}, {"m_title": "The Titans Bride", "r": "5.7", "year": "2020", "votes": "370", "genre": "Comedy, Fantasy", "desp": "Koichi, a student about to graduate, is summoned to a world of Titans. Its the land of Tildant, where the prince, Caius, appears in front of Koichi and says, I want you to be my bride and give birth to my baby!"}, {"m_title": "A Certain Scientific Railgun", "r": "7.5", "year": "2009", "votes": "1,694", "genre": "Action, Comedy", "desp": "In Toaru Kagaku no Railgun T Mikoto Misaka and her friends prepare for the Daihasei Festival, a seven-day sports competition between different Esper schools, which, of course, also include. watch the dude, no spoilers in summary!"}, {"m_title": "Wolfs Rain", "r": "7.9", "year": "2003", "votes": "7,993", "genre": "Action, Adventure", "desp": "In a post-apocalyptic future where humans live in domed cities surrounded by wasteland, wolves are assumed to be two hundred years extinct."}, {"m_title": "Chillin in My 30s after Getting Fired from the Demon Kings Army", "r": "6.7", "year": "2023", "votes": "375", "genre": "Adventure, Comedy", "desp": "A heros son, Dariel, was kidnapped and raised by his demon lord enemy to be his subordinate. But when Dariel is thrown out, he must find his own way to his own place in the world."}, {"m_title": "Happy Sugar Life", "r": "6.4", "year": "2018", "votes": "666", "genre": "Crime, Drama", "desp": "This is a shivering pure love psycho horror story of a girl named Satou Matsuzaka, who has someone she likes. She experiences a sweet feeling when she sleeps with her, and thought it must. watch the dude, no spoilers in summary!"}, {"m_title": "My Stepmoms Daughter Is My Ex", "r": "6.5", "year": "2022", "votes": "825", "genre": "Comedy, Romance", "desp": "Mizuto and Yume are a former couple who enjoyed a relationship in junior high school, but became more and more irritated at each other, and used their graduation as an opportunity to break. watch the dude, no spoilers in summary!"}, {"m_title": "Yuri Is My Job!", "r": "6.5", "year": "2023", "votes": "40", "genre": "Comedy, Drama", "desp": "Hime gives off the image as a sweet, helpful princess but on the inside only cares about herself. Against her will, Hime gets manipulated into working as a waitress at a place thats part. watch the dude, no spoilers in summary!"}, {"m_title": "Kidou Senshi Gundam: Tekketsu no Orphans - Urdr Hunt", "r": "8.8", "year": "2022", "votes": "39", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "City Hunter", "r": "7.7", "year": "1987", "votes": "1,477", "genre": "Action, Comedy", "desp": "The adventures of Ryo Saeba, a gun-for-hire living in the Tokyo metropolis, who will take on any dangerous job as long as it involves beautiful women."}, {"m_title": "Haganai: I Dont Have Many Friends", "r": "7.0", "year": "2011", "votes": "2,508", "genre": "Comedy, Drama", "desp": "A friendless teenager, feared by his fellow classmates, joins a club hoping to finally make some friends."}, {"m_title": "World Trigger", "r": "7.5", "year": "2014", "votes": "936", "genre": "Action, Sci-Fi", "desp": "A gate to another dimension has burst open, and from it emerge gigantic invincible creatures that threaten all of humanity. Earths only defense is a mysterious group of warriors who have. watch the dude, no spoilers in summary!"}, {"m_title": "The Asterisk War", "r": "6.8", "year": "2015", "votes": "2,270", "genre": "Action, Comedy", "desp": "The Academy City on the water, Rikka (also known as Asterisk) is the worlds largest stage for the integrated battle entertainment, Star Wars Festival. The young boys and girls of the. watch the dude, no spoilers in summary!"}, {"m_title": "Laid-Back Camp", "r": "8.1", "year": "2018", "votes": "1,586", "genre": "Adventure, Comedy", "desp": "Nadeshiko, a high school student who had moved from Shizuoka to Yamanashi, decides to see the famous, 1000 yen-bill-featured Mount Fuji. Even though she manages to bike all the way to. watch the dude, no spoilers in summary!"}, {"m_title": "Powerpuff Girls Z", "r": "5.0", "year": "2006", "votes": "626", "genre": "Action, Adventure", "desp": "Three ordinary girls gain superpowers and protect their home city from a variety of villains."}, {"m_title": "Record of the Lodoss War", "r": "7.6", "year": "1990", "votes": "2,716", "genre": "Action, Adventure", "desp": "A group of medieval adventurers must participate in a war against dark forces in the land of Lodoss."}, {"m_title": "Gleipnir", "r": "6.8", "year": "2020", "votes": "1,602", "genre": "Action, Fantasy", "desp": "Shuichi Kagaya is a high school student with an ability to transform into a monster costume. But when he rescues a fellow classmate, she forces him into a partnership to hunt down other monsters."}, {"m_title": "Zombieland Saga", "r": "7.1", "year": "2018", "votes": "1,291", "genre": "Comedy, Fantasy", "desp": "A young singer dies and becomes a zombie. She is put in a musical group with other zombie singer girls as part of a campaign to save the world."}, {"m_title": "Yowamushi Pedal", "r": "7.7", "year": "2013", "votes": "920", "genre": "Comedy, Drama", "desp": "Onoda is a cheerful otaku looking to join his new schools anime club, eager to finally make some friends. Unfortunately, the club has been disbanded and somehow he stumbled into the bicycle club."}, {"m_title": "The Millionaire Detective: Balance - Unlimited", "r": "7.3", "year": "2020", "votes": "2,000", "genre": "Crime, Drama", "desp": "A rich but eccentric detective and a middle-class cop team up to solve cases in Tokyo."}, {"m_title": "Combatants Will Be Dispatched!", "r": "6.7", "year": "2021", "votes": "1,035", "genre": "Action, Comedy", "desp": "Two agents of an evil corporation are assigned to conquer a distant planet, only to find themselves caught up in the wild and zany events of that world."}, {"m_title": "Dragonar Academy", "r": "6.6", "year": "2014", "votes": "659", "genre": "Action, Comedy", "desp": "In a land of dragons, citizens called breeders train their dragon pals. Races of dragons are born from breeders who are marked with Seikoku, also known as star brands. Learning to ride and. watch the dude, no spoilers in summary!"}, {"m_title": "Kuma Kuma Kuma Bear", "r": "6.7", "year": "2020", "votes": "467", "genre": "Action, Adventure", "desp": "Kuma Kuma Kuma Bear revolves around a 15-year-old Yuna. One day, upon returning home, she is hyped for a new update for her favorite worlds first VRMMO, World Fantasy Online. Ever since. watch the dude, no spoilers in summary!"}, {"m_title": "Yuragi-sou no Yuuna-san", "r": "6.8", "year": "2018", "votes": "601", "genre": "Comedy, Fantasy", "desp": "A young man with connections to the supernatural rents a room haunted by the ghost of a 16-year-old girl."}, {"m_title": "To Love-Ru - Darkness", "r": "7.0", "year": "2012", "votes": "997", "genre": "Comedy, Drama", "desp": "Lalas little sister, Momo Belia Deviluke, has fallen in love with Rito. Momo plots to steal Rito away from her sister, Lala. She hopes that if Rito marries Lala, he can legally marry every girl who is in love of him."}, {"m_title": "Blood+", "r": "7.6", "year": "2005", "votes": "4,692", "genre": "Action, Adventure", "desp": "In present day Okinawa, a young amnesiac girl named Saya combats vicious, shape-shifting vampires. Her adoptive family and friends help her track down their source and uncover her past."}, {"m_title": "The Fruit of Evolution: Before I Knew It, My Life Had It Made", "r": "6.1", "year": "2021", "votes": "884", "genre": "Adventure, Comedy", "desp": "A fat schoolkid is thrown into a fantasy world and must team up with a female gorilla to survive."}, {"m_title": "Akebis Sailor Uniform", "r": "7.3", "year": "2022", "votes": "524", "genre": "Comedy, Drama", "desp": "Komichi Akebi is starting junior high at the prestigious Robai Academy in the countryside. Its been her dream to wear a sailor-style school uniform and tomorrow her dream will come true."}, {"m_title": "Godzilla Singular Point", "r": "6.5", "year": "2021", "votes": "1,903", "genre": "Action, Mystery", "desp": "Brought together by a mysterious song, a grad student and an engineer lead the fight against an unimaginable force that may spell doom for the world."}, {"m_title": "Kemono Jihen", "r": "7.3", "year": "2021", "votes": "1,614", "genre": "Action, Fantasy", "desp": "Special detective Kohachi Inugami is sent to investigate a grisly phenomenon involving animal corpses near a remote mountain village. But after meeting a strange boy, he discovers cursed supernatural forces at work."}, {"m_title": "Dagashi kashi", "r": "6.1", "year": "2016", "votes": "820", "genre": "Comedy, Romance", "desp": "Shikada Kokonotsus father owns a rural sweets shop, and his plan is for Kokonotsu to take it over one day. However, Kokonotsu wants to be a manga author instead! One day in summer, the. watch the dude, no spoilers in summary!"}, {"m_title": "Infinite Stratos", "r": "6.6", "year": "2011", "votes": "2,021", "genre": "Action, Comedy", "desp": "A powerful exoskeleton, technologically ages beyond any current such tech, is found, dubbed Infinite Stratos and multiplied. Only women can use it. One lucky boy discovers he can too and. watch the dude, no spoilers in summary!"}, {"m_title": "Shigurui: Death Frenzy", "r": "7.7", "year": "2007", "votes": "2,513", "genre": "Action, Drama", "desp": "The series starts off at a tournament where a one armed samurai faces a blind one and quickly flashes back to reveal the history between the two fighters."}, {"m_title": "My Clueless First Friend", "r": "8.0", "year": "2023", "votes": "61", "genre": "Comedy", "desp": "One lonely, gloomy fifth-grade girl is the target of her classmates relentless bullying and teasing, that is, until a new kid arrives on the scene. Friendly Takada is as clueless as he is. watch the dude, no spoilers in summary!"}, {"m_title": "Moribito: Guardian of the Spirit", "r": "8.2", "year": "2007", "votes": "2,821", "genre": "Action, Adventure", "desp": "The Second Empress hires a spear-wielding woman to save her son from the Mikado (emperor) who believes the young prince is possessed by a water demon foretold to bring a terrible drought upon the land should he live."}, {"m_title": "Karakuri Circus", "r": "7.1", "year": "2018", "votes": "416", "genre": "Action, Fantasy", "desp": "The protagonist, Masaru, has just inherited a big fortune after the death of his father. However, people are trying to get their hands on it by any means necessary, even if it means killing. watch the anime dude!"}, {"m_title": "Love Flops", "r": "6.1", "year": "2022", "votes": "303", "genre": "Comedy, Romance", "desp": "Asahi Kashiwagi lives the normal life of a high school student until one day a TV fortune tellers predictions come true one after another. Fate ties his life and five beautiful girls together. But one day something happens."}, {"m_title": "Yu-Gi-Oh! 5Ds", "r": "6.9", "year": "2008", "votes": "1,820", "genre": "Action, Adventure", "desp": "This Yu-Gi-Oh spinoff explores more serious themes. The story takes place in a dystopian future where Domino City has been divided into Neo Domino and the Satellite sector after a tragic incident. Satellite residents are looked down upon."}, {"m_title": "Shinobi no Ittoki", "r": "6.1", "year": "2022", "votes": "452", "genre": "Action", "desp": "After surviving an attack from the rival Kouga clan, Ittoki Sakuraba, an ordinary boy, discovers that he is the 19th heir of the Iga ninja. Ittoki learns to become an Iga clan ninja and joins the war between the Iga and Kouga clans."}, {"m_title": "Terra Formars", "r": "6.4", "year": "2014", "votes": "1,220", "genre": "Action, Horror", "desp": "With the space program attempting to travel to Mars, 21st century scientists were tasked with warming up the planet so that humans could survive on its surface. They came up with an. watch the dude, no spoilers in summary!"}, {"m_title": "Kanokon", "r": "5.8", "year": "2008", "votes": "502", "genre": "Action, Comedy", "desp": "Kouta, a country boy who lives with his grandfather, is now moving to the city to attend high school. Changes come with his new home and new school, but nothing could have prepared the. watch the dude, no spoilers in summary!"}, {"m_title": "Plastic Memories", "r": "7.3", "year": "2015", "votes": "2,494", "genre": "Drama, Romance", "desp": "After failing his college exams, 18 year old Tsukasa Mizugaki is offered a position at the renowned SAI corporation due to his fathers connections. SAI corporation is known for its. watch the dude, no spoilers in summary!"}, {"m_title": "Asobi Asobase: Workshop of Fun", "r": "7.9", "year": "2018", "votes": "1,449", "genre": "Comedy", "desp": "Three girls form a school club. They play games, but the outcomes often alarm them."}, {"m_title": "Eureka Seven", "r": "7.8", "year": "2005", "votes": "3,206", "genre": "Adventure, Drama", "desp": "14-year-old Renton joins the rebel GekkoState ship, co-pilots the TypeZero with the mysterious Eureka, and unknowingly becomes part of a grand scheme."}, {"m_title": "My Next Life as a Villainess: All Routes Lead to Doom!", "r": "7.3", "year": "2020", "votes": "1,036", "genre": "Comedy, Drama", "desp": "A young girl dies and is reincarnated into a video game. But when she finds out that her role is the villainess destined for doom, she takes measures to change the game and avoid that fate."}, {"m_title": "Tensei kenja no isekai Life ~Dai ni no shokugyô wo ete, sekai saikyô ni nari mashita~", "r": "6.1", "year": "2022", "votes": "497", "genre": "Action, Adventure", "desp": "It tells the story of Yuji Sano who inadvertently accepts a summon to another world. Now, he has to get back to the mountain of work he left behind."}, {"m_title": "Lord Marksman and Vanadis", "r": "7.0", "year": "2014", "votes": "1,348", "genre": "Adventure, Fantasy", "desp": "Tigre, a young noble, is captured by a beautiful female general from an enemy country. Though enchanting, her curves arent the only dangerous thing about her. Shes backed by an army of desirable warrior-goddesses called the Vanadis!"}, {"m_title": "7Seeds", "r": "6.3", "year": "2019", "votes": "1,602", "genre": "Action, Adventure", "desp": "In the immediate future, a giant meteorite has collided with earth. All living organisms, including mankind, have been wiped off the face of the planet. The government, who had foreseen. watch the dude, no spoilers in summary!"}, {"m_title": "Digimon Adventure", "r": "7.8", "year": "1999", "votes": "6,630", "genre": "Action, Adventure", "desp": "An animated series based on the popular childrens toy Digimon, in which kids raise and train electronic monsters to fight against those raised by other kids. The show follows the. watch the dude, no spoilers in summary!"}, {"m_title": "Ace Attorney", "r": "6.5", "year": "2016", "votes": "466", "genre": "Comedy, Crime", "desp": "A rookie lawyer Ryuuichi Naruhodou stands up to save his defendants by proving their innocence from unusual cases!"}, {"m_title": "Maria the Virgin Witch", "r": "6.8", "year": "2015", "votes": "319", "genre": "Action, Adventure", "desp": "Maria, the most powerful witch who lives during the Hundred Years War in France. She despises war, so she obstructs battles with her strong magical powers. However not everyone loves her action, including the angel."}, {"m_title": "Ascendance of a Bookworm", "r": "7.9", "year": "2019", "votes": "1,230", "genre": "Adventure, Comedy", "desp": "Urano, a young Japanese soon-to-be librarian, dies in an earthquake. She is reincarnated in another world as a young girl named Mayne, but learns that books are scarce and only provided to. watch the dude, no spoilers in summary!"}, {"m_title": "The Fruit of Grisaia", "r": "7.0", "year": "2014", "votes": "1,055", "genre": "Drama, Romance", "desp": "Mihama Academy is a prison-like school. It is home to five female students who each have their own reasons. One day, Yuuji the institutes first male student arrives and throws the orderly rhythm of Mihama off balance."}, {"m_title": "Beyblade: Metal Fusion", "r": "6.4", "year": "2009", "votes": "2,021", "genre": "Action, Adventure", "desp": "The story of a teenager and his quest to become the best blader in the world."}, {"m_title": "Junjou Romantica", "r": "7.7", "year": "2008", "votes": "1,792", "genre": "Comedy, Drama", "desp": "Junjo Romantica follows the relationships of 3 very different couples. Romantica, Egoist, and Terrorist are all full of romance, longing, and of course, drama. Its complicated, but so very worth it."}, {"m_title": "Kubo-san wa Mobu o Yurusanai", "r": "6.9", "year": "2023", "votes": "197", "genre": "Comedy, Romance", "desp": "High school student Junta Shiraishi has a simple goal: To live a fulfilling youth. However, achieving this goal appears to be harder than expected, as everyone in his surroundings often. watch the dude, no spoilers in summary!"}, {"m_title": "High Card", "r": "6.9", "year": "2023", "votes": "243", "genre": "Action, Adventure", "desp": "The High Card is a secret club of people who own magical playing cards, which specializes in tracking down missing cards and retrieving them."}, {"m_title": "Masou Gakuen HxH", "r": "5.9", "year": "2016", "votes": "587", "genre": "Action, Comedy", "desp": "Hida Kizuna possesses the HHG (Heart Hybrid Gear) ability, but it is not strong enough to make him particularly important. His older sister calls him to transfer to a strategic defense. watch the dude, no spoilers in summary!"}, {"m_title": "Shinchan", "r": "8.4", "year": "1992", "votes": "1,195", "genre": "Comedy, Family", "desp": "Shinchan is the naughtiest 5 year old boy around. He is smitten with older women, an urge he never manages to saturate. This page covers the original Japanese version as aired in Japan."}, {"m_title": "The Big O", "r": "7.5", "year": "1999", "votes": "2,206", "genre": "Action, Adventure", "desp": "In a futuristic city with lost memories, an expert negotiator fights threats to the city with the help of an android and his own giant robot."}, {"m_title": "Zoids", "r": "7.4", "year": "2001", "votes": "1,042", "genre": "Action, Fantasy", "desp": "Story of a partnership between a Zoid Liger Zero and its pilot Bit Cloud as they battle to become the ultimate team."}, {"m_title": "Fate/Grand Order - Absolute Demonic Front: Babylonia", "r": "7.2", "year": "2019", "votes": "860", "genre": "Action, Adventure", "desp": "Chaldea agents Ritsuka Fujimnaru and Mash Kyrielight travel to Babylon and encounter the heroes and gods of that realm."}, {"m_title": "Dinosaur King", "r": "5.7", "year": "2007", "votes": "804", "genre": "Action, Adventure", "desp": "Max Taylor is the son of paleontologist Dr. Spike Taylor and is a fan of dinosaurs. Upon falling out of bed one morning, he sees something crashing into the woods. With his friends/fellow. watch the dude, no spoilers in summary!"}, {"m_title": "Street Fighter II: V", "r": "7.5", "year": "1995", "votes": "2,380", "genre": "Action, Adventure", "desp": "Based off of the video game, on an island off the coast of Japan, Ryu, a martial artist, was given an invitation and some American money from his friend Ken Masters, telling him to come to. watch the dude, no spoilers in summary!"}, {"m_title": "Radiant", "r": "6.7", "year": "2018", "votes": "560", "genre": "Adventure, Fantasy", "desp": "Synopsis Seth is an aspiring sorcerer from the Pompo Hills area. Like all wizards, he is an infected, one of the few living beings who survived the contact of the Nemesis--creatures. watch the dude, no spoilers in summary!"}, {"m_title": "Babylon", "r": "6.4", "year": "2019", "votes": "621", "genre": "Drama, Mystery", "desp": "Seizaki Zen is a prosecutor with the Tokyo District Public Prosecutors Office. While investigating illegal acts by a certain pharmaceutical company, Seizaki stumbles across a conspiracy. watch the dude, no spoilers in summary!"}, {"m_title": "After the Rain", "r": "7.4", "year": "2018", "votes": "1,438", "genre": "Drama, Romance", "desp": "A young student falls in love with her middle-aged boss."}, {"m_title": "Reborn to Master the Blade: From Hero-King to Extraordinary Squire", "r": "6.2", "year": "2023", "votes": "261", "genre": "Action, Fantasy", "desp": "From his deathbed, Hero-King Inglis, the divine knight and master of all he surveys, gazes down on the empire he built with his mighty hand. Having devoted his life to statecraft and his. watch the dude, no spoilers in summary!"}, {"m_title": "Astro Boy", "r": "6.9", "year": "2003", "votes": "519", "genre": "Action, Adventure", "desp": "Starring Astro, Japans first Superhero and a modern folk hero- an anime adaption for the new century based on the seminal manga Astro Boy by the God of Manga Osamu Tezuka."}, {"m_title": "My Wife Is the Student Council President", "r": "6.1", "year": "2015", "votes": "595", "genre": "Comedy, Romance", "desp": "The story begins with Izumi Hayato running to be student council president. But when a beautiful girl swings in promising the liberalization of love while flinging condoms into the audience. watch the anime dude!"}, {"m_title": "Tiger & Bunny", "r": "7.2", "year": "2011", "votes": "1,145", "genre": "Action, Adventure", "desp": "In a world where superheroes have corporate sponsors and are the stars of their own reality TV show, a new superhero team is born as the veteran hero Wild Tiger is assigned to be partners with rookie Barnaby Brooks Jr."}, {"m_title": "Akiba Maid War", "r": "7.3", "year": "2022", "votes": "425", "genre": "Action, Comedy", "desp": "Akihabara is the center of the universe for the coolest hobbies and quirkiest amusements. In the spring of 1999, bright-eyed Nagomi Wahira moves there with dreams of joining a maid café.. watch the dude, no spoilers in summary!"}, {"m_title": "Panty & Stocking with Garterbelt", "r": "7.3", "year": "2010", "votes": "2,694", "genre": "Action, Comedy", "desp": "Two Angels, Panty and Stocking, are kicked out of Heaven for bad behaviour and sent Ghost hunting by Garterbelt, a Priest of Daten City."}, {"m_title": "Super HxEros", "r": "5.4", "year": "2020", "votes": "359", "genre": "Action, Comedy", "desp": "Highschooler Retto Enjo is a member of the hero group, HXEROS, and together they fight to save the Earth, with or without clothes."}, {"m_title": "Bottom-Tier Character Tomozaki", "r": "7.0", "year": "2021", "votes": "1,096", "genre": "Comedy, Drama", "desp": "Tomozaki is one of the best gamers in Japan, and in his opinion, the game of real life is one of the worst. No clear-cut rules for success, horribly balanced, and nothing makes sense. But. watch the dude, no spoilers in summary!"}, {"m_title": "Black Rock Shooter: Dawn Fall", "r": "5.8", "year": "2022", "votes": "205", "genre": "Action, Drama", "desp": "In 2062, two decades after the failure of a massive workforce automation project. However, Artemis, the artificial intelligence at the core of that project, had decided to attack humanity, and the resulting war laid waste to Earth."}, {"m_title": "Remake Our Life!", "r": "7.0", "year": "2021", "votes": "948", "genre": "Comedy, Drama", "desp": "Hashiba Kyouya is a 28 year old game developer. With his company going bankrupt, and him losing his job, he returns to his hometown. Looking at the success of creators of his age, he finds. watch the dude, no spoilers in summary!"}, {"m_title": "Ranma ½", "r": "8.2", "year": "1989", "votes": "1,652", "genre": "Action, Comedy", "desp": "Ranma Saotome, the boy who turns into a girl with a splash of water, always has his hands full fending off crazed admirers. One of the most fanatical is Shampoo, a girl from a tribe of. watch the dude, no spoilers in summary!"}, {"m_title": "The Vision of Escaflowne", "r": "7.8", "year": "1996", "votes": "4,348", "genre": "Action, Adventure", "desp": "Hitomi is a girl with psychic abilities who gets transported to the magical world of Gaea. She and her friends find themselves under attack from the evil Zaibach empire, and the Guymelf. watch the dude, no spoilers in summary!"}, {"m_title": "Very Private Lesson", "r": "5.6", "year": "1998", "votes": "57", "genre": "Short, Comedy", "desp": "Tairaku, a local high school teacher, lives an ideal personal and professional life until Aya, the beautiful daughter of a very wealthy and powerful yakuza boss, enters his classroom."}, {"m_title": "Reborn!", "r": "7.7", "year": "2006", "votes": "1,605", "genre": "Action, Comedy", "desp": "The baby hitman Reborn, one of the cursed Arcabaleno arrives to teach Tsunayoshi No Good Tsuna Sawada how to become head of the Vongola, a power crime family."}, {"m_title": "Im Standing on a Million Lives", "r": "6.4", "year": "2020", "votes": "1,004", "genre": "Action, Adventure", "desp": "Yotsuya Yuusuke along with his classmates Shindou Iu and Hakozaki Kusue have been transported to a strange and unknown world inhabited by mythological creatures. As soon as they arrive,. watch the dude, no spoilers in summary!"}, {"m_title": "Nagi-Asu: A Lull in the Sea", "r": "7.4", "year": "2013", "votes": "1,288", "genre": "Drama, Fantasy", "desp": "Long ago, humanity lived in the sea, but some humans longed to go to land and eventually left the sea behind. Now in present day, the two human types co-exist though their thinking has. watch the dude, no spoilers in summary!"}, {"m_title": "X", "r": "7.5", "year": "2001", "votes": "1,767", "genre": "Action, Drama", "desp": "Kamui Shirou and Fuuma Monou have been best friends ever since they were kids. Fate, however, separates them when Fuumas mother died under a mysterious circumstances. Fast forward, 6 years. watch the anime dude!"}, {"m_title": "Raki suta: Lucky Star", "r": "7.4", "year": "2007", "votes": "2,322", "genre": "Comedy, Family", "desp": "Having fun in school, doing homework together, cooking and eating, playing video games, watching anime. All those little things make up the daily life of the anime-and. watch the dude, no spoilers in summary!"}, {"m_title": "Sirius the Jaeger", "r": "6.8", "year": "2018", "votes": "2,125", "genre": "Action, History", "desp": "Imperial Capital, 1930. A strange group of people carrying musical instrument cases landed on Tokyo station. They are called the Jaegers, who came to hunt vampires. Amongst them, there. watch the dude, no spoilers in summary!"}, {"m_title": "Back Street Girls", "r": "6.7", "year": "2018", "votes": "972", "genre": "Comedy", "desp": "Three gangsters are forced by their boss into becoming a trio of pop singers."}, {"m_title": "Blue Orchestra", "r": "7.7", "year": "2023", "votes": "32", "genre": "Drama, Music", "desp": "In the fall of his third and final year of middle school, Hajime Aono, a violin prodigy, stopped playing violin for his own personal reasons. But it was also that year when he got to know a. watch the anime dude!"}, {"m_title": "Diabolik Lovers", "r": "4.9", "year": "2013", "votes": "1,568", "genre": "Drama, Horror", "desp": "A girl arrives at an old mystical house to live there. Inside the house she quickly discovers that the habitants, 6 brothers, that are vampires and that she is meant to be their bridal slave."}, {"m_title": "Nadia: The Secret of Blue Water", "r": "7.8", "year": "1990", "votes": "1,908", "genre": "Action, Adventure", "desp": "A teen acrobat of mysterious origins and a young inventor find themselves entangled in a sinister plot that involves the fate of the world."}, {"m_title": "The Genius Princes Guide to Raising a Nation Out of Debt", "r": "7.1", "year": "2022", "votes": "829", "genre": "Comedy, Fantasy", "desp": "Once upon a time in a far away land there lived a prince, a genius prince. The genius prince fought alongside his people and led them to a great many triumphs. However, truth be told, he. watch the dude, no spoilers in summary!"}, {"m_title": "Kakuriyo: Bed & Breakfast for Spirits", "r": "7.6", "year": "2018", "votes": "924", "genre": "Fantasy, Romance", "desp": "A young girl with the ability to see ayakashi is sent to another realm to repay her grandfathers debt and marry an ogre."}, {"m_title": "Otaku Elf", "r": "7.6", "year": "2023", "votes": "30", "genre": "Comedy, Fantasy", "desp": "Koito Koganei works as the teenage shrine maiden at the Takamimi Shrine, catering to the whims of its resident: a centuries-old elf who loves video games as much as she hates going outside.. watch the anime dude!"}, {"m_title": "Kinos Journey", "r": "8.3", "year": "2003", "votes": "3,041", "genre": "Adventure, Drama", "desp": "Young world-weary sharpshooter girl Kino and her talking inquisitive motorcycle Hermes travel around her unusual world, visiting various city-states for three days each to learn about their culture, history and ruling philosophy."}, {"m_title": "Rilakkuma and Kaoru", "r": "8.3", "year": "2019", "votes": "1,791", "genre": "Short, Comedy", "desp": "Her life might be a little mundane, but Kaoru gets to go home to Rilakkuma, her endearingly lazy roommate who happens to be a fuzzy toy bear."}, {"m_title": "Pîtâ Pan no bôken", "r": "7.6", "year": "1989", "votes": "304", "genre": "Adventure, Comedy", "desp": "While in search of his own shadow, Peter arrives in London of our world, where he meets little Wendy and her brothers Michael and John. He takes his new friends back home to Neverland, where they participate in staggering adventures, and must confront Captain Hook and an even darker threat."}, {"m_title": "Pokémon Ultimate Journeys", "r": "7.8", "year": "2021", "votes": "37", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Aharen-san wa hakarenai", "r": "7.0", "year": "2022", "votes": "976", "genre": "Comedy", "desp": "Follows small and cute Reina Aharen, as she starts an odd friendship with her classmate Raidou Matsuboshi."}, {"m_title": "3D Kanojo Real Girl", "r": "6.9", "year": "2018", "votes": "933", "genre": "Comedy, Drama", "desp": "Hikaru Tsutsui is a quiet, unassuming high school student who forsakes the company of his classmates to spend his time reading manga, playing video games, and watching anime. Hikaru is. watch the dude, no spoilers in summary!"}, {"m_title": "Toriko", "r": "7.2", "year": "2009", "votes": "577", "genre": "Action, Adventure", "desp": "A master chef travels the world in search of ingredients to make fantastic dishes."}, {"m_title": "Flowers of Evil", "r": "7.6", "year": "2013", "votes": "1,274", "genre": "Drama, Thriller", "desp": "Takao worships the class beauty, Nanako, from afar. When he is alone in the classroom one day after school, he notices her bag of gym clothes on the floor. He cant resist picking it up. He. watch the anime dude!"}, {"m_title": "Re:ZERO - Starting Life in Another World (Re-Edit)", "r": "8.4", "year": "2020", "votes": "498", "genre": "Adventure, Fantasy", "desp": "This is a special version compiling the first season of Re:ZERO -Starting Life in Another World- and containing unseen scenes."}, {"m_title": "Rainbow: The Seven from Compound Two, Cell Six", "r": "8.3", "year": "2010", "votes": "2,684", "genre": "Crime, Drama", "desp": "Set in 1955, the anime follows the story of seven teenagers locked in a reformatory, waiting for a ray of light in a daily hell of suffering and humiliation, focusing especially on how they faced the life once regained freedom."}, {"m_title": "Beck: Mongolian Chop Squad", "r": "8.3", "year": "2004", "votes": "2,538", "genre": "Drama, Music", "desp": "Koyuki meets the guitarist, Ryuske, and discovers a love and talent for music."}, {"m_title": "Tawawa on Monday", "r": "5.7", "year": "2016", "votes": "259", "genre": "Comedy, Romance", "desp": "The anime follows a salary man who has a chance meeting with a girl named Ai on the train. They begin to meet every Monday on the train, with the man serving as her bodyguard on the crowded commute while they chat."}, {"m_title": "Owarimonogatari", "r": "8.3", "year": "2015", "votes": "1,798", "genre": "Comedy, Fantasy", "desp": "The end of the saga of Araragi Koyomi. What are they fighting against? What will happen to them? And who- or what- is the darkness?"}, {"m_title": "Lupin the 3rd Part 6", "r": "7.2", "year": "2021", "votes": "162", "genre": "Action, Adventure", "desp": "Lupin III challenges a shadowy British organization that has framed him for murder, and later searches for a mysterious woman from his past."}, {"m_title": "God Eater", "r": "6.9", "year": "2015", "votes": "2,134", "genre": "Action, Sci-Fi", "desp": "The year 2071, the Earth has been devastated by mysterious monsters that devour living creatures and are impervious to weapons. An organization called Fenrir is the only one with weapons capable of killing the monsters."}, {"m_title": "Kono Oto Tomare!", "r": "8.0", "year": "2019", "votes": "1,056", "genre": "Drama, Music", "desp": "The sole member of a high school koto club puts together an unlikely crew, including a misunderstood delinquent and a musical prodigy, to learn how to play the traditional string instrument for a music competition."}, {"m_title": "Havent You Heard? Im Sakamoto", "r": "7.1", "year": "2016", "votes": "2,395", "genre": "Comedy", "desp": "Sakamoto is a genius who is well liked by both girls and boys. Despite the strange situations he may be in, Sakamoto always manages to fix the problem in absolute perfection while still being over the top."}, {"m_title": "Gankutsuou: The Count of Monte Cristo", "r": "7.9", "year": "2004", "votes": "2,614", "genre": "Drama, Mystery", "desp": "Trying to escape his uneventful life, Albert, the son of a renowned general from Paris, makes a journey with his friend Franz. During his travels, he meets an immensely wealthy nobleman. watch the dude, no spoilers in summary!"}, {"m_title": "Talentless Nana", "r": "7.0", "year": "2020", "votes": "1,047", "genre": "Action, Fantasy", "desp": "A group of super-powered children are gathered to prepare for defending humanity. One of them has no powers, but is in fact the emissary of a grand mystery."}, {"m_title": "Dog Days", "r": "6.2", "year": "2011", "votes": "144", "genre": "Action, Adventure", "desp": "Following Cinques wild adventure in the Flonyard - an alternate world where its inhabitants look like humans but with animal ears - he is summoned back once again. However, this time he is. watch the anime dude!"}, {"m_title": "Ace of Diamond", "r": "8.2", "year": "2013", "votes": "1,261", "genre": "Sport", "desp": "Follows the life of Eijun Sawamura, a gifted baseball player who have his own extraordinary pitchers talent. His goal is one, to become the ace of Seidou High School."}, {"m_title": "Parallel World Pharmacy", "r": "7.0", "year": "2022", "votes": "679", "genre": "Comedy, Drama", "desp": "A pharmacist is reborn into a magical world and uses his knowledge of medicine from his previous and current lives to become a great medical expert."}, {"m_title": "We Never Learn", "r": "7.0", "year": "2019", "votes": "1,088", "genre": "Comedy, Romance", "desp": "Yuiga Nariyuki tutors three genius of different subjects in highschool to get a scholarship. Furuhashi Fumino is a genius on literature but horrible in math, Ogata Rizu is a genius on. watch the dude, no spoilers in summary!"}, {"m_title": "Keep Your Hands Off Eizouken!", "r": "8.2", "year": "I) (2020", "votes": "1,753", "genre": "Comedy, Drama", "desp": "A group of strangers come together to work on creating the greatest anime series ever."}, {"m_title": "Girls Last Tour", "r": "7.7", "year": "2017", "votes": "1,301", "genre": "Adventure, Drama", "desp": "Two girls wander a world destroyed by war."}, {"m_title": "Paripi Koumei", "r": "7.6", "year": "2022", "votes": "847", "genre": "Comedy, Music", "desp": "The music industry becomes a battlefield when ancient strategist Kongming is transported to modern Japan. Hell use military tactics to turn his new friend Eiko into a music star, and together theyll take Shibuya by storm."}, {"m_title": "Smile PreCure!", "r": "6.2", "year": "2012", "votes": "445", "genre": "Action, Adventure", "desp": "Once upon a time, there was a kingdom of fairy tales called Märchenland, where many fairy tale characters live together in joy. Suddenly, the evil emperor Pierrot made an invasion on. watch the dude, no spoilers in summary!"}, {"m_title": "Senran Kagura: Ninja Flash!", "r": "6.1", "year": "2013", "votes": "230", "genre": "Action, Comedy", "desp": "5 girls with abnormally sized breasts try to battle another group of girls who hate them for, reasons."}, {"m_title": "Ningen Fushin: Adventurers Who Dont Believe in Humanity Will Save the World", "r": "6.1", "year": "2023", "votes": "371", "genre": "Action, Adventure", "desp": "A group of adventurers who had all been betrayed and expelled from their own parties decide to come together as their own party."}, {"m_title": "Thermae Romae Novae", "r": "7.0", "year": "2022", "votes": "1,047", "genre": "Comedy, Fantasy", "desp": "The Roman architect Lucius is transported through time to modern-day Japan, where he learns about Japanese bathhouses and uses this to his gain back home."}, {"m_title": "Blast of Tempest", "r": "7.5", "year": "2012", "votes": "1,509", "genre": "Action, Fantasy", "desp": "Two teenage boys, Mahiro and Yoshino, try to find out more about the mysterious murder of Mahiros sister and at the same time save the world with the help of a mage girl whos stranded on a deserted island. Together they shall have to overcome logic itself."}, {"m_title": "Fena: Pirate Princess", "r": "7.0", "year": "2021", "votes": "839", "genre": "Action, Adventure", "desp": "Fena Houtman, a young orphan girl, has been raised on an island where there is no hope of becoming anything more than chattel, to be used and discarded by soldiers of the British Empire.. watch the dude, no spoilers in summary!"}, {"m_title": "The Magical Revolution of the Reincarnated Princess and the Genius Young Lady", "r": "7.0", "year": "2023", "votes": "405", "genre": "Adventure, Comedy", "desp": "Despite her supposed ineptitude with regular magic, Princess Anisphia defies the aristocracys expectations by developing magicology, a unique magical theory based on memories from her. watch the dude, no spoilers in summary!"}, {"m_title": "Are You Really the Only One Who Likes Me", "r": "7.0", "year": "2019", "votes": "1,081", "genre": "Comedy, Drama", "desp": "A high school students romantic expectations turn upside down, when the girls he likes go with someone else and a girl he hates makes a love confession to him. How will he be able to cope?"}, {"m_title": "Im the Villainess, So Im Taming the Final Boss", "r": "6.8", "year": "2022", "votes": "471", "genre": "Comedy, Fantasy", "desp": "A doomed villainess in a dating game realizes her fate and role, and seeks to prevent her doom by romantically connecting with the games other villain."}, {"m_title": "O Maidens in Your Savage Season", "r": "7.2", "year": "2019", "votes": "816", "genre": "Comedy, Drama", "desp": "Girls in a high school literature club answer the question Whats one thing you want to do before you die with sex, leading to a whirlwind of emotions."}, {"m_title": "Chihayafuru", "r": "8.1", "year": "2011", "votes": "1,422", "genre": "Comedy, Drama", "desp": "Chihaya meets a boy named Arata, a talented karuta player who thinks that she has a potential to become a great player. Chihaya then takes on a new dream of becoming Japans best karuta player."}, {"m_title": "Dakaichi: Im Being Harassed by the Sexiest Man of the Year-", "r": "7.3", "year": "2018", "votes": "1,114", "genre": "Comedy, Drama", "desp": "Saijou Takatos 5 year reign as the Most Huggable No. 1 has been snatched. Azumaya is a newbie actor with a 3-year debut, who uses his sincere sparkling smile to blackmail Takato into silence."}, {"m_title": "K-On!", "r": "7.8", "year": "2009", "votes": "4,100", "genre": "Comedy, Drama", "desp": "K-ON is about 5 high school girls who become friends through the Light Music Club. Yui the lead guitarist, Tsumugi the keyboardist, Mio the bassist, Azusa the rhythm guitarist, and Ritsu the drummer. K-ON is the story of 5 aspiring musicians and their journey through high school together."}, {"m_title": "The Marginal Service", "r": "5.3", "year": "2023", "votes": "20", "genre": "Action, Comedy", "desp": "A group of public safety workers secretly work to protect the world from aliens."}, {"m_title": "Space Battleship Yamato 2199", "r": "8.1", "year": "2012", "votes": "1,055", "genre": "Action, Adventure", "desp": "In the year 2199, the space battleship Yamato embarks on a voyage to the planet Iscandar, to retrieve a device that will rejuvenate a war-ravaged Earth."}, {"m_title": "Persona 4: The Animation", "r": "7.6", "year": "2011", "votes": "943", "genre": "Action, Adventure", "desp": "Yu Narukami has moved from Tokyo to Inaba to stay with his uncle and cousin, while his parents are away on business. However, after a murder shakes up the sleepy town, things get even more. watch the dude, no spoilers in summary!"}, {"m_title": "Initial D: Fifth Stage", "r": "8.0", "year": "2012", "votes": "786", "genre": "Action, Sport", "desp": "The final stage of Project Ds expeditions is coming, and is to be the hardest yet."}, {"m_title": "Knights & Magic", "r": "7.3", "year": "2017", "votes": "1,066", "genre": "Action, Fantasy", "desp": "Ernesti, a programmer and mecha fan in his previous life, is reincarnated in a fantasy world where he sets out to pilot a humanoid weapon."}, {"m_title": "Girlfriend, Girlfriend", "r": "6.6", "year": "2021", "votes": "1,036", "genre": "Comedy, Romance", "desp": "A honest young man is compelled to accept every girl who asks to be romantically involved with him."}, {"m_title": "Carole & Tuesday", "r": "7.8", "year": "2019", "votes": "2,266", "genre": "Drama, Music", "desp": "Two girls, from different backgrounds but sharing a love for music, meet and change history with their united musical talent."}, {"m_title": "Now and Then, Here and There", "r": "7.9", "year": "1999", "votes": "1,837", "genre": "Action, Adventure", "desp": "A Japanese boy named Shu tries to save a strange girl, Lala Ru, from kidnappers and is transported to an alternate Earth on the brink of being swallowed by the sun. There he meets an. watch the dude, no spoilers in summary!"}, {"m_title": "Hinamatsuri", "r": "7.9", "year": "2018", "votes": "2,193", "genre": "Comedy, Fantasy", "desp": "A vortex drops a being from another world into a yakuzas life - a life thats about to be greatly changed."}, {"m_title": "Miru Tights", "r": "5.9", "year": "2019", "votes": "300", "genre": "Short", "desp": "Short anime about high school girls who love tights."}, {"m_title": "Princess Connect! Re: Dive", "r": "6.4", "year": "2020", "votes": "557", "genre": "Action, Adventure", "desp": "After falling from the sky with no memories of his past, Yuuki forms an alliance with 3 new friends, and the hidden mysteries surrounding him and his allies begin to be revealed."}, {"m_title": "Aldnoah.Zero", "r": "7.1", "year": "2014", "votes": "2,338", "genre": "Action, Adventure", "desp": "In 1972, an ancient alien hypergate was discovered on the surface of the moon. Using this technology, humanity began migrating to Mars and settling there. After settlers discovered. watch the dude, no spoilers in summary!"}, {"m_title": "The 8th Son? Are You Kidding Me?", "r": "6.5", "year": "2020", "votes": "1,117", "genre": "Action, Adventure", "desp": "Shingo Ichinomiya, a 25-year-old man working at a firm company reincarnated as the youngest child of a poor noble family living in a back country. He is blessed with a very rare talent, but in his situation it only brought disaster."}, {"m_title": "Boarding School Juliet", "r": "7.3", "year": "2018", "votes": "806", "genre": "Action, Comedy", "desp": "To Love, or not to Love Juliet Percia and Inuzuka Romio are archenemies. They lead two high-school dormitories the Black Dogs and the White Cats from two rivaling countries. Both. watch the dude, no spoilers in summary!"}, {"m_title": "A Sisters All You Need", "r": "6.1", "year": "2017", "votes": "291", "genre": "Comedy, Romance", "desp": "The story revolves around Itsuki Hashima who is a 20 year old novelist who is obsessed with little sisters. Despite his nature, he is admired by a small group of friends that includes a. watch the dude, no spoilers in summary!"}, {"m_title": "Ore no Yubi de Midarero", "r": "5.1", "year": "2020", "votes": "117", "genre": "Drama, Romance", "desp": "At a hairdressers salon, a steamy romance develops between a mousy stylist and her handsome mentor."}, {"m_title": "Shadows House", "r": "7.4", "year": "2021", "votes": "963", "genre": "Fantasy, Mystery", "desp": "In a grandiose mansion, a young cheerful living doll begins to serve her shadow mistress, only to find out through their bonding that things are not what they appear to be."}, {"m_title": "Cells at Work!", "r": "7.6", "year": "2018", "votes": "2,581", "genre": "Action, Comedy", "desp": "One newcomer red blood cell just wants to do her job. Between bacteria incursions and meeting a certain white blood cell, shes got a lot to learn."}, {"m_title": "How Heavy Are the Dumbbells You Lift?", "r": "7.0", "year": "2019", "votes": "928", "genre": "Comedy, Sport", "desp": "A young girls life changes when she joins a gym to lose weight."}, {"m_title": "Sabikui Bisco", "r": "7.0", "year": "2022", "votes": "862", "genre": "Action, Adventure", "desp": "In a world where mushrooms have decimated the people of earth, a doctor gets caught up in a conflict between the survivors government and a band of revolutionaries."}, {"m_title": "Yu-Gi-Oh! Zexal", "r": "5.2", "year": "2011", "votes": "1,024", "genre": "Action, Comedy", "desp": "The story takes place in the near future in a city called Heartland. The story focuses on Yuma Tsukumo, a young duelist who strives to become the Duel Monsters champion, despite being an. watch the dude, no spoilers in summary!"}, {"m_title": "Ore Monogatari!!", "r": "7.8", "year": "2015", "votes": "1,938", "genre": "Comedy, Romance", "desp": "A pretty young girl and a sweet but plain-looking boy fall in love. Will their relationship prevail?"}, {"m_title": "The Twelve Kingdoms", "r": "8.0", "year": "2002", "votes": "2,092", "genre": "Adventure, Drama", "desp": "High school student Youko Nakajima is approached by a strange man who claims hes been searching for her and that she is the rightful ruler of his kingdom."}, {"m_title": "Tatami Time Machine Blues", "r": "7.6", "year": "2022", "votes": "312", "genre": "Comedy, Drama", "desp": "A sweltering midsummer day when Ozu, the protagonists terrible friend, accidentally drowns the only remote control for the air conditioning unit in their apartment. While they discuss. watch the dude, no spoilers in summary!"}, {"m_title": "Kokoro Connect", "r": "7.4", "year": "2012", "votes": "2,497", "genre": "Comedy, Drama", "desp": "Five school friends experience strange supernatural phenomena that put their relationships to test."}, {"m_title": "Taboo Tattoo", "r": "5.4", "year": "2016", "votes": "541", "genre": "Action, Comedy", "desp": "Tattoos - ancient weapons that drastically enhance the physical abilities of their users, known as the Sealed, allowing them to bring forth supernatural phenomena when activated through. watch the anime dude!"}, {"m_title": "Record of Grancrest War", "r": "7.1", "year": "2018", "votes": "1,053", "genre": "Action, Adventure", "desp": "In the Atlatan continent, demons from another dimension called Chaos invaded which the nobility fought back using Crests, which gave them superhuman powers. However, rather than join forces. watch the anime dude!"}, {"m_title": "Seven Mortal Sins", "r": "5.2", "year": "2017", "votes": "300", "genre": "Animation", "desp": "After Lucifer falls from Heaven and plans to become the Demon Lord, she must prove herself by defeating the Seven Deadly Sins."}, {"m_title": "Life Lessons with Uramichi Oniisan", "r": "7.3", "year": "2021", "votes": "630", "genre": "Comedy", "desp": "A popular TV childrens show is run by cheerful hosts, who struggle behind the scenes with production matters and their own bleak lives."}, {"m_title": "Orient", "r": "6.1", "year": "2022", "votes": "391", "genre": "Adventure, Fantasy", "desp": "In an era where Japan is under the control of demons, two warriors fight to overthrow their rule."}, {"m_title": "Freezing Vibration", "r": "6.9", "year": "2013", "votes": "754", "genre": "Action, Drama", "desp": "Ever since the 10th NOVA clash, the Chevalier had concluded that the NOVA have been appearing at a steadier rate and they are losing more Pandoras than they can produce. Their solution is. watch the dude, no spoilers in summary!"}, {"m_title": "The Saints Magic Power Is Omnipotent", "r": "7.1", "year": "2021", "votes": "672", "genre": "Fantasy", "desp": "Sei, a 20-year-old office worker, is whisked away to a whole new world. Unfortunately for Sei, the ritual that summoned her - meant to produce a Saint who would banish the dark magic -. watch the dude, no spoilers in summary!"}, {"m_title": "Play it Cool, Guys", "r": "7.1", "year": "2022", "votes": "144", "genre": "Comedy", "desp": "A slice of life comedy about cool but secretly clumsy guys."}, {"m_title": "Ladies Versus Butlers", "r": "6.5", "year": "2010", "votes": "378", "genre": "Comedy, Romance", "desp": "Heavenly school life? A young man joins a boarding school, surrounded by preppie girls and cute maids. What could go wrong?"}, {"m_title": "School-Live!", "r": "7.1", "year": "2015", "votes": "972", "genre": "Comedy, Drama", "desp": "A young girl pretends to live a normal school life with her friends, unable to perceive what is truly happening around her."}, {"m_title": "Ive Been Killing Slimes for 300 Years and Maxed Out My Level", "r": "6.4", "year": "2021", "votes": "1,057", "genre": "Action, Adventure", "desp": "Azusa Aizawa, a 27 years old woman, died on work site due to overworking. She has the possibility to reincarnate, and so she plans to live a chilling and calm life. But will she be able to be relaxed?"}, {"m_title": "Pop Team Epic", "r": "6.9", "year": "2018", "votes": "956", "genre": "Comedy", "desp": "Two high school girls, the short and exceptionally quick to anger Popuko, and the tall and much more calm Pipimi, do insane things and swear like sailors in this absurdist comedy filled with references and meta humor."}, {"m_title": "Tsurune: Kazemai koukou kyuudoubu", "r": "7.4", "year": "2018", "votes": "496", "genre": "Drama, Sport", "desp": "Minato Narumiya used to be in his middle schools archery club until a certain incident in his last tournament caused him to resolve to quit archery for good. When he attends high school,. watch the dude, no spoilers in summary!"}, {"m_title": "Game World Reincarnation", "r": "7.2", "year": "2022", "votes": "33", "genre": "Comedy, Drama", "desp": "An office lady is reincarnated as a dating game villainess and seduces the hero so that he will hate her, but he instead falls in love with her."}, {"m_title": "Hell Girl", "r": "7.4", "year": "2005", "votes": "1,871", "genre": "Drama, Fantasy", "desp": "When someone wants revenge, they post about it on a special website at midnight. Then Hell Girl appears to do their bidding."}, {"m_title": "Monster Musume no Oisha-san", "r": "6.2", "year": "2020", "votes": "536", "genre": "Comedy, Fantasy", "desp": "The town of Lindworm, here humans and monsters get along with each other. Dr.Glenn is a human doctor that runs his clinic to treat monster girls. Of course, many injured girls appreciate. watch the dude, no spoilers in summary!"}, {"m_title": "High School Prodigies Have It Easy Even in Another World!", "r": "6.2", "year": "2019", "votes": "904", "genre": "Adventure, Comedy", "desp": "Seven high school students that got caught in an airplane accident wake up, in a middle age like fantasy different world where magic and beast-man (juujin) exist - but they do not panic at. watch the dude, no spoilers in summary!"}, {"m_title": "Grimms Fairy Tale Classics", "r": "8.5", "year": "1987", "votes": "397", "genre": "Family, Fantasy", "desp": "An animated series retelling a different folk or fairy tale in each episode."}, {"m_title": "Tachibanakan Triangle", "r": "5.3", "year": "2018", "votes": "127", "genre": "Short, Comedy", "desp": "High school student Natsuno Hanabi moves into a dorm full of strange characters, including a childhood friend whos in love with her and a mysterious girl that Hanabis slowly falling for."}, {"m_title": "Initial D: Second Stage", "r": "8.2", "year": "1999", "votes": "1,517", "genre": "Action, Comedy", "desp": "Accumulating an impressive series of victories with his AE86, Takumi Fujiwara has imposed himself as street racings newest rising star. However, his newly found confidence of winning at. watch the dude, no spoilers in summary!"}, {"m_title": "Uzaki-chan Wants to Hang Out!", "r": "6.9", "year": "2020", "votes": "1,982", "genre": "Comedy, Drama", "desp": "Sakurai Shinichis one wish is for a little peace and quiet. But Uzaki Hana his boisterous, well-endowed underclassman has other plans. All she wants is to hang out and poke fun at him, this might be the start of a beautiful relationship."}, {"m_title": "The Yakuzas Guide to Babysitting", "r": "7.4", "year": "2022", "votes": "495", "genre": "Comedy, Crime", "desp": "Yakuza soldier Kirishima Tooru ha a very bloody, approach to life. The head of the Sakuragi family is so concerned about Kirishimas lack of restraint, he orders Kirishima to become the babysitter to his young daughter, Yaeka."}, {"m_title": "Super Dimension Fortress Macross", "r": "8.5", "year": "1982", "votes": "2,208", "genre": "Action, Adventure", "desp": "An alien spaceship crash lands on Earth and her secrets lead to a desperate war against an alien enemy sent to retrieve the ship."}, {"m_title": "The Faraway Paladin", "r": "6.7", "year": "2021", "votes": "1,003", "genre": "Adventure, Drama", "desp": "A knight raised by three righteous but monstrous guardians sets out on a quest to improve his land."}, {"m_title": "Bakuman.", "r": "8.0", "year": "2010", "votes": "2,555", "genre": "Comedy, Drama", "desp": "Mahiro and Takagi team up to pursue their dreams of making it in the manga industry while sharing experiences, successes and failures along the way."}, {"m_title": "Love After World Domination", "r": "7.1", "year": "2022", "votes": "653", "genre": "Action, Comedy", "desp": "In the public eye, the leaders of the heroic Gelato 5 and the villainous organization Gecko are sworn enemies. In reality? Theyre pursuing a forbidden romance."}, {"m_title": "Oreshura", "r": "6.4", "year": "2013", "votes": "788", "genre": "Comedy, Romance", "desp": "Eita Kido enters high school with the goal of graduating with marks high enough that he can earn a scholarship to medical school. With this goal in mind and the fact that his parents. watch the dude, no spoilers in summary!"}, {"m_title": "Hi Score Girl", "r": "8.0", "year": "2018", "votes": "2,175", "genre": "Comedy, Romance", "desp": "The funny and romantic misadventures of a couple of kids passionate about arcade games in 1991 Japan."}, {"m_title": "Michiko to Hatchin", "r": "7.5", "year": "2008", "votes": "1,094", "genre": "Action, Adventure", "desp": "Michiko is a free-willed sexy diva who escaped from prison, while Hatchin is a run away girl fleeing from her strict catholic foster family. The two join forces and set off on an unpredictable road trip seeking their own freedom."}, {"m_title": "Transformers: Energon", "r": "5.8", "year": "2004", "votes": "857", "genre": "Action, Adventure", "desp": "Years after Transformers: Armada, the Autobots continue to battle the Decepticons, but this time they must also prevent the resurrection of Unicron."}, {"m_title": "Captain Tsubasa: Road to 2002", "r": "7.7", "year": "2001", "votes": "1,339", "genre": "Adventure, Drama", "desp": "It shows the journey of a Japanese kid who wants to become the best footballer in the world and win the football World Cup for Japan."}, {"m_title": "Re: Creators", "r": "7.0", "year": "2017", "votes": "1,064", "genre": "Action, Fantasy", "desp": "People have created many stories. Joy, sadness, anger, deep emotion.Stories stir up emotion and captivate. However, those emotions are nothing more than the feelings of a spectator. What if. watch the anime dude!"}, {"m_title": "Robin Hood", "r": "7.2", "year": "1990", "votes": "880", "genre": "Adventure, Comedy", "desp": "Several centuries ago in Nottingham County, a cruel and greedy baron by the name of Alwine ordered the destruction of the Huntingtons castle - from the fire of which Robin and his cousins,. watch the anime dude!"}, {"m_title": "Yusibu: I couldnt become a hero, so I reluctantly decided to get a job.", "r": "6.3", "year": "2013", "votes": "314", "genre": "Action, Comedy", "desp": "In the aftermath of a war between heroes and demons, a hero in training and a demon lords daughter must work together as shop assistants."}, {"m_title": "Ronin Warriors", "r": "7.4", "year": "1988", "votes": "862", "genre": "Action, Adventure", "desp": "After a thousand years of exile, Talpa, ruler of the Netherworld, returns to claim the Earth as his own. The only thing that can stop him and his minions are five young men and their mystical suits of armor."}, {"m_title": "Air Gear", "r": "7.1", "year": "2006", "votes": "1,333", "genre": "Action, Comedy", "desp": "A couple of orphans form a gang called Storm Riders and try to become top dogs in the competitive world of professional and street sport that involves Air Treks, futuristic motorized rollerblades that can achieve super speed."}, {"m_title": "Alderamin on the Sky", "r": "7.4", "year": "2016", "votes": "1,181", "genre": "Adventure, Drama", "desp": "The mighty Katjvarna Empire is currently embroiled in a war with the neighboring Kiorka Republic. Within an unspecified corner of this empire, a certain young man was reluctantly preparing. watch the dude, no spoilers in summary!"}, {"m_title": "Cats Eye", "r": "7.2", "year": "1983", "votes": "982", "genre": "Action, Adventure", "desp": "The 3 Kisugi sisters, Rui, Hitomi and Ai run the Cats Eye Cafe. They are the daughters of Michael Heinz, a painter who mysteriously disappeared in the seventies. In order to find out what. watch the dude, no spoilers in summary!"}, {"m_title": "Gamers!", "r": "6.6", "year": "2017", "votes": "1,570", "genre": "Comedy, Romance", "desp": "This is a story that revolves around certain students and one hobby. Amano Keita is our lonely protagonist who has a passion for gaming and is friends with Uehara Tasuku, who is secretly a. watch the dude, no spoilers in summary!"}, {"m_title": "Dance in the Vampire Bund", "r": "6.5", "year": "2010", "votes": "838", "genre": "Action, Drama", "desp": "Vampire queen Mina Tepes comes out of hiding to offer peace between vampires and humans. Japan gives her a bund, where she goes to live with her people. However, extremist human and vampire groups do not want to coexist."}, {"m_title": "Phantom: Requiem for the Phantom", "r": "7.6", "year": "2009", "votes": "1,828", "genre": "Action, Crime", "desp": "The story of two brainwashed assassins, Ein and Zwei, who struggle to regain their memories as they work for the Inferno crime syndicate."}, {"m_title": "Tenchi Muyo! War on Geminar", "r": "7.0", "year": "2009", "votes": "317", "genre": "Animation", "desp": "Young Kenshi Masaki was kidnapped, transported to a distant galaxy, and captured by a gorgeous princess! This royal beauty claims Kenshi as her personal servant - and lends him out to all. watch the dude, no spoilers in summary!"}, {"m_title": "Super Lovers", "r": "6.8", "year": "2016", "votes": "1,154", "genre": "Comedy, Romance", "desp": "While spending summer with his mother, Haru meets Ren, a boy newly adopted by his mother. Getting along with Ren isnt easy as Harus goal is to civilize him by the time summer ends.. watch the dude, no spoilers in summary!"}, {"m_title": "Seiren", "r": "5.7", "year": "2017", "votes": "345", "genre": "Comedy, Drama", "desp": "Shoichi Kamita is an ordinary high school boy, who is faced with the university entrance exam and worried about his future. This campus romantic comedy, Seiren, which means honest in. watch the dude, no spoilers in summary!"}, {"m_title": "Love Hina", "r": "6.9", "year": "2000", "votes": "2,298", "genre": "Comedy, Drama", "desp": "When Keitaro was a child, he was friends with a little girl. When the girl had to move away, the two made a promise to meet each other at Tokyo University when they grew up. Years later,. watch the dude, no spoilers in summary!"}, {"m_title": "Bunny Drop", "r": "8.3", "year": "2011", "votes": "3,604", "genre": "Comedy, Drama", "desp": "A man adopts his six-year-old aunt and raises her as his own kid."}, {"m_title": "Undefeated Bahamut Chronicle", "r": "6.3", "year": "2016", "votes": "931", "genre": "Action, Fantasy", "desp": "Lux, a former prince of an empire named Arcadia that was overthrown via a rebellion five years earlier, accidentally trespasses in a female dormitorys bathing area, sees the kingdoms new. watch the dude, no spoilers in summary!"}, {"m_title": "Sound! Euphonium", "r": "7.9", "year": "2015", "votes": "1,257", "genre": "Comedy, Drama", "desp": "Kumiko, a girl with absolutely no ambition, decides to change herself now that she has started her high school life and try to leave her hobby, playing euphonium in orchestra aside but she totally cannot forget it anyway."}, {"m_title": "Love Tyrant", "r": "6.3", "year": "2017", "votes": "818", "genre": "Comedy, Drama", "desp": "When a strange girl named Guri comes knocking at Seiji Ainos door, he quickly finds himself thrust into a world of romantic troubles."}, {"m_title": "Durarara!!x2", "r": "7.7", "year": "2015", "votes": "1,974", "genre": "Action, Comedy", "desp": "6 Months Later...We havent changed at all - or it seems.... Half a year after the turmoil that rocked the entire city of Ikebukuro- Peace has once again returned to the city and people. watch the dude, no spoilers in summary!"}, {"m_title": "OniAi", "r": "5.8", "year": "2012", "votes": "300", "genre": "Animation", "desp": "After 6 years of living apart due to the death of their parents, the Himenokoji twins, Akito and Akiko, are finally reunited when Akito desires to live as a family once more. Although. watch the dude, no spoilers in summary!"}, {"m_title": "Beelzebub", "r": "7.5", "year": "2011", "votes": "2,161", "genre": "Action, Comedy", "desp": "The story follows the strongest juvenile delinquent, Oga Tatsumi, he is a first year in Ishiyama High, the school for delinquents. The story starts as he is telling the story of how he. watch the dude, no spoilers in summary!"}, {"m_title": "Digimon Frontier", "r": "6.7", "year": "2002", "votes": "2,781", "genre": "Action, Adventure", "desp": "Digimon Frontier introduces a new universe in the Digimon fandom. Five kids from the Real World have been called to the Digital World to acquire the legendary spirits to save the Digital. watch the dude, no spoilers in summary!"}, {"m_title": "World Break: Aria of Curse for a Holy Swordsman", "r": "6.8", "year": "2015", "votes": "1,047", "genre": "Action, Fantasy", "desp": "The scenery takes place in a private high school involving the concept of saviors. They are known as people who possess awakened memories of their past lives. The story tells of a young. watch the dude, no spoilers in summary!"}, {"m_title": "Onegai My Melody", "r": "8.3", "year": "2005", "votes": "77", "genre": "Comedy, Fantasy", "desp": "Kuromi, an escaped prisoner from Mari Land, has the Melody Key, an artifact with dream-corrupting powers, and she and her partner Baku have plans to gather 100 black notes to revive the Spirit of Dark Power."}, {"m_title": "Mobile Suit Gundam Seed", "r": "7.7", "year": "2002", "votes": "2,518", "genre": "Action, Drama", "desp": "The discovery of an advanced model of combat mecha on their space colony throws a young man and his friends into involvement in an interplanetary war as the crew of a powerful fighting ship."}, {"m_title": "Sugar Apple Fairy Tale", "r": "6.8", "year": "2023", "votes": "207", "genre": "Action, Adventure", "desp": "Anne Halford is a candy crafter determined to follow in her mothers footsteps and become a Silver Sugar Master, a title bestowed only by royalty. In order to travel to the capital and. watch the dude, no spoilers in summary!"}, {"m_title": "Detective Conan: The Culprit Hanzawa", "r": "6.5", "year": "2022", "votes": "297", "genre": "Comedy", "desp": "It is a spin-off of the original manga Detective Conan by Gosho Aoyama and stars the black-silhouetted criminal that appears in the main series to represent the mystery culprits."}, {"m_title": "Hetalia: Axis Powers", "r": "7.3", "year": "2009", "votes": "1,367", "genre": "Comedy", "desp": "An anime in which personified countries show history in a humorous fashion."}, {"m_title": "A Certain Magical Index", "r": "6.8", "year": "2008", "votes": "2,111", "genre": "Action, Comedy", "desp": "Index is still a fugitive and many powerful Magical organizations and individuals want to take her to use the books in her memory eliminating her in the process. War between those. watch the dude, no spoilers in summary!"}, {"m_title": "Texhnolyze", "r": "7.6", "year": "2003", "votes": "2,920", "genre": "Drama, Sci-Fi", "desp": "In a man-made underground society, descendants of a banished generation vie for control of the crumbling city of Lux. Ichise, an orphan turned prize fighter, loses a leg and an arm to. watch the dude, no spoilers in summary!"}, {"m_title": "Love Live!: School Idol Project", "r": "7.2", "year": "2013", "votes": "1,047", "genre": "Comedy, Drama", "desp": "When news gets out that the school is going to close due to poor numbers of new students Honoka and several of her friends rally together and form a school Idol group in order to prevent the school from closing."}, {"m_title": "Weather Report Girl", "r": "6.1", "year": "1994", "votes": "93", "genre": "Comedy", "desp": "Weather reporter Keiko Nakadai becomes a sensation by flashing her lingerie when on air. Behind the scenes she wages a war with envious and vicious competitors out to bring her down."}, {"m_title": "Fate/Extra Last Encore", "r": "5.3", "year": "2018", "votes": "888", "genre": "Action, Fantasy", "desp": "Waking up in a strange virtual world with no recollection of the past, Hakuno finds himself forced to fight for survival in a war he does not understand for a prize beyond value; the. watch the dude, no spoilers in summary!"}, {"m_title": "School Rumble", "r": "7.8", "year": "2004", "votes": "2,093", "genre": "Comedy, Romance", "desp": "A young high schooler is in love with an oblivious classmate; a delinquent classmate is in love with her. Both struggle to confess their feelings and their antics lead to crazy situations."}, {"m_title": "Code: Breaker", "r": "6.7", "year": "2012", "votes": "1,082", "genre": "Adventure, Crime", "desp": "An ordinary high school girl makes it her mission to stop supernatural assassins from killing morally corrupt people."}, {"m_title": "So Im a Spider, So What?", "r": "7.2", "year": "2021", "votes": "1,670", "genre": "Adventure, Comedy", "desp": "Something mysterious happens and an entire class finds themselves reincarnated in a RPG-game-like fantasy world. The main character, a girl, finds herself reincarnated in a spider. Will she able to survive and to find out what happened?"}, {"m_title": "Initial D: Final Stage", "r": "8.5", "year": "2014", "votes": "682", "genre": "Action, Sport", "desp": "After defeating every racing team, everything comes down to one last race. Will Project D succeed in the final and most difficult race of Initial D. And what is going to be the future of Project D after this final race."}, {"m_title": "Astra Lost in Space", "r": "8.0", "year": "2019", "votes": "1,561", "genre": "Action, Adventure", "desp": "Warped into outer space by a mysterious sphere, nine children find themselves in an assassination plot as they retrieve a spaceship and try to return home safely."}, {"m_title": "Blue Gender", "r": "7.2", "year": "1999", "votes": "1,268", "genre": "Action, Sci-Fi", "desp": "Yuji Kaido was diagnosed with a serious disease and since there was no cure for the unknown illness, he was cryogenically frozen. Twenty years later, he wakes up to see the Earth like it. watch the dude, no spoilers in summary!"}, {"m_title": "Brynhildr in the Darkness", "r": "6.7", "year": "2014", "votes": "1,031", "genre": "Action, Drama", "desp": "Brynhildr in the Darkness."}, {"m_title": "Digimon Tamers", "r": "7.6", "year": "2001", "votes": "2,169", "genre": "Action, Adventure", "desp": "An animated series based on the popular childrens toy Digimon, in which kids raise electronic monsters to fight against those raised by other kids. The show follows the adventures of a. watch the dude, no spoilers in summary!"}, {"m_title": "Mobile Fighter G Gundam", "r": "7.5", "year": "1994", "votes": "1,072", "genre": "Action, Adventure", "desp": "It is the year Future Century 60 and most of humanity has migrated to orbiting space colonies, leaving behind the polluted Earth. To prevent war, the colonies came up with the tournament. watch the dude, no spoilers in summary!"}, {"m_title": "Sword Gai: The Animation", "r": "5.8", "year": "2018", "votes": "1,007", "genre": "Action, Fantasy", "desp": "A boy bonds with a demon sword."}, {"m_title": "Eden", "r": "6.6", "year": "I) (2021", "votes": "1,315", "genre": "Action, Adventure", "desp": "A human girl secretly raised by robots starts to uncover the dark secrets behind her lush, utopian world where humanity has all but vanished."}, {"m_title": "Our Last Crusade or the Rise of a New World", "r": "6.4", "year": "2020", "votes": "822", "genre": "Action, Fantasy", "desp": "As a war rages on between a machine-based empire and a magic-focused empire, a witch and a soldier meet and fall in love. Will their romance bring an end to their nations conflict?"}, {"m_title": "Zatch Bell!", "r": "7.2", "year": "2003", "votes": "1,085", "genre": "Action, Adventure", "desp": "Kiyo Takamine meets a momoto (demons from another world) boy zatch bell and set out to compete in the momoto games held every 1000 years to become their king."}, {"m_title": "Cardcaptor Sakura: Clear Card Arc", "r": "7.8", "year": "2018", "votes": "737", "genre": "Adventure, Comedy", "desp": "Cardcaptor Sakura: Clear Card starts at the point where Cardcaptor Sakura ends, when Sakura Kinomoto starts junior high school alongside her friends, including her new boyfriend Syaoran,. watch the dude, no spoilers in summary!"}, {"m_title": "Bubblegum Crisis: Tokyo 2040", "r": "7.1", "year": "1998", "votes": "872", "genre": "Action, Adventure", "desp": "Set in 2039-2040, This series is set in Megalo City, a city built over the ruins of Tokyo. Boomers, a line of android workers created by the Genom Corporation, perform dozens of tasks, from. watch the anime dude!"}, {"m_title": "Kakegurui Twin", "r": "6.5", "year": "2022", "votes": "703", "genre": "Drama, Mystery", "desp": "A year before Yumeko Jabami graced the hallowed halls of Hyakkou Private Academy, Mary Saotome got her own start at the gambling-addicted school. Can this normal girl achieve her own rags-to-riches story through wits and luck?"}, {"m_title": "Ghost Hunt", "r": "7.4", "year": "2006", "votes": "1,544", "genre": "Comedy, Drama", "desp": "Shibuya Psychic Research Center investigates supernatural occurrences all over Japan."}, {"m_title": "Sword Art Online Alternative: Gun Gale Online", "r": "6.8", "year": "2018", "votes": "2,531", "genre": "Action, Adventure", "desp": "A university student tries out a VR battle royale shooter and quickly establishes herself as one of the most menacing players in the competitive scene thanks to her tiny, lightning-fast avatar LLENN."}, {"m_title": "Strawberry Panic", "r": "7.1", "year": "2006", "votes": "734", "genre": "Drama, Romance", "desp": "Aoi Nagisa, a young reddish haired girl, just transfers to an all-girls, catholic school, the Starrawberry Dorms. The school is divided in to three sections (Lulim, Miator, Spica) all in. watch the dude, no spoilers in summary!"}, {"m_title": "Pingu in the City", "r": "7.3", "year": "2017", "votes": "172", "genre": "Short, Comedy", "desp": "Pingu and his family move from their small village to the big city; in which there are many people with many different occupations. The ever-curious Pingu tries to join them at their jobs,. watch the dude, no spoilers in summary!"}, {"m_title": "Magmel of the Sea Blue", "r": "6.1", "year": "2019", "votes": "715", "genre": "Action, Adventure", "desp": "Unknown creatures. Deadly pitfalls. With so many reckless adventures out to make fortune, someones got to rescue them."}, {"m_title": "Monsuno", "r": "4.9", "year": "2011", "votes": "626", "genre": "Action, Adventure", "desp": "Monsuno hurtled through space eons before man existed. Upon burning through the planets atmosphere and crashing to Earth, it brought chaos and the extinction of the dinosaurs. Then, it vanished. Now, Monsuno is back. Is the Earth doomed?"}, {"m_title": "Kuro Gal ni Nattakara Shinyuu to Shite Mita.", "r": "6.8", "year": "2021", "votes": "97", "genre": "Fantasy, Romance", "desp": "Two womanizers and best friends take their relationship to a new level when one of them is cursed to become a pretty girl."}, {"m_title": "Dragonaut: The Resonance", "r": "6.2", "year": "2007", "votes": "127", "genre": "Action, Drama", "desp": "With a deadly asteroid hurtling towards Earth, sexy human Dragonauts in tight uniforms and their curvaceous dragon companions come together to create a formidable fighting force with all. watch the dude, no spoilers in summary!"}, {"m_title": "The Detective Is Already Dead", "r": "5.9", "year": "2021", "votes": "1,020", "genre": "Comedy, Drama", "desp": "Kimizuka Kimihiko is a crisis-magnet. From getting caught up in a crime scene to accidentally witnessing a drug deal, trouble seems to find him around every corner. So it is no surprise. watch the dude, no spoilers in summary!"}, {"m_title": "Danganronpa 3: The End of Kibougamine Gakuen - Zetsubou-hen", "r": "7.0", "year": "II) (2016", "votes": "844", "genre": "Drama, Horror", "desp": "Set before the events of Danganronpa: Trigger Happy Havoc, The Despair side of Danganronpa 3 shows the events leading to the rise of the Remnants of Despair and the fall of Hopes Peak and the world into anarchy."}, {"m_title": "Twin Star Exorcists", "r": "7.1", "year": "2016", "votes": "934", "genre": "Action, Fantasy", "desp": "Rokuro and Benio are named the future parents of a child of prophecy. But before they can have a baby, theyll have to raise a little hell."}, {"m_title": "The Legend of the Legendary Heroes", "r": "7.0", "year": "2010", "votes": "740", "genre": "Action, Adventure", "desp": "Also known as The Legend of the Legendary heroes, follows three main characters: A dongo-obsessed female, a peace-crazy half noble, and a lazy nap-taking mage. The mage is the main. watch the dude, no spoilers in summary!"}, {"m_title": "Love Live! Superstar!!", "r": "7.2", "year": "2021", "votes": "168", "genre": "Comedy, Drama", "desp": "Yuigaoka Girls High School is a newly established school that lies between the Omotesando, Harajuku, and Aoyama neighborhoods of Tokyo. The story centers on its first batch of students.. watch the dude, no spoilers in summary!"}, {"m_title": "Ookami shoujo to kuro ouji", "r": "6.9", "year": "2014", "votes": "2,755", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Last Exile", "r": "7.8", "year": "2003", "votes": "3,741", "genre": "Action, Adventure", "desp": "In a world of airships and civil war, two messengers come across a special girl who will change the course of history."}, {"m_title": "Science Fell in Love, So I Tried to Prove It", "r": "6.9", "year": "2020", "votes": "881", "genre": "Comedy, Romance", "desp": "Two scientists seek to find if love can be solved by a scientific theory. They will find the answer in their own hearts and feelings."}, {"m_title": "Trinity Blood", "r": "7.1", "year": "2005", "votes": "2,815", "genre": "Action, Drama", "desp": "In a futuristic world of an unsteady truce between humans and vampires a superhuman priest battles to secure a peaceful co-existence."}, {"m_title": "SoniAni: Super Sonico the Animation", "r": "5.6", "year": "2014", "votes": "136", "genre": "Comedy, Music", "desp": "Super Sonico is a klutzy, adorable, and well-endowed 18-year old girl. Shes a complete sucker for anything cute, especially stray cats, all of which she giddily adopts. However, though. watch the dude, no spoilers in summary!"}, {"m_title": "Wanna be the Strongest in the World!", "r": "5.3", "year": "2013", "votes": "164", "genre": "Action, Drama", "desp": "Hagiwara Sakura and Miyazawa Erena are the leading members of popular idol group, Sweet Diva. Erena is injured by the attack of a female pro-wrestler called Kazama Rio during the recording. watch the dude, no spoilers in summary!"}, {"m_title": "Full Dive: This Ultimate Next-Gen Full Dive RPG Is Even Shittier Than Real Life!", "r": "6.4", "year": "2021", "votes": "501", "genre": "Action, Comedy", "desp": "Yuuki Hiroshi has just picked up Kiwame Quest, the realest VRMMO ever made. But this RPG is TOO lifelike; he can smell, taste, feel, and when hes hurt in it, his physical body is hurt too.. watch the anime dude!"}, {"m_title": "Princess Tutu", "r": "8.1", "year": "2002", "votes": "1,092", "genre": "Adventure, Comedy", "desp": "A duck turned girl with magical powers must help save a prince from an unfinished fairy tale."}, {"m_title": "Campione!", "r": "6.8", "year": "2012", "votes": "1,364", "genre": "Action, Adventure", "desp": "Some people find religion, but for 16-year-old Godou Kusanagi, its religion thats found him! As the result of defeating the God of War in mortal combat, Godou received the position of. watch the dude, no spoilers in summary!"}, {"m_title": "Are You Lost?", "r": "6.2", "year": "2019", "votes": "388", "genre": "Adventure, Comedy", "desp": "Because of a plane crash - starting today, were spending the springtime of our lives on a desert island. Theres nothing here, so we have to make everything. And eat everything. (Ugh!). watch the dude, no spoilers in summary!"}, {"m_title": "Naruto SD: Rock Lee & His Ninja Pals", "r": "7.2", "year": "2012", "votes": "959", "genre": "Action, Comedy", "desp": "Naruto SD: Rock Lee & His Ninja Pals is story of Rock Lees adventures from Rock Lees perspective."}, {"m_title": "Magical Girl Site", "r": "5.9", "year": "2018", "votes": "474", "genre": "Action, Drama", "desp": "Aya Asagiri is a middle school girl who has problems both at school with bullying and at home from physical abuse by her brother. While browsing online, a website pops up on her computer. watch the dude, no spoilers in summary!"}, {"m_title": "Azumanga Daioh: The Animation", "r": "8.2", "year": "2002", "votes": "3,188", "genre": "Comedy", "desp": "A humorous and highly unpredictable animated series following a group of high school girls and their teachers."}, {"m_title": "Idaten Jump", "r": "8.2", "year": "2005", "votes": "621", "genre": "Adventure, Comedy", "desp": "The show follows the exciting adventures of Sho Yamato, a boy who loves to race his Idaten bike and ends up getting lost, along with his friends, in a mysterious place called the X-Zone.. watch the dude, no spoilers in summary!"}, {"m_title": "The Prince of Tennis II: U-17 World Cup", "r": "8.0", "year": "2022", "votes": "50", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Tokyo Ravens", "r": "7.3", "year": "2013", "votes": "1,319", "genre": "Action, Comedy", "desp": "Tsuchimikado Harutora was born into the prestigious Onmyoji family, but cant see spirit energy. Therefore, hes been enjoying a peaceful daily life with his friends at the Onmyo branch. watch the dude, no spoilers in summary!"}, {"m_title": "The Helpful Fox Senko-san", "r": "6.8", "year": "2019", "votes": "804", "genre": "Comedy, Fantasy", "desp": "The everyday life of Nakano, a salaryman working for an exploitative company, is suddenly intruded upon by the fox, Senko-san (800-year-old little girl). Whether it be cooking, cleaning, or. watch the anime dude!"}, {"m_title": "Strawberry Marshmallow", "r": "7.7", "year": "2005", "votes": "336", "genre": "Comedy, Drama", "desp": "An anime series about girls doing things in strange and cute ways."}, {"m_title": "Guyver: The Bioboosted Armor", "r": "7.5", "year": "1989", "votes": "943", "genre": "Action, Horror", "desp": "Sho and his friend Tetsurou stumble upon an odd alien artifact while walking through the woods. Then, the alien artifact breaks free of its metallic bonds and enters Shos body, turning him. watch the anime dude!"}, {"m_title": "When They Cry: Kai", "r": "8.1", "year": "2007", "votes": "2,562", "genre": "Drama, Horror", "desp": "In this continuation of the hit horror anime, secrets are revealed as Rika Furude desperately attempts to save Hinamizawa and her friends from their tragic fate"}, {"m_title": "Seitokai yakuindomo", "r": "7.1", "year": "2010", "votes": "679", "genre": "Comedy, Drama", "desp": "Takatoshi Tsuda joins Ousai Academy, a co-ed high school that until recently had been an all-girl school, where he is asked to join the student council as vice-president and a male representative."}, {"m_title": "The King of Fighters: Destiny", "r": "6.7", "year": "2017", "votes": "156", "genre": "Action, Adventure", "desp": "The show follows Kyo arriving in South Town where he must run into another formidable fighter, Terry. Ten years have passed since tragedy that changed his life forever."}, {"m_title": "Suppose a Kid from the Last Dungeon Boonies moved to a starter town?", "r": "6.4", "year": "2021", "votes": "1,037", "genre": "Adventure, Fantasy", "desp": "Considered a weakling his entire life, novice adventurer Lloyd leaves his village located at the fringe of the mortal world to fulfill his dream of becoming a soldier."}, {"m_title": "Ushio & Tora", "r": "7.6", "year": "2015", "votes": "964", "genre": "Action, Comedy", "desp": "Ushio Aotsuki unknowingly releases demonic energy that attracts monsters. The demon Tora, makes him an offer. He will help fight the monsters if Ushio removes the legendary Beast Spear that trapped him in Ushios basement for 500 years."}, {"m_title": "Love of Kill", "r": "6.7", "year": "2022", "votes": "692", "genre": "Action, Mystery", "desp": "Two assassins are romantically drawn to each other, even though their profession makes them rivals and targets to each other."}, {"m_title": "DanMachi: Is It Wrong to Try to Pick Up Girls in a Dungeon? On the Side - Sword Oratoria", "r": "7.1", "year": "2017", "votes": "1,582", "genre": "Action, Adventure", "desp": "The quests of Lefiya Viridis together with Aiz Wallenstein the Sword Princess as they enter the Dungeon to investigate new kinds of monsters that have been causing uproars among adventurers."}, {"m_title": "Forest of Piano", "r": "7.2", "year": "2018", "votes": "1,026", "genre": "Adventure, Drama", "desp": "Two different people - the son of a prostitute, and the son of a professional musician - develop a bond of friendship through their love of playing the piano."}, {"m_title": "Tenjo Tenge", "r": "6.8", "year": "2004", "votes": "1,286", "genre": "Action, Comedy", "desp": "An ongoing war takes place at Toudou Academy, where all the students and teachers are equipped in martial arts combat."}, {"m_title": "Life with an Ordinary Guy Who Reincarnated Into a Total Fantasy Knockout", "r": "6.7", "year": "2022", "votes": "469", "genre": "Adventure, Comedy", "desp": "Childhood friends Tachibana Hinata and Jinguji Tsukasa were living the everyday life of office workers. Then, on the way home from a mixer, they were sent flying into another world by a. watch the dude, no spoilers in summary!"}, {"m_title": "Lupin III", "r": "7.6", "year": "2015", "votes": "584", "genre": "Action, Adventure", "desp": "Lupin IIIs marriage to thrill-seeking entrepreneur Rebecca Rossellini is merely a pretext for him to commit a string of heists across Italy and San Marino."}, {"m_title": "Armed Girls Machiavellism", "r": "6.6", "year": "2017", "votes": "456", "genre": "Action, Comedy", "desp": "A boy, expelled from his old school, transfers to a private academy, only to find its female students armed and on the defense, with his only chance at survival being to crossdress."}, {"m_title": "Yu-Gi-Oh!", "r": "7.8", "year": "1998", "votes": "1,939", "genre": "Adventure, Comedy", "desp": "Yugi Mutou is a shy young boy who has finally completed his grandfathers Thousand-Year puzzle. Little does he realize that this puzzle unleashes an alter ego who challenges anyone who hurts Yugi to a game. The losers often pay dearly."}, {"m_title": "Ayakashi Triangle", "r": "6.1", "year": "2023", "votes": "62", "genre": "Action, Comedy", "desp": "Matsuri Kazamaki and Suzu Kanade are childhood friends, bonded by their shared ability to see ayakashi - youkai in search of a life force to devour. However, in high school, their. watch the dude, no spoilers in summary!"}, {"m_title": "Adachi to Shimamura", "r": "7.1", "year": "2020", "votes": "610", "genre": "Drama, Romance", "desp": "Adachi and Shimamura become friends, but soon new feelings blossom. They travel the sea of emotions as they learn each others feelings."}, {"m_title": "Yu-Gi-Oh! Arc-V", "r": "6.0", "year": "2014", "votes": "732", "genre": "Adventure, Fantasy", "desp": "Thanks to the Leo Corporation, headed by Reiji Akaba, new advancements to Solid Vision, the holographic system that gives life to the Duel Monsters card game, give physical mass to monsters. watch the anime dude!"}, {"m_title": "The Disastrous Life of Saiki K.: Reawakened", "r": "7.9", "year": "2019", "votes": "1,971", "genre": "Comedy, Fantasy", "desp": "Kusuo and his gaggle of self-proclaimed friends are back for more psychic mishaps. If he didnt have enough problems before, hes got even more now."}, {"m_title": "Uma Musume: Pretty Derby", "r": "6.9", "year": "2018", "votes": "234", "genre": "Comedy, Drama", "desp": "Horse girl Special Week has moved from the country to the city to attend Tracen Academy. There, she and her classmates compete to win the Twinkle Series and earn the title of The Countys #1 Horse Girl."}, {"m_title": "Bludgeoning Angel Dokuro-chan", "r": "6.8", "year": "2005", "votes": "847", "genre": "Comedy, Fantasy", "desp": "A boy is fond of an angel that kills him again and again."}, {"m_title": "Sergeant Keroro", "r": "7.7", "year": "2004", "votes": "715", "genre": "Comedy, Family", "desp": "An alien frog plotting the conquest of Earth is forced to work out of a not-so-typical Japanese household."}, {"m_title": "Osamake: Romcom Where the Childhood Friend Wont Lose", "r": "5.8", "year": "2021", "votes": "478", "genre": "Comedy, Romance", "desp": "High school student, Kuroha Shida, is convinced that her childhood friend and next door neighbor, Sueharu Maru, is in love with her. Kurohas dreams are shattered, however, when Sueharu. watch the dude, no spoilers in summary!"}, {"m_title": "Kiteretsu Daihyakka", "r": "7.2", "year": "1988", "votes": "527", "genre": "Sci-Fi", "desp": "Kiteretsu Daihyakka is the story of Eichii Kiteretsu, descendant of a great inventor named D. Kiteretsu which the authors say is the cousin of Nobita, Doraemon anime just that much smarter."}, {"m_title": "Showa Genroku Rakugo Shinju", "r": "8.3", "year": "2016", "votes": "1,334", "genre": "Comedy, Drama", "desp": "The life and career of Eighth Generation Yakumo Yurakutei, a legendary 20th century rakugo artist whose journey is greatly influenced by his incredibly gifted but hot-headed best friend."}, {"m_title": "Captain Tsubasa", "r": "8.2", "year": "2018", "votes": "1,776", "genre": "Drama, Family", "desp": "A 11-year-old boy who loves soccer and dreams to be the best."}, {"m_title": "Killing Bites", "r": "6.3", "year": "2018", "votes": "576", "genre": "Action, Sci-Fi", "desp": "People have been created that are human-animal hybrids, and powerful businesses bet on the outcome of their duels. College student Nomoto Yuuyas casual friends ask him to drive them around. watch the anime dude!"}, {"m_title": "Restaurant to Another World", "r": "7.2", "year": "2017", "votes": "601", "genre": "Fantasy, Sci-Fi", "desp": "A chef runs a restaurant that serves as a Portal between our World & a Fantasy land every 7 Days, where he serves Western Style Food to Elves, Demons, Dragons & more."}, {"m_title": "New Fist of the North Star", "r": "6.8", "year": "2003", "votes": "699", "genre": "Action, Drama", "desp": "In the 21st century, a nuclear war turned most of the Earths surface into a desert wasteland, which also resulted in the contamination of the Earths water supply. A man named Sanga has. watch the dude, no spoilers in summary!"}, {"m_title": "Kaze no Stigma", "r": "7.2", "year": "2007", "votes": "1,786", "genre": "Action, Comedy", "desp": "Yagami Kazuma was part of a family of fire users, the Kannagi family. But with no talent in fire magic he was banished and cast out from the family. After four years he returns to Japan as a powerful wind user."}, {"m_title": "Shakugan No Shana", "r": "7.0", "year": "2005", "votes": "1,125", "genre": "Action, Adventure", "desp": "Yuji is returning from school when suddenly time stops and monsters appear to devour people. He is not frozen, however. When one of the monsters spots him, he is saved by Shana, the great equalizer, who tells him hes a fading spirit."}, {"m_title": "Accel World", "r": "7.0", "year": "2012", "votes": "2,237", "genre": "Action, Romance", "desp": "Haruyuki is an overweight kid and at the bottom of the food chain at his middle school. Within the sci-fi setting of the real world he often seeks refuge in a virtual one. One day however. watch the dude, no spoilers in summary!"}, {"m_title": "Marvel Anime", "r": "6.6", "year": "2010", "votes": "353", "genre": "Action, Sci-Fi", "desp": "Series from Marvel set more to the environment of Anime than typical American fare. Plenty of lengthy dialog, emotion and angst to go around, as well as grossly-mutating monsters to please the most demanding anime fans."}, {"m_title": "Fate/Grand Carnival", "r": "6.8", "year": "2020", "votes": "70", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Ahiru no Sora", "r": "7.3", "year": "2019", "votes": "690", "genre": "Sport", "desp": "Hes short, weak, and has just transferred to a new school. Kurumatani Sora is the main character in this story who loves basketball. He has been shooting hoops ever since he was little and. watch the anime dude!"}, {"m_title": "Nanbaka", "r": "6.6", "year": "2016", "votes": "391", "genre": "Action, Adventure", "desp": "The adventures of the residents of Nanba prison."}, {"m_title": "Tamako Market", "r": "6.9", "year": "2013", "votes": "1,000", "genre": "Comedy, Romance", "desp": "Tamakos life becomes complicated with friendships, rivalries and the arrival of a peculiar bird."}, {"m_title": "Malevolent Spirits: Mononogatari", "r": "6.9", "year": "2023", "votes": "122", "genre": "Action, Fantasy", "desp": "When spirits cross over into the human world, they can possess old objects and gain a physical form: a tsukumogami. Tsukumogami can be gentle, violent, or somewhere in-between, so the. watch the dude, no spoilers in summary!"}, {"m_title": "Tsundere Akuyaku Reijou Liselotte to Jikkyou no Endou-kun to Kaisetsu no Kobayashi-san", "r": "6.6", "year": "2023", "votes": "131", "genre": "Comedy, Fantasy", "desp": "On the outside, otome villainess Liselotte seems foul in every way, a purely evil character. However, as revealed in her diary, she was always hiding both her pain and her true feelings for. watch the anime dude!"}, {"m_title": "Marriage of God & Soul Godannar!!", "r": "6.8", "year": "2003", "votes": "64", "genre": "Action, Comedy", "desp": "Go and Anna have to join the fight against aliens, on their wedding day."}, {"m_title": "Golgo 13", "r": "7.4", "year": "2008", "votes": "493", "genre": "Action, Adventure", "desp": "Duke Togo, a.k.a. Golgo 13, is a highly-skilled assassin hired only by those who pay top-dollar for his services - whether its the FBI, CIA, industrial corporations or mafias. Regardless. watch the dude, no spoilers in summary!"}, {"m_title": "Princess Lover!", "r": "5.9", "year": "2009", "votes": "319", "genre": "Comedy, Romance", "desp": "Teppei lost his parents in a traffic accident. His grandpa Isshin adopted him by force to succeed the head of Arima Group Corporation. At Shuuhou, a high school only for wealthy students, Teppeis new life as a celebrity starts."}, {"m_title": "Nura: Rise of the Yokai Clan", "r": "7.4", "year": "2010", "votes": "789", "genre": "Action, Comedy", "desp": "Rikuo Nura appears to be an average middle school student. No one knows that he comes home every day to a house full of youkai, nor that he transforms into the all-powerful leader of these. watch the dude, no spoilers in summary!"}, {"m_title": "The Pet Girl of Sakurasou", "r": "7.7", "year": "2012", "votes": "4,747", "genre": "Comedy, Drama", "desp": "Sorata had a cat living with her and is kicked out and forced to move into a small dorm call Sakuraso. All the residents are strange and very gifted."}, {"m_title": "Perman", "r": "7.9", "year": "1983", "votes": "631", "genre": "Action, Adventure", "desp": "A clumsy boy, Mitsuo is chosen as apprentice to a powerful superhero."}, {"m_title": "Recovery of an MMO Junkie", "r": "7.3", "year": "2017", "votes": "1,244", "genre": "Comedy, Drama", "desp": "A stressed office worker quits her job and begins spending all her time playing an MMORPG, becoming part of a tight-knit friend group that helps her rediscover the joy of living in the real world."}, {"m_title": "Manyuu hikenchou", "r": "6.1", "year": "2011", "votes": "150", "genre": "Action, Comedy", "desp": "In Edo period of Japan, a clan known as the Manyuu, with ability of a sword strike that could shrink the size of a womans breasts. This might not seem like an ability that could rule a. watch the dude, no spoilers in summary!"}, {"m_title": "Les Miserables: Shoujo Cosette", "r": "8.0", "year": "2007", "votes": "330", "genre": "Drama, Family", "desp": "The classic novel by Victor Hugo returns to the world of Japanese anime in Nippon Animations 52-episode adaptation Les Misérables: Shoujo Cosette. The plot shifts its focus to include. watch the dude, no spoilers in summary!"}, {"m_title": "Witch Craft Works", "r": "6.7", "year": "2014", "votes": "532", "genre": "Action, Comedy", "desp": "In a world where humans and witches coexist, the apocalypse is announced. Ayaka, the princess of fire, must protect the young Takayami."}, {"m_title": "No Guns Life", "r": "6.7", "year": "2019", "votes": "681", "genre": "Action, Sci-Fi", "desp": "With no memory of his previous life-or who replaced his head with a giant gun-Juzo Inui now scratches out a living in the dark streets of the city as a Resolver."}, {"m_title": "Myriad Colors Phantom World", "r": "6.3", "year": "2016", "votes": "841", "genre": "Fantasy", "desp": "When phantoms cause trouble, its up to Haruhiko and his Phantom Hunting Club teammates to take care of them! If they can, that is."}, {"m_title": "Spy Classroom", "r": "5.7", "year": "2023", "votes": "154", "genre": "Action, Comedy", "desp": "After a gruesome war wreaked havoc on the general populace, the governments of the world turned to a more elusive tactic to further their agendas, war fought in the shadows. Lily, a. watch the dude, no spoilers in summary!"}, {"m_title": "Initial D: Fourth Stage", "r": "8.2", "year": "2004", "votes": "1,274", "genre": "Action, Drama", "desp": "Takumi Fujiwara continues his undefeated racing streak, touring the mountain passes of Japan with rival driver Keisuke Takahashi as a part of ex-racer Ryosuke Takahashis racing team: Project D."}, {"m_title": "Noir", "r": "7.2", "year": "2001", "votes": "2,002", "genre": "Action, Crime", "desp": "Two beautiful female assassins travel the world accepting assignments to kill various culprits, and at the same time, having deadly traps set against them by their own employer."}, {"m_title": "Black Butler: Book of Circus", "r": "8.2", "year": "2014", "votes": "2,030", "genre": "Comedy, Fantasy", "desp": "Ciel Phantomhive, the queens watchdog, is given two tickets to go to a traveling circus for himself and his demon butler, Sebastian. His job is to enter the circus and look for multiple missing children that are believed to be abducted."}, {"m_title": "Ao-chan Cant Study", "r": "6.4", "year": "2019", "votes": "787", "genre": "Short, Comedy", "desp": "When Ao was in kindergarten, she smiled ear-to-ear as she told her classmates how her father (a bestselling erotic author) chose her name: A as in apple and O as in orgy! That day still. watch the dude, no spoilers in summary!"}, {"m_title": "Strike Witches", "r": "6.5", "year": "2008", "votes": "442", "genre": "Action, Comedy", "desp": "The story takes place in year 1944 and the world lives in fear of unidentified flying objects called neuroi. A young heroine Named Yoshika miyafuji joins the 501st joint wing, known as the. watch the dude, no spoilers in summary!"}, {"m_title": "New Game!", "r": "7.3", "year": "2016", "votes": "905", "genre": "Comedy, Drama", "desp": "Daily office lives of Aoba and her colleagues at the game development studio Eagle Jump."}, {"m_title": "The Wallflower", "r": "7.4", "year": "2006", "votes": "1,098", "genre": "Comedy, Drama", "desp": "Kyouhei, Ranmaru, Takenaga, and Yukinojou are promised free rent on a large mansion under one condition, to change the landlords niece into a respectable young lady. However if they fail, shell triple their rent."}, {"m_title": "Seinto Seiya: Meio Hades Elysion-hen", "r": "8.1", "year": "2008", "votes": "1,114", "genre": "Action, Adventure", "desp": "The twelve Gold Saints sacrificed their lives to destroy the Wailing Wall. Our heroes, the Bronze Saints now face the deepest of Hades, Elysion, where they must face The Twin Gods before they can approach Hades himself."}, {"m_title": "Hikaru no Go", "r": "8.3", "year": "2001", "votes": "1,322", "genre": "Adventure, Drama", "desp": "A sixth-grader Hikaru Shindo accidentally awakens the spirit of a deceased Go prodigy Fujiwara no Sai who then accompanies Hikaru and helps him develop his Go skills."}, {"m_title": "Pandora in the Crimson Shell: Ghost Urn", "r": "5.9", "year": "2016", "votes": "70", "genre": "Action, Sci-Fi", "desp": "A super-powered terrorist has threatened the island of Cenancle. To fight him off, Nene Nanakorobi, a cybernetic girl-next-door, will have to team up with famous inventor Delilah Uzal and her adorably grumpy cyborg assistant, Clarion."}, {"m_title": "Ga-Rei-Zero", "r": "7.3", "year": "2008", "votes": "734", "genre": "Action, Drama", "desp": "Follows two friends, daughters of exorcists, who are now members of the governments secret Supernatural Disaster Countermeasures Division that fights demons when they appear in Japan."}, {"m_title": "Harukana Receive", "r": "6.4", "year": "2018", "votes": "310", "genre": "Drama, Sport", "desp": "The story takes place in Okinawa, where Haruka forms a beach volleyball pair with Kanata. Kanata, who couldnt handle the pressure put on her by her short height, had given up on beach. watch the dude, no spoilers in summary!"}, {"m_title": "Arknights: Reimei Zensou", "r": "6.2", "year": "2022", "votes": "197", "genre": "Action, Adventure", "desp": "In the land of Terra, natural disasters of unknown causes have been occurring irregularly in many areas. Therefore, the majority of the people, in order to escape those natural disasters,. watch the dude, no spoilers in summary!"}, {"m_title": "Tensura Nikki: Tensei shitara Slime Datta Ken", "r": "6.9", "year": "2021", "votes": "474", "genre": "Action, Adventure", "desp": "Dear diary - I got reincarnated as a slime. Even in another world, lives arent always on the line. Theres plenty of work to be done, from feeding the community and forging the items the. watch the anime dude!"}, {"m_title": "Kyou no Asuka Show", "r": "6.6", "year": "2012", "votes": "69", "genre": "Short", "desp": "The show features Asuka, a cute but clueless high school girl who does innocent things that are sexually suggestive to the people around her. A new awkward situation is portrayed in every episode."}, {"m_title": "The Sacred Blacksmith", "r": "6.5", "year": "2009", "votes": "484", "genre": "Adventure, Drama", "desp": "The knight Cecily wants to defend the weak, but her lack of skill makes her an unlikely heroine - until the blacksmith Luke comes to her aid, using magic to forge blades of supernatural. watch the dude, no spoilers in summary!"}, {"m_title": "Fuuto PI", "r": "7.0", "year": "2022", "votes": "183", "genre": "Action, Adventure", "desp": "Small happiness, great misfortune, the town where wind always blows Futo. Shotaro Hidari and Philip work together in Narumi Detective Agency, solving various cases and bringing happiness to the people of this Windy City."}, {"m_title": "Kiss Him, Not Me!", "r": "6.6", "year": "2016", "votes": "1,388", "genre": "Comedy, Drama", "desp": "When her favorite character dies, Kae suddenly loses weight and gains attention. But she wants these boys to date each other, not her!"}, {"m_title": "Flame of Recca", "r": "7.3", "year": "1997", "votes": "365", "genre": "Action, Adventure", "desp": "In modern Japan where the way of ninja is still practiced secretly, Recca discovers that he has a power to manipulate fire. Eventually he meets people who also has strange power, and he soon has to overcome his past and face his destiny."}, {"m_title": "Ani*Kuri15", "r": "6.6", "year": "2007", "votes": "640", "genre": "Short, Comedy", "desp": "15 one-minute shorts created by various people from Japans animation industry. The title of the collection, Ani*Kuri15, is abbreviated from the words anime and creators."}, {"m_title": "Sherlock Hound", "r": "7.5", "year": "1984", "votes": "1,518", "genre": "Action, Adventure", "desp": "The adventures of the canine detective Sherlock Hound and his loyal companions Dr. Watson, Mrs. Hudson and Inspector Lestrade."}, {"m_title": "Yu-Gi-Oh!: Go Rush!!", "r": "5.8", "year": "2022", "votes": "22", "genre": "Adventure, Fantasy", "desp": "The Odo (Ohdo) twins, Yuhi and Yuamu, are elementary school students in charge of a company called Uchujin Toraburu Sodansha (UTS) that solves Earthling problems with aliens. During one of. watch the dude, no spoilers in summary!"}, {"m_title": "Sleepy Princess in the Demon Castle", "r": "7.6", "year": "2020", "votes": "621", "genre": "Comedy, Fantasy", "desp": "A princess is kidnapped by a demon king and held prisoner in his castle. But the princess just wants to spend her time either sleeping or causing trouble for the demons."}, {"m_title": "Jashin-chan Dropkick", "r": "6.5", "year": "2018", "votes": "194", "genre": "Comedy, Fantasy", "desp": "Yurine is a delusional gothic lolita who summons a demon girl with the tail of a cobra, Jashin-chan. Theyll have to live together - until Yurine finds a way to send Jashin-chan back to hell, or until Jashin-chan kills Yurine."}, {"m_title": "Hirogaru Sky! Precure", "r": "8.8", "year": "2023", "votes": "17", "genre": "Action, Adventure", "desp": "A major incident has occurred in the peaceful Sky Land!? The young Princess Eru has been kidnapped by the monsters of Underground Empire. A brave young girl, Sora, follows the princess. watch the dude, no spoilers in summary!"}, {"m_title": "Too Cute Crisis", "r": "6.1", "year": "2023", "votes": "21", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "The Duke of Death and His Maid", "r": "7.2", "year": "2021", "votes": "620", "genre": "Comedy, Drama", "desp": "A cursed duke who kills everyone he touches lives in a mansion in the forest, with only a butler and a seductive maid for company. Will he free himself of his curse?"}, {"m_title": "Magia Record: Puella Magi Madoka Magica Side Story", "r": "6.1", "year": "2020", "votes": "409", "genre": "Action, Drama", "desp": "A magical girl, Iroha Tamaki, searches for her missing sister in the mysterious city of Kamihama, where it is rumored that magical girls can achieve salvation."}, {"m_title": "Mobile Suit Zeta Gundam", "r": "8.1", "year": "1985", "votes": "1,397", "genre": "Action, Drama", "desp": "Eight years after the One Year War, the Earth Federation creates an elite force called the Titans to hunt Zeon remnants. When Titans proves to be no better than Zeon, the Anti Earth Union Group (AEUG) is formed to restore peace in space."}, {"m_title": "Kämpfer", "r": "6.1", "year": "2009", "votes": "409", "genre": "Action, Comedy", "desp": "The plot revolves around Natsuru, a normal high school boy with blue hair. One day he wakes up and found himself turned into a girl. A stuffed tiger Harakiri Tora comes to life and tells. watch the dude, no spoilers in summary!"}, {"m_title": "Tada Never Falls in Love", "r": "7.2", "year": "2018", "votes": "937", "genre": "Comedy, Romance", "desp": "Mitsuyoshi Tada is a quiet but friendly soul, but he has never known love. Then one day, he meets transfer student Teresa Wagner and his life changes forever."}, {"m_title": "His and Her Circumstances", "r": "7.7", "year": "1998", "votes": "1,611", "genre": "Comedy, Drama", "desp": "TV series that tells the every-day stories of a group of teenagers in high-school, focusing on their romances and personal problems."}, {"m_title": "Fate/kaleid liner Prisma Illya", "r": "6.0", "year": "2013", "votes": "422", "genre": "Action, Comedy", "desp": "Normal elementary schoolgirl Illya has her life turned upside down when a peculiar stick turns her to a magical girl whose job is to defeat stray Heroic Spirits and collect their cards."}, {"m_title": "The Master of Ragnarok and Blesser of Einherjar", "r": "5.3", "year": "2018", "votes": "340", "genre": "Adventure, Fantasy", "desp": "Suou Yuuto was transported into another world called Yggdrasil. after one year he became the wolf clan leader. now he must wage war against neighboring clans and simultaneously search for a way Back home."}, {"m_title": "School Babysitters", "r": "7.7", "year": "2018", "votes": "939", "genre": "Comedy, Drama", "desp": "After losing both parents in a fatal plane crash, teenager Ryuuichi Kashima must adjust to his new life as the guardian of his younger brother Kotarou. Although Ryuuichi is able to maintain. watch the anime dude!"}, {"m_title": "My Roommate Is a Cat", "r": "7.7", "year": "2019", "votes": "793", "genre": "Comedy, Drama", "desp": "A shy novelist and a stray cat describe the happiness of living together from both points of view."}, {"m_title": "Boku no Kanojo ga Majimesugiru Sho-bitch na Ken", "r": "5.7", "year": "2017", "votes": "343", "genre": "Comedy", "desp": "Haruka Shinozaki has been interested in the class representative, Akiho Kousaka, since his first year in high school. She is attractive, good at sports, and is an all-around model student."}, {"m_title": "Assasination Classroom", "r": "8.5", "year": "2015", "votes": "21", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Mobile Suit Gundam Seed Destiny", "r": "7.1", "year": "2004", "votes": "1,468", "genre": "Action, Drama", "desp": "Two years after the war between the Earth Alliance and ZAFT, conflicts between the two nations heat up again. Shin Asuka, the new main characters eyes are full of sorrow as his family was. watch the dude, no spoilers in summary!"}, {"m_title": "The Dungeon of Black Company", "r": "6.6", "year": "2021", "votes": "686", "genre": "Comedy, Fantasy", "desp": "Kinji, who lacks any kind of work ethic, is a layabout in his modern life. One day, he finds himself transported to another world, but not in a grand fantasy of a hero welcomed with open. watch the dude, no spoilers in summary!"}, {"m_title": "Nozo × Kimi", "r": "6.4", "year": "2014", "votes": "69", "genre": "Comedy, Romance", "desp": "Kimio finds himself hiding in the girls locker room. Nozomi, one of the shy girls in his class finds him, but surprisingly covers for him but with only one option, to showing each other body through their window."}, {"m_title": "Magic Knight Rayearth", "r": "7.6", "year": "1994", "votes": "961", "genre": "Action, Adventure", "desp": "When the princess of a land ruled by the power of ones will is kidnapped by a traitorous priest, she summons three young girls from Japan to potentially fulfill the legends of the Magic. watch the dude, no spoilers in summary!"}, {"m_title": "UQ Holder!", "r": "6.6", "year": "2017", "votes": "439", "genre": "Action, Fantasy", "desp": "In the decade since the world became aware of the existence of magic, the world has undergone a massive upheaval. However, a boy named Touta lives in seclusion in a rural town far removed. watch the dude, no spoilers in summary!"}, {"m_title": "Himouto! Umaru-chan", "r": "7.1", "year": "2015", "votes": "1,666", "genre": "Comedy", "desp": "In public, Umaru is the perfect vision of a beautiful and smart high school girl. At home, she morphs into the ultimate slob with the ability to game all night and binge on junk food and cola. Can her big brother, Taihei, change her?"}, {"m_title": "Gosick", "r": "7.3", "year": "2011", "votes": "1,384", "genre": "Comedy, Drama", "desp": "Gosick is set in the fictional European country of Sauville in the 1920s. Kazuya Kujo is a transfer student to St. Marguerite Academy, a place where myths and superstitions are abundant.. watch the dude, no spoilers in summary!"}, {"m_title": "Koikimo", "r": "7.1", "year": "2021", "votes": "828", "genre": "Comedy, Romance", "desp": "A total womanizer and handsome man, and a high school nerdy girl. She saves his life once, he falls for her, becoming her stalker. They do not match, but what will happen between them?"}, {"m_title": "Shine Post", "r": "7.6", "year": "2022", "votes": "44", "genre": "Music", "desp": "The idol unit TiNgS is on the verge of disbandment. However, if they can fill all two thousand seats in their upcoming anniversary concert, their idol careers will live to see another day.. watch the dude, no spoilers in summary!"}, {"m_title": "Senpai ga Uzai Kouhai no Hanashi", "r": "7.4", "year": "2021", "votes": "1,030", "genre": "Comedy, Romance", "desp": "Igarashi Futubas new job would be great if her senpai, Takeda Harumi, wasnt so incredibly annoying. Futuba hates his laugh, she hates how big he is, and she really hates that he treats. watch the dude, no spoilers in summary!"}, {"m_title": "Black Cat", "r": "7.0", "year": "2005", "votes": "1,151", "genre": "Action, Crime", "desp": "Train Heartnet is a famous assassin known as Black Cat. He works for Chronos, an organization that secretly controls the world. However, he begins to question the morality of his actions. watch the dude, no spoilers in summary!"}, {"m_title": "Youre Under Arrest!", "r": "7.1", "year": "1996", "votes": "350", "genre": "Action, Comedy", "desp": "The adventures of two female police officers in modern Japan."}, {"m_title": "The Maid I Hired Recently Is Mysterious", "r": "6.4", "year": "2022", "votes": "280", "genre": "Comedy, Mystery", "desp": "The beautiful maid Lilith has recently been hired to care for the whims of her employers son, Yuuri. However, Yuuris overzealous skepticism of her every action keeps him on edge as he. watch the dude, no spoilers in summary!"}, {"m_title": "Nakaimo: My Little Sister Is Among Them", "r": "6.1", "year": "2012", "votes": "246", "genre": "Comedy, Drama", "desp": "Is insanity hereditary? Shougo Mikadonos beginning to think so, because the terms of his late fathers will seem crazy and following them may drive Shougo bonkers as well. Oh, it sounds. watch the dude, no spoilers in summary!"}, {"m_title": "A Certain Scientific Accelerator", "r": "6.9", "year": "2019", "votes": "503", "genre": "Action, Comedy", "desp": "Accelerator is considered the most powerful psychic in Academy City and one of a few able to shift to the level 6. He, however, now lives a peaceful life and tries to recover from the past. watch the dude, no spoilers in summary!"}, {"m_title": "Im Quitting Heroing", "r": "6.7", "year": "2022", "votes": "797", "genre": "Action, Adventure", "desp": "After defeating an invading demon army, a magic-wielding war hero returns to discover that he is feared by his own people. Retiring as a hero, he seeks new opportunities."}, {"m_title": "Hyakka Ryoran: Samurai Bride", "r": "6.5", "year": "2013", "votes": "169", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Digimon Ghost Game", "r": "6.5", "year": "2021", "votes": "114", "genre": "Action, Adventure", "desp": "It is the ninth anime series in the Digimon franchise. This series has a darker tone than prior Digimon anime series and is the first series to utilize the horror genre. Story follows Hiro. watch the dude, no spoilers in summary!"}, {"m_title": "Kuromukuro", "r": "7.2", "year": "2016", "votes": "1,360", "genre": "Action, Sci-Fi", "desp": "When mecha attack a research center, its students, pilots, and researchers must fight back with the help of mysterious artifacts and a young samurai."}, {"m_title": "Kyô kara maô!", "r": "7.4", "year": "2004", "votes": "376", "genre": "Adventure, Comedy", "desp": "After being flushed down a toilet, 15-year-old Yuuri Shibuya is transported to an alternate world called the New Demon Kingdom, and is told that he is to be the new king of the demons.. watch the dude, no spoilers in summary!"}, {"m_title": "Just Because!", "r": "6.9", "year": "2017", "votes": "790", "genre": "Drama, Romance", "desp": "Four highschool kids are ready for their university life,but with a new transfer student it all changes."}, {"m_title": "Casshern Sins", "r": "7.0", "year": "2008", "votes": "895", "genre": "Action, Adventure", "desp": "The world is falling apart and Casshern is to blame. He is said to have killed a robot named Luna and by doing so unleashed a plague referred to as ruin. But Casshern has no. watch the dude, no spoilers in summary!"}, {"m_title": "The Legend of Heroes: Trails of Cold Steel", "r": "6.2", "year": "2023", "votes": "64", "genre": "Action, Adventure", "desp": "Septian Calendar Year 1205. Lavi was born in North Ambria, the poorest region in the northwestern Zemuria. She enlists in the Northern Jaegers, the largest in the continent, to protect her. watch the dude, no spoilers in summary!"}, {"m_title": "Maou-sama, Retry!", "r": "6.7", "year": "2019", "votes": "955", "genre": "Action, Fantasy", "desp": "On the night of his favorite games shutdown, Akira Oono awakens in the body of his online character: Demon Lord Hakuto Kunai."}, {"m_title": "Mahou Tsukai Reimeiki", "r": "5.8", "year": "2022", "votes": "301", "genre": "Action, Adventure", "desp": "A magician with zero magic is sent to investigate attacks on magicians, and uncovers a great mystery."}, {"m_title": "Cross Ange: Rondo of Angel and Dragon", "r": "7.1", "year": "2014", "votes": "544", "genre": "Action, Romance", "desp": "The first season follows the growth of a princess who first falls from grace but grows up to lead a rebellion once she learns the truth about the ignorance of racism in her land. Not. watch the dude, no spoilers in summary!"}, {"m_title": "Ore no imouto ga konna ni kawaii wake ga nai", "r": "6.4", "year": "2010", "votes": "1,594", "genre": "Comedy, Drama", "desp": "After years of resentment, a teenage girl begins warming up to her older brother when he discovers her secret hobbies."}, {"m_title": "Yu-No: A Girl Who Chants Love at the Bound of this World", "r": "6.3", "year": "2019", "votes": "248", "genre": "Drama, Mystery", "desp": "Arima Takuya is a young student whose father, a historian who has conducted various researches, disappeared recently. During a summer vacation Takuya receives a peculiar package from his. watch the dude, no spoilers in summary!"}, {"m_title": "Non Non Biyori", "r": "7.8", "year": "2013", "votes": "1,111", "genre": "Adventure, Comedy", "desp": "The lives of a group of children who live in the countryside village of Asahigaoka."}, {"m_title": "Rôgo ni sonaete Isekai de 8-man-mai no kinka wo tamemasu", "r": "6.5", "year": "2023", "votes": "216", "genre": "Fantasy", "desp": "One day, Mitsuha falls off a cliff and is transported to a medieval Europe-type world. After a near-death encounter with a pack of wolves, she then realizes that shes able to transport. watch the dude, no spoilers in summary!"}, {"m_title": "Cells at Work! Code Black", "r": "7.5", "year": "2021", "votes": "582", "genre": "Action, Comedy", "desp": "Humans are complex biological robots that are made up of trillion of cells, each with their own structure and function. Even a single cell cannot slack off. Scientists have come a long way. watch the dude, no spoilers in summary!"}, {"m_title": "Flying Witch", "r": "7.2", "year": "2016", "votes": "710", "genre": "Comedy, Fantasy", "desp": "Young witch Makoto moves to the beautiful countryside to live with her distant relatives and study witchcraft."}, {"m_title": "Shenmue", "r": "7.5", "year": "2022", "votes": "500", "genre": "Action, Adventure", "desp": "A 18 year old boy from Yokosuka, Japan named Ryo Hazuki, finds himself on the path of revenge after watching his fathers death right in front of him by a man named Lan Di, a ruthless. watch the dude, no spoilers in summary!"}, {"m_title": "Mischievous Kiss", "r": "7.3", "year": "2008", "votes": "1,681", "genre": "Comedy, Drama", "desp": "A cute girl (Aihara Kotoko) is in love with a smart and handsome guy whos a jerk in high school (Irie Naoki) and when her house is destroyed she moves with her father in to the Irie Family home where he starts to develop some feelings towards her."}, {"m_title": "Tales of Zestiria the X", "r": "6.8", "year": "2016", "votes": "911", "genre": "Action, Adventure", "desp": "The tale of the Shepherd, a savior who will bring peace to the seraphim and human worlds, becomes Soreys reality after a fateful encounter."}, {"m_title": "The Idaten Deities Know Only Peace", "r": "7.2", "year": "2021", "votes": "525", "genre": "Action, Adventure", "desp": "Gods defeated demons centuries ago. The demons have returned, but can the new generation of gods stop them?"}, {"m_title": "Problem Children Are Coming from Another World, Arent They?", "r": "7.0", "year": "2013", "votes": "980", "genre": "Action, Comedy", "desp": "Three teenagers receive mysterious letters that transport them to another world. This world is known as, Little Garden. Here, the inhabitants use their beyond human gifts to compete in. watch the dude, no spoilers in summary!"}, {"m_title": "SSSS.Gridman", "r": "7.0", "year": "2018", "votes": "767", "genre": "Action, Adventure", "desp": "A being calling itself Hyper Agent Gridman contacts high schooler Yuta through an old computer, recruiting him in the fight against impending terror."}, {"m_title": "My Daddy Long Legs", "r": "8.2", "year": "1990", "votes": "1,631", "genre": "Comedy, Drama", "desp": "Its an anime series based on the original novel of Daddy Long Legs written by Jean Webster. It takes you through the high school life of Judy Abbot and her mysterious benefactor called John Smith."}, {"m_title": "Rail Wars!", "r": "5.7", "year": "2014", "votes": "225", "genre": "Action, Romance", "desp": "The series takes place in an alternate version of Japan where the nationalized railway system was never privatized. Naoto Takayama is an ordinary high school student who aspires to a. watch the dude, no spoilers in summary!"}, {"m_title": "Children of the Whales", "r": "7.0", "year": "2017", "votes": "1,691", "genre": "Drama, Fantasy", "desp": "As a magic weilder, young archivist Chakuro knows his life will be short, but everything changes when a mysterious girl from the outside arrives on his island."}, {"m_title": "Pupa", "r": "2.6", "year": "2014", "votes": "1,160", "genre": "Short, Horror", "desp": "A boy, Utsutsu, and his little sister, Yume, are siblings who find themselves all alone. After Yumes body undergoes a strange metamorphosis-into a creature that eats humans, Utsutsu. watch the dude, no spoilers in summary!"}, {"m_title": "Handa-kun", "r": "7.2", "year": "2016", "votes": "883", "genre": "Comedy", "desp": "The school years of renowned calligrapher Sei Handa."}, {"m_title": "The Executioner and Her Way of Life", "r": "6.2", "year": "2022", "votes": "423", "genre": "Action, Adventure", "desp": "The Lost Ones are wanderers who come here from a distant world known as Japan. No one knows how or why they leave their homes. The only thing that is certain is that they bring disaster. watch the dude, no spoilers in summary!"}, {"m_title": "Welcome to the Ballroom", "r": "7.7", "year": "2017", "votes": "781", "genre": "Music, Sport", "desp": "Tatara is an average middle school student with no particular dreams until an unexpected incident draws him into the fascinating world of ballroom dancing. He dives into the world of dance, believing its his opportunity to change."}, {"m_title": "Raven of the Inner Palace", "r": "7.4", "year": "2022", "votes": "261", "genre": "Fantasy, History", "desp": "Deep within the inner palace lives a special consort who does not serve the emperor despite her position, known as the Raven Consort. People who have seen her say she looks like an old. watch the dude, no spoilers in summary!"}, {"m_title": "Ikebukuro West Gate Park", "r": "6.4", "year": "2020", "votes": "220", "genre": "Drama, Mystery", "desp": "Follows Makoto Majima and the interactions he has with the gangs that are located in his area of Japan."}, {"m_title": "To the Abandoned Sacred Beasts", "r": "6.2", "year": "2019", "votes": "409", "genre": "Action, Drama", "desp": "During a protracted civil war that pitted the North against the South, the outnumbered Northerners used dark magical arts to create monstrous super-soldiers-Incarnates. Now that the war has. watch the anime dude!"}, {"m_title": "Weird Anime Excel Saga", "r": "7.5", "year": "1999", "votes": "1,396", "genre": "Comedy, Drama", "desp": "Ilpalazzo wants one thing: to rule world so he can better it. Excel just wants to help. Too bad she sucks at her job."}, {"m_title": "Maison Ikkoku", "r": "8.2", "year": "1986", "votes": "1,160", "genre": "Comedy, Drama", "desp": "The misadventures of a young student and his landladys romance."}, {"m_title": "Anti-Magic Academy: The 35th Test Platoon", "r": "6.3", "year": "2015", "votes": "431", "genre": "Action, Comedy", "desp": "The 35th Test Platoon of Antimagic Academy is one of the squads enrolled at an academy specializing in raising the witch hunters, whose job is to find and neutralize any magical threats!. watch the dude, no spoilers in summary!"}, {"m_title": "Please Teacher!", "r": "6.9", "year": "2002", "votes": "1,219", "genre": "Comedy, Romance", "desp": "An alien who becomes a teacher, and a student start a relationship and have to keep it all secret."}, {"m_title": "XXXHOLiC", "r": "7.6", "year": "2006", "votes": "1,965", "genre": "Comedy, Drama", "desp": "Kimihiro Watanuki, a high-schooler, is troubled by monsters and spirits, but meets Yuko Ichihara, a beautiful witch, who grants his wish to not be able to see them. However, there is a price."}, {"m_title": "The Orbital Children", "r": "6.4", "year": "2022", "votes": "734", "genre": "Adventure, Drama", "desp": "In 2045, two children born on the moon and three kids from Earth try to survive after an accident on their space station leaves them stranded."}, {"m_title": "Hakuouki", "r": "7.3", "year": "2010", "votes": "717", "genre": "Action, Drama", "desp": "When her father has disappeared, Chizuru Yukimura goes searching for him in Kyoto. She is attacked by strange men, and then rescued by the Shinsengumi."}, {"m_title": "Transformers: The Headmasters", "r": "6.9", "year": "1987", "votes": "769", "genre": "Action, Adventure", "desp": "With the aid of the Headmasters the mighty Cybertrons (Autobots) continue to wage war against their evil counterparts the Destrons (Decepticons). But with Convoy (Optimus Prime) now gone can the Cybertrons win?"}, {"m_title": "Outbreak Company", "r": "6.7", "year": "2013", "votes": "637", "genre": "Comedy, Drama", "desp": "Shinichi Kanou is a young secluded Otaku who is offered a job thanks to his vast knowledge of anime, video games and manga and just after meeting his new employer, he is kidnapped,. watch the dude, no spoilers in summary!"}, {"m_title": "Infinite Dendrogram", "r": "6.1", "year": "2020", "votes": "455", "genre": "Adventure, Fantasy", "desp": "In the year 2043, Infinite Dendrogram, the worlds first successful full-dive VRMMO was released. In addition to its ability to perfectly simulate the five senses, along with its many other. watch the anime dude!"}, {"m_title": "Mermaid Melody Pichi Pichi Pitch", "r": "7.2", "year": "2003", "votes": "496", "genre": "Comedy, Drama", "desp": "Lucia Nanami, the mermaid princess of the North Pacific Ocean, sets out to land to find the boy she saved from a tsunami wave seven years ago. She cannot tell him that she is a mermaid, and she must also fight off water demons."}, {"m_title": "The Qwaser of Stigmata II", "r": "5.7", "year": "2011", "votes": "155", "genre": "Action, Comedy", "desp": "Hana is Sashas newer Maria. Together they concoct a plan to infiltrate the all-girl school, which necessitates Sasha to cross dress. They seek an object related to the Qwaser, which is. watch the dude, no spoilers in summary!"}, {"m_title": "Revisions", "r": "5.9", "year": "2019", "votes": "667", "genre": "Action, Sci-Fi", "desp": "When Shibuya City is time-warped to 2388, a group of high school students is conscripted by an AHRV agent to fight off the hostile cyborg race called revisions, with the help of giant robot monsters."}, {"m_title": "My Sister, My Writer", "r": "4.2", "year": "2018", "votes": "162", "genre": "Comedy, Romance", "desp": "The story centers around the younger sister Nagami Suzuka and the older brother Nagami. Suzuka is a beautiful third-year middle school student who has excellent grades and is the student. watch the dude, no spoilers in summary!"}, {"m_title": "If Its for My Daughter, Id Even Defeat a Demon Lord", "r": "7.0", "year": "2019", "votes": "698", "genre": "Adventure, Comedy", "desp": "Dale is a cool, composed, and highly skilled adventurer whos made quite a name for himself despite his youth. One day on a job deep in the forest, he comes across a little devil girl whos. watch the anime dude!"}, {"m_title": "Kantai Collection: Kan Colle", "r": "6.0", "year": "2015", "votes": "292", "genre": "Action, Comedy", "desp": "Set in a world where humanity has lost control of the oceans to the deep sea fleet, the only hope to counter this threat are the Kanmusu, a group of girls who possess the spirit of. watch the dude, no spoilers in summary!"}, {"m_title": "Lupin the Third: The Woman Called Fujiko Mine", "r": "7.8", "year": "2012", "votes": "837", "genre": "Action, Adventure", "desp": "The story of how the fashionable femme fatale Fujiko Mine first met Lupin III, her sometimes-lover and sometimes-rival, and the rest of his gang."}, {"m_title": "Taisho Otome Fairy Tale", "r": "7.6", "year": "2021", "votes": "456", "genre": "Comedy, Drama", "desp": "Tamahiko finds himself exiled to the countryside by his wealthy family because of his disability. His lonely life is turned upside down by the arrival of Yuzuki, his arranged bride."}, {"m_title": "One Outs", "r": "8.3", "year": "2008", "votes": "1,730", "genre": "Drama, Sport", "desp": "Hiromichi Kojima, the star batter of the Lycaons, heads to the southern Japanese island of Okinawa to train and find a new pitcher for the team. There, he meets Toa Tokuchi, a 134-kmph. watch the dude, no spoilers in summary!"}, {"m_title": "D.Gray-man Hallow", "r": "7.4", "year": "2016", "votes": "688", "genre": "Action, Drama", "desp": "Allen Walker, an exorcist fights Akuma to save the world. Sequel of the 2006 D.Gray-Man series."}, {"m_title": "The Day I Became a God", "r": "6.6", "year": "2020", "votes": "697", "genre": "Drama, Fantasy", "desp": "Yota Narukami decides to spend his possible last days with Hina, a young girl who declares that the world will end in only 30 days!"}, {"m_title": "A.I.C.O. Incarnation", "r": "6.3", "year": "2018", "votes": "1,304", "genre": "Action, Sci-Fi", "desp": "Everything Aiko knew was a lie. Now shes joining a team of Divers to reach the place where the Burst began to stop it for good and save her family."}, {"m_title": "Sengoku Basara", "r": "6.7", "year": "2009", "votes": "523", "genre": "Action, Drama", "desp": "In the age of the Warring Countries, Japan was fragmented in semi-independent provinces, governed by feudal lords or daimyo, who confronted each other in an endless struggle for power.. watch the dude, no spoilers in summary!"}, {"m_title": "Valkyria Chronicles", "r": "7.3", "year": "2009", "votes": "353", "genre": "Action, Drama", "desp": "When Imperial forces attack the small border town of Bruhl, Welkin Gunther, son of late General Belgen Gunther, is forced to fight for his life alongside Town Watch captain Alicia Melchiott. watch the anime dude!"}, {"m_title": "Seinto Seiya: Omega", "r": "5.8", "year": "2012", "votes": "638", "genre": "Animation", "desp": "After many years since the last battle, Mars, the god of war, resurrect and wants to dominate earth again. The new generation of saint soldiers join together to fight against newly emerged dark forces."}, {"m_title": "Chobits", "r": "7.3", "year": "2002", "votes": "4,561", "genre": "Comedy, Drama", "desp": "Hideki finds the discarded and malfunctioning Persocom Chi, a personal computer that looks like a girl. While trying to fix and care for Chi, Hideki discovers that she might be a Chobits, a robot of urban legend that has free will."}, {"m_title": "Run with the Wind", "r": "8.4", "year": "2018", "votes": "1,473", "genre": "Comedy, Sport", "desp": "Kakeru, a former elite runner at high school, is chased for stealing food. He is saved by a Kansei University student Haiji, who is also a runner. Haiji persuades Kakeru to live in the old. watch the dude, no spoilers in summary!"}, {"m_title": "Chain Chronicle - The Light of Haecceitas -", "r": "6.6", "year": "2016", "votes": "169", "genre": "Adventure, Fantasy", "desp": "In the land of Yuguto a grope of warrior accompany by their armies strives to defeat the dark king before his black magic and dark soldiers consume everything."}, {"m_title": "Bobobo-bo Bo-bobo", "r": "7.1", "year": "2003", "votes": "933", "genre": "Action, Adventure", "desp": "The impossible adventures of Bobobo-bo Bo-bobo, a warrior who fights using his nose hairs to defeat an evil empire that wants to turn the human race bald."}, {"m_title": "Somali and the Forest Spirit", "r": "7.5", "year": "2020", "votes": "939", "genre": "Adventure, Drama", "desp": "When a golem discovers a young human girl in the forest that he protects, the two set out on a journey in search of other humans, whom are considered to be nearly extinct."}, {"m_title": "Pui Pui Molcar", "r": "8.1", "year": "2021", "votes": "501", "genre": "Short, Comedy", "desp": "Pui Pui Molcar centers in the wonderful soft world where cars are guinea pigs and humans are occasionally plastic dolls. This stop-motion animated short series brings all you ever wanted in a TV show to life."}, {"m_title": "Blue Exorcist: Kyoto Saga", "r": "7.3", "year": "2017", "votes": "2,584", "genre": "Action, Drama", "desp": "Assiah, the realm of humans, and Gehenna, he realm of demons. Normally, these two dimensions would never intersect, but having possessed all material substances, the demons are now intruding on the material world."}, {"m_title": "Gilgamesh", "r": "6.7", "year": "2003", "votes": "415", "genre": "Drama, Sci-Fi", "desp": "Set in a future where the sky has been changed into a giant mirror, two teenage children are caught in a struggle between a group of super-powered beings and a mysterious womans team of children aiming to stop them."}, {"m_title": "Romeo and the Black Brothers", "r": "8.6", "year": "1995", "votes": "1,221", "genre": "Adventure, Drama", "desp": "The adventures of a young Swiss boy working as a chimney sweeper in Milan."}, {"m_title": "Battle Game in 5 Seconds", "r": "6.8", "year": "2021", "votes": "660", "genre": "Action, Fantasy", "desp": "It was just a usual morning. Akira Shiroyanagi, a high schooler who loves games and Konpeito (Japanese sweets), has suddenly been dragged into a battlefield by a mysterious girl who calls. watch the dude, no spoilers in summary!"}, {"m_title": "The File of Young Kindaichi", "r": "8.0", "year": "1997", "votes": "199", "genre": "Action, Crime", "desp": "Kindaichi Hajime, a lazy, perverted student with an IQ of 180, and the grandson of famous detective Kindaichi Kosuke, is often put into situations where he must discover the killer and their trick."}, {"m_title": "Nokemono-tachi no Yoru", "r": "6.2", "year": "2023", "votes": "125", "genre": "Action, Fantasy", "desp": "Wisteria is an orphan girl living in a corner of the British Empire at the end of the 19th century. Her life is desolate and bleak, until she encounters Malbus, a powerful but equally. watch the dude, no spoilers in summary!"}, {"m_title": "Conception: Please Give Birth to My Child!", "r": "3.8", "year": "2018", "votes": "172", "genre": "Action, Comedy", "desp": "A high school student is transported to a fantasy world under danger, where he must sire superhuman children with mystical females that will save the world."}, {"m_title": "Astarottes Toy", "r": "6.2", "year": "2011", "votes": "97", "genre": "Comedy, Fantasy", "desp": "A man and his daughter are pulled into a magical world with a 10-year-old succubus princess that hates men, her S&MA-loving elf lady-in-waiting who is concerned with her hatred for men, a female guard and many more colorful characters."}, {"m_title": "Fuuka", "r": "6.3", "year": "2017", "votes": "707", "genre": "Comedy, Drama", "desp": "When introvert Yuu Haruna transfers to a school in Tokyo, his dull life takes a sharp turn when Fuuka Akitsuki brings music into his world."}, {"m_title": "Charger Girl Ju-den Chan", "r": "6.5", "year": "2009", "votes": "141", "genre": "Action, Comedy", "desp": "Plug is a Charger Girl from a parallel dimension. Her job is to find people on Earth that are depressed and charge them up to restore their energy."}, {"m_title": "Sorcerous Stabber Orphen: Battle of Kimluck", "r": "5.6", "year": "2020", "votes": "222", "genre": "Action, Adventure", "desp": "A sorcerer who was once the top student of the famous Tower of Fang, now spends his time chasing around his hopeless clients as a moneylender, at least until his client comes up with a plan to make money: marriage fraud."}, {"m_title": "Guyver: The Bioboosted Armor", "r": "7.8", "year": "2005", "votes": "807", "genre": "Action, Adventure", "desp": "Sho Fukamachi, a normal teenager accidentally found an alien object called Unit and thus, changed his life forever. The Unit bonded with Sho, resulting a powerful fighting life form called Guyver."}, {"m_title": "Days", "r": "7.3", "year": "2016", "votes": "429", "genre": "Drama, Sport", "desp": "Tsukushi Tsukamoto is a freshman at Seiseki High School. That school is known for its football team. After meeting Jin Kazama and playing a football match with him, Tsukamoto decides to. watch the dude, no spoilers in summary!"}, {"m_title": "Irozuku Sekai No Ashita Kara", "r": "7.3", "year": "2018", "votes": "722", "genre": "Drama, Fantasy", "desp": "Set in the city of Nagasaki, the story takes place in a world where magic exists in everyday life. Hitomi Tsukishiro is a 17-year-old mage who grew up with stale emotions, as she lost her. watch the dude, no spoilers in summary!"}, {"m_title": "Dusk Maiden of Amnesia", "r": "7.3", "year": "2012", "votes": "1,102", "genre": "Comedy, Drama", "desp": "An old school building rumoured to be haunted is the place where a young woman was left to die. A freshman at the school, Niija Teiichi, stumbles upon the ghost of this woman and seems to. watch the dude, no spoilers in summary!"}, {"m_title": "DokiDoki! PreCure", "r": "6.2", "year": "2013", "votes": "211", "genre": "Action, Adventure", "desp": "5 teenage girls unleash their inner superheroes to stop King Mercenare from taking over the world."}, {"m_title": "Shuffle!", "r": "6.8", "year": "2005", "votes": "829", "genre": "Comedy, Drama", "desp": "Rin is a seemingly normal young man in school. Orphaned as a young boy he has since lived with his childhood friend Kaede who lost her mother in the same accident that he lost both of his. watch the dude, no spoilers in summary!"}, {"m_title": "Rave Master", "r": "6.8", "year": "2001", "votes": "331", "genre": "Action, Adventure", "desp": "Haru Glory is a boy who lives with his sister, Cattleya, in Garage Island. After fishing out a thing called Plue, he meets Shiba who entrusted him with Rave Stone or more known as Holy. watch the dude, no spoilers in summary!"}, {"m_title": "Working!!", "r": "7.3", "year": "2010", "votes": "661", "genre": "Comedy, Drama", "desp": "Due to his love for small, cute things, Souta Takanashi cannot turn childlike Popura Taneshima down when she recruits him to work for Wagnaria, a family restaurant located in Hokkaido.. watch the dude, no spoilers in summary!"}, {"m_title": "A Little Princess Sara", "r": "7.8", "year": "1985", "votes": "1,469", "genre": "Drama, History", "desp": "Anime adaption of 1905s childrens novel A Little Princess. Sara Crewe arrives in London with her wealthy father to enter Miss Minchins boarding school for young ladies. Despite an. watch the dude, no spoilers in summary!"}, {"m_title": "Honey and Clover", "r": "7.8", "year": "2005", "votes": "1,320", "genre": "Comedy, Drama", "desp": "Yuuta Takemoto is a poor college student who lives in a dilapidated apartment building with his two friends, Shinobu Morita and Takumi Mayama. The series chronicles the students story as they struggle through life and love."}, {"m_title": "Crayon Shin-chan Gaiden: Alien vs. Shinnosuke", "r": "7.4", "year": "2016", "votes": "87", "genre": "Comedy", "desp": "On the 25th anniversary of the start of the animated series Shinchan was presented in a new style for four seasons. Available now in Japanese and English dubbing on Amazons Prime Video."}, {"m_title": "Didnt I Say to Make My Abilities Average in the Next Life?!", "r": "6.5", "year": "2019", "votes": "709", "genre": "Comedy, Drama", "desp": "When she turns ten years old, Adele von Ascham is hit with a horrible headache-and memories of her previous life as an eighteen-year-old Japanese girl named Kurihara Misato. That life. watch the dude, no spoilers in summary!"}, {"m_title": "Isekai Quartet", "r": "7.0", "year": "2019", "votes": "843", "genre": "Comedy, Fantasy", "desp": "The worlds of isekai sagas collide and cross over, resulting in comical chaos."}, {"m_title": "The Demon Girl Next Door", "r": "7.0", "year": "2019", "votes": "271", "genre": "Comedy, Drama", "desp": "In order to end a curse, a weakling Demon girl must defeat an overpoweringly strong Magic girl who always treats her kindly."}, {"m_title": "Eyeshield 21", "r": "7.6", "year": "2005", "votes": "683", "genre": "Comedy, Drama", "desp": "Kobayakawa Sena is a little guy who always get picked on by the bullies in school. Since elementary school, he always play the role of the errand boy to the extent that he has become. watch the dude, no spoilers in summary!"}, {"m_title": "The Jungle Book: The Adventures of Mowgli", "r": "7.5", "year": "1989", "votes": "1,462", "genre": "Adventure, Drama", "desp": "The adventures of Mowgli, an orphan boy raised by wolves, and his friends in the jungles of India."}, {"m_title": "Silver Spoon", "r": "7.9", "year": "2013", "votes": "1,314", "genre": "Comedy, Drama", "desp": "In order to escape his stressful city life, Hachiken enrolls in an agricultural school where he must learn a whole new way of life alongside a colorful cast of characters."}, {"m_title": "Turn-A Gundam", "r": "7.5", "year": "1999", "votes": "371", "genre": "Action, Adventure", "desp": "In the distant future, technology on Earth has regressed to World War I levels. Mankinds enormous twenty-kilometer-long space colonies and advanced weapons such as mobile suits have long. watch the dude, no spoilers in summary!"}, {"m_title": "In the Land of Leadale", "r": "6.6", "year": "2022", "votes": "572", "genre": "Adventure, Comedy", "desp": "New beginnings in familiar lands. After a horrific accident put her on life support, the last vestige of freedom Keina Kagami had was in the VRMMORPG World of Leadale. When she wakes up in. watch the dude, no spoilers in summary!"}, {"m_title": "Maoyu ~ Archenemy & Hero", "r": "6.9", "year": "2013", "votes": "727", "genre": "Fantasy, Romance", "desp": "After a long and treacherous journey, our Hero finally arrives at the Dark Ladys castle only to find himself being asked for help. The Hero explains how the war that the demons have. watch the dude, no spoilers in summary!"}, {"m_title": "DRAMAtical Murder", "r": "5.7", "year": "2014", "votes": "411", "genre": "Action, Sci-Fi", "desp": "Nothing on Midorijima has been the same since the powerful Tour Konzern forced the original population into the old residential district and converted the rest of the island into the. watch the dude, no spoilers in summary!"}, {"m_title": "Yu-Gi-Oh! Sevens", "r": "3.9", "year": "2020", "votes": "125", "genre": "Adventure, Action", "desp": "Yuga Ohdo, a fifth grade student, creates a new set of dueling rules."}, {"m_title": "Black Rock Shooter", "r": "6.3", "year": "2012", "votes": "690", "genre": "Action, Drama", "desp": "On the first day of junior high school, Mato Kuroi happens to run into Yomi Takanashi, a shy, withdrawn girl whom she immediately takes an interest in. Mato tries her best to make. watch the dude, no spoilers in summary!"}, {"m_title": "She Professed Herself Pupil of the Wise Man", "r": "5.7", "year": "2021", "votes": "274", "genre": "Adventure, Fantasy", "desp": "Sakimori Kagami plays a VRMMORPG called Arch Earth Online. He is a veteran player with the name and appearance similar to an elderly bearded wizard from a certain book about a ring. One day. watch the anime dude!"}, {"m_title": "The Prince of Tennis", "r": "7.4", "year": "2001", "votes": "862", "genre": "Action, Comedy", "desp": "Prince of Tennis follows the story of a 12 year old boy Ryoma Echizen who won four consecutive titles in the American Junior Tennis tournaments. Hes an amazing tennis player and is. watch the dude, no spoilers in summary!"}, {"m_title": "Azur Lane", "r": "5.7", "year": "2019", "votes": "315", "genre": "Action, Drama", "desp": "The seas were taken over by a mysterious enemy known only as sirens. Humans then created humanoid female embodiments of real life warships in order to combat the sirens and take back. watch the dude, no spoilers in summary!"}, {"m_title": "Fairy gone", "r": "5.7", "year": "2019", "votes": "388", "genre": "Action, Fantasy", "desp": "Able to summon fairies as alter-ego weapons, soldiers go rogue in search of purpose beyond the battlefield. Who will maintain the peace?"}, {"m_title": "Cobra the Animation", "r": "7.9", "year": "2010", "votes": "241", "genre": "Action, Adventure", "desp": "Cobra, a famous charismatic space pirate with psychogun for a hand, decides to renounce his ways, and become an ordinary person with a new identity. His memories do come back and he becomes a space adventurer fighting villains."}, {"m_title": "Recently, My Sister is Unusual", "r": "5.6", "year": "2014", "votes": "188", "genre": "Comedy, Fantasy", "desp": "Mitsuki and Yuya become stepsister and stepbrother because of their parents marriage. Mitsuki cant get along with her new family and Yuya doesnt know how to approach his new stepsister.. watch the dude, no spoilers in summary!"}, {"m_title": "Akibas Trip the Animation", "r": "6.0", "year": "2017", "votes": "215", "genre": "Animation", "desp": "Otaku siblings Tamotsu and Niwaka Denkigai are shopping in Akihabara when it is overrun by vampiric cosplaying monsters! These creatures, known as Bugged Ones, can possess anyone they. watch the dude, no spoilers in summary!"}, {"m_title": "When Supernatural Battles Became Commonplace", "r": "6.5", "year": "2014", "votes": "515", "genre": "Action, Comedy", "desp": "The story focuses on Senko High Schools Literature Club, whose five members; Jurai, Tomoyo, Hatoko, Sayumi, and Chifuyu, have all somehow developed superpowers. The superpowers have now. watch the dude, no spoilers in summary!"}, {"m_title": "The Honor at Magic High School", "r": "5.8", "year": "2021", "votes": "183", "genre": "Action, Drama", "desp": "A century has passed since magic - true magic, the stuff of legends - has returned to the world. It is spring, the season of new beginnings, and a new class of students is about to begin. watch the dude, no spoilers in summary!"}, {"m_title": "The Squid Girl: The Invader Comes from the Bottom of the Sea!", "r": "7.5", "year": "2010", "votes": "480", "genre": "Comedy, Drama", "desp": "Furious about ocean pollution, Squid Girl launches a one-squid land invasion to subjugate all humanity."}, {"m_title": "Sweetness and Lightning", "r": "7.2", "year": "2016", "votes": "737", "genre": "Comedy, Drama", "desp": "Six months after his wifes death, a highschool teacher struggles to raise his daughter due to his inability to cook. When he meets an introverted student, whos also a part-time cook, he. watch the dude, no spoilers in summary!"}, {"m_title": "Rewrite", "r": "6.0", "year": "2016", "votes": "178", "genre": "Action, Drama", "desp": "Kazamatsuri, a modern, well-developed city renowned for its burgeoning greenery and rich Japanese culture, is home to Kotarou Tennouji, a high schooler least privy to the places shared. watch the dude, no spoilers in summary!"}, {"m_title": "Witch Hunter Robin", "r": "7.2", "year": "2002", "votes": "2,115", "genre": "Action, Adventure", "desp": "Witches exist, and it is the mysterious organization STNJs job to hunt them down. Robin is a recent trainee who joins up, and begins unravelling the secrets of both the witches and STNJ..."}, {"m_title": "My Bride Is a Mermaid", "r": "7.2", "year": "2007", "votes": "820", "genre": "Comedy, Fantasy", "desp": "Nagasumis in hot water after a beautiful, young mermaid named Sun saves him from drowning. The deep-sea sweethearts dad is a merman yakuza prone to executing anyone who learns his. watch the dude, no spoilers in summary!"}, {"m_title": "Pandora Hearts", "r": "7.1", "year": "2009", "votes": "798", "genre": "Action, Fantasy", "desp": "Oz Vessalius, heir to one of the duke houses, has just turned fifteen. His life is rich and carefree, darkened only by the constant absence of his father. At his coming-of-age ceremony,. watch the dude, no spoilers in summary!"}, {"m_title": "Maou Evelogia ni Mi o Sasageyo", "r": "7.3", "year": "2021", "votes": "48", "genre": "Animation", "desp": "An aimless young man is killed and reborn in a video game world that he used to play as a child, as an NPC. Changing fate, he teams up with the games Demon King, his crush named Eve, to take over the world."}, {"m_title": "Carnival Phantasm", "r": "7.2", "year": "2011", "votes": "361", "genre": "Comedy", "desp": "Once every ten years, an event called The Carnival Moment occurs, where tales from other dimensions and worlds cross over paths, allowing characters from Kinoko Nasus works to encounter each other."}, {"m_title": "Tokyo Majin", "r": "6.2", "year": "2007", "votes": "147", "genre": "Action, Drama", "desp": "A force of supernatural darkness is unleashing the walking dead on Tokyo, and five high school students with extraordinary powers - and nothing in common - must band together to fight the scourge."}, {"m_title": "Osomatsu-san", "r": "7.5", "year": "2015", "votes": "426", "genre": "Comedy", "desp": "The adventures of young Osomatsu Matsuno and his five sextuplet brothers."}, {"m_title": "Ah! My Goddess", "r": "7.2", "year": "2005", "votes": "1,223", "genre": "Comedy, Drama", "desp": "Nice Guy Keiichi is doing his best as a poor student at a tech college, but his good nature is taken advantage of relentlessly. One day he is attempting to order a pizza over the phone (. watch the anime dude!"}, {"m_title": "Lovely Complex", "r": "7.9", "year": "2007", "votes": "2,977", "genre": "Comedy, Romance", "desp": "Risa and Otani are always being laughed at as the comedy duo. Risa, a girl, is taller than the average girl, and Otani, a boy, is shorter than the average boy. However, Risa has feelings for Otani that shocks him."}, {"m_title": "Queens Blade 2: The Evil Eye", "r": "6.6", "year": "2009", "votes": "342", "genre": "Action, Adventure", "desp": "The sequel that follows Queens Blade: The Exiled Virgin (2009). Countless powerful women from around the continent join a tournament called the Queens Blade tournament to challenge one another and eventually try their skills at defeating the current queen."}, {"m_title": "The World Is Still Beautiful", "r": "7.3", "year": "2014", "votes": "942", "genre": "Adventure, Fantasy", "desp": "In the almighty Sun Kingdom, it never rains. In the Rain Dukedom, select women of the royal family have the ability to call the rain with song. So when the most powerful ruler in the world,. watch the anime dude!"}, {"m_title": "Descending Stories: Showa Genroku Rakugo Shinju", "r": "8.5", "year": "2017", "votes": "910", "genre": "Drama", "desp": "Now a rakugo artist himself, Yotarou is challenged to keep the art form alive, meanwhile witnessing the final years of his master, Yakumo."}, {"m_title": "Kemono Michi: Rise Up", "r": "6.5", "year": "2019", "votes": "525", "genre": "Action, Adventure", "desp": "A pro wrestler is summoned into a fantasy world, with a destiny to slay monsters inhabiting the realm. But he loves animals, so he rejects that destiny and takes to raising the monsters as pets."}, {"m_title": "Amagami SS", "r": "6.8", "year": "2010", "votes": "516", "genre": "Drama, Romance", "desp": "High school student Junichi Tachibana attempts to find love once again after having his heart broken last Christmas. This series follows his romantic endeavors in a sequence of six separate arcs, each involving a different heroine."}, {"m_title": "Wandering Witch: The Journey of Elaina", "r": "7.0", "year": "2020", "votes": "978", "genre": "Adventure, Fantasy", "desp": "The anime centers on a girl named Elaina. When she was a kid, she loved reading books. And among all the books she read, there was a series called Niikes Adventure, which was her. watch the dude, no spoilers in summary!"}, {"m_title": "Xamd: Lost Memories", "r": "7.3", "year": "2008", "votes": "510", "genre": "Adventure, Sci-Fi", "desp": "A young man named Akiyuki is a victim of a terrorist attack perpetrated on his peaceful island during wartime. As a result of the attack, he is imbued with an indefinable entity known as a. watch the dude, no spoilers in summary!"}, {"m_title": "Ôyukiumi no Kaina", "r": "6.5", "year": "2023", "votes": "155", "genre": "Adventure, Fantasy", "desp": "In a world that is enveloped in snow and has canopies reaching to the sky, two people from different communities make an encounter that changes history."}, {"m_title": "Sakura Trick", "r": "6.9", "year": "2014", "votes": "725", "genre": "Comedy, Romance", "desp": "Haruka and Yuu are best friends. But after starting high school together and realising the need to meet new people, they decide to share a special moment most other friends would not: a kiss. From there, their relationship blossoms."}, {"m_title": "Kiyo in Kyoto: From the Maiko House", "r": "7.2", "year": "2021", "votes": "60", "genre": "Comedy", "desp": "The story of 2 students becoming a cook and a promising one-in-a-century Maiko in a Maiko house in Kyoto"}, {"m_title": "Mobile Police Patlabor: The Early Days", "r": "7.5", "year": "1988", "votes": "761", "genre": "Short, Action", "desp": "The adventures of a giant robot police force unit."}, {"m_title": "Vampire Princess Miyu", "r": "7.1", "year": "1997", "votes": "400", "genre": "Action, Drama", "desp": "Miyu, a vampire girl, acts as a guardian, sending stray demons known as Shinma back to the darkness while publically posing as a high school student."}, {"m_title": "Kakushigoto", "r": "7.4", "year": "2020", "votes": "891", "genre": "Comedy, Drama", "desp": "A writer of lewd stories tries to keep his work a secret from his innocent and unknowing daughter."}, {"m_title": "Final Fantasy: Unlimited", "r": "5.9", "year": "2001", "votes": "683", "genre": "Adventure, Fantasy", "desp": "Two siblings try to find their missing parents. Based on the best selling video game series by Square Enix."}, {"m_title": "Papa Datte, Shitai", "r": "5.6", "year": "2019", "votes": "115", "genre": "Romance", "desp": "Asumi Kouya is a fourth-year college student who works as a housekeeper. Kouya starts working for single father Naruse Keiichi, and a relationship develops between the two."}, {"m_title": "Nekopara", "r": "6.1", "year": "2020", "votes": "377", "genre": "Comedy, Romance", "desp": "Whats NEKOPARA? Why, its a cat paradise. Kashou Minazuki, the son of a long line of Japanese confection makers moved out to open his own shop La Soleil as a patisserie."}, {"m_title": "Nananas Buried Treasure", "r": "6.4", "year": "2014", "votes": "176", "genre": "Action, Adventure", "desp": "Juugo Yama moves into an apartment on an artificial island and discovers it is haunted by the ghost of Nanana Ryugajo, who was assassinated a decade ago. He goes on a treasure hunt for the mysterious Nanana Collection."}, {"m_title": "Senran Kagura Shinovi Master: Tokyo Yoma-hen", "r": "6.5", "year": "2018", "votes": "76", "genre": "Action, Comedy", "desp": "Where there is light, there is shadow. The strongest and most well-endowed ninjas in history, both good and evil, will collide in a battle to determine true justice."}, {"m_title": "Basquash!", "r": "7.2", "year": "2009", "votes": "70", "genre": "Adventure, Sci-Fi", "desp": "The story takes place in the world of Earth Dash, an alternate, futuristic version of Earth where human society split between the far more advanced lunar society and that of the planets. watch the dude, no spoilers in summary!"}, {"m_title": "Tokyo Mew Mew New", "r": "6.9", "year": "2022", "votes": "71", "genre": "Action, Adventure", "desp": "Follows Ichigo Momomiya, a girl who transforms into Mew Ichigo with the power of the Iriomote leopard cat to save Earth from parasitic Chimera Anima aliens."}, {"m_title": "Kodocha", "r": "7.5", "year": "1996", "votes": "507", "genre": "Comedy, Drama", "desp": "The story of super hyper child star Sana Kurata. Her mother is an award winning writer who has strange hairstyles and hats in which lives the family pet, Maro-chan the squirrel. Her manager. watch the anime dude!"}, {"m_title": "Iketeru futari", "r": "6.9", "year": "1999", "votes": "28", "genre": "Comedy, Romance", "desp": "Keisuke Saji is a horndog looking for love. Akira Koizumi is an ice queen with a hidden naughty streak. Together they make a cool (iketeru in Japanese) couple."}, {"m_title": "Drifting Dragons", "r": "6.1", "year": "2020", "votes": "656", "genre": "Adventure, Fantasy", "desp": "Dragons are on the menu as the crew of the airship Quin Zaza sets out on a hunt. If they fail, empty stomachs will be the least of their worries."}, {"m_title": "Assassins Pride", "r": "5.7", "year": "2019", "votes": "662", "genre": "Action, Drama", "desp": "An assassin sent to kill an illegitimate girl has a change of heart and becomes her protector. How long can he manage the job?"}, {"m_title": "Mobile Suit Gundam 0083: Stardust Memory", "r": "7.7", "year": "1991", "votes": "1,125", "genre": "Action, Drama", "desp": "After the One Year War, Peace was signed between Zeon and the Earth Federation. Three years later, however, some remaining forces of Zeon who never gave up steal the new Federation mobile. watch the dude, no spoilers in summary!"}, {"m_title": "Ookami-san wa Taberaretai", "r": "6.9", "year": "2020", "votes": "64", "genre": "Short", "desp": "sorry no summary not available!"}, {"m_title": "NTR: Netsuzou Trap", "r": "5.0", "year": "2017", "votes": "477", "genre": "Short, Drama", "desp": "Yuma and Hotaru have been friends since childhood, so its only natural that when Yuma is nervous about her new boyfriend, she asks Hotaru for advice. But when Hotaru starts coming onto. watch the dude, no spoilers in summary!"}, {"m_title": "Gabriel DropOut", "r": "7.1", "year": "2017", "votes": "1,022", "genre": "Comedy, Fantasy", "desp": "Angel-girls and demon-girls are sent to the same high-school on Earth. What could possibly go wrong?"}, {"m_title": "Space Brothers", "r": "8.2", "year": "2012", "votes": "524", "genre": "Comedy, Sci-Fi", "desp": "One evening two brothers see what appears to be a UFO in the sky. Whether or not that was the case becomes irrelevant, but it does spark an interest in space and the two brothers make a. watch the dude, no spoilers in summary!"}, {"m_title": "Danganronpa 3: The End of Hopes Peak Academy - Hope Arc", "r": "7.0", "year": "III) (2016", "votes": "537", "genre": "Animation", "desp": "As Makoto races to stop Ryota from broadcasting his Hope video, he and the others are assisted by the arrival of the 77th Class, who Hajime managed to restore to normal using Izurus. watch the dude, no spoilers in summary!"}, {"m_title": "Blue Dragon", "r": "6.3", "year": "2007", "votes": "368", "genre": "Action, Adventure", "desp": "Shu and his fellow shadow wielders are determined to seek help at Jibral castle. When they are finally granted an audience with the king, Zola requests permission to view the all-important. watch the dude, no spoilers in summary!"}, {"m_title": "The Night When Me and My Boss Spent the Night at the Capsule Hotel After the Last Train and He Passed His Fever to Me", "r": "5.6", "year": "2018", "votes": "54", "genre": "Romance", "desp": "The anime centers on Minori, an office worker, and her boss Hadano, who are constantly arguing with each other. One night, after a company drinking party, Minori and Hadano are having their. watch the anime dude!"}, {"m_title": "Ace of Diamond: Act II", "r": "8.2", "year": "2019", "votes": "247", "genre": "Comedy, Sport", "desp": "After winning the regional championship Seido High School leading by captain Miyuki makes appearance in Koshien stadium at the spring invitational with both second year pitcher Sawamura and Furuya ."}, {"m_title": "Nodame kantâbire", "r": "8.1", "year": "2007", "votes": "1,321", "genre": "Comedy, Drama", "desp": "Shinichi Chiaki, is a talented musician who finds a rather strange companion in Megumi Noda. As they get to know each other they also meet a variety of crazy characters and face some challenging opportunities."}, {"m_title": "Seinto Seiya: Meiou Hades Meikai Hen", "r": "8.2", "year": "2005", "votes": "1,476", "genre": "Action, Adventure", "desp": "Seiya and the rest of the Bronze and Gold Saints who have survived the Sanctuary battle have entered the world of the Dead. They are fighting to get to Hades and defeat him."}, {"m_title": "Hanasaku iroha", "r": "7.6", "year": "2011", "votes": "767", "genre": "Comedy, Drama", "desp": "Ohana Matsumae, a girl from Tokyo, is forced to leave her hometown to live at an inn with her grandmother whom shes never met. When she arrives, she finds out shell have to work at the. watch the dude, no spoilers in summary!"}, {"m_title": "Gargantia on the Verdurous Planet", "r": "7.4", "year": "2013", "votes": "1,730", "genre": "Adventure, Drama", "desp": "While fighting an intense inter-galactic war, a mecha pilot was accidentally warped into a space-time neither he nor the computer of his mecha could recognize. After waking up from a. watch the dude, no spoilers in summary!"}, {"m_title": "Punch Line", "r": "6.1", "year": "2015", "votes": "282", "genre": "Adventure, Comedy", "desp": "High schooler Yuta Iridatsu has experienced astral projection i.e. his spirit has been separated from his body. He woke up in a mansion named Koraikan and met a cat spirit named. watch the dude, no spoilers in summary!"}, {"m_title": "Ie naki ko Remi", "r": "8.2", "year": "1996", "votes": "816", "genre": "Adventure, Drama", "desp": "Remi, Nobodys Girl tells the story of Remi, a cheerful and tender-hearted girl, who is an excellent singer and lives in the French country town of Chavanon with her mother. One day her. watch the dude, no spoilers in summary!"}, {"m_title": "Rage of Bahamut: Manaria Friends", "r": "5.4", "year": "2019", "votes": "189", "genre": "Fantasy", "desp": "The adventures of students at the prestigious magic school, Mysteria Academy."}, {"m_title": "Samurai Girls", "r": "6.5", "year": "2010", "votes": "411", "genre": "Action, Comedy", "desp": "A light in the sky. A falling body. A Kiss. Unimagable power is awakened, and with it a series of events will unfold that will change Great Japan forever."}, {"m_title": "Chaika: The Coffin Princess", "r": "6.9", "year": "2014", "votes": "685", "genre": "Adventure, Comedy", "desp": "A mysterious girl carrying a coffin is on a quest to gather magical remains scattered throughout the land."}, {"m_title": "Ganbare Douki-chan", "r": "6.2", "year": "2021", "votes": "294", "genre": "Comedy, Romance", "desp": "An office worker struggles to reach her dream man."}, {"m_title": "Majikoi ~ Oh! Samurai Girls", "r": "5.7", "year": "2011", "votes": "323", "genre": "Comedy, Romance", "desp": "In Kawakami City, having a samurai ancestor and fighting spirit isnt just an interesting fact, it dictates how youre ranked on the social totem pole. Fortunately, thats something kids. watch the dude, no spoilers in summary!"}, {"m_title": "Mahoromatic", "r": "6.7", "year": "2001", "votes": "369", "genre": "Action, Comedy", "desp": "A military robot is decommissioned and works as a loyal maid for the young son of her late creator."}, {"m_title": "Kashimashi: Girl Meets Girl", "r": "6.7", "year": "2006", "votes": "237", "genre": "Comedy, Fantasy", "desp": "After being rejected by the girl he loves, Hazumu goes to the mountains to console himself. A crashing alien spaceship lands on him. The aliens regenerate his body. However, they could only make him into a girl."}, {"m_title": "Reservoir Chronicle: Tsubasa", "r": "7.3", "year": "2005", "votes": "1,325", "genre": "Action, Adventure", "desp": "Syaoran, a boy who wants to become an archeologist, and Sakura, a princess from the Clow Kingdom, are childhood friends with a close relationship. On a fateful night, Sakura lost all her. watch the dude, no spoilers in summary!"}, {"m_title": "Aggressive Retsuko", "r": "8.2", "year": "2016", "votes": "1,185", "genre": "Comedy, Musical", "desp": "In this series of shorts, a polite accountant deals with her annoying co-workers by chugging beer and singing death metal."}, {"m_title": "Murenase! Seton Gakuen", "r": "6.6", "year": "2020", "votes": "311", "genre": "Comedy, Drama", "desp": "Seton Academy, a school full of animals where, thanks to population decline, there are fewer humans than any other creature. Mazama Jin, an animal hater and the only human male in his class, falls in love with Hino Hitomi,"}, {"m_title": "Granblue Fantasy: The Animation", "r": "6.3", "year": "2017", "votes": "436", "genre": "Adventure, Fantasy", "desp": "This is a world of the skies, where many islands drift in the sky. A boy named Gran and a speaking winged lizard named Vyrn lived in Zinkenstill. One day, they come across a girl named. watch the dude, no spoilers in summary!"}, {"m_title": "Pretty Cure", "r": "6.7", "year": "2004", "votes": "250", "genre": "Action, Adventure", "desp": "Two young girls, Nagisa and Honoka, find mysterious fairies that grant the girls the ability to transform into the legendary warriors known as Pretty Cure and combat the forces of darkness."}, {"m_title": "Mobile Suit V Gundam", "r": "7.0", "year": "1993", "votes": "302", "genre": "Action, Drama", "desp": "Uso Ewin gets himself tangled in a battle between the Yellow Jacket and guerrilla resistance League Militaire. Following his heart, Uso pilots the resistances mobile suit and goes against the Zanscare Empire of space."}, {"m_title": "Zipang", "r": "7.4", "year": "2004", "votes": "149", "genre": "Drama, Sci-Fi", "desp": "The story of Zipang is basically the same as that of the movie The Final Countdown, only with a few twists. The Japanese AEGIS Destroyer Mirai of the Japanese Maritime Self Defense Force. watch the dude, no spoilers in summary!"}, {"m_title": "Dragon Pilot: Hisone and Masotan", "r": "6.8", "year": "2018", "votes": "560", "genre": "Fantasy, Sci-Fi", "desp": "When a young girl with a personality disorder meets a dragon, her life is set on a new course. To learn how to fly this dragon and work on her disorder (to accidentally say innocent things. watch the dude, no spoilers in summary!"}, {"m_title": "Kinnikuman", "r": "7.0", "year": "1983", "votes": "176", "genre": "Action, Adventure", "desp": "The lost prince of Planet Kinniku must take his place among his people. Theres a few problems: hes lazy, cowardly and unskilled."}, {"m_title": "Chrome Shelled Regios", "r": "7.0", "year": "2009", "votes": "555", "genre": "Action, Adventure", "desp": "The world of tomorrow is an arid wasteland. Humans must live in domed cities to escape the threat of mutant Contaminoid monsters. In this future, mysterious Layfon struggles to keep his. watch the dude, no spoilers in summary!"}, {"m_title": "Danganronpa 3: The End of Kibougamine Gakuen - Mirai-hen", "r": "6.9", "year": "I) (2016", "votes": "797", "genre": "Drama, Horror", "desp": "Makoto Naegi, the Ultimate Hope who defeated the Ultimate Despair, Junko Enoshima, is under suspicion of treason against the Future Foundation for harboring the Remnants of Despair who. watch the dude, no spoilers in summary!"}, {"m_title": "Digimon Adventure 02", "r": "7.4", "year": "2000", "votes": "1,786", "genre": "Action, Adventure", "desp": "3 years after the DigiDestined beat Apocalymon, a new evil has arrived. Ken the Digimon Kaizer! Using his Black Digivice and Evil Rings, he takes control of the digimon. To complete this,. watch the dude, no spoilers in summary!"}, {"m_title": "Akagi", "r": "7.8", "year": "2005", "votes": "707", "genre": "Sport, Thriller", "desp": "Genius gambler Shigeru Akagi competes with members of the mafia in mahjong."}, {"m_title": "Duel Masters", "r": "5.3", "year": "2002", "votes": "662", "genre": "Action, Family", "desp": "A ten year old boy wants to become the best duelist, like his father in the duel masters world."}, {"m_title": "Opus Colors", "r": "4.9", "year": "2023", "votes": "11", "genre": "Drama", "desp": "Centers on the relationship between an artist and a grader, in an immersive field of digital art known as perception art that was developed a decade ago."}, {"m_title": "Soredemo Ayumu wa yosetekuru", "r": "6.6", "year": "2022", "votes": "287", "genre": "Comedy, Romance", "desp": "On a whim, first-year Ayumu Tanaka quits the Kendo Club to join the illegitimate Shogi Club. Urushi Yaotome, the president of the club and a master of shogi, is thrilled to finally have a. watch the dude, no spoilers in summary!"}, {"m_title": "Mushikaburi-hime", "r": "6.9", "year": "2022", "votes": "67", "genre": "Drama, Romance", "desp": "For as long as she could read, Eliana Bernstein-more commonly known as the Bibliophile Princess-has preferred the company of books to that of people. However, as a noble, it is her duty. watch the dude, no spoilers in summary!"}, {"m_title": "Corpse Princess: Aka", "r": "6.9", "year": "2008", "votes": "447", "genre": "Action, Adventure", "desp": "An undead school girl must hunt down 108 other undeads with the help of specially trained Buddhist monks, in order to gain entry into Heaven."}, {"m_title": "Stars Align", "r": "7.4", "year": "2019", "votes": "533", "genre": "Drama, Sport", "desp": "Tomas summer challenge is to save his tennis team. But theres no hope unless he can recruit Makis talent to the court!"}, {"m_title": "The Fire Hunter", "r": "6.7", "year": "2023", "votes": "102", "genre": "Adventure, Drama", "desp": "When a Fire Hunter dies in her presence, Touko must make a treacherous journey to the capital for redemption, and just may change the destiny of the destitute world."}, {"m_title": "Kimagure Orange Road", "r": "7.6", "year": "1985", "votes": "880", "genre": "Comedy, Drama", "desp": "Kyousuke Kasuga is a completely normal Japanese high school student, with two very big problems. The first is his complete and utter inability to choose between two girls, the bright,. watch the dude, no spoilers in summary!"}, {"m_title": "Actually, I Am...", "r": "6.4", "year": "2015", "votes": "350", "genre": "Comedy, Fantasy", "desp": "When a boy trys to confess to his crush that he loves her, he accidentally finds out shes a vampire and has to try and keep her secret..."}, {"m_title": "Deltora Quest", "r": "7.1", "year": "2007", "votes": "270", "genre": "Adventure, Family", "desp": "Leif must restore the seven gems and the belt of Deltora to save his homeland. Along with his companions Barda and Jasmine, Lief must face devious enemies and dangerous beasts to oust the Shadow Lord and return peace to Deltora."}, {"m_title": "Interviews with Monster Girls", "r": "7.4", "year": "2017", "votes": "991", "genre": "Comedy, Fantasy", "desp": "In a modern alternate reality, a teacher studies the unusual lives of his schools demi-humans."}, {"m_title": "Zoids Wild", "r": "7.3", "year": "2018", "votes": "37", "genre": "Animation", "desp": "Arashi longs to follow his fathers footsteps and become a Zoid Hunter. With Wild Liger, and team Freedom by his side, he sets out to find the Great Ancient Treasure Z."}, {"m_title": "Mujaki no Rakuen", "r": "3.8", "year": "2014", "votes": "37", "genre": "Comedy", "desp": "Shouta, a 25-year-old NEET. A mysterious incident at the school pool throws him back in time and lets him deal with those same classmates back when they were fifth grade girls."}, {"m_title": "Ah My Buddha!!", "r": "5.9", "year": "2005", "votes": "214", "genre": "Comedy, Romance", "desp": "Ikkou turns into a super-powerful Buddhist monk capable of vanquishing evil spirits when he gets turned on. Ikkous parents send him to a Temple to train where hes surrounded by beautiful nun trainees."}, {"m_title": "Kizuna No Allele", "r": "5.5", "year": "2023", "votes": "18", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Meiji Tokyo Renka", "r": "7.3", "year": "I) (2019", "votes": "236", "genre": "Comedy, Fantasy", "desp": "On the night of a crimson full moon, high school girl Mei Ayazuki, lured by the magic of mysterious magician Charlie, enters into a box. When she awakens, shes in Tokyo during the Meiji Period."}, {"m_title": "Val x Love", "r": "5.2", "year": "2019", "votes": "143", "genre": "Comedy, Drama", "desp": "High schooler Akutsu Takuma has learned to accept his lonely lot in life and is content surrounded by his studies, but when the god Odin taps him to save the world alongside nine Valkyries. watch the dude, no spoilers in summary!"}, {"m_title": "Wataten! An Angel Flew Down to Me", "r": "6.7", "year": "2019", "votes": "355", "genre": "Comedy, Romance", "desp": "Miyako is a shy college student who is also an otaku. One day she happens to meet some angelic grade school kids. When Miyako sees her little sisters new friend Hana-chan, Miyakos heart. watch the dude, no spoilers in summary!"}, {"m_title": "Flip Flappers", "r": "7.3", "year": "2016", "votes": "389", "genre": "Adventure, Comedy", "desp": "Cocona is an average middle schooler living with her grandmother. And she who has yet to decide a goal to strive for, soon met a strange girl named Papika who invites her to an organization called Flip Flap."}, {"m_title": "Ground Control to Psychoelectric Girl", "r": "6.6", "year": "2011", "votes": "252", "genre": "Comedy, Drama", "desp": "Makoto Niwa moves in with his eccentric aunt and cousin. As it so happens, his cousin believes herself to be an alien. Makoto takes it upon himself to reintroduce her to society."}, {"m_title": "Mars Red", "r": "6.1", "year": "2021", "votes": "362", "genre": "Action, Fantasy", "desp": "This is a tale about humans and vampires. Taishou Romantism: the blooming era. Ladies and gentlemen chat in cafes about the avant-garde and other European trends. But this tranquil. watch the dude, no spoilers in summary!"}, {"m_title": "Stella of the Theater: World Dai Star", "r": "6.9", "year": "2023", "votes": "14", "genre": "Drama", "desp": "After Dai Star stage performers exploded in worldwide popularity in the 20th century, 16-year-old Kokona Ootori follows her dream of becoming a World Dai Star by auditioning for the Sirius theatrical troupe."}, {"m_title": "Ex-Arm", "r": "1.7", "year": "2021", "votes": "830", "genre": "Action, Crime", "desp": "In 2014, Akira dies in an accident. 16 years later a policewoman and her android partner retrieve an advanced AI called EX-ARM. Turns out the AI is actually Akiras brain."}, {"m_title": "Riddle Story of Devil", "r": "6.3", "year": "2014", "votes": "521", "genre": "Action, Thriller", "desp": "Tokaku Azuma is a heartless teenage assassin on a mission of death. Shes been sent to a prestigious, all-girls boarding school to compete in an assassination game against eleven other. watch the dude, no spoilers in summary!"}, {"m_title": "Holo no Graffiti", "r": "7.7", "year": "2019", "votes": "35", "genre": "Comedy", "desp": "From unraveling the secrets of opening and closing doors to defusing surprise packages more commonly known as bombs, there is never a dull day at the Hololive Production office. Holo no. watch the dude, no spoilers in summary!"}, {"m_title": "Gintama", "r": "8.9", "year": "2006", "votes": "28", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "YuruYuri", "r": "7.6", "year": "2011", "votes": "680", "genre": "Comedy, Drama", "desp": "A Comedy Series that revolves around the daily lives of the Amusement Club, consisting of Akari Akaza, her childhood friends, Kyoko Toshino and Yui Funami, and her classmate Chinatsu Yoshikawa"}, {"m_title": "Ippon Again!", "r": "7.0", "year": "2023", "votes": "35", "genre": "Action, Drama", "desp": "Michi Sonoda planned to quit judo after her final tournament in junior high until her best friend Sanae Takigawa invites her to continue judo in high school."}, {"m_title": "Shônan Bakusôzoku", "r": "7.4", "year": "1986", "votes": "12", "genre": "Action, Comedy", "desp": "In Japan, the country that inspired the slogan Born to be Mild, the blood of young men still runs hot. In the old days theyd go off into the country and hack each other up with swords,. watch the dude, no spoilers in summary!"}, {"m_title": "Z/X: Code Reunion", "r": "6.2", "year": "2019", "votes": "19", "genre": "Animation", "desp": "The signing of a peace treaty has secured a tenuous ceasefire between mankind and the Zex, beings who emerged from space/time rifts connected to mysterious, distant worlds. Partnered with a. watch the anime dude!"}, {"m_title": "Mobile Suit Gundam: The Origin - Advent of the Red Comet", "r": "7.8", "year": "2019", "votes": "113", "genre": "Action, Drama", "desp": "The tragic story of Char Aznable, the ace pilot known as the Red Comet, and his sister Sayla Mass unfolds against the backdrop of the events leading to the One Year War. The Zabi familys. watch the dude, no spoilers in summary!"}, {"m_title": "We Without Wings: Under the Innocent Sky", "r": "6.2", "year": "2011", "votes": "95", "genre": "Comedy, Drama", "desp": "Haneda Takashi has a secret he cannot speak of. To leave his dull school-life, he was supposed to have escaped to another world. However, he is reeled in by certain ties on his heart. One. watch the dude, no spoilers in summary!"}, {"m_title": "Shin Ikkitousen", "r": "6.8", "year": "2022", "votes": "34", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Peach Boy Riverside", "r": "6.2", "year": "2021", "votes": "453", "genre": "Action, Fantasy", "desp": "A hoard of vicious demons known as Oni come knocking on her doorstep, threatening the lives of everyone in the Kingdom, they are saved by a lone traveler named Kibitsu Mikoto who slays these monsters with a mysterious Peach Eye."}, {"m_title": "Blade Dance of the Elementalers", "r": "6.5", "year": "2014", "votes": "510", "genre": "Action, Comedy", "desp": "In a world where spirits exist and only pure maidens have the privilege of contracting with the spirits maidens from noble families gather at Areishia Spirit Academy where they are trained. watch the dude, no spoilers in summary!"}, {"m_title": "Queens Blade Unlimited", "r": "6.3", "year": "2018", "votes": "27", "genre": "Action, Fantasy", "desp": "A reboot of the Queens Blade anime series, featuring revamped character designs, and a new storyline."}, {"m_title": "Black Butler II", "r": "7.4", "year": "2010", "votes": "2,351", "genre": "Comedy, Fantasy", "desp": "In late-nineteenth century England, one year and three months after the dramatic conclusion of the original series, Kuroshitsuji II introduces a new butler and young master. The sadistic. watch the dude, no spoilers in summary!"}, {"m_title": "Mix: Meisei Story", "r": "7.3", "year": "2019", "votes": "77", "genre": "Drama, Romance", "desp": "Stepbrothers Touma and Suichirou are ace baseball players following in the tragic legacy of their fathers."}, {"m_title": "Tokyo Twenty Fourth Ward", "r": "6.0", "year": "2022", "votes": "179", "genre": "Mystery, Sci-Fi", "desp": "Enter the 24th Ward, a man-made island inside Tokyo Bay. Three of its inhabitants: Shuta, Ran, and Koki, have been best friends since childhood, but after a deadly incident, everything. watch the dude, no spoilers in summary!"}, {"m_title": "Burst Angel", "r": "6.5", "year": "2004", "votes": "706", "genre": "Action, Comedy", "desp": "In a chaotic future Tokyo, a young aspiring chef is employed by four beautiful mercenaries."}, {"m_title": "The Heike Story", "r": "7.6", "year": "2021", "votes": "409", "genre": "Drama, History", "desp": "The Taira clan, also known as the Heike, holds immense authority over Japan. When a young girl, gifted with an odd eye that allows her to see the future, foolishly disrespects the clan, her. watch the anime dude!"}, {"m_title": "I Want You to Show Me Your Panties with a Disgusted Face", "r": "5.5", "year": "2018", "votes": "112", "genre": "Animation", "desp": "A project where a number of women in different costumes show their panties while they make a disgusted face."}, {"m_title": "Dirty Pair Flash", "r": "6.9", "year": "1995", "votes": "196", "genre": "Short, Action", "desp": "Yuri and Kei are given accommodations at an all-girls boarding school from the 20th century. Yuri finds the life as an exchange student a lot of fun."}, {"m_title": "Otherside Picnic", "r": "5.9", "year": "2021", "votes": "369", "genre": "Adventure, Fantasy", "desp": "Moments from death, Sorawo is saved by the mysterious Toriko in the world of the Otherside. Entranced by its beauty, she discovers that this dimension is inhabited by monsters once thought imaginary."}, {"m_title": "The New Prince of Tennis", "r": "6.7", "year": "2012", "votes": "130", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Karafuru", "r": "6.0", "year": "1999", "votes": "125", "genre": "Comedy", "desp": "An anime series of 16 short episodes of perverted situations."}, {"m_title": "Aikatsu!", "r": "7.0", "year": "2012", "votes": "212", "genre": "Adventure, Comedy", "desp": "Ichigo Hoshimiya is a regular, ordinary middle school girl. But when her best friend, Aoi, invites her to join the idol training academy, Starlight Academy, her whole world is turned upside. watch the anime dude!"}, {"m_title": "How to Keep a Mummy", "r": "7.2", "year": "2018", "votes": "218", "genre": "Comedy, Fantasy", "desp": "One day high school student Sora Kashiwagi receives a coffin sent by his adventurer father and inside it is a little mummy that just wants to be friends."}, {"m_title": "After War Gundam X", "r": "7.2", "year": "1996", "votes": "306", "genre": "Action, Adventure", "desp": "After the colonies crash into the Earth, the few surviving communities are preyed on by those few who retain technology. A scrappy young boy is hired to rescue a girl from one of these. watch the dude, no spoilers in summary!"}, {"m_title": "Hero Mask", "r": "6.2", "year": "2018", "votes": "726", "genre": "Action, Crime", "desp": "Detective James Blood investigates a string of grisly crimes all involving a mysterious mask."}, {"m_title": "Strawberry 100%", "r": "6.3", "year": "I) (2005", "votes": "216", "genre": "Comedy, Romance", "desp": "One day, Manaka Junpei walks to the roof of his school and encounters a beautiful girl falling down from above him and accidentally exposing her strawberry panties. The embarrassed girl. watch the dude, no spoilers in summary!"}, {"m_title": "Kenzen Robo Daimidaler", "r": "6.3", "year": "2014", "votes": "107", "genre": "Action, Comedy", "desp": "Madanbashi Kouichi, a high school boy who can emit Hi-ERO particles by doing perverted things, is chosen to operate the robotic weapon Daimidaler, as Earth receives threats from the Penguin Empire."}, {"m_title": "Dies Irae", "r": "4.4", "year": "2017", "votes": "149", "genre": "Action, Horror", "desp": "A teenager gets involved with a group of sorcerers from World War 2 who unleashed a force beyond their understanding."}, {"m_title": "Laughing Under the Clouds", "r": "6.9", "year": "2014", "votes": "266", "genre": "Action, Drama", "desp": "The story of three brothers who live in a place that is always cloudy because of the curse of a great demon snake that is reborn every 300 years. Tears will be shed and smiles will be shed. watch the dude, no spoilers in summary!"}, {"m_title": "Dennou Coil", "r": "7.8", "year": "2007", "votes": "952", "genre": "Adventure, Comedy", "desp": "Eleven years after the introduction of internet-connected, augmented reality eyeglasses and visors, Yuuko Okonogi moves with her family to Daikoku City, the technological center of the. watch the dude, no spoilers in summary!"}, {"m_title": "Inazuma Eleven Go", "r": "6.9", "year": "2011", "votes": "424", "genre": "Sport", "desp": "The main character, Matsukaze Tenma tries out for the Raimon soccer team and passes, but soon finds out that soccer has changed. Tenma, along with the other members, now aim to free soccer so everyone can enjoy it freely."}, {"m_title": "Rumbling Hearts", "r": "7.6", "year": "2003", "votes": "978", "genre": "Drama, Romance", "desp": "Follows complicated love life of Takayuki Narumi, a male high school student."}, {"m_title": "Last Hope", "r": "5.8", "year": "2018", "votes": "465", "genre": "Action, Adventure", "desp": "After causing the near extinction of mankind seven years ago, genius scientist Leon Lau must now fight the ecological disaster he unwittingly created."}, {"m_title": "Princess Principal", "r": "7.3", "year": "2017", "votes": "487", "genre": "Action, Adventure", "desp": "The stage is set during the 19th century London, in its capital where a wall divides the east and west of the Kingdom of Albion. Five high school girls, who enrolled in the prestigious. watch the dude, no spoilers in summary!"}, {"m_title": "Mr Love: Queens Choice", "r": "6.0", "year": "2020", "votes": "152", "genre": "Action, Drama", "desp": "A corporate heiress inherits her fathers failing company and in her efforts to save it gets romantically involved with a variety of attractive superpowered men."}, {"m_title": "Dragon Quest: Dais Great Adventure", "r": "7.8", "year": "1991", "votes": "882", "genre": "Action, Adventure", "desp": "The story begins with a young boy named Dai .Dai, the young protagonist of the series, is the only human living on the island. Having been raised by Brass and with his best friend, the. watch the dude, no spoilers in summary!"}, {"m_title": "Beyblade: Shogun Steel", "r": "5.1", "year": "2012", "votes": "207", "genre": "Animation", "desp": "A new era of Beyblade has begun, bringing with it new Blades. Zyro, champion of his hometown, heads to Metal Bey City to find greater opponents. Hell test his might against the Cyclone Stadium, and discover how to become a true Blader."}, {"m_title": "Rio: Rainbow Gate", "r": "5.6", "year": "2011", "votes": "249", "genre": "Comedy", "desp": "The Howard Resort Hotel is a famous gambling resort where people gather from around the world in hopes of winning huge fortunes overnight. In the hotels casino is a beautiful female dealer. watch the anime dude!"}, {"m_title": "Nazca", "r": "6.6", "year": "1998", "votes": "53", "genre": "Action, Drama", "desp": "The sleeping souls of ancient warriors awaken to fight for the destiny of earth! Nearly 500 years ago, a bloody civil war erupts within the Inca Empire. The losing side decides to unleash a. watch the anime dude!"}, {"m_title": "Descendants of Darkness", "r": "7.1", "year": "2000", "votes": "387", "genre": "Comedy, Drama", "desp": "A Guardian of Death is investigating a series of what seem to be vampire attacks in Nagasaki."}, {"m_title": "Izetta: The Last Witch", "r": "6.7", "year": "2016", "votes": "357", "genre": "Action, Fantasy", "desp": "In 1939 C.E., the imperialist nation of Germania invaded a neighboring country. All at once, that war spread throughout Europe, and the era was dragged into a spiral of a great war. Then,. watch the dude, no spoilers in summary!"}, {"m_title": "Noozles", "r": "7.6", "year": "1984", "votes": "185", "genre": "Adventure, Comedy", "desp": "The adventures of a 12-year-old girl named Sandy and her koala friends, Blinky and Pinky, who are from the extra-dimensional realm of Koalawalla Land."}, {"m_title": "Dear Brother", "r": "7.6", "year": "1991", "votes": "252", "genre": "Drama, Romance", "desp": "Nanako and her childhood friend Tomoko enter a special all-girl high school where students are rich upper class. The series is about events that take place during her schooldays, which she chronicles in letters she sends to her brother."}, {"m_title": "Birdy the Mighty Decode", "r": "7.3", "year": "2008", "votes": "353", "genre": "Action, Comedy", "desp": "Birdy is an undercover, interstellar federation agent charged with capturing the most dangerous criminals in the galaxy. A famous model by day, Birdy spends her nights hunting deadly alien. watch the dude, no spoilers in summary!"}, {"m_title": "Pokémon Evolutions", "r": "7.2", "year": "2021", "votes": "232", "genre": "Action, Fantasy", "desp": "An anthology series about 8 generations of stories in the world of Pokémon."}, {"m_title": "Cyclops Shojo Saipu", "r": "5.6", "year": "2013", "votes": "82", "genre": "Short", "desp": "Cyclops Shoujo Saipu is a four-panel manga created by Yasu Tora. It was made into an original net animation in 2013. Fuuka Saito, nicknamed Saipu, once made a promise to marry her older. watch the dude, no spoilers in summary!"}, {"m_title": "The Great Jahy Will Not Be Defeated!", "r": "6.8", "year": "2021", "votes": "414", "genre": "Comedy, Fantasy", "desp": "Once respected as the Demon Realms second greatest authority, the Demon Kings aide, Jahy, ruled her subjects with fear. But when a magical girl attacks and destroys a mystic gem. watch the dude, no spoilers in summary!"}, {"m_title": "Lupin III: Part 5", "r": "7.8", "year": "2018", "votes": "399", "genre": "Adventure, Comedy", "desp": "While visiting his ancestral homeland of France, Lupin III becomes entangled with a dark web bank and a worldwide hunt for him spurred by social media."}, {"m_title": "Higurashi: When They Cry - SOTSU", "r": "6.3", "year": "2021", "votes": "358", "genre": "Drama, Horror", "desp": "Sequel and answer arcs to Higurashi no Naku Koro ni Gou."}, {"m_title": "Soul Eater Not!", "r": "5.6", "year": "2014", "votes": "552", "genre": "Action, Comedy", "desp": "Soul Eater Not! is a spin off series from the main series Soul Eater. Tsugumi Harudori from the NOT class must choose between Anya and Meme for a partner, while fighting a dangerous foe."}, {"m_title": "The Strongest Sage with the Weakest Crest", "r": "6.2", "year": "2022", "votes": "831", "genre": "Adventure, Fantasy", "desp": "His strength limited by the magical crest with which he was born, Mathias, the worlds most powerful sage, decides reincarnation is necessary to become the strongest of all. Upon his. watch the dude, no spoilers in summary!"}, {"m_title": "Shomin Sample", "r": "6.4", "year": "2015", "votes": "488", "genre": "Comedy, Romance", "desp": "A private all-girls school forcibly recruits an ordinary male student to help its female graduates adjust to the outside world."}, {"m_title": "The Worlds Greatest First Love", "r": "7.6", "year": "2011", "votes": "1,490", "genre": "Comedy, Romance", "desp": "Ritsu Onodera and Masamune Takano try to sort their feelings from their past relationship."}, {"m_title": "Isuca", "r": "5.8", "year": "2015", "votes": "190", "genre": "Action, Romance", "desp": "Shinichirou decides to look for a job in order to pay his rent, and his teacher suggests that he work for someone at their house doing housekeeping; however, his job is to help exorcise and eliminate unwanted creatures."}, {"m_title": "Ginga Densetsu Weed", "r": "7.8", "year": "2005", "votes": "539", "genre": "Action, Adventure", "desp": "After finding out he is the son of the great leader of Ohu, Gin, Weed sets out on a journey to find his father. However, with many friends to find, there are many enemies."}, {"m_title": "Kanon", "r": "7.5", "year": "2006", "votes": "1,586", "genre": "Comedy, Drama", "desp": "As a young child, Aizawa Yuuichi had often visited his cousin in the city; however, something drastic happened to keep him away for seven long years. Now, Yuuichi returns, his memories of those days are simply gone."}, {"m_title": "Patlabor: The TV Series", "r": "7.7", "year": "1989", "votes": "677", "genre": "Action, Comedy", "desp": "The misadventures of a police robot pilot squad and its support crew in a futuristic Tokyo."}, {"m_title": "Nightwalker: Midnight Detective", "r": "6.9", "year": "1998", "votes": "208", "genre": "Action, Comedy", "desp": "Shido Tatsuhiko is not only a private eye, he is also a vampire with no real memory of his past. Joined by a female government agent, Shido must face demonic creatures known as Nightbreed."}, {"m_title": "Gegege no Kitaro", "r": "7.5", "year": "2018", "votes": "106", "genre": "Action, Adventure", "desp": "The adventures of a young spirit boy and his friends."}, {"m_title": "Bokura no", "r": "7.4", "year": "2007", "votes": "479", "genre": "Drama, Sci-Fi", "desp": "In a summer vacation, a group of 15 children discovered a mysterious man living in a seashore cave, along with his high-tech gadgets. The man claimed to be a game developer creating a video. watch the anime dude!"}, {"m_title": "Wasteful Days of High School Girls", "r": "7.4", "year": "2019", "votes": "290", "genre": "Comedy, Drama", "desp": "One day out of boredom, Tanaka decided to give her classmates nicknames based on their quirks. Her friend Sakuchi became Ota for her nerdy interests, and her other friend Saginomiya. watch the dude, no spoilers in summary!"}, {"m_title": "Garo the Animation", "r": "6.7", "year": "2014", "votes": "320", "genre": "Action, Adventure", "desp": "17 years ago, in the name of witch-hunting, many Makai-Knights and Makai-Priests were murdered at Valiante Kingdom. A Makai-Priestess who is a blood relative of Golden Knight Garo, burned. watch the dude, no spoilers in summary!"}, {"m_title": "Speed Grapher", "r": "7.1", "year": "2005", "votes": "1,001", "genre": "Action, Adventure", "desp": "One man will capture the heart of a woman, and destroy the foundations of society."}, {"m_title": "Shugo Chara!", "r": "7.1", "year": "2007", "votes": "217", "genre": "Comedy, Family", "desp": "Young girl Amu is blessed with three tiny guardian spirits and gains the power to transform and fight alongside a group of fellow students to free others hearts from corruption."}, {"m_title": "Ars no kyojû", "r": "6.3", "year": "2023", "votes": "92", "genre": "Action, Adventure", "desp": "The great beasts created the land, but humans stole it. Angered, the beasts began eating humans, who in turn called upon the gods to fight the beasts. In the age of the sword, heroes and. watch the dude, no spoilers in summary!"}, {"m_title": "Mahou Tsukai no Yome: Nishi no Shounen to Seiran no Kishi", "r": "6.9", "year": "2021", "votes": "83", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Queens Blade: Rebellion", "r": "5.9", "year": "2012", "votes": "206", "genre": "Action, Adventure", "desp": "The story is set four years after the first story-line and revolves around Annelotte, the rebelling knight princess who stands up to the tyranny of the new queen."}, {"m_title": "Assault Lily: Bouquet", "r": "6.3", "year": "2020", "votes": "173", "genre": "Action, Fantasy", "desp": "In the near future, humanity faces imminent destruction from mysterious creatures known as Huge. The world wages war against the Huge, and develops the anti-weapon CHARM (Counter Huge. watch the dude, no spoilers in summary!"}, {"m_title": "Gokinjo monogatari", "r": "8.1", "year": "1995", "votes": "73", "genre": "Romance", "desp": "The protagonist, Kouda Mikako, is a student of Yaza Gaku. Specialising in fashion design, Mikako dreams of becoming a fashion designer with her own brand. Living next to her is her. watch the dude, no spoilers in summary!"}, {"m_title": "Grisaia: Phantom Trigger", "r": "7.4", "year": "2019", "votes": "101", "genre": "Action, Drama", "desp": "Following the Heath Oslo incident, the existence of the US-Japanese anti-terror organization CIRS has become a matter of public knowledge. CIRS has been rebuilt from the ground up, and its. watch the dude, no spoilers in summary!"}, {"m_title": "Hoshizora e kakaru hashi", "r": "6.2", "year": "2011", "votes": "200", "genre": "Romance", "desp": "Kazuma and Ayumu, two brothers, move to a rural mountain village from the city to allow for Ayumu to recover from an illness. Until their parents catch up to them they will be living at a. watch the dude, no spoilers in summary!"}, {"m_title": "Scarlet Nexus", "r": "5.6", "year": "2021", "votes": "218", "genre": "Action, Sci-Fi", "desp": "In a distant future dominated by the breakthrough of people with superpowers, Yuito Sumeragi joins a special taskforce to combat a growing threat and mystery."}, {"m_title": "Capeta", "r": "8.3", "year": "2005", "votes": "240", "genre": "Sport", "desp": "One day capetas father brought junk parts home from his workplace and made a racing cart. However, Capeta found that its frame was bent and it could hardly run straight. But, with his technique he managed the cart."}, {"m_title": "Ive Somehow Gotten Stronger When I Improved My Farm-Related Skills", "r": "5.5", "year": "2022", "votes": "276", "genre": "Action, Adventure", "desp": "Al Wayne, a young man who aims to be a top-notch farmer, has refined his farming skills and finally reached the MAX level. However, once he achieved the pinnacle of farming skills, for some. watch the anime dude!"}, {"m_title": "Aria: The Animation", "r": "7.2", "year": "2005", "votes": "423", "genre": "Comedy, Drama", "desp": "Akari, Aika, and Alice are three girls who share a single dream: to become the most talented gondoliers in all of Neo-Venezia! Every day they train while exploring all the wondrous sights. watch the dude, no spoilers in summary!"}, {"m_title": "Kaleido Star", "r": "7.8", "year": "2003", "votes": "941", "genre": "Comedy, Drama", "desp": "Sora, a young girl from Japan, comes to America in search of her dream. She wants, with all her heart, to be a member of the famous Kaleido Stage, a combination of musicals, acrobatics and. watch the dude, no spoilers in summary!"}, {"m_title": "Star Blazers 2202", "r": "7.0", "year": "2017", "votes": "279", "genre": "Adventure, Drama", "desp": "The continuing adventures of the space battleship Yamato."}, {"m_title": "Beast Wars II: Super Life-Form Transformers", "r": "6.8", "year": "1998", "votes": "148", "genre": "Action, Family", "desp": "A new generation of Maximals and Predacons appear on Earth to continue the Beast Wars."}, {"m_title": "Isekai Cheat Magician", "r": "5.4", "year": "2019", "votes": "924", "genre": "Action, Adventure", "desp": "As regular high school students Taichi and Rin disappeared in a beam of light. When they came to, the two of them were already in a world of swords and magic. Finally getting away after. watch the dude, no spoilers in summary!"}, {"m_title": "Gunslinger Girl", "r": "7.3", "year": "2003", "votes": "2,261", "genre": "Action, Drama", "desp": "The Storyline section on the main page contains the phrase the country of Europe. This implies that the Social Welfare Agency is a sort of EU function, but it is part of Italian government."}, {"m_title": "Boogiepop and Others", "r": "6.4", "year": "2019", "votes": "477", "genre": "Drama, Horror", "desp": "There is an urban legend that tells of a shinigami that can release people from the pain they are suffering. This Angel of Death has a name: Boogiepop. And the legends are true. Boogiepop. watch the anime dude!"}, {"m_title": "Zenki", "r": "6.6", "year": "1995", "votes": "162", "genre": "Action, Adventure", "desp": "Chiaki Enno is just your average high school student, who spends her spare time studying to be a Shinto exorcist. One day, however, the forces of evil are unleashed in her backyard. Quite. watch the dude, no spoilers in summary!"}, {"m_title": "Super Cub", "r": "7.3", "year": "2021", "votes": "382", "genre": "Adventure, Drama", "desp": "A lonely girls eyes are opened to the many possibilities when she buys a Honda Super Cub. In her old life she had nothing, but now she has a Cub and many opportunities before her."}, {"m_title": "Re-Main", "r": "7.2", "year": "2021", "votes": "419", "genre": "Comedy, Drama", "desp": "The story centers on Minato Kiyoumizu, a boy who quit playing water polo after an incident during the winter of his third year of junior high school. After entering high school, he makes a. watch the dude, no spoilers in summary!"}, {"m_title": "Gun x Sword", "r": "7.2", "year": "2005", "votes": "582", "genre": "Action, Sci-Fi", "desp": "There are a lot of crime in Planet of Endless Illusion, but people dont give up their own hope. A man who was killed his bride searches for the mysterious killer, the Claw, to revenge. Its not only him searching for the Claw."}, {"m_title": "Love Live! Nijigasaki High School Idol Club", "r": "7.0", "year": "2020", "votes": "184", "genre": "Comedy, Music", "desp": "The story centers on the members of the School Idol Club in Nijigasaki, and their attempt to prevent the club from being abolished."}, {"m_title": "Mobile Suit Gundam ZZ", "r": "6.8", "year": "1986", "votes": "646", "genre": "Action, Comedy", "desp": "Following the decisive battle against the Titans, AEUGs Argama crew (along with a new bunch of kids), has little time to regroup as a new threat looms in the horizon: Hamans Neo Zeon!"}, {"m_title": "THE IDOLM@STER CINDERELLA GIRLS U149", "r": "7.9", "year": "2023", "votes": "14", "genre": "Comedy, Drama", "desp": "A spin-off of THE iDOLM@STER Cinderella Girls set in a different canon, it features almost all of the younger idols under 12 years old (as the title implies, who are under 149cm in height) of the series and a new, short producer."}, {"m_title": "Ghost Hound", "r": "7.4", "year": "2007", "votes": "812", "genre": "Drama, Horror", "desp": "The experiences of three boys who have had traumatic experiences in childhood from which they have learned to transfer their souls to a parallel world known as the Unseen World."}, {"m_title": "Marvel Disk Wars: The Avengers", "r": "4.4", "year": "2014", "votes": "323", "genre": "Action, Adventure", "desp": "A group of teenagers join forces with the Avengers, Earths mightiest heroes, to fight against the tyrannical Loki and his mighty empire."}, {"m_title": "Gundam Build Fighters", "r": "7.2", "year": "2013", "votes": "279", "genre": "Action, Comedy", "desp": "In a world where Gundam is a very popular T.V. show, Sei Iori wishes to be just like his father, who was a Gunpla champion years ago, unfortunately, Sei only has skill in building Gunpla,. watch the dude, no spoilers in summary!"}, {"m_title": "Miss Caretaker of Sunohara-sou", "r": "6.3", "year": "2018", "votes": "259", "genre": "Comedy, Romance", "desp": "Shiina Aki is at odds with his masculinity. So, hes heading to Tokyo to live with three female classmates and his voluptuous caretaker!"}, {"m_title": "The Familiar of Zero", "r": "7.0", "year": "2006", "votes": "2,272", "genre": "Action, Adventure", "desp": "When Saito, an ordinary teen living in Japan, is suddenly whisked away to the magical kingdom of Halkeginia, he becomes the familiar servant of Louise, a young girl enrolled in an academy for aristocrat magicians."}, {"m_title": "I Want You to Show Me Your Panties with a Disgusted Face 2", "r": "6.4", "year": "2019", "votes": "57", "genre": "Animation", "desp": "A project where a number of women in different costumes show their panties while they make a disgusted face."}, {"m_title": "Engaged to the Unidentified", "r": "7.0", "year": "2014", "votes": "475", "genre": "Comedy, Romance", "desp": "Kobeni has lived with her Mother and Sister, Benio, for most of her life. When shes 16, Hakuya, a tall, quiet boy, comes to stay with her and its revealed that theyre actually engaged.. watch the dude, no spoilers in summary!"}, {"m_title": "Kakumeiki Valvrave", "r": "6.6", "year": "2013", "votes": "308", "genre": "Action, Drama", "desp": "True Calendar year 71-a new age where 70% of the human population lives in space thanks to the development of Dyson Sphere, a city in space. The world is divided into two main powers: The. watch the anime dude!"}, {"m_title": "The World Ends with You: The Animation", "r": "5.6", "year": "2021", "votes": "200", "genre": "Action, Fantasy", "desp": "An asocial teenager, Neku Sakuraba, mysteriously finds himself a part of the Reapers Game, and must make allies to complete the games missions if he wants to survive."}, {"m_title": "s-CRY-ed", "r": "6.9", "year": "2001", "votes": "1,146", "genre": "Action, Adventure", "desp": "22 years ago, a bizarre uprising occurred in Japans Yokohama region, destroying civilization, killing many, and causing the area to break off from Japans mainland. This region came to be. watch the dude, no spoilers in summary!"}, {"m_title": "The Hentai Prince and the Stony Cat", "r": "6.8", "year": "2013", "votes": "583", "genre": "Comedy, Fantasy", "desp": "A sex obsessed teenage boy wishes to be able to express his thoughts while a girl from his school wishes to be able to conceal her feelings. Both wishes are granted - only for them to want to return to how things were."}, {"m_title": "Armored Trooper VOTOMS", "r": "7.6", "year": "1983", "votes": "205", "genre": "Action, Drama", "desp": "The end of the 100 year war between the planets Bavalant and Melkia does not allow Melkian soldier Chirico Cuvie a moment of peace. His accidental involvement in a clandestine mission. watch the dude, no spoilers in summary!"}, {"m_title": "Bungou to Alchemist: Shinpan no Haguruma", "r": "6.9", "year": "2020", "votes": "71", "genre": "Action, Adventure", "desp": "You are in a world where you can meet famous Japanese historical writers. With a party composed of other writers, you delve into tainted books to purify them, unlock secrets, and gain new allies."}, {"m_title": "Miss Kuroitsu from the Monster Development Department", "r": "6.3", "year": "2022", "votes": "131", "genre": "Comedy, Fantasy", "desp": "Touka Kuroitsu is a research assistant who belongs to the Monster Development Department of the evil organization Agastia. However, the enemy she has to fight right now isnt the heroes of. watch the dude, no spoilers in summary!"}, {"m_title": "Nobunaga teachers young bride", "r": "5.1", "year": "2019", "votes": "92", "genre": "Comedy, Fantasy", "desp": "Nobunaga is a teacher who has waited his whole life to be in a situation like this, which only happens in video games. One day, a 14-year-old girl named Kicho appears and tells you that she is your wife??"}, {"m_title": "SSSS.Dynazenon", "r": "6.8", "year": "2021", "votes": "299", "genre": "Action, Adventure", "desp": "After a chance encounter with a self-proclaimed kaiju tamer, a high school student is drawn into a world of fighting mechas and monsters."}, {"m_title": "Sukippu bîto!", "r": "7.8", "year": "2008", "votes": "1,544", "genre": "Comedy, Drama", "desp": "Kyouko went with her childhood friend, Shou to Tokyo to pursue his dream of becoming an idol. Despite she is supporting him, Shou deceive her. Kyouko vows to get her revenge by defeating him in the entertainment world too."}, {"m_title": "Ai no wakakusa monogatari", "r": "7.5", "year": "1987", "votes": "812", "genre": "Drama, Family", "desp": "Louisa May Alcotts autobiographical account of her life with her three sisters in Concord, Massachusetts in the 1860s. With their father fighting in the American Civil War, sisters Jo, Meg. watch the anime dude!"}, {"m_title": "Clean Freak! Aoyama Kun", "r": "6.5", "year": "2017", "votes": "287", "genre": "Comedy, Sport", "desp": "Aoyama is a first-year student and is a genius football player, and is wanted by talents scouts, but is also obsessed with cleanliness."}, {"m_title": "Rilakkumas Theme Park Adventure", "r": "6.6", "year": "2022", "votes": "218", "genre": "Adventure, Comedy", "desp": "The series depicts an active day of incidents and meetings that take place when Rilakkuma, Korilakkuma, Kiiroitori, and Kaoru go play in an amusement park that is about to close."}, {"m_title": "Kemonozume", "r": "7.6", "year": "2006", "votes": "900", "genre": "Action, Drama", "desp": "Kemonozume Beast Claw is an anime series written and directed by Masaaki Yuasa, animated by Studio Madhouse. Since ancient times a race of flesh eating monsters called shokujinki that can. watch the anime dude!"}, {"m_title": "Dance Dance Danseur", "r": "7.6", "year": "2022", "votes": "220", "genre": "Comedy, Drama", "desp": "Junpei Muraos love affair with ballet began at a young age during his sisters ballet recital, when a male ballet dancer roused his interest with a gripping performance. But for Junpei,. watch the dude, no spoilers in summary!"}, {"m_title": "Virtua Fighter", "r": "7.4", "year": "1995", "votes": "319", "genre": "Action, Adventure", "desp": "An anime adaption of the fighting video game series, Virtua Fighter."}, {"m_title": "Hamtaro", "r": "6.3", "year": "2000", "votes": "2,442", "genre": "Adventure, Comedy", "desp": "The story of hamsters who get together at meetings to talk about their adventures."}, {"m_title": "Hoshi no samidare", "r": "5.4", "year": "2022", "votes": "167", "genre": "Action, Adventure", "desp": "The world is in mortal peril. A high school boy has suddenly given a mission of finding the princess Samidare and protecting the planet from incredible Biscuit Hammer."}, {"m_title": "Magical Warfare", "r": "5.8", "year": "2014", "votes": "310", "genre": "Action, Fantasy", "desp": "Takeshi Nanase is an ordinary high school boy who has a somewhat dark past. Due to certain circumstances, he formed a fake couple with his childhood friend Kurumi Isoshima, but otherwise,. watch the anime dude!"}, {"m_title": "Listen to Me, Girls. Im Your Father!", "r": "7.0", "year": "2012", "votes": "277", "genre": "Comedy, Romance", "desp": "When their parents are killed in a plane crash, three sisters go to live with their college student uncle Yuuta."}, {"m_title": "Kandagawa Jet Girls", "r": "6.4", "year": "2019", "votes": "100", "genre": "Action, Sport", "desp": "Jet Racing is a popular sport where teams of two girls work together. They consist of a Jetter, who pilots the watercraft, and a Shooter, who fires a water gun at rival teams. Aspiring to. watch the dude, no spoilers in summary!"}, {"m_title": "Muv-Luv Alternative", "r": "6.0", "year": "2021", "votes": "64", "genre": "Action, Drama", "desp": "In 1973, the invasion of an extra terrestrial life form, the BETA, began a war that has driven mankind to the brink of extinction. In an attempt to counter the BETAs overwhelming strength. watch the dude, no spoilers in summary!"}, {"m_title": "Dog & Scissors", "r": "6.1", "year": "2013", "votes": "214", "genre": "Comedy", "desp": "Kazuhito Harumi is a boy whose entire life revolves around books. Every day he craves nothing more than to delve into a good book in the comforts of his own apartment. One day, as hes. watch the dude, no spoilers in summary!"}, {"m_title": "Tribe Nine", "r": "5.2", "year": "2022", "votes": "153", "genre": "Action, Adventure", "desp": "In a futuristic era overrun by gangs, a tournament of super-powered baseball takes place to determine which gang shall rule over all."}, {"m_title": "Full Metal Panic! Invisible Victory", "r": "7.5", "year": "2018", "votes": "414", "genre": "Action, Comedy", "desp": "The story picks up after The Second Raid as Amalgams fight against Mithril heats up."}, {"m_title": "Master Keaton", "r": "8.0", "year": "1998", "votes": "196", "genre": "Action, Adventure", "desp": "Taichi Keaton is a half-British half-Japanese archaeologist and SAS veteran of the Falklands War. He solves mysteries and investigates insurance fraud for Lloyds around the world."}, {"m_title": "Princess Jellyfish", "r": "7.8", "year": "2010", "votes": "950", "genre": "Comedy, Romance", "desp": "An insecure young woman, obsessed with jellyfish, lives in a house full of awkward women. Meeting a stylish princess turns her life upside down."}, {"m_title": "Irina: The Vampire Cosmonaut", "r": "6.7", "year": "2021", "votes": "337", "genre": "Drama, Fantasy", "desp": "In an alternate 1960s, a space program recruits a vampire to serve as an astronaut and go into space."}, {"m_title": "Unlimited Psychic Squad", "r": "7.0", "year": "2008", "votes": "299", "genre": "Action, Drama", "desp": "An ex-soldier attempts to infiltrate P.A.N.D.R.A., a supposed criminal organization run by ESPers."}, {"m_title": "Naka no Hito Genome", "r": "6.3", "year": "2019", "votes": "192", "genre": "Comedy, Drama", "desp": "Iride Akatsuki and seven other charismatic game streamers wake up in the world of a mysterious free-to-play game called Naka no Hito Genome. Now they are competing in real-life games with. watch the dude, no spoilers in summary!"}, {"m_title": "Love Live! Sunshine!!", "r": "7.0", "year": "2016", "votes": "542", "genre": "Comedy, Drama", "desp": "The second term has started and the members of Aqours work harder than ever to save their school from being shut down."}, {"m_title": "Basilisk: Ouka Ninpouchou", "r": "4.7", "year": "2018", "votes": "186", "genre": "Action, Drama", "desp": "The battle for succession that continued for three generations of shogun in the Keichou era culminated in a gruesome battle of ninja arts between the Kouga and Iga clans. Amidst a rain of. watch the dude, no spoilers in summary!"}, {"m_title": "Frankenstein Family", "r": "5.8", "year": "2018", "votes": "36", "genre": "Comedy, Drama", "desp": "The series centers on a family where both parents are mad scientists, and where both brother and sister are modded humans called creatures. The family members aim to become a normal family."}, {"m_title": "Lady Georgie", "r": "7.4", "year": "1983", "votes": "503", "genre": "Adventure, Comedy", "desp": "Georgie lives happily in an Australian farm with her family, but she doesnt know she was found in a wood and adopted. A gold bracelet is her only clue to the past. When her adoptive mother. watch the anime dude!"}, {"m_title": "Yakunara Mug Cup mo", "r": "6.7", "year": "2021", "votes": "77", "genre": "Animation", "desp": "The story revolves around high school student Himeno Toyokawa, who joins pottery club to learn more about her mother an incredibly skilled potter renowned for creating exceptional ceramic objects."}, {"m_title": "Adventures of the Little Koala", "r": "7.3", "year": "1984", "votes": "143", "genre": "Family", "desp": "Roobear, a smart and athletic koala, had various adventures along with his friends and family."}, {"m_title": "The Seven Heavenly Virtues", "r": "5.9", "year": "2018", "votes": "69", "genre": "Animation", "desp": "A group of angels sent from Heaven look for a potential Savior to counter-attack the demonic influence being spread by Lucifer and the other emissaries of Hell."}, {"m_title": "Tsuujou Kougeki ga Zentai Kougeki de Ni-kai Kougeki no Okaasan wa Suki Desuka?", "r": "5.2", "year": "2019", "votes": "480", "genre": "Adventure, Comedy", "desp": "Do You Love Your Mom and Her Two-Hit Multi-Target Attacks?"}, {"m_title": "Heroic Age", "r": "7.1", "year": "2007", "votes": "555", "genre": "Action, Adventure", "desp": "Set in a distant future, Heroic Age tells the story of the quest of the human race to find the messianic individual that holds the power to save mankind. This savior, named Age, is the only. watch the anime dude!"}, {"m_title": "Magicians Academy!", "r": "6.7", "year": "2008", "votes": "71", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Do It Yourself!!", "r": "7.1", "year": "2022", "votes": "244", "genre": "Comedy, Drama", "desp": "High school girls working on DIY projects, as they struggle, fail, yet dont give up and see the projects through to their end. The story depicts the lives of those girls as they take the first step toward the future."}, {"m_title": "Heavens Memo Notepad", "r": "6.6", "year": "2011", "votes": "273", "genre": "Drama, Mystery", "desp": "Narumi is an assistant to detective Alice, a NEET teenage girl who stays in her pajamas and operates within her bedroom. She uses her vast network of people and the internet to solve cases."}, {"m_title": "ACCA: 13-Territory Inspection Dept.", "r": "7.0", "year": "2017", "votes": "519", "genre": "Crime, Drama", "desp": "The kingdom of Dowa, which is subdivided into 13 states, is celebrating its monarchs 99th birthday. These 13 states have many agencies that are controlled by the giant organization known. watch the dude, no spoilers in summary!"}, {"m_title": "Aria the Scarlet Ammo", "r": "6.5", "year": "2011", "votes": "739", "genre": "Action, Comedy", "desp": "Kinji, a student of an elite academy that produces Butei - highly skilled soldiers who fight crime in an anarchic society, gets attacked by a criminal who hunts Butei members. Aria, an elite Butei, saves him and they team up."}, {"m_title": "Reign: The Conqueror", "r": "7.0", "year": "1997", "votes": "526", "genre": "Action, Adventure", "desp": "A unique re-telling of the story of Alexander the Great; a powerful warrior King who was prophesied to conquer and destroy the world."}, {"m_title": "Arakawa Under the Bridge", "r": "7.1", "year": "2010", "votes": "639", "genre": "Comedy, Fantasy", "desp": "A young man is drawn by his eccentric new girlfriend into her home, a village of eccentrics who live under a bridge on the Arakawa River."}, {"m_title": "Medabots", "r": "6.7", "year": "1999", "votes": "2,401", "genre": "Action, Adventure", "desp": "10-year-old Ikki Tenrio has always dreamed about becoming a top-ranked medafighter, and now that he has his own medabot he can finally pursue his dream."}, {"m_title": "Mysterious Girlfriend X", "r": "7.1", "year": "2012", "votes": "895", "genre": "Comedy, Mystery", "desp": "A newly transferred girl to Akiras school happens to drool on the desk. The otherwise normal Akira decides to taste it and it is now that things start to change as he will start going out with the girl, Mikoto, whom he seems destined for."}, {"m_title": "Sakigake!! Cromartie Kôkô", "r": "7.6", "year": "2003", "votes": "847", "genre": "Comedy", "desp": "A satire comedy of a misfit high school with punks, robots, gorillas and much more."}, {"m_title": "Sabagebu! Survival Game Club!", "r": "7.3", "year": "2014", "votes": "145", "genre": "Action, Comedy", "desp": "It follows the adventures of the survival game club in an all girls school . The male voice-over gives a lot of funny remarks."}, {"m_title": "Legend of the Galactic Heroes Gaiden", "r": "8.3", "year": "1998", "votes": "259", "genre": "Action, Drama", "desp": "Two military strategic geniuses battle each other in the backdrop of a centuries long galactic civil war."}, {"m_title": "BEM", "r": "6.1", "year": "2019", "votes": "228", "genre": "Horror", "desp": "Three monsters, Bem, who looks like a gangster, Belo, who looks like a boy, and Bela, a witch, act as agents of justice, hoping that they will be allowed to become human. Several episodes. watch the dude, no spoilers in summary!"}, {"m_title": "Dragon Goes House Hunting", "r": "6.1", "year": "2021", "votes": "150", "genre": "Action, Comedy", "desp": "When a dragon fails to live up to the fearsome standards set for him, his family kicks him out. He embarks on a quest to find a new home, but soon finds that life on the road is no place. watch the dude, no spoilers in summary!"}, {"m_title": "This Art Club Has a Problem!", "r": "6.6", "year": "2016", "votes": "393", "genre": "Comedy, Romance", "desp": "Focuses on an art club in a certain middle school, and its members: Subaru Uchimaki, who is a genius at drawing faces, but only wants to draw the perfect 2D wife; Colette, a rich. watch the dude, no spoilers in summary!"}, {"m_title": "Big Order", "r": "4.7", "year": "2016", "votes": "371", "genre": "Action, Drama", "desp": "Ten years ago, a child wished for the destruction of the world. Now, a group of people has the ability to grant their wishes depending on their power. They are called Order User. Before. watch the dude, no spoilers in summary!"}, {"m_title": "Requiem from the Darkness", "r": "7.0", "year": "2003", "votes": "273", "genre": "Horror, Mystery", "desp": "A young aspiring writer sets out on a cross country journey to collect 100 tales of the supernatural. On the way he meets some very strange characters, and learns about much more than folktales."}, {"m_title": "Magical Girl Raising Project", "r": "7.1", "year": "2016", "votes": "348", "genre": "Action, Adventure", "desp": "A social game entitled Magical Girl Raising Project secretly turns some of its players into actual magical girls. But when Nabuka city ends up having sixteen magical girls an. watch the dude, no spoilers in summary!"}, {"m_title": "All Out", "r": "6.7", "year": "2016", "votes": "191", "genre": "Sport", "desp": "Kenji Gion joins the rugby club with his classmate, Iwashimizu, who has a complicated past and sub-captain Hachiouji, who always takes good care of his Club members. Lastly, there is Captain Sekizan, who has overwhelming powers."}, {"m_title": "Magic Kaito", "r": "7.3", "year": "2010", "votes": "344", "genre": "Adventure, Comedy", "desp": "Kaito Kuroba, a normal teenage high school student, stumbles across the secret of his fathers death: his father was not only a magician, but the famous phantom thief Kaitou Kid, and was. watch the dude, no spoilers in summary!"}, {"m_title": "DNA²: Dokokade nakushita aitsu no aitsu", "r": "6.5", "year": "1994", "votes": "357", "genre": "Comedy, Sci-Fi", "desp": "A female time traveler must work with a teenage boy who is in danger of becoming a future menace because of her interference."}, {"m_title": "Peach Girl", "r": "6.9", "year": "2005", "votes": "459", "genre": "Drama, Romance", "desp": "Momo, a high school student, is torn between two love interests, each of whom possess great qualities."}, {"m_title": "Yuki Yuna Is a Hero", "r": "6.8", "year": "2014", "votes": "246", "genre": "Comedy, Drama", "desp": "Yuuna Yuuki lives an ordinary life as a second year middle school student, but shes also a member of the Hero Club, where club activities involve dealing with a mysterious being called Vertex."}, {"m_title": "Space Patrol Luluco", "r": "7.0", "year": "2016", "votes": "656", "genre": "Comedy, Fantasy", "desp": "A normal girl and an alien meet and spark an interstellar romance."}, {"m_title": "Meitantei Conan: Zero no Tea Time", "r": "6.2", "year": "2022", "votes": "270", "genre": "Comedy, Crime", "desp": "An enigmatic man who weaves through light and shadow, he is the strict and demanding superior, the eager apprentice, the manipulative informant. Who is the man known as Zero and which side of himself does he reveal when no one is watching?"}, {"m_title": "White Album 2", "r": "7.3", "year": "2013", "votes": "383", "genre": "Music, Romance", "desp": "White Album 2 orchestrates Harukis final semester with complex romance and exhilarating music, as the curtains of the stage he so desired begin to open."}, {"m_title": "Cardfight!! Vanguard", "r": "5.8", "year": "2011", "votes": "136", "genre": "Action, Fantasy", "desp": "There are now billions of card game players in the world, to an extent where card games have become a necessity, a part of our lives. The protagonist of this story, Aichi Sendo, is a timid. watch the dude, no spoilers in summary!"}, {"m_title": "DearS", "r": "6.2", "year": "2004", "votes": "482", "genre": "Comedy, Drama", "desp": "Aliens girls who crash land on Earth, try to keep the fact that they are slaves a secret by forming a peaceful friendship and homestay program with humanity. This begins to change when one boy meets a certain unusual DearS."}, {"m_title": "Karin", "r": "6.8", "year": "2005", "votes": "436", "genre": "Comedy, Fantasy", "desp": "In the 21st century, a family of vampires settled down on the grounds of Japan and lived a modest life whilst sucking on the blood of humans discreetly. However, the eldest daughter of the. watch the dude, no spoilers in summary!"}, {"m_title": "Hinomaru Sumo", "r": "7.4", "year": "2018", "votes": "233", "genre": "Sport", "desp": "Ushio may be small but hes ready to prove hes got enough heft to throw his weight around in the sumo ring!"}, {"m_title": "Uchi no Maid ga Uzasugiru", "r": "6.2", "year": "2018", "votes": "182", "genre": "Comedy", "desp": "Takanashi Misha is a second grade elementary school girl whose father is Japanese and mother was Russian. After she lost her mother at an early age, she has lived alone with her father. One. watch the anime dude!"}, {"m_title": "2.43 Seiin Koukou Danshi Volley Bu", "r": "5.8", "year": "2021", "votes": "344", "genre": "Drama, Sport", "desp": "Kimichika Haijima was a strong junior high school volleyball team member. After getting into trouble with the team, he moved to his mothers hometown and reunites with his childhood friend, Yuni Kuroba."}, {"m_title": "Magical Sempai", "r": "6.2", "year": "2019", "votes": "620", "genre": "Comedy, Drama", "desp": "Our MC finds out that his school requires him to join a club and during his reluctant search he stumbles upon Tejina-senpai attempting magic tricks in her clubroom. Tejina-senpai has. watch the dude, no spoilers in summary!"}, {"m_title": "Unbreakable Machine Doll", "r": "6.7", "year": "2013", "votes": "411", "genre": "Action, Fantasy", "desp": "During the early 20th century, along with technological advancements, scientists were able to develop sophisticated magic. The combination of science and sorcery was Makinot, circuits made. watch the dude, no spoilers in summary!"}, {"m_title": "Macross Frontier", "r": "7.9", "year": "2008", "votes": "829", "genre": "Action, Music", "desp": "Taking place in 2059 A.D., Macross Frontier chronicles the events of the 25th Frontier fleet, located near the center of the Galaxy. The Frontier comes under attack by a mysterious and. watch the dude, no spoilers in summary!"}, {"m_title": "Pokémon: Diamond and Pearl", "r": "7.6", "year": "2006", "votes": "173", "genre": "Action, Adventure", "desp": "Covering the continuing adventures of series protagonist Ash Ketchum and Pikachu, and his best friend Brock, the two meet a new coordinator named Dawn, who travels with them through Sinnoh and enters Pokemon Contests."}, {"m_title": "Hyperdimension Neptunia", "r": "6.3", "year": "2013", "votes": "216", "genre": "Action, Comedy", "desp": "In the world of Gamindustri, four competing CPU goddesses rule the lands of Lastation, Leanbox, Lowee, and Planeptune. When dark forces threaten their power, theyll have to learn to work. watch the dude, no spoilers in summary!"}, {"m_title": "Atashin chi", "r": "7.8", "year": "2002", "votes": "91", "genre": "Comedy", "desp": "Funny everyday life of a family of four, a middle class working man, a lazy housewife, Mikan, a high school girl who enjoys life but is fed up from her family, Yuzuhiko, Mikans elder brother, intelligent but popular and plays games."}, {"m_title": "World Conquest Zvezda Plot", "r": "6.4", "year": "2014", "votes": "128", "genre": "Comedy, Fantasy", "desp": "Tells the story of how a little girl was able to achieve world conquest with the use of her magical powers and faithful followers."}, {"m_title": "The Book of Bantorra", "r": "6.6", "year": "2009", "votes": "209", "genre": "Action, Fantasy", "desp": "Tatakau Shisho. War and struggle between two seemingly totally opposite groups. The Armed Librarians and the followers of Shindeki Kyoudan. The Armed Librarians are exceptional people with. watch the dude, no spoilers in summary!"}, {"m_title": "Air", "r": "7.0", "year": "2005", "votes": "1,464", "genre": "Drama, Fantasy", "desp": "Yukito Kunisaki is on a journey, seeking a girl with wings who flies in the sky, as mentioned in a childhood tale. During this journey, he settles down in a small town, trying to make some. watch the dude, no spoilers in summary!"}, {"m_title": "Because I Dont Like My Big Brother at All!", "r": "6.0", "year": "2011", "votes": "106", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Sugar Sugar Rune", "r": "6.6", "year": "2005", "votes": "119", "genre": "Comedy, Fantasy", "desp": "Chocolate and Vanilla are two best friends since they were born, they live in a magical place not on earth. When Vanillas mother, the queen, is about to give her throne to Vanilla, Vanilla. watch the anime dude!"}, {"m_title": "Akasaka", "r": "5.7", "year": "2008", "votes": "161", "genre": "Comedy, Drama", "desp": "Juniichi Nagase is a high school sophomore who saves new student Yuuhi from harassment, only to find out that she is to be his future wife!"}, {"m_title": "Nintama Rantarô", "r": "7.1", "year": "1993", "votes": "111", "genre": "Action, Adventure", "desp": "A series of anime featuring the Nintamas and others, centered around the trio of Rantarou, Kirimaru and Shinbei, focusing on their adventures, fun, daily activities of Ninjutsu Academy."}, {"m_title": "Kiba", "r": "7.2", "year": "2006", "votes": "288", "genre": "Action, Adventure", "desp": "Zed, and his crippled friend Noah, are dragged from their dystopian city of Calm to another world, where they become Shard casters, and must prevent the end of the world."}, {"m_title": "Kageki Shojo!!", "r": "7.4", "year": "2021", "votes": "194", "genre": "Comedy, Drama", "desp": "Founded in the Taisho era, the Red Flower Opera Company attracts peoples hearts across generations on a beautiful and gorgeous stage created only by unmarried women. The 100th generation. watch the anime dude!"}, {"m_title": "LBX: Little Battlers eXperience", "r": "7.0", "year": "2011", "votes": "46", "genre": "Action", "desp": "The story is about a boy named yamano ban who wishes to own a LBX toy like his friends. An assistant of his father-who presumed to be dead rushes to him and gives an LBX toy saying it holds both hope and despair of humanity."}, {"m_title": "Fate/kaleid liner Prisma Illya 3rei", "r": "6.3", "year": "2016", "votes": "204", "genre": "Action, Comedy", "desp": "Following the decisive battle with the 8th Class Card, Illya must face the worst when the Ainsworth family comes from a parallel world and takes Miyu back with them."}, {"m_title": "Mama Is a 4th Grader", "r": "7.6", "year": "1992", "votes": "107", "genre": "Drama, Sci-Fi", "desp": "Natsumi Mizuki is a 4th grader whose parents have gone to London. Her Aunt Izumi comes to live with her. Izumi is an aspiring manga writer who is trying to get her first big break, so she. watch the dude, no spoilers in summary!"}, {"m_title": "Captain Tsubasa J", "r": "7.6", "year": "1994", "votes": "1,681", "genre": "Drama, Romance", "desp": "Oliver Atton wants to be a professional football player..."}, {"m_title": "The Lost Village", "r": "5.5", "year": "2016", "votes": "485", "genre": "Drama, Mystery", "desp": "A group of 30 individuals go on a bus tour to Nanaki Village in order to leave behind their normal lives but arrive to find that the village is hiding many secrets."}, {"m_title": "Concrete Revolutio", "r": "5.9", "year": "2015", "votes": "114", "genre": "Action, Adventure", "desp": "In an alternate 1960s history where extraterrestrial, supernatural and paranormal beings reside in Japan, a special bureau works to maintain peace and order."}, {"m_title": "Ghost Sweeper Mikami", "r": "6.9", "year": "1993", "votes": "153", "genre": "Action, Comedy", "desp": "The adventures of a group of exorcists for hire."}, {"m_title": "The Legend of Snow White", "r": "6.7", "year": "1994", "votes": "125", "genre": "Adventure, Fantasy", "desp": "This anime based on the Grimm Brothers most famous fairy tale considerably expands Snow Whites story. Enjoy this fairytale fantasy."}, {"m_title": "Daphne in the Brilliant Blue", "r": "6.9", "year": "2004", "votes": "88", "genre": "Action, Adventure", "desp": "In the distant future, the earth is almost entirely covered by the ocean, with only small islands and floating cities on the surface. 15-year-old Maia Mizuki is turned down by the. watch the dude, no spoilers in summary!"}, {"m_title": "Marmalade Boy", "r": "7.1", "year": "1994", "votes": "488", "genre": "Comedy, Drama", "desp": "Two sets of parents divorce and marry each other, while their children Miki and Yuu are caught in the middle and start falling for each other."}, {"m_title": "Take My Brother Away!", "r": "7.1", "year": "2017", "votes": "188", "genre": "Short, Comedy", "desp": "Teenage siblings Fen and Shi Mao fight about everything every single day. However, when disaster strikes, theyve got each others backs. Hopefully."}, {"m_title": "Sakura Diaries", "r": "6.3", "year": "1997", "votes": "170", "genre": "Drama, Romance", "desp": "Urara likes Touma. Touma likes Meiko. Meiko might like Touma, but Meiko will only date a college boy. So whats an under achiever like Touma to do?"}, {"m_title": "Bâtendâ", "r": "7.1", "year": "2006", "votes": "228", "genre": "Drama", "desp": "At the Eden Hall bar, genius bartender Ryu Sasakura serves divine drinks and helps out his customers with whatever troubles they have."}, {"m_title": "Nyaruko: Crawling with Love", "r": "6.4", "year": "2012", "votes": "326", "genre": "Action, Comedy", "desp": "Actual H.P. Lovecrafts alien god of chaos from Cthulhu Mythos appears to a high school boy in the form of a whimsical girl Nyaruko claiming that its there to protect him from other alien godlike races who want to abduct him."}, {"m_title": "Kings Raid: Successors of the Will", "r": "5.8", "year": "2020", "votes": "162", "genre": "Action, Adventure", "desp": "Its been 100 years since King Kyle buried Ang Mundo. Now in a time of peace, the apprentice knight Kasel enjoys his life without worry. His fate begins to change, however, with news of. watch the dude, no spoilers in summary!"}, {"m_title": "Angolmois: Genkou Kassenki", "r": "7.1", "year": "2018", "votes": "279", "genre": "Action, Drama", "desp": "The story follows a group of prisoners who are exiled to Tsushima Island to help form a first line of defense against the first Mongol invasion of Japan in 1274. They first join the So clan. watch the anime dude!"}, {"m_title": "Needless", "r": "6.6", "year": "2009", "votes": "275", "genre": "Action, Adventure", "desp": "In post apocalyptic Japan, teenager Cruz joins a group of superpower-wielding idiots in hope of avenging his sisters death."}, {"m_title": "Giant Killing", "r": "7.6", "year": "2010", "votes": "255", "genre": "Drama, Sport", "desp": "After years of poor performances for several years in the Japanese professional football league, East Tokyo United (ETU) hire the eccentric Takeshi Tatsumi as manager to try to return this small Tokyo club to its former glory days."}, {"m_title": "Gibiate", "r": "4.2", "year": "2020", "votes": "189", "genre": "Action, Adventure", "desp": "A disease called Gibia has infected the world, turning people into various types of monsters, and a ninja and samurai from Japans Edo Period have traveled through time to stop it."}, {"m_title": "Gravion", "r": "7.1", "year": "2002", "votes": "68", "genre": "Adventure, Comedy", "desp": "The year is now 2041 AD. A new enemy called Zeravire suddenly appears in the solar system, destroying all military installations it comes across. However, a wealthy man named Klein Sandman. watch the dude, no spoilers in summary!"}, {"m_title": "Nabari no Ou", "r": "6.5", "year": "2008", "votes": "312", "genre": "Action, Comedy", "desp": "In the shadows of this world, ninjas fight for control of an ancient technique which holds untold strength. This coveted power dwells within apathetic Miharu, a fact the guy really couldnt. watch the anime dude!"}, {"m_title": "Zoids: Chaotic Century", "r": "7.9", "year": "1999", "votes": "658", "genre": "Action, Adventure", "desp": "In the far reaches of the milkyway, on a planet called Zi, there are powerful machines with superior fighting skills called Zoids."}, {"m_title": "Gon", "r": "6.9", "year": "2012", "votes": "106", "genre": "Animation", "desp": "The Gon series features the adventures of the irascible title character, a tiny fictional dinosaur, as he interacts with the natural world. Gon somehow survived the extinction of his fellow dinosaurs and interacts with paleolithic animals."}, {"m_title": "Mai-HiME", "r": "7.4", "year": "2004", "votes": "812", "genre": "Action, Adventure", "desp": "A group a school girls discover that they have been given the ability to materialize weapons and control robotic beasts called Children to stop an evil organization from their plans of domination. Meanwhile, a darker plot thickens."}, {"m_title": "When Seagulls Cry", "r": "6.4", "year": "2009", "votes": "574", "genre": "Fantasy, Horror", "desp": "A family reunion is held on a secluded island. The dying patriarch wants to see his lost love one last time, so he tries summoning her with black magic and thats when the murders begin. Third installment in the When They Cry series."}, {"m_title": "Ghost in the Shell Arise: Alternative Architecture", "r": "7.3", "year": "2015", "votes": "878", "genre": "Action, Drama", "desp": "A recompilation of the 4 original Ghost in the Shell: Arise movies plus 2 episodes that tie in the story with Ghost in the Shell: The New Movie (2015)."}, {"m_title": "Horizon in the Middle of Nowhere", "r": "5.8", "year": "2011", "votes": "169", "genre": "Action, Comedy", "desp": "The students of the Ariadust Academy must use their unique abilities to gain allies during a reenactment of the 1588 A.D. invasion of England by the Spanish Armada. Meanwhile the class must. watch the anime dude!"}, {"m_title": "Arte", "r": "6.9", "year": "2020", "votes": "427", "genre": "Drama, Romance", "desp": "Arte was born into an aristocratic family in Florence during the 16th century Renaissance era. Being female, it was unheard of for one to become an artist. She is willing to cast aside her aristocratic lifestyle to become an artist."}, {"m_title": "RahXephon", "r": "7.2", "year": "2002", "votes": "1,717", "genre": "Action, Drama", "desp": "Kamina Ayato lives an ordinary life. He goes to school. He lives with his mother. He has schoolmates. But all of that changes one day when massive machines, known as the Dolem, attack the. watch the dude, no spoilers in summary!"}, {"m_title": "Lupin Zero", "r": "7.2", "year": "2022", "votes": "82", "genre": "Adventure, Comedy", "desp": "In 1960s Japan, a thiefs rebellious grandson and a gun-toting delinquent meet for the first time, forging a friendship that will one day achieve global infamy."}, {"m_title": "Blame!", "r": "6.3", "year": "2003", "votes": "464", "genre": "Sci-Fi", "desp": "In the post-apocalyptic future, the earth is buried in an enormous mega-structure created by machines. The last hope for mankind is information on a data disc from an engineer called Cibo."}, {"m_title": "Tonari no Seki-kun: The Master of Killing Time", "r": "7.2", "year": "2014", "votes": "526", "genre": "Comedy", "desp": "In the middle of class at a certain school, a diligent student, has her in-class life turned upside-down when her next-door desk mate Seki creates intricate activities to play at his desk during class."}, {"m_title": "Nyankoi!", "r": "7.1", "year": "2009", "votes": "436", "genre": "Comedy, Fantasy", "desp": "A high-schooler is allergic to cats. One day he accidentally breaks a cat statue and can suddenly understand meows. Cats tell him that he must atone for the broken statue by granting them 100 wishes or hell turn into a cat and die."}, {"m_title": "Little Busters!", "r": "6.7", "year": "2012", "votes": "357", "genre": "Comedy, Drama", "desp": "The adventures of five inseparable childhood friends living in a boarding high school."}, {"m_title": "Genesis of Aquarion", "r": "6.5", "year": "2005", "votes": "273", "genre": "Action, Adventure", "desp": "Humanity is dominated by fear when the Shadow Angels invade after 12,000 years of slumber. There is hope, however - Mechanical Angel Aquarion, a fighting machine powered by the intertwined souls of pilots known as Elements."}, {"m_title": "C", "r": "6.6", "year": "2011", "votes": "492", "genre": "Action, Fantasy", "desp": "The Japanese government was rescued from the brink of financial collapse by the Sovereign Wealth Fund. For its citizens, however, life has not improved, and unemployment, crime, suicide,. watch the dude, no spoilers in summary!"}, {"m_title": "Ulysses: Jeanne dArc and the Alchemist Knight", "r": "5.2", "year": "2018", "votes": "143", "genre": "Action, Fantasy", "desp": "During the Hundred Years War, Montmorency, the son of a nobleman, immerses himself in the study of alchemy and magic at Paris school for knights. However, his everyday life comes to an. watch the dude, no spoilers in summary!"}, {"m_title": "Gregory Horror Show", "r": "7.7", "year": "1999", "votes": "85", "genre": "Comedy, Fantasy", "desp": "A series centered on a creepy hotel, called Gregory House, ran by an anthropomorphic mouse, named Gregory. The hotel has a list of strange guests all wanting to meet you."}, {"m_title": "Kiddy Grade", "r": "6.9", "year": "2002", "votes": "357", "genre": "Action, Adventure", "desp": "Two women carry out missions and fight crime across the galaxy."}, {"m_title": "Yukikaze", "r": "7.2", "year": "2002", "votes": "345", "genre": "Action, Adventure", "desp": "Thirty-three years ago, an alien force known as the JAM invaded Earth through a dimensional portal over Antarctica. Earths forces managed to drive the JAM away to a distant planet. watch the dude, no spoilers in summary!"}, {"m_title": "Pretty Boy Detective Club", "r": "6.4", "year": "2021", "votes": "174", "genre": "Mystery", "desp": "Mayumi Dojima, a second-year student at the exclusive Yubiwa Academy middle school, has lost something - a star she glimpsed just once, ten long years ago. But help is on the way, in the. watch the dude, no spoilers in summary!"}, {"m_title": "Battle Doll Angelic Layer", "r": "6.8", "year": "2001", "votes": "329", "genre": "Action, Comedy", "desp": "A young girl battles her way through the exciting games of Angelic Layer, where players fight with tiny robotic dolls to win the national championship."}, {"m_title": "Persona 4 the Golden Animation", "r": "6.3", "year": "2014", "votes": "235", "genre": "Action, Adventure", "desp": "A young man named Yu Narukami arrives in the town of Yasoinaba, where his uncle lives. For family reasons, he will be transferring into the local high school, Yasogami High. And so begins. watch the dude, no spoilers in summary!"}, {"m_title": "Pet", "r": "6.2", "year": "2020", "votes": "254", "genre": "Crime, Mystery", "desp": "There are people who can get into peoples minds and control their memories. This power is used to eradicate mysteries and cases or worse, to assassinate. The potential of said power is. watch the dude, no spoilers in summary!"}, {"m_title": "Kabukichou Sherlock", "r": "6.2", "year": "2019", "votes": "219", "genre": "Comedy, Drama", "desp": "Shinjuku ward east side - The center of the street with most chaos, Theres a Kabukicho with full of neon lights. When the light is stronger, the shadow is deeper. The story begins when. watch the dude, no spoilers in summary!"}, {"m_title": "Negima!? Magister Negi Magi", "r": "6.1", "year": "2006", "votes": "274", "genre": "Comedy, Drama", "desp": "The story is about negi the son of the thousand master who has to teach a class of an all girls school as a test that proves that he can blend in with society and become a wizard like his father"}, {"m_title": "Tiger Mask W", "r": "6.9", "year": "2016", "votes": "148", "genre": "Action, Drama", "desp": "Two young wrestlers have the same goal; destroy the Tigers Den. One tiger walks the path of light, while the other walks the path of shadows."}, {"m_title": "Deep Insanity: The Lost Child", "r": "5.0", "year": "2021", "votes": "73", "genre": "Action, Sci-Fi", "desp": "Madness and unawakening sleep, Randolph syndrome. This new illness is slowly but steadily approaching humanity, caused by the huge underground world Asylum that appeared in Antarctica.. watch the dude, no spoilers in summary!"}, {"m_title": "The Great Passage", "r": "7.4", "year": "2016", "votes": "480", "genre": "Drama, Romance", "desp": "A major publishing company Genbu Shobo has moved ahead with a publish plan of a new Japanese dictionary named Daitokai. A young sales staff in Genbu Shobo, Mitsuya Majime is poor at. watch the dude, no spoilers in summary!"}, {"m_title": "Igano Kabamaru", "r": "8.9", "year": "1983", "votes": "659", "genre": "Action, Adventure", "desp": "A young ninja who has lived in the country comes to the big city for the first time."}, {"m_title": "Space Cobra", "r": "8.4", "year": "1982", "votes": "814", "genre": "Action, Adventure", "desp": "In the 24th Century, space pirate Cobra, a womanizing vagabond with a mysterious past and a powerful psychogun for a hand, fights the galaxy spanning crime syndicate, The Pirate Guild."}, {"m_title": "If Her Flag Breaks", "r": "6.1", "year": "2014", "votes": "126", "genre": "Comedy, Romance", "desp": "Hatate Souta is a high school boy, who has an ability to visualize flags for future events such as death flag, friendship flag, and romance flag. Despite the flag-viewing ability,. watch the dude, no spoilers in summary!"}, {"m_title": "Gakuen Alice", "r": "7.2", "year": "2004", "votes": "237", "genre": "Comedy, Family", "desp": "Best friends Mikan and Hotaru are separated when Hotaru is scouted by a special school called Alice Academy. Mikan determines to enroll too but Alice Academy is a mysterious place."}, {"m_title": "Girls High", "r": "6.2", "year": "2006", "votes": "119", "genre": "Comedy", "desp": "Eriko, Yuma and Ayano are so thrilled to attend Yamasaki Academy, a private high school. High school is not always what they expected. They are excited, naught, break the rules and give the. watch the anime dude!"}, {"m_title": "Shironeko Project: Zero Chronicle", "r": "5.6", "year": "2020", "votes": "195", "genre": "Action, Fantasy", "desp": "Follow In a world consisting of numerous isles, a young hero from the Astora Isle encounters the adventurer Kyle and follows him on an expedition on the isle."}, {"m_title": "My Sweet Tyrant", "r": "6.5", "year": "2018", "votes": "206", "genre": "Short, Comedy", "desp": "The romantic comedy follows the everyday life of an extremely tsundere (initially aloof and abrasive, but later kind-hearted) boy named Atsuhiro Akkun Kagari and his girlfriend Non. watch the dude, no spoilers in summary!"}, {"m_title": "Emily of New Moon", "r": "7.8", "year": "2007", "votes": "98", "genre": "Drama, History", "desp": "Emily is an orphan who gets sent to live with her relatives on Prince Edward Island after her father dies. In New Moon, she lives with her Aunt Elizabeth, Aunt Laura, and Cousin Jimmy, and. watch the dude, no spoilers in summary!"}, {"m_title": "Tamagotchi!", "r": "8.5", "year": "2009", "votes": "54", "genre": "Comedy", "desp": "Follows the lives of Mametchi and his friends as they progress through each day in TamaTown."}, {"m_title": "The Disappearance of Nagato Yuki-chan", "r": "6.2", "year": "2015", "votes": "442", "genre": "Comedy, Romance", "desp": "A spin-off series based on The Disappearance of Haruhi Suzumiya movie, set in the alternate world where Nagato is just a normal human girl."}, {"m_title": "Chis Sweet Home", "r": "7.8", "year": "2008", "votes": "337", "genre": "Comedy, Family", "desp": "The story follows a cute little cat named Chi and her fun family friendly adventures."}, {"m_title": "Chibi Maruko-chan", "r": "8.1", "year": "1990", "votes": "646", "genre": "Comedy, Drama", "desp": "Maruko Sakura is a young elementary school student growing up with her parents, grandparents and elder sister in this animated series based on the producers childhood in the 60s. As. watch the dude, no spoilers in summary!"}, {"m_title": "Sorcerer Hunters", "r": "6.6", "year": "1995", "votes": "162", "genre": "Adventure, Comedy", "desp": "In Spurna, the Sorcerers are the rulers and tiranize the people. The people have only one hope: The Sorcerers Hunters, a group of people devoted to free Spurna from the evil sorcerers.. watch the dude, no spoilers in summary!"}, {"m_title": "Cop Craft", "r": "6.6", "year": "2019", "votes": "445", "genre": "Action, Crime", "desp": "15 years ago, an unknown hyperspace gate opened over the Pacific. Beyond this gate lies Reto Semaani, a strange alternate world where fairies and demons live."}, {"m_title": "Azur Lane: Slow Ahead!", "r": "7.0", "year": "2021", "votes": "76", "genre": "Comedy, Romance", "desp": "This is a slightly peculiar world where the shipgirls live in. A lively and enjoyable school life for girls from different factions will begin at the naval port of Azur Lane. Living out. watch the dude, no spoilers in summary!"}, {"m_title": "He Is an Ultimate Teacher", "r": "6.1", "year": "2015", "votes": "169", "genre": "Comedy", "desp": "Junichiro Kagami, a prodigy in physics to an ultimate otaku, is forced to work as a teacher. Many adventures follow as he helps each of his students with their personal problems as well as his own."}, {"m_title": "Those Snow White Notes", "r": "7.5", "year": "2021", "votes": "435", "genre": "Drama, Music", "desp": "After the death of Setsus legendary musician grandfather, the guy sets out a journey to Tokyo in order to find his missing sound leaving everything behind."}, {"m_title": "Hortensia Saga", "r": "5.4", "year": "2021", "votes": "167", "genre": "Action, Adventure", "desp": "Hortensia Saga is a medieval fantasy tale of war and chaos."}, {"m_title": "Thunderbolt Fantasy", "r": "7.9", "year": "2016", "votes": "195", "genre": "Action, Adventure", "desp": "Set in an Eastern fantasy setting, Dan Fei and her brother, guardians of a sword known as the Tian Xíng Jin, are pursued by the evil Xuan Gui Zong clan, who seek to obtain the sword for. watch the dude, no spoilers in summary!"}, {"m_title": "MÄR: Märchen Awakens Romance", "r": "7.0", "year": "2005", "votes": "151", "genre": "Action, Adventure", "desp": "Ginta Tiramizu is an average school student but keeps on having weird dreams about an amazing land. One day, something strange happens and Ginta appears in his fantasy. Together with. watch the dude, no spoilers in summary!"}, {"m_title": "Number 24", "r": "6.7", "year": "2020", "votes": "127", "genre": "Animation", "desp": "Natsusa Yuzuki entered university expecting to be the rugby clubs ace, but he can no longer play rugby due to certain circumstances. Ibuki Ueoka is a senior who also quit playing rugby."}, {"m_title": "Pokémon Chronicles", "r": "7.2", "year": "1998", "votes": "543", "genre": "Action, Adventure", "desp": "Various adventures of Trainers in the amazing world Pokémon."}, {"m_title": "Zetman", "r": "6.4", "year": "2012", "votes": "563", "genre": "Action, Sci-Fi", "desp": "The story starts off with showing what players are and showing how Kouga and Jin were friends when they were young than the story makes a leap forward in time and follows the 2 main. watch the dude, no spoilers in summary!"}, {"m_title": "11 Eyes", "r": "5.7", "year": "2009", "votes": "400", "genre": "Action, Fantasy", "desp": "11 eyes follows the childhood friendship of Satsuki Kakeru and Minase Yuka, who grew up together living ordinary lives - until one unexpected day where they are both transported to a different world."}, {"m_title": "Gravitation", "r": "7.1", "year": "1999", "votes": "943", "genre": "Comedy, Drama", "desp": "ll Shuichi ever dreamed about was following in the footsteps of his pop idol, Ryuichi Sakuma and the band Nittle Grasper. Together with his best friend Hiro, Shuichis formed a band called. watch the dude, no spoilers in summary!"}, {"m_title": "Kekkaishi", "r": "7.5", "year": "2006", "votes": "524", "genre": "Action, Adventure", "desp": "Yoshimori and Tokine are Kekkaishis tasked to protect the land of Karusomari. They are constantly pushed to their limits, but they always persevere."}, {"m_title": "Magic Kaito 1412", "r": "7.3", "year": "2014", "votes": "485", "genre": "Adventure, Comedy", "desp": "High school student Kaito loves magic. His father was a genius magician with a secret identity as the Phantom Thief Kid. Unfortunately his father died in an unforeseen accident in a magic. watch the dude, no spoilers in summary!"}, {"m_title": "Martian Successor Nadesico", "r": "7.5", "year": "1996", "votes": "500", "genre": "Action, Comedy", "desp": "Some unknown presence is attacking Earth with dangerous robots and the only thing that can save the day is the crew of the Nadesico: a powerful spaceship."}, {"m_title": "Key: The Metal Idol", "r": "7.0", "year": "1994", "votes": "327", "genre": "Drama, Fantasy", "desp": "Tokiko Key Mimas is a completely human looking robot that lives in a farming valley in present day Japan. Her grandfather has just died in an accident. He makes a tape recording. watch the dude, no spoilers in summary!"}, {"m_title": "Brave 10", "r": "6.4", "year": "2012", "votes": "415", "genre": "Action, Comedy", "desp": "A girl being pursued by assassins runs into a ronin samurai. He saves her and subsequently cant get rid of her, as she begs for him to escort her to a nearby castle. Once there the lord is. watch the anime dude!"}, {"m_title": "Gakuen Heaven", "r": "5.9", "year": "2006", "votes": "301", "genre": "Romance", "desp": "Keita Ito is your average high school boy, but he suddenly finds out that he has been enrolled into Bell Liberty Academy, an elite all boy boarding school. But with all the handsome boys. watch the dude, no spoilers in summary!"}, {"m_title": "Muv-Luv Alternative: Total Eclipse", "r": "6.4", "year": "2012", "votes": "188", "genre": "Action, Drama", "desp": "Since 1973, an invasion of aliens known as BETA upon Earth is driving human civilization into nearly collapse. Facing an enormous mass of enemy force, mankind has developed large humanoid. watch the dude, no spoilers in summary!"}, {"m_title": "Utawarerumono", "r": "7.0", "year": "2006", "votes": "453", "genre": "Action, Adventure", "desp": "An injured man wakes up in a small village after being rescued by an apprentice physician. He has no memory of who he is and a mask he cannot take off, he then decides to live with the. watch the dude, no spoilers in summary!"}, {"m_title": "Mejaa", "r": "8.1", "year": "2004", "votes": "361", "genre": "Sport", "desp": "Honda Goro the son of a famous baseball player loves nothing more than baseball itself. His biggest dream is to show his father that he can become the best pitcher in the world despite all. watch the dude, no spoilers in summary!"}, {"m_title": "Heroman", "r": "7.1", "year": "2010", "votes": "92", "genre": "Action, Sci-Fi", "desp": "One day Joey rescues and fixes a broken toy robot. Joey allows it be in his room. One stormy evening, the toy is hit by a lightning bolt, it gives it life and a reason to exist. To fight for Joey and the dangerous ordeals that shall come."}, {"m_title": "Sweet Blue Flowers", "r": "7.0", "year": "2009", "votes": "340", "genre": "Drama, Romance", "desp": "The trials and tribulations of a gang of schoolgirls as they walk through life together."}, {"m_title": "The Aquatope on White Sand", "r": "7.3", "year": "2021", "votes": "322", "genre": "Drama", "desp": "The anime takes place at Gama Gama Aquarium, a small aquarium in Okinawa, an hours bus ride from Naha. Kukuru Misakino is an 18-year-old high school student who works there, and she knows. watch the dude, no spoilers in summary!"}, {"m_title": "Appare-Ranman!", "r": "6.8", "year": "2020", "votes": "420", "genre": "Adventure", "desp": "In the late 19th century, two Japanese, an inventor and a samurai guard, get stranded in America. They enter a cross-country race from Los Angeles to New York to earn money to get home."}, {"m_title": "Suzuka", "r": "6.8", "year": "2005", "votes": "393", "genre": "Drama, Romance", "desp": "Suzuka is a sports-themed romance comedy that intertwines the pursuit of love and athletics. The story is based around Yamato Akitsuki, a young man from rural Hiroshima Prefecture moving to. watch the anime dude!"}, {"m_title": "Kunoichi Tsubaki no mune no uchi", "r": "6.4", "year": "2022", "votes": "168", "genre": "Comedy, Drama", "desp": "In an isolated female-only village of ninjas, a young girl wants to know what men are like."}, {"m_title": "Joshiraku", "r": "7.1", "year": "2012", "votes": "182", "genre": "Comedy", "desp": "The everyday lives of a group of female rakugo comedians."}, {"m_title": "The Morose Mononokean", "r": "7.1", "year": "2016", "votes": "318", "genre": "Comedy, Drama", "desp": "The start of Hanae Ashiyas high school career has not been easy-he has spent all of the first week in the infirmary, and his inexplicable condition is only getting worse. The cause of his. watch the dude, no spoilers in summary!"}, {"m_title": "Eve no jikan", "r": "7.8", "year": "2008", "votes": "1,381", "genre": "Drama, Sci-Fi", "desp": "In the not too distant future androids have come into common usage. Taking the androids for granted, humans treat them as if they were common everyday tools, while on the other hand, some. watch the dude, no spoilers in summary!"}, {"m_title": "Yo-Kai Watch", "r": "6.2", "year": "2014", "votes": "232", "genre": "Comedy, Family", "desp": "Primary school student Keita Amanos curiosity is as innocent as any other childs his age. But when one day he decides to venture deeper into the forest, he encounters a small and. watch the dude, no spoilers in summary!"}, {"m_title": "Pani poni dasshu!", "r": "6.6", "year": "2005", "votes": "207", "genre": "Comedy", "desp": "Rebecca Miyamoto, an 11-year-old child prodigy, accepts a teaching job at Momotsuki Gakuen (Peach Moon Academy), a high school that can be best described as bizarre!"}, {"m_title": "Romio x Jurietto", "r": "7.1", "year": "2007", "votes": "558", "genre": "Drama, Fantasy", "desp": "Roughly based on Shakespeares original, this fantastical tale is set in Neo Verona that houses the Capulets and their rivals, the Montagues. Amidst the rivalry, Romeo and Juliet fall in love."}, {"m_title": "Psychic Squad", "r": "7.1", "year": "2008", "votes": "62", "genre": "Action, Comedy", "desp": "Kouichi Minamoto has to watch over three adorable young girls: Kaoru, Shiho, and Aoi, who all have super powers called ESP. The Children work for B.A.B.E.L.; an organization dedicated to stopping crimes before they happen."}, {"m_title": "Hanamaru Kindergarten", "r": "7.0", "year": "2010", "votes": "139", "genre": "Comedy", "desp": "Three kindergartners grapple for the attention of their new young teacher."}, {"m_title": "Polar Bears Café", "r": "8.1", "year": "2012", "votes": "293", "genre": "Comedy", "desp": "Many animals work at a local zoo. When they arent working, they relax at a polar bears cafe."}, {"m_title": "Nagasarete Airantou", "r": "6.6", "year": "2007", "votes": "149", "genre": "Comedy, Fantasy", "desp": "A young boy ends up trapped on an deserted island. But hes not alone, as the island hosts a community of beautiful girls who are all after him."}, {"m_title": "Photo Kano", "r": "5.7", "year": "2013", "votes": "137", "genre": "Drama, Romance", "desp": "Kazuya Maeda is a second-year high school student who takes pictures of girls with his camera that he received from his dad. The game focuses on his romantic encounters with various girls. watch the dude, no spoilers in summary!"}, {"m_title": "Steel Angel Kurumi", "r": "6.7", "year": "1999", "votes": "188", "genre": "Adventure, Comedy", "desp": "During Japans Taisho Era (1912-1926), a scientist named Ayanokoji developed the Steel Angel-an artificial humanoid with superhuman physical abilities. While the Imperial Army wanted to use. watch the anime dude!"}, {"m_title": "Drugstore in Another World: The Slow Life of a Cheat Pharmacist", "r": "5.7", "year": "2021", "votes": "189", "genre": "Comedy, Fantasy", "desp": "A salary worker comes to another world and takes it easy with a slow life running a drugstore."}, {"m_title": "Tales of Symphonia: The Animation", "r": "7.2", "year": "2007", "votes": "94", "genre": "Short, Fantasy", "desp": "Tales of Symphonia focuses on the two worlds; one world prospers at the cost of another. In one of these worlds, Colette Brunel and her guardians embark on a journey to save their world. watch the dude, no spoilers in summary!"}, {"m_title": "Hen Zemi", "r": "5.5", "year": "2011", "votes": "37", "genre": "Comedy", "desp": "Nanako Matsutaka is a freshman in college, who has enrolled in a course just so she can get closer to the guy she likes. Unfortunately, shes now taking the Pervert Physiology Course and her classmates are far from ordinary."}, {"m_title": "Seven Senses of the ReUnion", "r": "5.0", "year": "2018", "votes": "164", "genre": "Action, Adventure", "desp": "In the popular MMORPG world Union there existed a legendary party named Subaru. This party, made up of a group of childhood friends and elementary schoolers, exceeded the limits of the. watch the dude, no spoilers in summary!"}, {"m_title": "World Fairy Tale", "r": "8.0", "year": "1994", "votes": "56", "genre": "Adventure, Family", "desp": "Aladdin, Beauty and the Beast, Cinderella, etc. this series includes the many widely known 26 world fairy tales."}, {"m_title": "Requiem of the Rose King", "r": "4.6", "year": "2022", "votes": "283", "genre": "Drama, History", "desp": "The war for the throne of England, as seen through the eyes of the deformed anti-hero Richard III."}, {"m_title": "I Cant Understand What My Husband Is Saying", "r": "6.7", "year": "2014", "votes": "840", "genre": "Comedy, Romance", "desp": "When a normal beautiful woman and a geek get married, how could both lead their life normally like everyone else anymore?"}, {"m_title": "Joshi Kausei", "r": "5.3", "year": "2019", "votes": "90", "genre": "Comedy", "desp": "Who needs dialog when youre this cute? The beautiful (but unlucky) Momoko, the cool, collected Shibumi, and the refreshingly innocent Mayumi star in a silent manga. No speeches, no. watch the dude, no spoilers in summary!"}, {"m_title": "Beatless", "r": "5.6", "year": "2018", "votes": "261", "genre": "Action, Drama", "desp": "With the introduction of an ultra-advanced AI that surpasses human intelligence, beings that mankind is yet to fully comprehend made from materials far too advanced for human technology. watch the dude, no spoilers in summary!"}, {"m_title": "Lord El-Melloi IIs Case Files: Rail Zeppelin Grace Note", "r": "6.9", "year": "2018", "votes": "353", "genre": "Action, Fantasy", "desp": "When Lord El-Melloi II, the later Waver Velvet, gets asked to claim his inheritance over Adra Castle, he travels there with his apprentice Gray to do so. But they are not the only ones. watch the dude, no spoilers in summary!"}, {"m_title": "A3! Season Spring & Summer", "r": "7.0", "year": "2020", "votes": "73", "genre": "Music", "desp": "Mankai Company is a far cry from its glory days as an all-male theater. With only one member left and debt collectors at the door, its no wonder Izumi Tachibana finds herself in over her. watch the dude, no spoilers in summary!"}, {"m_title": "BanG Dream!", "r": "6.7", "year": "2017", "votes": "223", "genre": "Comedy, Drama", "desp": "The series revolves around Kasumi Toyama, a high school student who looking for the Star Beat, a sparkling and exciting sound which she heard while looking up at the night sky during her. watch the dude, no spoilers in summary!"}, {"m_title": "HeartCatch PreCure!", "r": "8.5", "year": "2010", "votes": "79", "genre": "Action, Adventure", "desp": "Tsubomi Hanasaki is determined to become more outgoing. Upon becoming a PreCure, she is ecstatic - until her shyness proves an obstacle. However, with her new friend Erika, she is determined to eventually become a stronger person."}, {"m_title": "Touken Ranbu: Hanamaru", "r": "6.8", "year": "2016", "votes": "68", "genre": "Comedy, Drama", "desp": "The daily lives of warriors who work together to protect evil forces from changing history."}, {"m_title": "Akuma-kun", "r": "7.0", "year": "1989", "votes": "8", "genre": "Adventure, Family", "desp": "The age of the demons has begun. Dr Faust has foreseen this rise of evil. Unfortunately, he is near death and is unable to personally battle this upcoming threat. Faust entrusts a young boy. watch the anime dude!"}, {"m_title": "Revue Starlight", "r": "7.4", "year": "2018", "votes": "255", "genre": "Drama, Music", "desp": "Starlight is a song and dance revue troupe loved throughout the world. Karen and Hikari make a promise with each other when theyre young that one day theyll stand on that stage together.. watch the dude, no spoilers in summary!"}, {"m_title": "Dorei-ku The Animation", "r": "5.8", "year": "2018", "votes": "128", "genre": "Animation", "desp": "What if you could enslave anyone you ever wanted? Well, this comes close. The SCM lets you enslave anyone who is also wearing an SCM, at a price. One must win over the other, at anything, in order for the other to become their slave."}, {"m_title": "She, the Ultimate Weapon", "r": "6.9", "year": "2002", "votes": "710", "genre": "Drama, Romance", "desp": "A teenage boy and girl struggle to stay in love when the girls body is transformed into Japans ultimate weapon."}, {"m_title": "White Album", "r": "5.7", "year": "2009", "votes": "120", "genre": "Drama, Romance", "desp": "Touya Fujii is a normal college student with normal worries. He is also concerned about the amount of time he has to spend with Yuki Morikawa, or rather, the lack of it. Being an up and. watch the dude, no spoilers in summary!"}, {"m_title": "Gugure! Kokkuri-san", "r": "7.3", "year": "2014", "votes": "268", "genre": "Comedy, Fantasy", "desp": "The original manga revolves around Kohina, a little girl who ends up summoning by a Kokkuri-san, a lower-ranking ghost in Japanese folklore. The Kokkuri-san she calls ends up being a. watch the dude, no spoilers in summary!"}, {"m_title": "Spy Room", "r": "7.0", "year": "2023", "votes": "30", "genre": "Action, Comedy", "desp": "A spy recruits a group of inexperienced girls for a dangerous mission."}, {"m_title": "Is the Order a Rabbit?", "r": "7.0", "year": "2014", "votes": "343", "genre": "Comedy", "desp": "Cocoa Hoto is a positive and energetic girl who becomes friends with anyone in just three seconds. After moving in with the Kafuu family in order to attend high school away from home, she. watch the dude, no spoilers in summary!"}, {"m_title": "Green Green", "r": "5.9", "year": "2003", "votes": "333", "genre": "Comedy, Romance", "desp": "A group of guys get sent to what they think, is an all boys school, however they soon realize it is inhabited by women for the summer."}, {"m_title": "Shura no Toki: Age of Chaos", "r": "7.4", "year": "2004", "votes": "144", "genre": "Action, Comedy", "desp": "This anime is about a family called Mutsu they use their fists to fight and there three heroes Mutsu Yakmu and his son Takto and Mutsu Izumi"}, {"m_title": "True Tears", "r": "7.1", "year": "2008", "votes": "531", "genre": "Drama, Romance", "desp": "Living together with the girl he love should be a dream for Nakagami, but the reality is closer to a nightmare. At school, Hiromi is kind and popular. But at home, she is a completely different person. How can he unlock her heart?"}, {"m_title": "Wedding Peach", "r": "6.3", "year": "1995", "votes": "237", "genre": "Comedy, Fantasy", "desp": "Young teenager Momoko Hanasaki is given the power to turn into Wedding Peach, a battle angel charged with protecting love, as well as the Four Sacred Somethings; Four items that hold all. watch the dude, no spoilers in summary!"}, {"m_title": "Kodomo no jikan", "r": "6.8", "year": "2007", "votes": "339", "genre": "Comedy, Drama", "desp": "The story revolves around a twenty-three-year-old grade school teacher named Daisuke Aoki whose main problem is that one of his students named Rin Kokonoe has a crush on him."}, {"m_title": "The Night Beyond the Tricornered Window", "r": "6.0", "year": "2021", "votes": "127", "genre": "Fantasy, Mystery", "desp": "Two psychics team up to solve a string of murders, and along their investigations develop a romantic connection."}, {"m_title": "009-1", "r": "6.4", "year": "2006", "votes": "217", "genre": "Action, Crime", "desp": "What would happen if the Cold War was still at large?"}, {"m_title": "Please Save My Earth", "r": "6.9", "year": "1993", "votes": "262", "genre": "Drama, Fantasy", "desp": "A group of students discover they are aliens reborn on Earth. But their past lives start to overlap with their present ones, leading to drama and heartache."}, {"m_title": "Rozen Maiden", "r": "6.9", "year": "2004", "votes": "718", "genre": "Adventure, Comedy", "desp": "A young boys life is about to change when he finds a doll."}, {"m_title": "Inazuma Eleven: Orion no Kokuin", "r": "7.3", "year": "2018", "votes": "73", "genre": "Sport", "desp": "Orion no Kokuin revolves around a team called Inazuma Japan--composed of the best players of the previous Inazuma Eleven installments--that will act as a Japanese national team and will. watch the dude, no spoilers in summary!"}, {"m_title": "Robotics;Notes", "r": "6.9", "year": "2012", "votes": "399", "genre": "Action, Comedy", "desp": "2019, Divergence Ratio 1.048596. The near future, where a device popularly called the PokeCom, packaged with PhoneDroid OS, spreads, bringing the Augmented Reality world close to existence.. watch the anime dude!"}, {"m_title": "The Ryuos Work Is Never Done!", "r": "6.3", "year": "2018", "votes": "160", "genre": "Action, Comedy", "desp": "The story is about a teenage boy who happens to be a shogi master. One day, a nine-year-old girl turns up at his house, requesting to be taken as his disciple. From there, all kinds of wacky hijinks ensue."}, {"m_title": "Ensemble Stars", "r": "6.8", "year": "2019", "votes": "45", "genre": "Animation", "desp": "Yumenosaki private Academy is a school. That specializes in raising male idols Hokuto Hidaka and Subaru Akehoshi and Makoto Yuuki and Mao Isara together they all form trickstar aiming to be top idols someday and take lesson everyday"}, {"m_title": "Bluer Than Indigo", "r": "7.0", "year": "2002", "votes": "513", "genre": "Comedy, Drama", "desp": "A naive, traditional girl from the country goes to the city to find the boy she was engaged to as a young girl, and the two childhood friends fall in love."}, {"m_title": "Delicious Party Pretty Cure", "r": "6.2", "year": "2022", "votes": "37", "genre": "Action, Adventure", "desp": "In the happy and delicious realm responsible for food all over the world, Cookingdom, the Recipebons are stolen from the Bundles, which protect the preparation procedures of each type of. watch the dude, no spoilers in summary!"}, {"m_title": "One Week Friends", "r": "7.0", "year": "2014", "votes": "542", "genre": "Comedy, Drama", "desp": "Kaori has distanced herself from her peers due to a condition that causes her memories to reset every week, but one of her classmates is determined to become friends with her no matter what."}, {"m_title": "Vampire Knight Guilty", "r": "7.3", "year": "2008", "votes": "1,514", "genre": "Action, Drama", "desp": "Vampire Knight is a story about how Vampires Survive without drinking blood"}, {"m_title": "Digimon Xros Wars", "r": "7.0", "year": "2010", "votes": "286", "genre": "Action, Adventure", "desp": "The sixth generation of Digimon follows a boy and his friends, who have the ability to fuse Digimon together to fight off the evil plaguing our world and the Digimon World."}, {"m_title": "Dolls Frontline", "r": "6.1", "year": "2022", "votes": "73", "genre": "Action, Sci-Fi", "desp": "In a world where female androids have become a new military weapon to use, a squad is tasked to hunt down any girls who have rebelled and turned on humanity."}, {"m_title": "Monster Hunter Stories: Ride On", "r": "7.7", "year": "2016", "votes": "55", "genre": "Action, Adventure", "desp": "In a world where people and monsters are at odds, some learn to coexist. Known as Riders, they awaken the powers of monsters and live alongside them in secrecy. Now, one young boy journeys. watch the dude, no spoilers in summary!"}, {"m_title": "Shadow Star Narutaru", "r": "6.3", "year": "2003", "votes": "149", "genre": "Drama, Fantasy", "desp": "A twelve-year-old girl named Shiina Tamai bonds with a starfish-shaped creature whom she calls Hoshimaru. But she isnt the first one with a creature like that!"}, {"m_title": "Himegoto", "r": "5.0", "year": "2014", "votes": "184", "genre": "Comedy", "desp": "This is a story about a boy who was dressed up as a girl by these creepy men, because his parents had made a large debt under his name. The people of the student council at his high school. watch the dude, no spoilers in summary!"}, {"m_title": "Goodbye, Mr. Despair", "r": "7.5", "year": "2007", "votes": "962", "genre": "Comedy, Drama", "desp": "A pessimistic high school teacher must somehow manage a class of eccentric students."}, {"m_title": "Sanrio Boys", "r": "6.6", "year": "2018", "votes": "25", "genre": "Animation", "desp": "A high school boy meets other high school boys who enjoy the cute characters that make up the Sanrio line of merchandise. It a slice-of-life type of show."}, {"m_title": "Heavy Object", "r": "6.4", "year": "2015", "votes": "198", "genre": "Action, Comedy", "desp": "In the future, warfare has been revolutionized by the invention of mechanical weapons called Objects. These Objects are overwhelmingly powerful, rendering traditional weapons like guns, tanks and fighter jets completely useless."}, {"m_title": "Scan2Go", "r": "6.3", "year": "2012", "votes": "45", "genre": "Action, Adventure", "desp": "Sometime in the near future, in an age in which we have established contact and communications with planets outside our galaxy, Scan2Go has become a huge phenomenon throughout all of outer. watch the dude, no spoilers in summary!"}, {"m_title": "Inazuma Eleven Go 2: Chrono Stone", "r": "7.0", "year": "2012", "votes": "203", "genre": "Animation", "desp": "When soccer comes under threat by a shady organization from the future, Tenma and his team traverse through time to stop the eradication of soccer from history."}, {"m_title": "Transformers: Super-God Masterforce", "r": "7.0", "year": "1988", "votes": "235", "genre": "Adventure, Family", "desp": "A new threat to Earth arises when powerful giant Decepticon Pretenders shaped like Earths beasts escape their ancient prison at the bottom of the ocean. Autobot Pretenders who can turn into human and live among them fight back."}, {"m_title": "The Tower of DRUAGA: The Aegis of URUK", "r": "6.6", "year": "2008", "votes": "279", "genre": "Action, Adventure", "desp": "A demon lord commands his forces from a tower. During the Summer of Anu, they lose their power and human army invades the tower to end their threat. Adventurers who have heard about a. watch the dude, no spoilers in summary!"}, {"m_title": "Beautiful Bones: Sakurakos Investigation", "r": "6.8", "year": "2015", "votes": "396", "genre": "Crime, Drama", "desp": "Shoutarou Tatewaki, a high schooler living in Asahikawa City, Hokkaido, meets Sakurako Kujou, a bone-loving ojou-san who has an ability to analyze specimen. Because of meeting her, he has to accompany her to various cases."}, {"m_title": "Yakitate!! Japan", "r": "7.7", "year": "2004", "votes": "463", "genre": "Comedy, Drama", "desp": "Azuma Kazuma isnt terribly clever, but hes got a good heart and great skill - at baking. Since childhood, hes been on a quest to create the perfect bread to represent Japan. watch the dude, no spoilers in summary!"}, {"m_title": "Bakuten!", "r": "7.3", "year": "2021", "votes": "256", "genre": "Sport", "desp": "The story follows Shoutarou Futaba, a boy whos fascinated with mens rhythmic gymnastics and enrolled at the Ao High School to join the schools team. The story follows a team of unique. watch the dude, no spoilers in summary!"}, {"m_title": "Fist of the North Star 2", "r": "8.1", "year": "1987", "votes": "870", "genre": "Action, Adventure", "desp": "Having disappeared years earlier, Kenshiro returns to help his old friends Lin and Bat confront a string of new threats, including masters of Gento Koken and Hokuto Ryuken."}, {"m_title": "Plastic Model Wrestling Sanshiro", "r": "8.7", "year": "1983", "votes": "399", "genre": "Action", "desp": "A young boy named Sanshiro Sugata battles against other PlaWrestlers using his miniature PlaWrestler pocket robot named Juohmaru."}, {"m_title": "The Racing Brothers, Lets & Go", "r": "7.1", "year": "1996", "votes": "535", "genre": "Adventure, Comedy", "desp": "Two competitive brothers, Go and Retsu, meet a man named Dr. Tsuchiya who gives them two cars and tells them to enter the Mini 4WD competition."}, {"m_title": "Lost Song", "r": "7.0", "year": "2018", "votes": "403", "genre": "Adventure, Drama", "desp": "Rin, a young girl blessed with the power of song goes through on a journey trying to get to the capital to sing, little does she know, theres another girl who also has the power of song named finis."}, {"m_title": "Dogeza de Tanondemita", "r": "4.3", "year": "2020", "votes": "146", "genre": "Comedy", "desp": "The adventures of Suwaru Doge, a shameless pervert who kowtows to beautiful ladies to make them show him their underwear."}, {"m_title": "Thumbelina: A Magical Story", "r": "7.9", "year": "1992", "votes": "74", "genre": "Adventure, Fantasy", "desp": "When a bratty little girl named Maya is read the story of Thumbelina by her mother, she is suddenly shrunk and transported into the world of the book and must find her way back home."}, {"m_title": "Good Luck! Ninomiya-kun", "r": "6.3", "year": "2007", "votes": "106", "genre": "Comedy, Romance", "desp": "Shungo Ninomiyas life takes a turn for the weird when Mayu Tsukimura is sent to live with him. Mayu may seem like a normal teen, but shes a succubus with powers she cant control, and. watch the dude, no spoilers in summary!"}, {"m_title": "Makai Ouji: Devils and Realist", "r": "6.5", "year": "2013", "votes": "140", "genre": "Comedy, History", "desp": "The story revolves around William, an aristocratic familys progeny with rare intellect. One day, his uncle lost his possessions after his business failed. Fearing that his familys name. watch the dude, no spoilers in summary!"}, {"m_title": "Fushigi no kuni no Arisu", "r": "7.0", "year": "1983", "votes": "361", "genre": "Adventure, Family", "desp": "Alices journey to Wonderland begins with a shopping trip to London where she buys a dusty top-hat. Inside this hat she discovers Benny Bunny, a rabbit who escorts the little nosy girl to. watch the dude, no spoilers in summary!"}, {"m_title": "The Irresponsible Captain Tylor", "r": "7.7", "year": "1993", "votes": "469", "genre": "Adventure, Comedy", "desp": "The adventures of a space navy captain who is either an incredibly lucky idiot, or an unorthodox genius in tactics."}, {"m_title": "Daitoshokan no Hitsujikai", "r": "6.3", "year": "2014", "votes": "137", "genre": "Comedy, Drama", "desp": "Kakei goes to library to enjoy reading by himself. He then meets a second-year girl named Shirasaki through the efforts of a so-called shepherd who fulfills peoples wishes. Eventually, his boring life change."}, {"m_title": "Transformers: Victory", "r": "7.3", "year": "1989", "votes": "320", "genre": "Adventure, Family", "desp": "Transformers: Victory begins in the year 2025 when the Cybertron Leader Star Saber defeats the leader of the Destrons, Deszaras and traps his space headquarters in intergalactic dust. The. watch the dude, no spoilers in summary!"}, {"m_title": "A Centaurs Life", "r": "5.4", "year": "2017", "votes": "147", "genre": "Comedy, Fantasy", "desp": "High school is a complicated time for any young centaur, but shy, sweet Himeno wont have to clop through it alone!"}, {"m_title": "Ninjala Anime", "r": "7.6", "year": "2022", "votes": "17", "genre": "Action", "desp": "The year is 20XX. The ninja, who once forged the history of Japan, were scattered across the country during the Meiji Restoration. As these ninja mingled with the other clans, their. watch the dude, no spoilers in summary!"}, {"m_title": "Release the Spyce", "r": "6.6", "year": "2018", "votes": "137", "genre": "Action, Comedy", "desp": "Momo, a high school student in Sorasaki, becomes the newest member of Tsukikage, an intelligence agency working to keep the peace in the city. Tsukikages nemesis is the evil organization Moryo, and it seems a traitor is feeding them info."}, {"m_title": "Captain Tsubasa", "r": "7.8", "year": "1983", "votes": "779", "genre": "Comedy, Drama", "desp": "A talented football-loving 11-year-old elementary school student begins his journey as he moves to a new city and a new school."}, {"m_title": "F-Zero GP Legend", "r": "6.8", "year": "2003", "votes": "139", "genre": "Action, Adventure", "desp": "Based on the classic Nintendo series, GP Legend tells the story of Rick Wheeler, a detective who wakes up from a cryogenic sleep from an accident while pursuing an escaped villain named. watch the dude, no spoilers in summary!"}, {"m_title": "Midori Days", "r": "7.2", "year": "2004", "votes": "696", "genre": "Comedy, Drama", "desp": "A boy wakes up one day to find that his hand has been replaced by a miniature version of a girl who has a crush on him."}, {"m_title": "Dimension W", "r": "6.8", "year": "2016", "votes": "941", "genre": "Action, Sci-Fi", "desp": "In the year 2072, the worlds energy problems seem solved by a network of cross-dimensional electric-field inductors called coils that extract energy from a seemingly infinite source.. watch the dude, no spoilers in summary!"}, {"m_title": "Hozukis Coolheadedness", "r": "7.2", "year": "2014", "votes": "279", "genre": "Comedy, Fantasy", "desp": "The dark comedy revolves around the fierce god aide to the Great King Yama, Hoozuki. Calm and super-sadistic, he tries to resolve problems that often occur in Hell."}, {"m_title": "Hakkenden: Eight Dogs of the East", "r": "6.8", "year": "2013", "votes": "201", "genre": "Animation", "desp": "A boy who is meant to be 18 years old is trapped in a 13 year old kids body after accepting a demon sword after his home village was burnt down. He must find 8 special crystals while trying to survive with the demon in him."}, {"m_title": "Shinmai renkinjutsushi no tenpo keiei", "r": "5.9", "year": "2022", "votes": "126", "genre": "Adventure, Fantasy", "desp": "When young girl Sarasa graduated from the Royal Alchemist Academy, her teacher gifted her a house for her to set up shop. Aiming to become a masterclass alchemist, she gathers materials. watch the dude, no spoilers in summary!"}, {"m_title": "Samurai Harem", "r": "5.9", "year": "2009", "votes": "290", "genre": "Comedy, Drama", "desp": "After years training in the martial arts on remote mountains, teenager Yoichi Karasuma is sent down to begin new training. Hes going to learn about more practical things, like appliances, clothing, socializing and, most especially, girls."}, {"m_title": "Magical Girl Special Ops Asuka", "r": "6.4", "year": "2019", "votes": "174", "genre": "Action, Drama", "desp": "A former magical girl must return to duty when an invading alien army she had defeated years ago comes back for another assault."}, {"m_title": "R-15", "r": "5.7", "year": "2011", "votes": "81", "genre": "Comedy, Romance", "desp": "Takuto Akutagawa has a special ability for writing porn novels. He applies to Hirameki Gakuen a high school which only accepts people with Unique talent. Now he must chase his dream and. watch the dude, no spoilers in summary!"}, {"m_title": "Ginga e Kickoff", "r": "7.7", "year": "2012", "votes": "31", "genre": "Sport", "desp": "After his team was disbanded, Sho Ota is 6th grader determined on finding new members to reassemble the Momotaro Predators."}, {"m_title": "Gokujo.: Gokurakuin Joshikou Ryou Monogatari", "r": "5.5", "year": "2012", "votes": "52", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "No-Rin", "r": "6.3", "year": "2014", "votes": "196", "genre": "Comedy, Romance", "desp": "A farm boy named Kousaka Hata meets his famous platonic love when Yuka Kusakabe retires and moves to his agriculture school."}, {"m_title": "Fairy Ranmaru: Anata no Kokoro Otasuke Shimasu", "r": "6.4", "year": "2021", "votes": "40", "genre": "Action, Fantasy", "desp": "Fairies come to Earth looking to save the hearts of women for their fairy queen."}, {"m_title": "Big Windup!", "r": "7.6", "year": "2007", "votes": "285", "genre": "Comedy, Drama", "desp": "Follows Ren Mihashi, who had previously excelled as his middle school baseball teams ace pitcher, due to his grandfathers connections as its manager. As he graduates to high school, whose. watch the anime dude!"}, {"m_title": "Katsugeki/Touken Ranbu", "r": "6.8", "year": "2017", "votes": "97", "genre": "Action, Adventure", "desp": "A sage travels to the past and collects a set of legendary swords, which he brings to life as a gang of talented but dysfunctional young men, to fight a battle against evil."}, {"m_title": "THE IDOLM@STER CINDERELLA GIRLS", "r": "6.2", "year": "2014", "votes": "143", "genre": "Comedy, Drama", "desp": "Cinderella... a fairytale that every girl dreamed of will come to life in the world of Idols. 14 rookie idols are gathered at a new idol project by 346 Production House called Cinderella. watch the dude, no spoilers in summary!"}, {"m_title": "Warukyûre romantse", "r": "6.2", "year": "2013", "votes": "97", "genre": "Romance, Sport", "desp": "Horses dashing gallantly - Knights challenging their own limits - Magnificent, heroic battles highlight the horseback lance sport of jousting. One of the many people fascinated by this. watch the dude, no spoilers in summary!"}, {"m_title": "Appleseed XIII", "r": "5.2", "year": "2011", "votes": "355", "genre": "Sci-Fi", "desp": "Following World War V, a global-scale conflict fought with non-nuclear weapons that almost halved the earths population, the city-nation of Olympus stands as a beacon of hope in a world of chaos."}, {"m_title": "Yozakura Quartet: Hana no Uta", "r": "7.1", "year": "2013", "votes": "76", "genre": "Action, Fantasy", "desp": "There is a town where humans and demons co-exist. In order to maintain peace there is a fantastic foursome of unique teenagers, each gifted with an amazing super power named Hiizumi Life Counseling Office."}, {"m_title": "No Game No Life Specials", "r": "5.4", "year": "2014", "votes": "326", "genre": "Short, Fantasy", "desp": "Picture drama specials included on the Blu-ray/DVD volumes. Showing the main protagonists of the series No Game, No Life in various interesting, funny and sometimes even suggestive every. watch the dude, no spoilers in summary!"}, {"m_title": "Rymans Club", "r": "7.1", "year": "2022", "votes": "172", "genre": "Sport", "desp": "Mikoto Shiratori is a childhood prodigy at badminton that lost in high school; after joining the sales department of the Sunlight Beverage company, he starts playing badminton again."}, {"m_title": "Chios School Road", "r": "7.5", "year": "2018", "votes": "382", "genre": "Comedy", "desp": "Chio makes her way to school but encounters abnormal situations resulting in her creating unique solutions."}, {"m_title": "Emma: A Victorian Romance", "r": "7.7", "year": "2005", "votes": "282", "genre": "Drama, History", "desp": "Emma, a maid, falls in love with an upper-class man, against his familys wishes."}, {"m_title": "Rainbow Days", "r": "6.8", "year": "2016", "votes": "375", "genre": "Short, Comedy", "desp": "The series details the lives of four main characters named Natsuki, Tomoya, Keiichi, and Tsuyoshi. They spend their days trying to have fun by finishing their studies and debating about. watch the dude, no spoilers in summary!"}, {"m_title": "Agatha Christies Great Detectives Poirot and Marple", "r": "7.7", "year": "2004", "votes": "44", "genre": "Adventure, Crime", "desp": "A teenage girl divides her time between working in London for the famed Belgian sleuth, Hercule Poirot, and staying in the countryside with her shrewd great-aunt, Jane Marple."}, {"m_title": "Ghastly Prince Enma Burning Up", "r": "6.6", "year": "2011", "votes": "264", "genre": "Action, Comedy", "desp": "In 1970s Japan demons are attacking the human world, Prince Enma and the Demon Patrol are sent from Hell to stop them."}, {"m_title": "Corpse Princess: Kuro", "r": "7.1", "year": "2009", "votes": "288", "genre": "Action, Drama", "desp": "sorry no summary not available!"}, {"m_title": "The Landlord is in Puberty", "r": "6.0", "year": "2016", "votes": "85", "genre": "Comedy", "desp": "Maeda has just moved into his new apartment to live by himself. While unpacking, a cute middle school student named Chie Satonaka suddenly appears in his room. She introduces herself as his. watch the anime dude!"}, {"m_title": "Cosplay Complex", "r": "5.3", "year": "2002", "votes": "33", "genre": "Comedy", "desp": "Chako and her friends are members of the struggling Cosplay Association club at East Oizuka Academy. They dream of going all the way to the Cosplay World Series. Can these girls, with the. watch the dude, no spoilers in summary!"}, {"m_title": "Gundam Build Divers", "r": "7.0", "year": "2018", "votes": "101", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Green Legend Ran", "r": "6.7", "year": "1992", "votes": "329", "genre": "Drama, Fantasy", "desp": "Set against the backdrop of a post-apocalyptic wasteland caused by an alien invasion, a young orphan must overcome warfare, revenge and basic survival following his mothers murder."}, {"m_title": "High School Fleet", "r": "6.6", "year": "2016", "votes": "117", "genre": "Action, Drama", "desp": "Living by the sea, protecting the sea, and journeying across the sea, that is the life of the Blue Mermaid, group of women trained to operate navy warships to protect international peace. watch the dude, no spoilers in summary!"}, {"m_title": "Blade & Soul", "r": "6.0", "year": "2014", "votes": "243", "genre": "Adventure, Fantasy", "desp": "Alka is an assassin for the Clan of the Sword. Shes on a journey to find the woman Jin Valel, who killed her master Hon. Jin is demon-like woman who controls the Energy of Chaos and kills. watch the dude, no spoilers in summary!"}, {"m_title": "Ef: A Tale of Memories.", "r": "7.4", "year": "2007", "votes": "815", "genre": "Comedy, Drama", "desp": "A story of love, heartbreak and youth, as the stories of 6 different characters, which seem initially unrelated, begin to intertwine."}, {"m_title": "Soreike! Anpanman", "r": "7.5", "year": "1988", "votes": "61", "genre": "Action, Family", "desp": "sorry no summary not available!"}, {"m_title": "The Idolm@ster", "r": "6.6", "year": "2011", "votes": "209", "genre": "Drama, Music", "desp": "The arduous struggle of the 13 girls from Talent Production 765 who dream of one day becoming idols in the competitive Japanese entertainment industry."}, {"m_title": "Clockwork Planet", "r": "5.3", "year": "2017", "votes": "201", "genre": "Fantasy, Sci-Fi", "desp": "A black box crashes through the roof of Naoto Miura, a brilliant amateur tinkerer. Inside the box is a mysterious female automaton. They must work together to avoid the destruction of their city."}, {"m_title": "The Eccentric Family", "r": "7.6", "year": "2013", "votes": "390", "genre": "Comedy, Drama", "desp": "In Kyoto there are three kinds of residents: humans, tanuki and tengu. Shimogamo Yasaburou is the third son of the Shimogamo tanuki family. His father was eaten by members of the Friday. watch the dude, no spoilers in summary!"}, {"m_title": "Sasamekikoto", "r": "6.9", "year": "2009", "votes": "278", "genre": "Comedy, Romance", "desp": "The story takes place in a high school. The main character,Murasame Sumika is secretly in love with her best friend,Kazama Ushio. Kazama is interested in other girls for years but they. watch the dude, no spoilers in summary!"}, {"m_title": "Uta no prince-sama - maji love 1000%", "r": "6.5", "year": "2011", "votes": "365", "genre": "Comedy, Drama", "desp": "With dreams of becoming a composer and someday writing a song for her favorite idol, Haruka Nanami enters the Saotome Academy, a prestigious performing arts school. If successful there,. watch the dude, no spoilers in summary!"}, {"m_title": "Bananya", "r": "6.3", "year": "2016", "votes": "189", "genre": "Comedy, Family", "desp": "The daily adventures of Bananya, a cat with dreams to become a chocolate-covered banana, and his friends."}, {"m_title": "Video Girl Ai", "r": "7.5", "year": "1992", "votes": "711", "genre": "Comedy, Drama", "desp": "Yotas heart is thoroughly broken one afternoon, when Moemi, the love of his life confesses her love for his best friend Takashi. With a hope of cheering himself up, he rents a video,. watch the dude, no spoilers in summary!"}, {"m_title": "Cutie Honey F", "r": "7.0", "year": "1997", "votes": "76", "genre": "Action, Adventure", "desp": "Honey Kisaragi is your everyday beautiful high school student but her father is kidnapped by the evil organization Panther Claw. Her scientist father left her a device that lets her transform into Cutie Honey to battle the forces of evil."}, {"m_title": "Mekakucity Actors", "r": "6.5", "year": "2014", "votes": "490", "genre": "Action, Comedy", "desp": "Shintaro Kisaragi was a NEET who shut himself inside his room for 2 years with a cyber girl named Ene live inside his computer. One day, Ene messes Shintaros computer, which forces. watch the dude, no spoilers in summary!"}, {"m_title": "Gundam Build Divers Re: Rise", "r": "7.7", "year": "2019", "votes": "63", "genre": "Action, Adventure", "desp": "Set two years after the story of Gundam Build Divers, a new version of the Gunpla Battle Nexus Online (GBN) game launches."}, {"m_title": "First Love Monster", "r": "4.6", "year": "2016", "votes": "154", "genre": "Comedy, Drama", "desp": "Nikaido Kaho is a daughter of a rich family and her whole life everyones been nice to her. When moving to her new high school dormitory, she almost gets hit by a truck but is saved by a. watch the dude, no spoilers in summary!"}, {"m_title": "BlazBlue: Alter Memory", "r": "5.8", "year": "2013", "votes": "134", "genre": "Action, Fantasy", "desp": "In the year 2199, humanity eagerly waits for the dawn of the new century, following the end of a series of devastating magic wars. When the worlds most wanted man, Ragna the Bloodedge (. watch the anime dude!"}, {"m_title": "Chibi Revenger", "r": "7.2", "year": "2021", "votes": "14", "genre": "Comedy", "desp": "The short two minute long specials center on the daily lives of the various Tokyo Revengers (2021) characters."}, {"m_title": "Megazone 23 III", "r": "6.4", "year": "1989", "votes": "249", "genre": "Action, Adventure", "desp": "Eiji Takanaka is a B Level hacker who has been accepted into EX. However, before his first day on the job, he becomes embroiled in a network war, a legendary figure named EVE, and a plot to re-inhabit the Earth."}, {"m_title": "07-Ghost", "r": "6.9", "year": "2009", "votes": "577", "genre": "Action, Drama", "desp": "Young amnesiac finds out that the army he was forced to join destroyed his fathers kingdom. He finds shelter in a church and discovers that demons are trying to invade the world. He must find his past, vengeance and save the world."}, {"m_title": "World War Blue", "r": "5.4", "year": "2012", "votes": "82", "genre": "Action, Fantasy", "desp": "A parody of the console wars, the series tells the story of two nations, the Segua Kingdom and Ninteldo Empire, locked in a struggle for dominance over the land of Consume."}, {"m_title": "The Law of Ueki", "r": "7.2", "year": "2005", "votes": "397", "genre": "Action, Adventure", "desp": "A young student named Kousuke Ueki gets chosen to battle in a tournament. Kousuke has the power to turn garbage into trees and will fight against other students."}, {"m_title": "Detective School Q", "r": "8.1", "year": "2003", "votes": "302", "genre": "Adventure, Comedy", "desp": "A young and enthusiastic student anticipates joining the countries most proven detective school where he finds incredibly talented friends and various mysteries lurking ahead."}, {"m_title": "Oneechan ga Kita", "r": "5.7", "year": "2014", "votes": "137", "genre": "Comedy", "desp": "The Tomoyas Father get married again after his first wife (and Tomoyas biologic mother) past away. But with this, he gets a big sister named Ichika. And she, falls in love with her new little brother."}, {"m_title": "Phoenix", "r": "7.5", "year": "2004", "votes": "77", "genre": "Action, Drama", "desp": "Throughout history, from the distant past to the far future, men seek to capture the Phoenix, the source of eternal life. But what does success truly mean?"}, {"m_title": "S.A: Supesharu ê", "r": "7.2", "year": "2008", "votes": "999", "genre": "Comedy, Drama", "desp": "Her whole life, Hikari Hanazono has been consumed with the desire to win against her school rival in S.A class, Kei Takishima at anything. However, Keis feelings towards Hikari are different."}, {"m_title": "The Case Files of Jeweler Richard", "r": "6.8", "year": "2020", "votes": "202", "genre": "Mystery", "desp": "The novel follows Richard Ranashinha Dvorpian, a handsome British jewelry appraiser, and Seigi Nakata, a bright and upright Japanese college student. The duo solves various jewel-related cases by unraveling the hidden ."}, {"m_title": "Orenchi no Furo Jijou", "r": "6.4", "year": "2014", "votes": "225", "genre": "Comedy, Fantasy", "desp": "A lonely man meets a handsome but homeless merman and brings him home to live with him in his bathtub."}, {"m_title": "Anonymous Noise", "r": "6.2", "year": "2017", "votes": "220", "genre": "Drama, Music", "desp": "Every day, a young girl wearing a mask stands by the beach and sings a nostalgic melody. After experiencing two sudden heart-wrenching partings when she was only a child, Nino Arisugawa has. watch the anime dude!"}, {"m_title": "Hungry Heart: Wild Striker", "r": "8.1", "year": "2002", "votes": "303", "genre": "Sport", "desp": "The story of Kano Kyosuke, a teenage high school student who at the beginning of the series, has just transferred into Jyoyo Orange High School. Kyosukes older brother is the illustrious. watch the dude, no spoilers in summary!"}, {"m_title": "Iwa Kakeru! Sport Climbing Girls", "r": "6.2", "year": "2020", "votes": "225", "genre": "Sport", "desp": "Konomi Kasahara, a junior high school girl, has made a big name after winning numerous competitive puzzle game tournaments. The story begins when she finds a sports climbing club in her. watch the dude, no spoilers in summary!"}, {"m_title": "Kiteretsu Daihyakka", "r": "8.4", "year": "1987", "votes": "50", "genre": "Comedy, Drama", "desp": "Kiteretsu is the genius kid who makes new invention. Accompanied by robot korusùkè and his 3 friends."}, {"m_title": "Visual Prison", "r": "7.1", "year": "2021", "votes": "66", "genre": "Music", "desp": "A powerful vampire musical contest under the Scarlet Moon. The winner is bestowed the ultimate power that can change the world."}, {"m_title": "+tic Elder Sister", "r": "6.6", "year": "2011", "votes": "264", "genre": "Short, Comedy", "desp": "The crazy antics of the girls belonging to a plastic model club."}, {"m_title": "Eureka Seven: Ao", "r": "6.7", "year": "2012", "votes": "420", "genre": "Action, Drama", "desp": "In a technologically advanced future, a young boy accidentally activates the strange giant robot his mother piloted before she went missing. He becomes a pilot with only one mission - to find his mother."}, {"m_title": "Samurai Deeper Kyo", "r": "6.7", "year": "2002", "votes": "511", "genre": "Action", "desp": "Renowned and feared swordsman Demon Eyes Kyo endures harsh duels with powerful enemies in search for his original body, as he struggles to maintain power of the body he is temporarily in,. watch the dude, no spoilers in summary!"}, {"m_title": "Assassination Classroom: Koro-sensei Q!", "r": "6.7", "year": "2016", "votes": "275", "genre": "Action, Comedy", "desp": "A spin-off of Assassination Classroom."}, {"m_title": "Maria Watches Over Us", "r": "6.8", "year": "2004", "votes": "271", "genre": "Comedy, Drama", "desp": "Sachiko has offered to be Yumis soeur, the most popular girl at the school as her sister and guide for all her years at the academy. The whole idea has Yumi completely flustered because they hardly know each other."}, {"m_title": "Wizard Barristers: Benmashi Cecil", "r": "6.4", "year": "2014", "votes": "90", "genre": "Crime, Fantasy", "desp": "The story takes place in Tokyo in 2018, where normal humans and those who can use magic coexist. As a result, there are laws against using magic, and a magic courtroom exists to preside. watch the dude, no spoilers in summary!"}, {"m_title": "Hamatora", "r": "6.8", "year": "2014", "votes": "365", "genre": "Action, Fantasy", "desp": "The detective duo of Nice and Murasaki, otherwise known as Hamatora, wait again today for work with their friends at a table at Cafe Nowhere that they use as their agency. Suddenly, a job. watch the anime dude!"}, {"m_title": "Qualidea Code", "r": "6.3", "year": "2016", "votes": "90", "genre": "Action, Drama", "desp": "Approximately thirty years ago,the Earth is suddenly attacked by the- so-called Unknown, tried to overthrow the human race and forced the world to collapse."}, {"m_title": "Getter Robo Armageddon", "r": "7.4", "year": "1998", "votes": "158", "genre": "Sci-Fi", "desp": "Sent to jail after being accused of murdering Getter Robo creator Dr. Saotome, Ryoma Nagare must once again team up with his former comrades and pilot Getter Robo when Saotome returns from. watch the dude, no spoilers in summary!"}, {"m_title": "Battle Girls: Time Paradox", "r": "6.3", "year": "2011", "votes": "74", "genre": "Sci-Fi", "desp": "Yoshino Hide, known as Hideyoshi to her friends, is a typical middle-school student, until shes thrown into another world. This new world is just like the Sengoku period of Japan, except one big difference: There are no men."}, {"m_title": "Dragon Warrior", "r": "7.9", "year": "1989", "votes": "107", "genre": "Action, Adventure", "desp": "A young adventurer and his friends set out to resurrect the Dragon of Legend and defeat the evil Baramos."}, {"m_title": "Chuka ichiban", "r": "7.5", "year": "1997", "votes": "244", "genre": "Action, Comedy", "desp": "After the death of his mother, a young boy travels to China and become a master chef, in his attempt to become head chef of his late mothers restaurant"}, {"m_title": "Najica: Blitz Tactics", "r": "5.9", "year": "2001", "votes": "180", "genre": "Comedy, Crime", "desp": "Najica Hiragi spends her days dreaming up the most alluring scents known to man... and her nights fighting the worlds criminal element as special agent for CRI."}, {"m_title": "Sakura Quest", "r": "7.3", "year": "2017", "votes": "264", "genre": "Comedy, Drama", "desp": "Career plans that didn’t work out lead five different girls to start working at a local tourism bureau where their lives become intertwined."}, {"m_title": "Garo: Vanishing Line", "r": "6.8", "year": "2017", "votes": "190", "genre": "Action, Drama", "desp": "In a prosperous city named Russell City, an omen that threatens to shake its world begins to move within it. A man named Sword is the first to hear the first stirrings of the plot, and. watch the dude, no spoilers in summary!"}, {"m_title": "Destiny of the Shrine Maiden", "r": "6.6", "year": "2004", "votes": "360", "genre": "Action, Drama", "desp": "The highborn Himemiya Chikane is cool and aloof, the perfect young lady of good breeding. Kurusegawa Himeko is devoted to her, and secretly her friend. Their lives are undisturbed in their. watch the dude, no spoilers in summary!"}, {"m_title": "Okami-san and Her Seven Companions", "r": "7.0", "year": "2010", "votes": "478", "genre": "Comedy, Fantasy", "desp": "Ryoko-san and her friends at the Otogi Bank help classmates with their problems, in exchange for their help later on."}, {"m_title": "Parappa the Rapper", "r": "7.6", "year": "2001", "votes": "54", "genre": "Action, Comedy", "desp": "A young rapping dog and his crazy friends go on various adventures."}, {"m_title": "Yatogame-chan Kansatsu Nikki", "r": "5.9", "year": "2019", "votes": "37", "genre": "Animation", "desp": "Kaito Jin is a second year of high school student who moved to Aichi from Tokyo. He meets Monaka Yatogami, who has a heavy Nagoya dialect, and he learns about Nagoya through joining photography club."}, {"m_title": "Cross Game", "r": "8.2", "year": "2009", "votes": "480", "genre": "Romance, Sport", "desp": "5th grader Koh Kitamura lives next door to the Tsukishima family, owners of a batting cage and the Clover cafe. His girlfriend, Wakaba, is the 2nd Tsukishima daughter; the 3rd daughter, the. watch the anime dude!"}, {"m_title": "Genesis Climber Mospeada", "r": "7.6", "year": "1983", "votes": "261", "genre": "Action, Adventure", "desp": "A warrior named Stig Bernard and a bunch of other rebels and soldiers must fight a deadly alien race and track down their ruler."}, {"m_title": "Jouran: The Princess of Snow and Blood", "r": "6.1", "year": "2021", "votes": "294", "genre": "Action, Fantasy", "desp": "In an alternative 1931, in which the Shogunate still exists, a young woman participates in a group of secret agents of the Japanese government with the aim of avenging the murder of her family."}, {"m_title": "Tenchi Muyô! GXP", "r": "7.0", "year": "2002", "votes": "279", "genre": "Adventure, Comedy", "desp": "The misadventures of Tenchis unlucky nephew."}, {"m_title": "Cyborg 009: Call of Justice", "r": "5.9", "year": "2016", "votes": "373", "genre": "Action, Sci-Fi", "desp": "The cyborgs regroup. Donning their combat armor, they fight seriously once again to defeat the power of the storm and its cause, the Cowboy."}, {"m_title": "Le Chevalier DEon", "r": "7.0", "year": "2006", "votes": "379", "genre": "Action, Adventure", "desp": "Its 1742 in Paris, France. Louis XV is still king of France. A woman was found dead in a coffin floating in the Seine River. The brother of the deceased woman is DEon, who is determined. watch the dude, no spoilers in summary!"}, {"m_title": "Mermaid Forest", "r": "7.3", "year": "2003", "votes": "172", "genre": "Adventure, Drama", "desp": "A young man unknowingly eats a piece of mermaids flesh and becomes immortal. For a long time he searches for other mermaids hoping theyll grant him mortality. Finally, he finds them, but theyre about to eat a girl and he cant let that happen."}, {"m_title": "Oshi ga Budôkan ittekuretara shinu", "r": "7.2", "year": "2020", "votes": "105", "genre": "Animation", "desp": "Two years ago, Eri was hypnotized by Maina Ichii and her pop group ChamJam. Now the desire to see them at the Budokan burns inside of her."}, {"m_title": "Hanebado: The Badminton Play of Ayano Hanesaki!", "r": "6.2", "year": "2018", "votes": "377", "genre": "Drama, Sport", "desp": "Ayano Hanesaki, a first-year student at Kanagawa Prefectural Kitakomachi High School, has a badminton ability able to surpass others effortlessly yet avoids playing the sport. She meets. watch the dude, no spoilers in summary!"}, {"m_title": "Magical Girl Lyrical Nanoha", "r": "6.8", "year": "2004", "votes": "202", "genre": "Action, Drama", "desp": "Nanoha Takamachi has a very ordinary life as a third grader at Seishou Elementary School. She lives with her parents who run a cafe, and her older brother and sister."}, {"m_title": "The Ambition of Oda Nobuna", "r": "6.5", "year": "2012", "votes": "499", "genre": "Action, Comedy", "desp": "Suddenly finding himself in the Sengoku era, regular high school student Yoshiharu Sagara is about to be killed in the battlefield. He is saved by none other than the man who later becomes. watch the dude, no spoilers in summary!"}, {"m_title": "Super Dimensional Fortress Macross II: Lovers Again", "r": "6.0", "year": "1992", "votes": "648", "genre": "Action, Music", "desp": "A new splinter race of the Zentraedi show up: the Marduk. While the Zentraedi were defeated by Lynn Minmays music, the Marduk have their own singers (emulators), spur their soldiers into. watch the dude, no spoilers in summary!"}, {"m_title": "Mach Go Go Go", "r": "6.9", "year": "1997", "votes": "36", "genre": "Animation", "desp": "Remake of the anime Mach Go Go Go (1967)."}, {"m_title": "In The Beginning: The Bible Stories", "r": "7.8", "year": "1997", "votes": "33", "genre": "Adventure, Biography", "desp": "An anime series based on the Old Testament in the Bible, up until the birth of Jesus Christ."}, {"m_title": "Le portrait de petite cossette", "r": "6.6", "year": "2004", "votes": "527", "genre": "Fantasy, Horror", "desp": "Eiri, a talented art student, works part-time in an antique shop in a quiet part of town. One afternoon, he uncovers a delicate Venetian glass that holds a startling secret. When Eiri peers. watch the anime dude!"}, {"m_title": "Konohana Kitan", "r": "7.2", "year": "2017", "votes": "253", "genre": "Drama, Fantasy", "desp": "A fox girl starts working at a hot spring inn for spirits."}, {"m_title": "The File of Young Kindaichi Returns", "r": "7.7", "year": "2014", "votes": "82", "genre": "Mystery", "desp": "One day, Hajime and his girlfriend Miyuki visit Hong Kong for a fashion event, where Miyuki is mistaken for someone else and kidnapped. Trying to save Miyuki will lead to an altogether different criminal case."}, {"m_title": "Asteroid in Love", "r": "6.8", "year": "2020", "votes": "226", "genre": "Comedy, Drama", "desp": "When she was little, Mira Kinohata met a boy named Ao at a campsite in town. While gazing at the starry sky together, Mira learns that theres a star with the same name as herself, but no. watch the dude, no spoilers in summary!"}, {"m_title": "Kannagi: Crazy Shrine Maidens", "r": "6.9", "year": "2008", "votes": "212", "genre": "Comedy, Drama", "desp": "For his art project Mikuriya Jin carved a statue from the recently chopped local sacred tree. As he is about to bring it to school, the goddess of the tree, Nagi, descends upon the statue,. watch the dude, no spoilers in summary!"}, {"m_title": "Occultic;Nine", "r": "6.4", "year": "2016", "votes": "297", "genre": "Mystery, Sci-Fi", "desp": "This paranormal science story follows nine idiosyncratic individuals, linked by the Choujou Kagaku Kirikiri Basara occult summary blog run by 17-year-old second-year high school student. watch the anime dude!"}, {"m_title": "Saiyuki Reload: Zeroin", "r": "6.4", "year": "2022", "votes": "35", "genre": "Action, Adventure", "desp": "when their latest assailants turn out to be humans, the group is left dumbfounded. The mystery only deepens when they encounter a wandering priest, Hazel Grouse, and his imposing bodyguard."}, {"m_title": "Baby & Me", "r": "7.8", "year": "1996", "votes": "582", "genre": "Comedy, Drama", "desp": "After his mothers death, 11-year-old Takuya is forced to undertake the enormous task of raising his baby brother Minoru while their father toils away at work to provide for them."}, {"m_title": "New Dominion Tank Police", "r": "6.8", "year": "1993", "votes": "554", "genre": "Short, Action", "desp": "The year is 2016. The place is New Port City. The Tank Police are still waging a desperate war on crime, and their tactics are still highly questionable. Mayor Weatherby is trying to pass a. watch the anime dude!"}, {"m_title": "Mai otome", "r": "7.2", "year": "2005", "votes": "207", "genre": "Action, Adventure", "desp": "My-Otomes story takes place in the distant future on the planet Earl, colonized by immigrants from Earth centuries ago. Old technology has survived in the form of nanomachines that allow. watch the anime dude!"}, {"m_title": "Shakunetsu Kabaddi", "r": "7.0", "year": "2021", "votes": "504", "genre": "Action, Sport", "desp": "Based on the manga by Hajime Musashino about a former soccer player who gets drawn into the highly competitive world of a full-contact sport that originates in India."}, {"m_title": "ChäoS;HEAd", "r": "5.9", "year": "2008", "votes": "752", "genre": "Horror, Mystery", "desp": "A high school student who stumbles on a murder scene spirals into paranoid delusions as he becomes convinced the police are after him."}, {"m_title": "Buso Renkin", "r": "6.8", "year": "2006", "votes": "567", "genre": "Action, Adventure", "desp": "A high-schooler saves a strange girl from a monster, but gets killed. The girl revives him with a magic medallion that also creates a weapon for its owner to fight these monsters. They team up to stop the monsters and their master."}, {"m_title": "Sakura Wars", "r": "7.0", "year": "2000", "votes": "109", "genre": "Action, Adventure", "desp": "The year is around the 1920s and the darkness of the demons have arrived. Now, only the people with large amounts of spirit energy can save the Earth. The Flower Division is now going to. watch the dude, no spoilers in summary!"}, {"m_title": "Mitsudomoe", "r": "7.4", "year": "2010", "votes": "189", "genre": "Comedy", "desp": "Three sisters create chaos at their elementary school."}, {"m_title": "Idoly Pride", "r": "6.5", "year": "2021", "votes": "121", "genre": "Drama, Fantasy", "desp": "Mana Nagase, who worked with her friend Makino, became one of the greatest aspiring idols during her time. Now, a few years later, her younger sister, Kotono, joins a girl named Sakura and ten others on an idol journey of a lifetime."}, {"m_title": "The Comic Artist and Assistants", "r": "6.4", "year": "2014", "votes": "384", "genre": "Comedy", "desp": "Yuuki Aito is a perverted man obsessed with love and drawing panties, putting his assistant, Ashisu, in awkward situations."}, {"m_title": "Zombie-Loan", "r": "6.1", "year": "2007", "votes": "288", "genre": "Action, Drama", "desp": "This clever, modern take on the zombie genre follows Michiru Kita, who possesses a gift that allows her to see whos about to die. When she discovers that two of her classmates are both. watch the dude, no spoilers in summary!"}, {"m_title": "Pocos Udon World", "r": "7.5", "year": "2016", "votes": "264", "genre": "Comedy, Drama", "desp": "Souta Tawara, A young web designer comes to his hometown to escape the busy life of the city. He then stumbles upon a Young kid only to be surprised by the fact that he is a Tanuki (a. watch the dude, no spoilers in summary!"}, {"m_title": "Jyu oh sei", "r": "6.9", "year": "2006", "votes": "478", "genre": "Drama, Mystery", "desp": "Thor and his younger brother Rai get sent to a planet for the worst criminals in the Bulken system, where they must survive to see the future of their life"}, {"m_title": "Heroines Run the Show: The Unpopular Girl and the Secret Task", "r": "6.7", "year": "2022", "votes": "110", "genre": "Music", "desp": "Hiyori left her hometown to continue pursuing her passion of track, enrolls in Tokyo High School. In searching for a part-time job, she ends up an apprentice manager for classmates Someya and Shibasaki, members of the idol group LIPxLIP."}, {"m_title": "The Silver Guardian", "r": "5.8", "year": "2017", "votes": "113", "genre": "Action, Adventure", "desp": "High school students and pro-level gamers Riku Suigin and Riku Rei are about to get swept up in an RPG unlike any other theyve played before."}, {"m_title": "Attacker You!", "r": "6.6", "year": "1984", "votes": "443", "genre": "Action, Family", "desp": "Mila is a girl who grew up in the countryside and moves with her father to Tokyo where she joins the Hikawa team, discovering that she has a lot of talent for volleyball."}, {"m_title": "Momo Kyun Sword", "r": "6.0", "year": "2014", "votes": "35", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "D4DJ", "r": "8.3", "year": "2020", "votes": "94", "genre": "Comedy, Music", "desp": "Upon returning to Japan, freshman high school student Rinku Aimoto enrolls at Yoba Girls Academy and meets Maho Akashi, the schools broadcaster and DJ. Rinku soon develops an interest in. watch the dude, no spoilers in summary!"}, {"m_title": "Genshiken", "r": "7.5", "year": "2004", "votes": "746", "genre": "Comedy", "desp": "The misadventures of the close-knit members of a small anime/manga club on a college campus."}, {"m_title": "A.D. Police", "r": "5.7", "year": "1999", "votes": "137", "genre": "Action, Crime", "desp": "Set a few years before Bubblegum Crisis Tokyo 2040, A.D. Police chronicles the tales of Mega-Tokyos special police division designed to control rogue Boomers in the city. A.D. Police. watch the dude, no spoilers in summary!"}, {"m_title": "Crush Gear Turbo", "r": "7.6", "year": "2001", "votes": "80", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Tatchi", "r": "7.9", "year": "1985", "votes": "380", "genre": "Comedy, Romance", "desp": "The bond between three kids; the talented Kazuya, his lazy do-nothing brother Tatsuya and their childhood friend, the adoring Minami. Baseball, brotherhood, love, and high school challenges and strengthens their connection to one another."}, {"m_title": "Back Arrow", "r": "6.0", "year": "2021", "votes": "218", "genre": "Action, Adventure", "desp": "Lingalind is a land enclosed by the Wall. The Wall covers, protects, cultivates, and nurtures the land. One day in Edger, a village on the outskirts of Lingalind, a mysterious man named Back Arrow appears."}, {"m_title": "Shachiku-san wa Yôjo-yûrei ni iyasaretai.", "r": "7.2", "year": "2022", "votes": "74", "genre": "Comedy, Fantasy", "desp": "From sunup to sundown, Fushihara-san works round the clock. She spends so much time there that even the ghosts worry about her well-being. One night, Fushihara-san was working late as. watch the dude, no spoilers in summary!"}, {"m_title": "Cutie Honey Universe", "r": "5.8", "year": "2018", "votes": "79", "genre": "Action, Comedy", "desp": "An android girl capable of changing her appearance at will helps an investigative agency deal with the criminal organization Panther Claw, whose leader seeks the androids unique shape-changing device."}, {"m_title": "mm!", "r": "6.8", "year": "2010", "votes": "231", "genre": "Comedy, Romance", "desp": "Taro Sado is a masochist and goes to a high school with his male friend Tatsukichi Hayama. Taro falls in love for the first time with a girl and wants to be cured of his masochism so that. watch the dude, no spoilers in summary!"}, {"m_title": "Puraore! Pride of Orange", "r": "6.3", "year": "2021", "votes": "38", "genre": "Sport", "desp": "The story takes place in Nikko city, Tochigi Prefecture, where girls aim for the top of the world through ice hockey."}, {"m_title": "Bodacious Space Pirates", "r": "6.8", "year": "2012", "votes": "163", "genre": "Adventure, Comedy", "desp": "Marika, a high school girl, has been living a normal life in a distant galaxy many years into the future. But this changes when she is told that her recently deceased father was captain of. watch the dude, no spoilers in summary!"}, {"m_title": "Welcome to Irabus Office", "r": "7.4", "year": "2009", "votes": "253", "genre": "Comedy, Drama", "desp": "Many patients with different problems visit the psychiatric ward of Irabu General Hospital; a trapeze artist suffering from insomnia after suddenly failing his jumps, a gangster afraid of. watch the dude, no spoilers in summary!"}, {"m_title": "Sket Dance", "r": "7.5", "year": "2011", "votes": "385", "genre": "Comedy, Drama", "desp": "A high school group Sket-Dan (Helper Group) is dedicated to helping students and staff alike with literally any problem they may have. Partly due to this ambiguity and strange set of. watch the dude, no spoilers in summary!"}, {"m_title": "Karas", "r": "6.9", "year": "2005", "votes": "114", "genre": "Action, Fantasy", "desp": "The world of the humans and the world of the demons (youkai) have overlapped one another, leaving humans to walk the streets of life as they normally would, while demons walk, hidden from. watch the dude, no spoilers in summary!"}, {"m_title": "Lucy-May of the Southern Rainbow", "r": "7.1", "year": "1982", "votes": "302", "genre": "Adventure, Drama", "desp": "A family of English immigrants overcomes many hardships in order to fulfill their dream of starting their own farm in South Australia."}, {"m_title": "Sakigake!! Otokojuku", "r": "7.9", "year": "1988", "votes": "72", "genre": "Action, Comedy", "desp": "Otokojuku: a private school for juvenile delinquents that were previously expelled from normal schools. At this school, Japanese chivalry is taught through feudal and military fundamentals.. watch the anime dude!"}, {"m_title": "Sumomomo Momomo: Chijô Saikyô no Yome", "r": "6.6", "year": "2006", "votes": "123", "genre": "Action, Comedy", "desp": "Koshi, the young heir of a martial arts clan, wishes to lead a peaceful life, but he gets engaged to Momoko, a foolish fighting prodigy. But when a family feud starts, she is the only one who can protect him."}, {"m_title": "Shinderera monogatari", "r": "7.3", "year": "1996", "votes": "183", "genre": "Adventure, Drama", "desp": "The Story of Cinderella opens as Cinderellas life changes for the worse when her father leaves on a business trip. No sooner is he out of sight than Cinderellas stepmother has. watch the dude, no spoilers in summary!"}, {"m_title": "Chis Sweet Adventure", "r": "8.2", "year": "2016", "votes": "65", "genre": "Comedy", "desp": "Chi is a sweet playful kitten whos always full of energy. She lives with the loving Yamada family."}, {"m_title": "Midnight Occult Civil Servants", "r": "6.7", "year": "2019", "votes": "219", "genre": "Fantasy, Mystery", "desp": "Miyako Arata is newly assigned to the Shinjuku Ward Offices Nighttime Regional Relations Department. Each of Tokyos 23 wards has one such department, established to mitigate paranormal. watch the dude, no spoilers in summary!"}, {"m_title": "Castle Town Dandelion", "r": "6.7", "year": "2015", "votes": "144", "genre": "Comedy, Drama", "desp": "The Sakurada family live a normal life just like their father who is the king, wants for them. As members of a royal family, each sibling possesses a superpower and have been designated as a successor to become king through an election."}, {"m_title": "Chaos;Child", "r": "5.6", "year": "2017", "votes": "163", "genre": "Mystery, Sci-Fi", "desp": "Six years after the deadly earthquake and events of ChäoS;HEAd, Tokyos reconstructed Shibuya district becomes home to a familiar series of abnormal events and a string of mysterious deaths. watch the anime dude!"}, {"m_title": "Healer Girl", "r": "7.0", "year": "2022", "votes": "56", "genre": "Comedy, Drama", "desp": "Those that use voice medicine,  the ability to heal injuries and cure illnesses with song, are called healers. There are three first year high school girls are training after school to become full-fledged Healers."}, {"m_title": "Jigoku sensei Nube", "r": "6.9", "year": "1996", "votes": "100", "genre": "Action, Comedy", "desp": "A new teacher who also happens to be an exorcist with a demon hand finds his new school is a hotbed of paranormal and must balance his teaching duties with protecting his students and friends from all manner of supernatural threats."}, {"m_title": "Bokura ga ita", "r": "6.9", "year": "2006", "votes": "441", "genre": "Drama, Romance", "desp": "Nanami Takahashi is a high school student entering her first year. Immediately on her first day however, she becomes the victim of a practical joke by the schools most popular guy,. watch the dude, no spoilers in summary!"}, {"m_title": "Ninja Robots", "r": "8.3", "year": "1985", "votes": "197", "genre": "Action, Adventure", "desp": "Ninja Senshi Tobikage tells the story of a boy named Joe Maya. One day, Joe, who lives on Mars, witnesses a battle between aliens."}, {"m_title": "Dr. Slump", "r": "7.8", "year": "1997", "votes": "471", "genre": "Comedy, Fantasy", "desp": "Professor Senbe Norimaki, aka Doctor Slump, is a bumbling inventor living with his wife and his out-of-control robot daughter, Arale. The trio frequently get involved in surreal adventures. watch the dude, no spoilers in summary!"}, {"m_title": "Rinshi!! Ekoda-chan", "r": "6.8", "year": "2019", "votes": "13", "genre": "Comedy", "desp": "Ekoda-chan is from the countryside, living alone in the city. Shes a temp worker and a strong, carefree soul, with no solid plan for the future, who doesnt care much about guys. She talks. watch the anime dude!"}, {"m_title": "Seven Knights Revolution: The Heros Successor", "r": "6.0", "year": "2021", "votes": "146", "genre": "Action, Fantasy", "desp": "An ordinary boy named Nemo is saved by a girl named Faria from the forces of destruction. During the ensuing battle, Nemo summons the power of a hero and becomes a successor and so an epic journey where the past and present collide begins."}, {"m_title": "Zero no tsukaima F", "r": "7.4", "year": "2012", "votes": "639", "genre": "Comedy, Drama", "desp": "In this fourth and final season of the series, Saito and Louise feel their bond deepening as the days go on, and they feel it might be time to take the next step in their relationship, but. watch the dude, no spoilers in summary!"}, {"m_title": "Smile at the Runway", "r": "6.9", "year": "2020", "votes": "251", "genre": "Comedy, Drama", "desp": "Fujito Chiyuki has a dream: to become a Paris Collection model. The problem is, shes too short to be a model, and everyone around her tells her so. But no matter what they say, she wont. watch the dude, no spoilers in summary!"}, {"m_title": "Scrapped Princess", "r": "7.4", "year": "2003", "votes": "765", "genre": "Adventure, Comedy", "desp": "The adventures of a princess prophesized to destroy the world and her magic using sibling warrior bodyguards determined to protect her."}, {"m_title": "Holmes of Kyoto", "r": "6.6", "year": "2018", "votes": "178", "genre": "Drama, Mystery", "desp": "There is an antique shop in Kyotos Teramachi Sanjou shopping district. High school girl Mashiro Aoi unexpectedly runs into Yagashira Kiyotaka, the son of the shops owner, and ends up. watch the dude, no spoilers in summary!"}, {"m_title": "Haruchika: Haruta & Chika", "r": "6.0", "year": "2016", "votes": "77", "genre": "Mystery, Romance", "desp": "Its Chikas first day of high school, and shes so excited to start a brand new life! Her goal is to become her image of a cute girl. She plans to quit volleyball and start playing the. watch the dude, no spoilers in summary!"}, {"m_title": "Chaos Dragon: Sekiryuu Seneki", "r": "4.5", "year": "2015", "votes": "123", "genre": "Action, Fantasy", "desp": "Based on a table-top roleplaying session involving Gen Urobuchi (Psycho-Pass, Fate/Zero), Ryohgo Narita (Baccano!, Durarara!!), Kinoko Nasu (Kara no Kyōkai, Tsukihime) and more, Chaos. watch the anime dude!"}, {"m_title": "You and Me", "r": "7.7", "year": "2011", "votes": "101", "genre": "Comedy, Drama", "desp": "The high school students in Kimi to Boku. consider their school life to be anything but exciting. A repetitive journey through classes, arguments, and orientations for future careers that. watch the dude, no spoilers in summary!"}, {"m_title": "Utawarerumono: The False Faces", "r": "6.2", "year": "2015", "votes": "126", "genre": "Action, Adventure", "desp": "Anime based on the second game in the Utawarerumono series features the unnamed protagonist who wakes up awkwardly in the middle of a vast snowy plain that he doesnt recognize and cant. watch the dude, no spoilers in summary!"}, {"m_title": "Dive!!", "r": "6.0", "year": "2017", "votes": "67", "genre": "Animation", "desp": "Tomoki Sakai, a 14 years, whole life dedicated to diving. All began upon meeting Yoichi Fujitani, a genius diver, older, three years. Tomoki and his friends attend the MDC, and as Kayoko. watch the dude, no spoilers in summary!"}, {"m_title": "The Flying House", "r": "6.7", "year": "1982", "votes": "169", "genre": "Adventure, Drama", "desp": "The adventures of an inventor whose time-travelling house enables him and his friends to encounter people/events from the Bible."}, {"m_title": "Divergence Eve", "r": "5.8", "year": "2003", "votes": "80", "genre": "Adventure, Comedy", "desp": "In the future, interstellar travel takes place via inflation gates that pass through a parallel universe. Unfortunately, denizens of another universe, called ghouls, want to invade our. watch the dude, no spoilers in summary!"}, {"m_title": "The Kabocha Wine", "r": "8.3", "year": "1982", "votes": "37", "genre": "Comedy, Romance", "desp": "On Transfer student, Aoba Shunsukes first day at Sunshine School he meets a big-sized L(Eru) beauty by the name of Natsumi appeared before him on that day and soon the two of them hit off together."}, {"m_title": "Code: Realize - Guardian of Rebirth", "r": "6.6", "year": "2017", "votes": "116", "genre": "Adventure, Romance", "desp": "When a dashing thief sweeps her away, Cardia will go on an adventure to find her father and answers about her mysterious illness."}, {"m_title": "ClassicaLoid", "r": "7.2", "year": "2016", "votes": "29", "genre": "Comedy, Music", "desp": "Kanae and Sosuke are high school students who live in the medium-sized Japanese city of Hamamatsu. One day, Classicaloid versions of Beethoven and Mozart appear at Otowa Mansion, a large. watch the dude, no spoilers in summary!"}, {"m_title": "Anime san jushi", "r": "7.6", "year": "1987", "votes": "272", "genre": "Action, Adventure", "desp": "Dartagnan leaves Gascone and his grandparents to go to Paris to be a musketeer like his father was..."}, {"m_title": "Arupusu monogatari watashi no Annetto", "r": "7.2", "year": "1983", "votes": "667", "genre": "Drama, Family", "desp": "Little Swiss girl Annettes relationship with her best friend Lucien hits the rocks when he is accused of hurting her brother."}, {"m_title": "Those Who Hunt Elves", "r": "7.0", "year": "1996", "votes": "238", "genre": "Adventure, Comedy", "desp": "An actor, a martial artist, a gun-crazy high school student, and their tank are transported from earth to a world of elves and magic. However, the spell to return them home was botched. watch the dude, no spoilers in summary!"}, {"m_title": "Is Pure", "r": "6.7", "year": "2005", "votes": "87", "genre": "Adventure, Drama", "desp": "On Christmas Eve, a shy schoolboy is caught between the ideal girl and a childhood friend."}, {"m_title": "Police in a Pod", "r": "7.0", "year": "2022", "votes": "182", "genre": "Comedy, Crime", "desp": "A story about two policewomen working at a small police station. Every day is a new challenge."}, {"m_title": "Love Lab", "r": "6.8", "year": "2013", "votes": "202", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Karneval", "r": "6.7", "year": "2013", "votes": "375", "genre": "Fantasy, Mystery", "desp": "Nai searches for someone important to him, with only an abandoned bracelet as a clue. Gareki steals and pick-pockets to get by from day to day. The two meet in a strange mansion where they. watch the dude, no spoilers in summary!"}, {"m_title": "Uragiri wa Boku no Namae o Shitteiru", "r": "6.7", "year": "2010", "votes": "215", "genre": "Adventure, Fantasy", "desp": "Yuki Sakuragi is a soft-hearted high-schooler who was abandoned by his parents as a child and was raised in an orphanage. While cursed by mysterious dreams and the strange ability to read. watch the dude, no spoilers in summary!"}, {"m_title": "Deaimon", "r": "7.2", "year": "2022", "votes": "228", "genre": "Comedy, Drama", "desp": "Nagomu Irino returns to his Kyoto home for the first time in ten years when his father is hospitalized. Nagomu is eager to take over Ryokushou, the familys Japanese sweet shop, but hes. watch the dude, no spoilers in summary!"}, {"m_title": "Macross Delta", "r": "5.9", "year": "2015", "votes": "182", "genre": "Action, Sci-Fi", "desp": "In the year 2067, the Tactical Sound Unit, Walküre,  and the Valkyrie fighter team Delta Flight,  join together to combat a disease that threatens the galaxy, the Var Syndrome."}, {"m_title": "Inazuma Eleven Go: Galaxy", "r": "6.7", "year": "2013", "votes": "170", "genre": "Adventure, Sci-Fi", "desp": "Tenma becomes the captain of Japans national youth team, but for some reason the other members picked are people who know nothing about soccer. Consequently, Tenma and his team must work their way up from the very bottom."}, {"m_title": "Kamichama Karin", "r": "6.8", "year": "2007", "votes": "35", "genre": "Comedy, Fantasy", "desp": "Karin Hanazono is an orphan, living with her relatives. Her life has never been easy, her studies are not well, she is not the best in sports and technically she is miserable. Left with. watch the dude, no spoilers in summary!"}, {"m_title": "Strike Witches: Road to Berlin", "r": "6.8", "year": "2020", "votes": "50", "genre": "Action", "desp": "sorry no summary not available!"}, {"m_title": "Trickster", "r": "6.8", "year": "2016", "votes": "79", "genre": "Drama, Mystery", "desp": "It is the 2030s. The Boy Detectives Club gathers under Kogoro Akechi, the mysterious detective. The group solves cases great and small using their unique skills. One day, a member of the. watch the dude, no spoilers in summary!"}, {"m_title": "Comic Girls", "r": "6.8", "year": "2018", "votes": "249", "genre": "Comedy", "desp": "Moeta Kaoruko ia a 15-year-old high school student and manga creator, who uses the pen name Chaos. After ranking at the bottom of a reader survey, Kaorukos editor recommends that she enter. watch the anime dude!"}, {"m_title": "Toward the Terra", "r": "7.6", "year": "2007", "votes": "294", "genre": "Action, Drama", "desp": "In the future, mankinds seemingly utopian society is strictly controlled by the government, and anything that threatens to disrupt the status quo is ruthlessly suppressed. When 14 year-old. watch the anime dude!"}, {"m_title": "Ninja Slayer", "r": "6.0", "year": "2015", "votes": "89", "genre": "Action, Fantasy", "desp": "When a salarymans wife and child are slain in a ninja war, he fuses with a supernatural entity to take vengeance on all ninjas."}, {"m_title": "Phantom Thief Jeanne", "r": "7.3", "year": "1999", "votes": "387", "genre": "Adventure, Comedy", "desp": "A young Japanese girl discovers she is the reincarnation of Joan dArc and takes on the task of destroying the evil possessing certain artifacts."}, {"m_title": "Divine Gate", "r": "4.9", "year": "2016", "votes": "332", "genre": "Action, Fantasy", "desp": "The Divine Gate ushered in an era of chaos. Now, a select few have gathered to attempt to reach the Gate and remake the world."}, {"m_title": "Bloodivores", "r": "4.4", "year": "2016", "votes": "118", "genre": "Action, Horror", "desp": "Also called Time Prisoners, this is based on a Chinese webtoon by Bai Xiao. Its an alternative world where people researched and found the cure for a mysterious sleep-deprivation sickness. watch the dude, no spoilers in summary!"}, {"m_title": "Hangyakusei Million Arthur", "r": "6.3", "year": "2018", "votes": "49", "genre": "Action, Adventure", "desp": "A squad of avatars of King Arthur are assembled to travel through time and protect history."}, {"m_title": "Black Blood Brothers", "r": "6.9", "year": "2006", "votes": "1,240", "genre": "Action, Comedy", "desp": "Remnants of a vampire clan that wanted to rule the world alone, but got defeated in a war by joined forces of other vampires and humans, kidnap the brother of a vampire war veteran. He looks for them in a secret vampire city."}, {"m_title": "Maria Holic", "r": "6.5", "year": "2009", "votes": "324", "genre": "Comedy, Drama", "desp": "Its not easy to find the perfect girl, even when youre the same sex! Kanako thought attending an all-girls school would be the perfect way to meet the girl of her dreams, but after her. watch the dude, no spoilers in summary!"}, {"m_title": "Tales of Luminaria the Fateful Crossroad", "r": "6.4", "year": "2022", "votes": "20", "genre": "Action, Fantasy", "desp": "When war erupts between two groups drawn to the ancient remains of primordial beasts, both sides will quest for the true meaning of justice."}, {"m_title": "Level E", "r": "6.8", "year": "2011", "votes": "345", "genre": "Comedy, Sci-Fi", "desp": "In the present day, hundreds of extraterrestrial species walk the Earth. Some are pacifistic, others violent. Some are here for research purposes, others are career criminals. However,. watch the dude, no spoilers in summary!"}, {"m_title": "The Ping-Pong Club", "r": "7.6", "year": "1995", "votes": "147", "genre": "Comedy, Sport", "desp": "A schools ping pong club tries to take back their ping pong practice room from rivals despite their differences."}, {"m_title": "Zoids Genesis", "r": "7.9", "year": "2005", "votes": "98", "genre": "Adventure, Sci-Fi", "desp": "A thousand years after natural disasters have devastated the planet Zi humans have re-established civilization, salvaging the ancient Zoids. When his village is attacked a boy activates an ancient Zoid to defend it."}, {"m_title": "Gensomaden Saiyuki", "r": "7.3", "year": "2000", "votes": "244", "genre": "Action, Adventure", "desp": "Demons and man lived together in harmony until a band of rogue demonic forces seek to resurrect a diabolical monster and a dark spiritual power covered the land. Demons no longer see. watch the dude, no spoilers in summary!"}, {"m_title": "Humanity Has Declined", "r": "7.3", "year": "2012", "votes": "309", "genre": "Comedy, Fantasy", "desp": "An unnamed girl acts as a mediator between humans and fairies after they have become the most prosperous species following the decline of the human population."}, {"m_title": "Lady Lady", "r": "7.5", "year": "1987", "votes": "222", "genre": "Drama, History", "desp": "Lynns mother passed away when Lynn was only five and she now lives with her father in England who is not always able to be with her so she tries to be a strong person on her own."}, {"m_title": "Spirit Blade Mountain", "r": "6.7", "year": "2016", "votes": "107", "genre": "Comedy, Fantasy", "desp": "A comet falling will bring about a great calamity, and in order to choose a child fated by the falling comet, the family of the Reiken clan with a long storied history is once again. watch the dude, no spoilers in summary!"}, {"m_title": "Senyuu", "r": "6.7", "year": "2013", "votes": "190", "genre": "Action, Adventure", "desp": "Long ago, there lived an evil being who reigned over the world through fear, called Satan Rchimedes. He invaded the human world with his many demon underlings, spreading madness and chaos. watch the dude, no spoilers in summary!"}, {"m_title": "Cuticle Detective Inaba", "r": "6.5", "year": "2013", "votes": "104", "genre": "Comedy, Crime", "desp": "In a world where half-human, half-animal chimeras live and work alongside normal people, there are sure to be a few bad apples in the bunch. Unfortunately, half-human criminals means. watch the dude, no spoilers in summary!"}, {"m_title": "Arata the Legend", "r": "6.5", "year": "2013", "votes": "125", "genre": "Animation", "desp": "The stage is the world of Amawakuni, where mankind coexists with the many gods. There, a boy called Arata is wrapped up in a coup détat led by the Twelve Shinsho, wielders of swords called. watch the anime dude!"}, {"m_title": "Chibi Maruko-chan", "r": "7.9", "year": "1995", "votes": "172", "genre": "Comedy, Family", "desp": "An animated series following the life of young Maruko Sakura, as she grows up with her parents, grandparents and elder sister. In most episodes, Maruko and her friends hatch odd schemes and. watch the anime dude!"}, {"m_title": "Vividred Operation", "r": "5.7", "year": "2013", "votes": "94", "genre": "Action, Sci-Fi", "desp": "A team of middle school girls fight inter-dimensional beings using the powers of friendship and technology."}, {"m_title": "Super Life-Form Transformers: Beast Wars Neo", "r": "6.7", "year": "1999", "votes": "95", "genre": "Action, Fantasy", "desp": "Big Convoy leads rookie Autobot Maximals against Magmatron and his Decepticon Predacons who are after energy source called Angolmois. However, when ancient evil transformer Unicron and his Blendtrons appear, everything changes."}, {"m_title": "Senki Zessho Symphogear", "r": "6.4", "year": "2012", "votes": "129", "genre": "Action, Music", "desp": "Tsubasa Kazanari and Kanade Amou-the idol duo known as Zwei Wing-use their songs to power ancient weapons known as symphogears to combat a deadly alien race called the Noise. While the. watch the dude, no spoilers in summary!"}, {"m_title": "Aoki Densetsu Shoot!", "r": "7.3", "year": "1993", "votes": "213", "genre": "Action, Comedy", "desp": "In the life as in soccer, one must fight until the end to win."}, {"m_title": "Project ARMS", "r": "6.7", "year": "2001", "votes": "91", "genre": "Action, Adventure", "desp": "Several teenagers discover they have supernatural abilitys, however a top secret government organization wants the power for themselves so they decide to fight back."}, {"m_title": "Pumpkin Scissors", "r": "6.9", "year": "2006", "votes": "256", "genre": "Action, Adventure", "desp": "A ceasefire put an end to the long war between the Royal Empire and the Republic of Frost. Three years later, the Empire is still plagued by starvation, pestilence, and soldiers turning. watch the dude, no spoilers in summary!"}, {"m_title": "Ayashi no Ceres", "r": "7.1", "year": "2000", "votes": "494", "genre": "Adventure, Comedy", "desp": "Now, in modern day time, Aya Mikage is a descendent of Ceres, and has quite an amount of tennyo blood."}, {"m_title": "Kemono Friends", "r": "6.8", "year": "2017", "votes": "279", "genre": "Adventure, Comedy", "desp": "In the large animal park named Japari Park, animals have lived in the shape of human girls by the power of mysterious substances called Sandstar. However, Japari Park has started to. watch the dude, no spoilers in summary!"}, {"m_title": "Servant X Service", "r": "7.0", "year": "2013", "votes": "308", "genre": "Comedy", "desp": "The ward office where Yamagami Lucy has been hired employs many unique characters on its staff. Yamagami and her colleagues are full of energy and secrets as theyre hard at work (?!) in this workplace comedy."}, {"m_title": "Pilot Candidate", "r": "5.7", "year": "2000", "votes": "209", "genre": "Action, Drama", "desp": "The planet Zion is mankinds last hope for survival. But the legions of Victim are intent on destroying humanity! Five humanoid fighting weapons called Ingrids are capable of stopping the. watch the dude, no spoilers in summary!"}, {"m_title": "Madame Peppermint", "r": "6.9", "year": "1983", "votes": "497", "genre": "Comedy, Fantasy", "desp": "Mrs. Pepperpot has a magic spoon that can make her shrink."}, {"m_title": "Vlad Love", "r": "6.0", "year": "2020", "votes": "73", "genre": "Comedy, Fantasy", "desp": "A vampire girl goes to school for the first time and makes friends with humans."}, {"m_title": "Kono Healer, Mendokusai", "r": "5.9", "year": "2022", "votes": "145", "genre": "Action, Adventure", "desp": "A fantasy and adventure comedy starring Karla, a dark elf cleric who invested too much in her insulting skills, and Alvin, a swordsman who has to put up with her antics."}, {"m_title": "Ixion Saga DT", "r": "6.7", "year": "2012", "votes": "149", "genre": "Adventure, Comedy", "desp": "Kon Hokaze was transferred into a game like world, after agreeing to help an unknown woman. now he must go on a journey to find a way back to his home world."}, {"m_title": "Yatterman Night", "r": "6.3", "year": "2015", "votes": "46", "genre": "Action, Adventure", "desp": "Once upon a time, Yattarman who is a heroes of the justice. After having won the fight with Doronbo who is a gang of the evil, Yattarman established their country Yattar-Kingdam. The. watch the dude, no spoilers in summary!"}, {"m_title": "Pokemon Black and White", "r": "6.7", "year": "2010", "votes": "23", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Torappu ikka monogatari", "r": "7.7", "year": "1991", "votes": "297", "genre": "Adventure, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Onigiri", "r": "5.4", "year": "2016", "votes": "36", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "D.N. Angel", "r": "7.0", "year": "2003", "votes": "901", "genre": "Adventure, Comedy", "desp": "Not-so-average teenager, Daisuke Niwa, has had an abnormal life. His mother and grandfather have trained him his whole life by setting up traps that can kill him if he doesnt figure them. watch the dude, no spoilers in summary!"}, {"m_title": "Sound of the Sky", "r": "7.0", "year": "2010", "votes": "264", "genre": "Comedy, Drama", "desp": "15-year old Kanata Sorami enlists in the 1121st Platoon of the Helvetians Army to become a great bugle player, and experiences missions and adventures with her fellow platoon members."}, {"m_title": "Ashita no Nadja", "r": "7.2", "year": "2002", "votes": "117", "genre": "Adventure, Comedy", "desp": "Nadja is a bright, cheerful girl who was raised in an orphanage near London. Before her thirteenth birthday, she found out that her mother might be alive and she sets out on a journey to find her mother."}, {"m_title": "Aura Battler Dunbine", "r": "6.9", "year": "1983", "votes": "128", "genre": "Action, Adventure", "desp": "An aspiring Japanese motocross racer is pulled into the world of Byston Well, where he is forcefully recruited into becoming the pilot of a Dunbine, a giant robot guided by his aura."}, {"m_title": "Wave, Listen to Me!", "r": "7.0", "year": "2020", "votes": "261", "genre": "Comedy, Drama", "desp": "From manga legend Hiroaki Samura, creator of Blade of the Immortal and Die Wergelder, comes something completely different, the journey of one disgruntled young womans journey into the wild rapids of talk radio."}, {"m_title": "Anime-Gataris", "r": "6.1", "year": "2017", "votes": "108", "genre": "Comedy", "desp": "A girl who knows nothing about anime gets roped into joining her schools anime club."}, {"m_title": "Hanaukyo Maid Team: La Verite", "r": "6.6", "year": "2004", "votes": "44", "genre": "Comedy, Romance", "desp": "When Taro Hanaukyos mother passes away, gets to stay at his grandfathers home where he meets a gorgeous maid named Mariel and he finds out his grandfather has gone to a island and is. watch the dude, no spoilers in summary!"}, {"m_title": "KADO: The Right Answer", "r": "6.6", "year": "2017", "votes": "183", "genre": "Drama, Mystery", "desp": "Kojirou Shindou. Cabinet Office Director-General for Policy Planning, is at Haneda Airport for a business trip. While the plane is on the runway, a giant structure suddenly appears out of. watch the dude, no spoilers in summary!"}, {"m_title": "Cerberus", "r": "6.0", "year": "2016", "votes": "79", "genre": "Adventure, Fantasy", "desp": "Sword and magic rule in the continent of Kunaahn. In this continent are three powerful nations: the Holy Kingdom of Amoria, the Kingdom of Ishilfen, and the Kingdom of Vanrodis, which. watch the dude, no spoilers in summary!"}, {"m_title": "Ms. Vampire Who Lives in My Neighborhood", "r": "6.4", "year": "2018", "votes": "163", "genre": "Comedy, Fantasy", "desp": "Amano Akari was looking for the vampires mansion in the woods because people say that there is a young girl there who hasnt aged, but she gets lost. Shes saved by a young girl, Sophie, who turns out to be the vampire shes looking for!"}, {"m_title": "Ganbare! Kikkâzu", "r": "7.4", "year": "1986", "votes": "1,141", "genre": "Comedy, Drama", "desp": "Takeru is a young boy who joins a soccer team who is doing poorly to an all star team."}, {"m_title": "Kai Byoui Ramune", "r": "6.7", "year": "2021", "votes": "183", "genre": "Fantasy", "desp": "As long as hearts exist inside people, there will always be those who suffer. And then something strange enters their mind and causes a strange disease to manifest itself in the body. The. watch the anime dude!"}, {"m_title": "New Adventures of Kimba the White Lion", "r": "6.5", "year": "1989", "votes": "140", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Star Driver", "r": "6.1", "year": "2010", "votes": "191", "genre": "Action, Romance", "desp": "A young boy named Takuto arrives at the island of Southern Cross, and begins to investigate the islands deep-set secrets, to which he is somehow connected."}, {"m_title": "Mirmo Zibang!", "r": "6.6", "year": "2002", "votes": "108", "genre": "Adventure, Comedy", "desp": "This Anime based upon a Magical journey of school girl. Her life plays a twisted when she found a magic cup Of mirmo the prince of magical world."}, {"m_title": "Hitori Bocchis OO Lifestyle", "r": "7.1", "year": "2019", "votes": "445", "genre": "Comedy, Drama", "desp": "In order to reconcile with her childhood friend, a very shy girl aims to befriend everyone in her class."}, {"m_title": "Wangan Midnight", "r": "7.3", "year": "2007", "votes": "237", "genre": "Action, Sport", "desp": "After a crushing defeat to the legendary racer Blackbird, Akio Asakura finds the enigmatic Devil Z car, and is determined to become a street racing legend that will surpass even Blackbird."}, {"m_title": "Kyousogiga", "r": "7.8", "year": "2013", "votes": "94", "genre": "Action, Comedy", "desp": "Hidden within Kyoto is the mirror capital where youkai and humans live together and no one ever dies. It is protected by three people - Kurama, Yase, and Myoue - as they await the return. watch the dude, no spoilers in summary!"}, {"m_title": "Amagami SS+ Plus", "r": "6.7", "year": "2012", "votes": "183", "genre": "Drama, Romance", "desp": "New events begin to take place between each of the girls and Junichi. Tsukasa Ayatsuji, the class representative, runs for student council president; Rihoko Sakurai, who has taken over the. watch the dude, no spoilers in summary!"}, {"m_title": "Gatchaman Crowds", "r": "6.8", "year": "2013", "votes": "242", "genre": "Action, Adventure", "desp": "The story focuses on the Gatchaman, a group of superheroes, who protect earth. The series also focuses on the question on what it means to be a hero."}, {"m_title": "Island", "r": "5.4", "year": "II) (2018", "votes": "163", "genre": "Drama, Romance", "desp": "Urashima, an island far from the mainland. The people who live there lead carefree lives. But five years ago, the islands three great families suffered a series of misfortunes and. watch the dude, no spoilers in summary!"}, {"m_title": "Cross Fight B-Daman ES", "r": "6.6", "year": "2012", "votes": "14", "genre": "Adventure", "desp": "The show takes place in Crest Land, set after the mysterious B-Crystal went berserk at the WBMA Headquarters. The main protagonist, Kamon Day, lives in the south area of Crest Land, and. watch the anime dude!"}, {"m_title": "Koya no Kotobuki Hikotai", "r": "6.8", "year": "2019", "votes": "52", "genre": "Action", "desp": "The adventures of team Kotobuki, a group of girls flying fighter planes."}, {"m_title": "Senryuu Girl", "r": "7.0", "year": "2019", "votes": "385", "genre": "Comedy, Drama", "desp": "A mute girl and a former delinquent bond through their love of senryu poetry."}, {"m_title": "Kyôsôgiga", "r": "7.4", "year": "2013", "votes": "199", "genre": "Fantasy, Sci-Fi", "desp": "Everything was peaceful until Myoue and Koto suddenly vanished. Their three children are left to take care of the city, and Yakushimaru inherits Myoues name and duties. Stranded in this. watch the dude, no spoilers in summary!"}, {"m_title": "Upotte!!", "r": "6.7", "year": "2012", "votes": "97", "genre": "Action, Comedy", "desp": "In this anime, the girls are personifications of firearms. Assault Rifles primarily. The story follows the FNC, M16A4, L85A2, and Sig 550. While the plot itself isnt deep or motivating,. watch the dude, no spoilers in summary!"}, {"m_title": "Rec", "r": "7.0", "year": "2006", "votes": "333", "genre": "Comedy, Romance", "desp": "Fumihiko Matsumaru is an average salaryman with no girlfriend. He invited his colleague Miss Tanaka to a movie but was stood up. Right at the time he was about to toss tickets into a trash. watch the dude, no spoilers in summary!"}, {"m_title": "Psychic Detective Yakumo", "r": "6.6", "year": "2010", "votes": "337", "genre": "Mystery, Thriller", "desp": "In order to help a friend possessed by a spirit, Ozawa Haruka knocks on the door of her universitys movie research association. There she meets Saitou Yakumo, a rude, unkempt young man. watch the dude, no spoilers in summary!"}, {"m_title": "Tottemo! Luckyman", "r": "8.5", "year": "1994", "votes": "23", "genre": "Action, Comedy", "desp": "Focuses on the adventures of a bizarre superhero and his fights against various aliens and other enemies threatening the planet Earth."}, {"m_title": "Kyô kara Ore wa!!", "r": "7.6", "year": "1994", "votes": "146", "genre": "Action, Comedy", "desp": "Two high school students, Mitsuhashi Takashi and Ito Shinji, set out to become the strongest delinquents in Japan."}, {"m_title": "Tegamibachi", "r": "7.1", "year": "2009", "votes": "133", "genre": "Adventure, Fantasy", "desp": "Its an anime where a boy dreams to became a letter bee.....he meets people,obstacles and so on"}, {"m_title": "Samurai Gun", "r": "6.0", "year": "2004", "votes": "89", "genre": "Action, Adventure", "desp": "Samurais use guns to redeem justice."}, {"m_title": "Mobile Suit Gundam AGE", "r": "5.6", "year": "2011", "votes": "147", "genre": "Action, Fantasy", "desp": "A colonized space had finally achieved peace, until a mysterious enemy, the UE, enters the scene and attacks colonies and other targets. Our main protagonist is from a destroyed colony,. watch the dude, no spoilers in summary!"}, {"m_title": ".hack//Roots", "r": "6.1", "year": "2006", "votes": "326", "genre": "Adventure, Fantasy", "desp": "In 2015, CC Corporations data-center caught on fire destroying The World. By splicing in data from what would have been another project, CC Corp. was able to rebuild the game. In 2016,. watch the dude, no spoilers in summary!"}, {"m_title": "Togainu no chi", "r": "5.7", "year": "2010", "votes": "198", "genre": "Action, Drama", "desp": "An undefeated street fighting champion is imprisoned for a crime he didnt commit, but is soon approached by two strangers who promise him freedom. The terms? That he signs up for a deadly battle game called Igura."}, {"m_title": "Chicchana yuki tsukai Shugâ", "r": "7.5", "year": "2001", "votes": "104", "genre": "Adventure, Comedy", "desp": "Season Fairies create and control the weather using special musical instruments. Sugar, an apprentice Snow Fairy, and her friends Salt and Pepper, all want to become full-fledged Season. watch the dude, no spoilers in summary!"}, {"m_title": "Kaze No Yojimbo", "r": "7.4", "year": "2001", "votes": "85", "genre": "Action, Mystery", "desp": "In this modern day reimagining of Akira Kurosawas film Yojimbo, Jyouji (George) Kodama has arrived in Kimujuku, a town distinctly divided by two rival factions, in order to search for a. watch the dude, no spoilers in summary!"}, {"m_title": "Katana Maidens", "r": "5.7", "year": "2018", "votes": "93", "genre": "Action, Fantasy", "desp": "The story is centered around a group of shrine maiden who wield swords while wearing school uniforms known as Toji, and they serve as a unit in the police force in order to exorcise. watch the dude, no spoilers in summary!"}, {"m_title": "Chu-Bra!!", "r": "6.3", "year": "2010", "votes": "35", "genre": "Comedy", "desp": "A middle school girl with a strange but innocent fascination with womens undergarments tries to start a school club for underwear."}, {"m_title": "Occult Academy", "r": "6.6", "year": "2010", "votes": "208", "genre": "Action, Comedy", "desp": "The story revolves around Maya, the daughter of the former headmaster of a private Japanese academy dedicated to the study of the occult who died in 1999; and a time traveling agent Uchida,. watch the anime dude!"}, {"m_title": "The Secret Garden Animated", "r": "7.6", "year": "1991", "votes": "132", "genre": "Drama", "desp": "At a very young age, Mary loses her parents to the disease, leaving her traumatized and orphaned. From India, she is brought to England with her uncle who has taken on the role of her new guardian."}, {"m_title": "Hakugei: Legend of the Moby Dick", "r": "7.3", "year": "1997", "votes": "32", "genre": "Adventure, Sci-Fi", "desp": "The year in 4699 and spaceships are drifting in space that go by the name Whales. Lucky is a young child who go to Ahab and want shim to rescue Mohd, which is his home planet."}, {"m_title": "Popotan", "r": "6.4", "year": "2003", "votes": "131", "genre": "Comedy, Drama", "desp": "Three sisters are bound to a mysterious house capable of teleporting through time and space. In their travels through twentieth century Japan with their robotic maid, they encounter the. watch the dude, no spoilers in summary!"}, {"m_title": "Yume no Hoshi no Button Nose", "r": "8.2", "year": "1985", "votes": "39", "genre": "Adventure, Family", "desp": "The series focused on Button Nose, a little girl who had been left in charge of her home while her parents were away. One night, a robot came to her house to pick up some strawberries. When. watch the anime dude!"}, {"m_title": "The Familiar of Zero: Knight of the Twin Moons", "r": "7.2", "year": "2007", "votes": "884", "genre": "Comedy, Drama", "desp": "In this continuation of the previous season, not long after the war, Louise and Saito seemingly getting along now since she knows that he had given up his only chance to return to Tokyo for. watch the anime dude!"}, {"m_title": "NieA under 7", "r": "7.1", "year": "2000", "votes": "313", "genre": "Comedy, Sci-Fi", "desp": "A poor Earth girl and NieA, a hotheaded alien girl of low class (under 7), bond in a world where aliens are part of every day society."}, {"m_title": "Hidamari Sketch", "r": "7.0", "year": "2007", "votes": "161", "genre": "Comedy", "desp": "An anime shows life of four students of artistic separation of school of Yamabuki taking off an accommodation in Hidamari."}, {"m_title": "Persona: Trinity Soul", "r": "6.3", "year": "2008", "votes": "152", "genre": "Action, Adventure", "desp": "The stage is Ayanagi City, a city near the Japan Sea. It is a futuristic city that was built to carry out the recovery from the calamity caused by the Apathy Syndrome ten years previous.. watch the dude, no spoilers in summary!"}, {"m_title": "Asura Cryin", "r": "6.0", "year": "2009", "votes": "118", "genre": "Action, Comedy", "desp": "A boy whose friend died and became a ghost, starts high school. On his first day, a trunk is sent to him by his brother. From that moment on everyone are after him believing that he holds the mysterious powerful Asura Machine."}, {"m_title": "The Twins at St. Clares", "r": "7.9", "year": "1991", "votes": "77", "genre": "Adventure, Comedy", "desp": "The day by day of two twin sisters in a college for girls in the England of 19th Century."}, {"m_title": "SD Gundam Force", "r": "4.1", "year": "2003", "votes": "129", "genre": "Comedy, Family", "desp": "A Gundam series with super deformed robots."}, {"m_title": "Sekkô Boys", "r": "6.1", "year": "2016", "votes": "78", "genre": "Comedy", "desp": "The statues of Medici, Mars, Hermes and St. Giorgio are trying to become professional idols. Are they going to make it to the top or will they fail?"}, {"m_title": "C³: C Cube", "r": "6.1", "year": "2011", "votes": "137", "genre": "Action, Comedy", "desp": "Yachi Haruaki is a high school boy who is naturally resistant to curses. After his father sends him a mysterious black cube, Haruaki awakes to find a nude girl named Fear standing in his. watch the dude, no spoilers in summary!"}, {"m_title": "Mujin wakusei Survive", "r": "8.2", "year": "2003", "votes": "182", "genre": "Adventure, Fantasy", "desp": "A group of teenagers from a deep space colony become lost when their shuttle is sucked into a gravity storm, and end up on a habitable, but, for some reason, uninhabited planet. Now its up. watch the anime dude!"}, {"m_title": "22/7", "r": "6.6", "year": "2020", "votes": "57", "genre": "Drama, Music", "desp": "One day, Miu Takigawa suddenly receives a letter notifying her that she has been chosen as a member of a brand-new project. Half in disbelief, she heads over to the location stated on the. watch the dude, no spoilers in summary!"}, {"m_title": "Yamibo: Darkness, the Hat & the Travelers of the Books", "r": "6.5", "year": "2003", "votes": "66", "genre": "Fantasy, Romance", "desp": "This Japanese anime series chronicles the incredible journey of Hazuki through many fantasy worlds in search of her one true love. Hazuki is a high school girl who lives with her adopted. watch the dude, no spoilers in summary!"}, {"m_title": "Encouragement of Climb", "r": "6.9", "year": "2013", "votes": "144", "genre": "Adventure, Comedy", "desp": "As a child, Aoi Yukimura adored mountains and was passionate about climbing. However, a playground accident has since left her afraid of heights, turning her toward indoor hobbies.. watch the dude, no spoilers in summary!"}, {"m_title": "Baby Steps", "r": "7.7", "year": "2014", "votes": "394", "genre": "Comedy, Romance", "desp": "Maruo Eiichirou (Ei-Chan), a first year honor student, one day decides hes unhappy with the way things are and lacks exercise. He finds a flyer for the Tennis Club and decides to check it. watch the dude, no spoilers in summary!"}, {"m_title": "New Captain Tsubasa", "r": "7.6", "year": "1989", "votes": "969", "genre": "Drama, Sport", "desp": "Captain tsubasa in japan national team"}, {"m_title": "The SoulTaker", "r": "6.0", "year": "2001", "votes": "151", "genre": "Action, Horror", "desp": "Kyosuke Date is stabbed by his mother and left for dead. He is buried alive, but is soon dug up by a mysterious girl. He searches for answers about why he was stabbed, looks for a long lost. watch the anime dude!"}, {"m_title": "Girl Beats Boys", "r": "6.6", "year": "2017", "votes": "104", "genre": "Comedy", "desp": "Hinako Nakayama is a high school girl, that by the request of her twin brother Hikaro to switch places with him in his school, she enters an all-boys school which is filled with delinquents."}, {"m_title": "Gloomy the Naughty Grizzly", "r": "5.0", "year": "2021", "votes": "31", "genre": "Comedy", "desp": "Gloomy, a bear was rescued by a little boy named Pity. At first, the bear was very cute but he was getting wilder as he grow up and forgot that he shouldnt attack humans. Thats why he started attacking his owner Pity."}, {"m_title": "Magical Angel Creamy Mami", "r": "7.2", "year": "1983", "votes": "248", "genre": "Adventure, Comedy", "desp": "Yuu Morisawa, an ordinary schoolgirl, leads a double life as the magical singer Creamy Mami."}, {"m_title": "Tiâzu to tiara", "r": "6.5", "year": "2009", "votes": "194", "genre": "Action, Adventure", "desp": "In a world resembling the middle age a girl Riannon is set to be sacrificed to appease a ressurected demon Lord arawn as her brother attempts to rescue her"}, {"m_title": "Kyôran kazoku nikki", "r": "6.8", "year": "2008", "votes": "23", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Glasslip", "r": "5.0", "year": "2014", "votes": "191", "genre": "Drama, Romance", "desp": "Touko often watches a hallucination in the glitters. In her last summer holidays of a school, Kakeru who is strange transfer student appears before her and her friends. Kakeru tells Touko that he often hears voice from future."}, {"m_title": "Master of Martial Hearts", "r": "4.4", "year": "2008", "votes": "109", "genre": "Action, Comedy", "desp": "Several hot girls with big boobs fight in a tournament."}, {"m_title": "Shutsugeki! Machine Robo Rescue", "r": "7.6", "year": "2003", "votes": "30", "genre": "Family, Sci-Fi", "desp": "Twelve children are selected to become part of Machine Robo Rescue, an organization that rescue people from a variety of dangers."}, {"m_title": "Jewelpet", "r": "8.1", "year": "2009", "votes": "44", "genre": "Fantasy", "desp": "Ruby, a mischievous Japanese Hare from Jewel Land, is sent to Earth to retrieve all her friends."}, {"m_title": "Magical Princess Minky Momo", "r": "7.9", "year": "1982", "votes": "95", "genre": "Comedy, Family", "desp": "A fairytale princess is sent to Earth to revive the hopes and dreams of its populace."}, {"m_title": "Sohryuden: Legend of the Dragon Kings", "r": "7.3", "year": "1991", "votes": "83", "genre": "Drama, Mystery", "desp": "The story features four divine Dragon Kings, rulers of the four parts of the world according to Chinese myths."}, {"m_title": "VS. Knight Ramune & 40 Fresh", "r": "6.8", "year": "1997", "votes": "43", "genre": "Comedy, Sci-Fi", "desp": "Two Holy Virgins, Parfait and Cacao, are in search for the mysterious warrior. Coming along with them is an advisor named PQ, who is a small red tomato-like creature. At a space station,. watch the dude, no spoilers in summary!"}, {"m_title": "Bakuon!!", "r": "6.4", "year": "2016", "votes": "112", "genre": "Comedy, Drama", "desp": "The story revolves around high school girls who discover the appeal of motorcycles. Sakura Hane is a high school student who looks a little bit like an airhead. On the way to her all-female. watch the anime dude!"}, {"m_title": "Infinite Ryvius", "r": "7.0", "year": "1999", "votes": "264", "genre": "Adventure, Drama", "desp": "Over 400 teenagers are stranded without adult supervision aboard a prototype space battleship after their astronaut training school is sabotaged by an outside group."}, {"m_title": "Ushio & Tora", "r": "7.1", "year": "1992", "votes": "18", "genre": "Action, Fantasy", "desp": "Ushio thinks his fathers tale of an ancient ancestor impaling a demon on a temple altar stone with the legendary Beast Spear is nuts, but when he finds the monster in his own basement,. watch the dude, no spoilers in summary!"}, {"m_title": "Crest of the Stars", "r": "7.6", "year": "1999", "votes": "513", "genre": "Action, Drama", "desp": "Jinto is a polite young man whose world was overrun by the biggest empire in the galaxy canned the Abh Empire. He meets the lovely Lafiel who he befriends."}, {"m_title": "Library Wars", "r": "7.1", "year": "2008", "votes": "181", "genre": "Action, Comedy", "desp": "In a world where information is heavily censored, Kasahara Iku joins the Library Force to protect the freedom of literature and follow in the footsteps of a man who once saved her favorite book from being destroyed."}, {"m_title": "Armor Hunter Mellowlink", "r": "7.7", "year": "1988", "votes": "76", "genre": "Short, Sci-Fi", "desp": "The sole survivor of a sacrificed squad sets out for revenge on corrupt officers."}, {"m_title": "Fist of the Blue Sky", "r": "6.5", "year": "2006", "votes": "114", "genre": "Action", "desp": "Kasumi Kenshiro, also known as Yan Wang or the King of Death, is a Tokyo professor and the 62nd successor to the deadly assassin art known as Hokuto Shinken. He leaves his teaching job and. watch the dude, no spoilers in summary!"}, {"m_title": "Dai Shogun - Great Revolution", "r": "5.8", "year": "2014", "votes": "56", "genre": "History", "desp": "sorry no summary not available!"}, {"m_title": "Cream Lemon: Lemon Angel", "r": "6.6", "year": "1987", "votes": "9", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Eat-Man", "r": "7.4", "year": "1997", "votes": "80", "genre": "Action, Adventure", "desp": "Bolt Crank is a man who has the specials ability to turn almost anything he eats into a weapon of choice."}, {"m_title": "Ultraviolet: Code 044", "r": "6.5", "year": "2008", "votes": "67", "genre": "Action, Sci-Fi", "desp": "In 2140, rebels infected with Phage are fighting for independence from corporate government on a space-station-turned-metropolis. Genetically modified melancholic 19-year-old dying corporate female assassin will decide the outcome."}, {"m_title": "Ultra B", "r": "6.7", "year": "1987", "votes": "26", "genre": "Animation", "desp": "There is a little baby in the town who has incredible powers. He use these powers to help his brothers who know about it but he also uses these powers as naughty boy and he not shows his powers anybody else ."}, {"m_title": "Aria the Origination", "r": "8.3", "year": "2008", "votes": "279", "genre": "Comedy, Drama", "desp": "In Neo Venezia, Akari, Aika, and Alice work diligently toward becoming Prima Undine (pro tour guide gondolier). The girls have come far since they began their training, and are forming. watch the dude, no spoilers in summary!"}, {"m_title": "My Sexual Harassment", "r": "6.5", "year": "1994", "votes": "71", "genre": "Drama, Romance", "desp": "Mochizuki performs sexual favors in order to help his mentor and lover, to climb up the company ladder of a computer firm they work for in Tokyo. He easily attracts the attention of both men and women, the first of which is his boss Honma."}, {"m_title": "New Getter Robo", "r": "7.6", "year": "2004", "votes": "103", "genre": "Sci-Fi", "desp": "Creatures called the Oni have attacked the planet earth, so a scientist creates three mecha robots. The strongest of these robots is called Getter Robo, which three men pilot it, a martial artist, criminal leader and a monk."}, {"m_title": "Lord of Vermilion: The Crimson King", "r": "5.2", "year": "2018", "votes": "77", "genre": "Action, Adventure", "desp": "In Tokyo, a cataclysm creates a new breed of human who can wield superpowers. Lines are drawn and sides are taken."}, {"m_title": "Inferno Cop", "r": "6.6", "year": "2012", "votes": "580", "genre": "Action, Comedy", "desp": "A cop with a skeleton head towards evil around the world"}, {"m_title": "Shakugan no Shana II", "r": "7.2", "year": "2007", "votes": "560", "genre": "Action, Adventure", "desp": "The heated bond between Shana and Yuji is tested as their paranormal adventures continue. The Flame Haze fends off supernatural foes by night. By day, she contends with a classmate whos. watch the dude, no spoilers in summary!"}, {"m_title": "Aquarion EVOL", "r": "6.7", "year": "2012", "votes": "152", "genre": "Action, Romance", "desp": "The protagonists live on a planet called Vega, while most of the antagonists originate from its sister planet, Altair. The main character, Amata Sora, a young man that has kept his. watch the dude, no spoilers in summary!"}, {"m_title": "The Severing Crime Edge", "r": "5.7", "year": "2013", "votes": "93", "genre": "Action, Fantasy", "desp": "Kiri Haimura has an obsession with beautiful hair--specifically, cutting it. This bizarre trait is what leads him to seek out a rumored long-haired ghost who lives in an abandoned house on. watch the dude, no spoilers in summary!"}, {"m_title": "Tari Tari", "r": "7.0", "year": "2012", "votes": "211", "genre": "Drama, Music", "desp": "A story of 5 high school students on the cusp of graduation, brought together by music, and uncertain of their futures."}, {"m_title": "Cyborg Kuro-chan", "r": "7.3", "year": "1999", "votes": "85", "genre": "Action, Adventure", "desp": "Kuro is a cat who looks his elderly owners. One day he is captured by a scientist who turns him into a cyborg to be used in an army. Kuro leaves and returns home to continue living as a normal cat despite his new appearance and abilities."}, {"m_title": "UFO Baby", "r": "8.0", "year": "1998", "votes": "98", "genre": "Comedy, Sci-Fi", "desp": "two teenagers take care of an alien baby and fun events happen around them as they try to hide the baby and their feelings for each other."}, {"m_title": "Bucchigire!", "r": "6.8", "year": "2022", "votes": "105", "genre": "Action, Fantasy", "desp": "To ensure law and order in Kyoto, a top-secret replacement operation is set in motion."}, {"m_title": "Zillion", "r": "7.5", "year": "1987", "votes": "73", "genre": "Action, Adventure", "desp": "Invaders have arrived on a second Earth called Maris in the year 2387. The natives are helpless as their weapons are ineffective against the invaders. The exception is three weapons. watch the dude, no spoilers in summary!"}, {"m_title": "Soul Link", "r": "6.1", "year": "2006", "votes": "49", "genre": "Adventure, Comedy", "desp": "AD 2045: Aizawa Ryota and his classmates of the Central Military Academy must find their way back to Earth after a terrorist attack."}, {"m_title": "Kyou kara Maou", "r": "6.7", "year": "2004", "votes": "14", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Mahô no idol Pastel Yumi", "r": "6.7", "year": "1986", "votes": "53", "genre": "Family, Comedy", "desp": "Yumi is a little girl who loves flowers. She is presented with a flower award on the festival of flowers and is given a magic-wand that materializes any drawing she makes."}, {"m_title": "Hand Shakers", "r": "3.0", "year": "2017", "votes": "189", "genre": "Action, Sci-Fi", "desp": "This is a story about hand shakers, partners have superpoers and compete with similar people in a empty world called Ziggurat. Shows journey of protagonist who is Mecha geek , to meet god...Though is doesnt want to..."}, {"m_title": "Hourou Musuko", "r": "7.6", "year": "2011", "votes": "325", "genre": "Drama", "desp": "Yoshino and Shuichi are two friends, both starting at a new school, both starting puberty, and both transgender. This series follows them as they come out and find acceptance."}, {"m_title": "Hakyu Hoshin Engi", "r": "6.0", "year": "2018", "votes": "40", "genre": "Action, Adventure", "desp": "The story is set in legendary China. A beautiful female fox spirit named Dakki is controlling the emperor Chu-o and the ruling In dynasty, and is using her power over him to do evil in the. watch the dude, no spoilers in summary!"}, {"m_title": "Maburaho", "r": "6.4", "year": "2003", "votes": "277", "genre": "Comedy, Drama", "desp": "His name is Kazuki Shikimori, 17 years old. He attends Aoi Academy, a school for elite magicians. Each magician has a set limit of spells, and performing more spells than the limit allows will cause the user to turn into dust."}, {"m_title": "In Search of the Lost Future", "r": "6.9", "year": "2014", "votes": "83", "genre": "Drama, Mystery", "desp": "An astronomy club gets caught up in a bizarre series of events at an old school building while trying to unravel the circumstances revolving around the sudden appearance of a mysterious transfer student named Yui."}, {"m_title": "Magical Girl Ore", "r": "6.4", "year": "2018", "votes": "77", "genre": "Comedy, Fantasy", "desp": "In this over-the-top parody of the entire Magical Girl genre, Saki must protect her crush, Mohiro, from being kidnapped by demons with the help of her best friend Sakuyo."}, {"m_title": "Tokko - 特公", "r": "6.5", "year": "2006", "votes": "482", "genre": "Action, Crime", "desp": "When 108 demons free themselves and start killing people, Special Public Safety Task Force, or Tokko for short, is formed to stop them. Some of the members have demons inside of them and can use them to enhance their own abilities."}, {"m_title": "Dragon Crisis!", "r": "6.0", "year": "2011", "votes": "172", "genre": "Action, Fantasy", "desp": "A normal high school boy Ryuji Kisaragis peaceful life is turned into an adventure by the return of his second cousin, Eriko. Ryuji and Eriko seize a relic box from a black broker named. watch the dude, no spoilers in summary!"}, {"m_title": "Please Tell Me! Galko-chan", "r": "7.0", "year": "2016", "votes": "307", "genre": "Comedy", "desp": "Three teenage girls discuss subjects that are usually of an embarrassing nature. Their interactions are gently humorous and often informative."}, {"m_title": "Mitsuteru yokoyamas sangokushi", "r": "7.6", "year": "1991", "votes": "213", "genre": "Action, Drama", "desp": "Because of the civil war, the country now is divided into three kingdoms. A villager with his two companions try to unite all the kingdoms into one."}, {"m_title": "Koihime musô", "r": "6.9", "year": "2008", "votes": "73", "genre": "Action, Adventure", "desp": "Orphaned by murdering bandits, Kanu becomes a well-known traveling bandit-slayer. At one village, she stumbles upon an orphaned warrior-child. Kanu adopts her as her little sister and continues her journey to change the world."}, {"m_title": "Blue Reflection Ray", "r": "5.5", "year": "2021", "votes": "43", "genre": "Action, Drama", "desp": "With opposite personalities, Hiori and Ruka are magical girls called Reflectors who use their powers to help resolve struggles and heal hearts."}, {"m_title": "Dame x Prince Anime Caravan", "r": "5.7", "year": "2018", "votes": "17", "genre": "Adventure, Comedy", "desp": "Inako is a minor country, enclosed by two superior countries: the militaristic Milidonia, with its goal of conquering as much land as possible, and the monotheistic Selenfalen, devoted to. watch the dude, no spoilers in summary!"}, {"m_title": "Koi Koi Seven", "r": "6.2", "year": "2005", "votes": "34", "genre": "Comedy, Drama", "desp": "Tetsuro is the only boy at an all girl high school, there he gets into trouble with the powerful student council leadership who are in conflict with a group of schoolgirl warriors named the Koi Koi Seven which includes his friend Yayoi."}, {"m_title": "Macross 7", "r": "6.5", "year": "1994", "votes": "384", "genre": "Action, Adventure", "desp": "A rock band must play a key role in the defense of a space colony fleet."}, {"m_title": "My Wife Is a High School Girl", "r": "6.6", "year": "2005", "votes": "64", "genre": "Comedy, Drama", "desp": "Asami Onohara is a seventeen-year old high-school student with a secret which has not been revealed to anyone: She is already married. Her husband, Kyosuke Ichimaru, is a Physics teacher in the same high school as her."}, {"m_title": "Otome wa boku ni koishiteru", "r": "6.8", "year": "2006", "votes": "88", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Hell Girl: Three Vessels", "r": "7.5", "year": "2008", "votes": "297", "genre": "Drama, Fantasy", "desp": "After the events of the second season, Ais three helpers, Ichimoku Ren, Wanyuudou, and Hone-Onna live out their lives in relative peace. This is suddenly shattered when Kikuri returns to. watch the dude, no spoilers in summary!"}, {"m_title": "Patalliro!", "r": "7.2", "year": "1982", "votes": "32", "genre": "Adventure, Comedy", "desp": "The adventures of Patalliro, the wild and uncontrollable infant king of Malynera."}, {"m_title": "The Knight in the Area", "r": "6.8", "year": "2012", "votes": "224", "genre": "Comedy, Drama", "desp": "Kakeru Aizawa loves soccer, yet always lived in the shadow of his exceptionally talented brother and hence relegated himself to the position of manager. Kakerus life changed however after a tragic accident."}, {"m_title": "Aishiteruze baby", "r": "6.9", "year": "2004", "votes": "301", "genre": "Comedy, Drama", "desp": "When little Yuzuyu Sakashitas mother runs away from her family, the little girl has to live with her teenage nephew Kippei. Through taking care of Yuzuyu, Kippei learns a lot about life."}, {"m_title": "Super GALS!", "r": "7.0", "year": "2001", "votes": "119", "genre": "Comedy, Drama", "desp": "Ran Kotobuki is the self-proclaimed worlds greatest gal. She lives to shop, eat, and have as much fun as possible, because Girls Just Wanna Have Fun. In the midst of all of her goofing off. watch the anime dude!"}, {"m_title": "Dragon Drive", "r": "7.2", "year": "2002", "votes": "73", "genre": "Action, Fantasy", "desp": "A teen boy named Reiji is introduced to a virtual reality game called Dragon Drive, where he meets Chibisuke, a dragon who the creators of the game have no info in their database on. Reiji. watch the anime dude!"}, {"m_title": "Teekyu", "r": "6.2", "year": "2012", "votes": "109", "genre": "Comedy, Sport", "desp": "The frenetic adventures of a girls tennis club that seldom plays tennis."}, {"m_title": "Minami-ke", "r": "7.2", "year": "2007", "votes": "305", "genre": "Comedy, Drama", "desp": "There are three of the Minami sisters: Haruka, Kana and Chiaki, who have an average life. The girls only have each other to depend on and help each other get through everything from love confessions to cooking."}, {"m_title": "Onihei", "r": "7.5", "year": "2017", "votes": "347", "genre": "Action, Drama", "desp": "Heizou Hasegawa, chief officer charged with sustaining safety, suppresses evils that are threatening a peaceful Edo."}, {"m_title": "Zoids: Fuzors", "r": "6.4", "year": "2003", "votes": "110", "genre": "Action, Adventure, Animation", "desp": "In a futuristic society, People called Z-Fighters pilot Mechanical Combat Units known as Zoids. The Young Pilot, R.D. and his Liger ZERO, along with the rest of Team Mach Storm pilot these. watch the dude, no spoilers in summary!"}, {"m_title": "Kurogane no rainbareru", "r": "5.8", "year": "2008", "votes": "142", "genre": "Action, Comedy", "desp": "Kouichi dreams of being a hero, but in reality, hes a punching bag for every bully on the block. His life flatlines when the mysteriously naked Emi crash lands on his pitiful existence,. watch the dude, no spoilers in summary!"}, {"m_title": "Muromi-san", "r": "6.6", "year": "2013", "votes": "53", "genre": "Comedy, Fantasy", "desp": "A boy goes fishing and catches a wacky mermaid."}, {"m_title": "4-nin wa Sorezore Uso wo Tsuku", "r": "7.1", "year": "2022", "votes": "61", "genre": "Comedy", "desp": "The story of four friends who each have a secret they cannot share with each other."}, {"m_title": "Mr. Tonegawa: Middle Management Blues", "r": "7.0", "year": "2018", "votes": "152", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Sasami-san@Ganbaranai", "r": "6.0", "year": "2013", "votes": "90", "genre": "Comedy, Romance", "desp": "The Japanese call them hikikomori-people whove become so withdrawn socially that they refuse to leave their homes for weeks and even months at a time. For Sasami Tsukuyomi, whos. watch the dude, no spoilers in summary!"}, {"m_title": "Technoroid: Overmind", "r": "5.5", "year": "2023", "votes": "19", "genre": "Music, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Kira Kira PreCure A La Mode", "r": "8.1", "year": "2017", "votes": "33", "genre": "Action, Adventure", "desp": "8th Grader Usami Ichika becomes a precure through the power of baking. With the help of fairy friends, she and 4 buddies fight evil as precures while running a patisserie."}, {"m_title": "UniteUp!", "r": "5.8", "year": "2023", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Kimi ga aruji de shitsuji ga ore de", "r": "6.1", "year": "2008", "votes": "140", "genre": "Comedy, Drama", "desp": "A teenager and his protective pretty sister are hired by a wealthy family as help. They primarily serve the familys three pretty young daughters who all have their eye on the boy which irritates the sister."}, {"m_title": "Fate/kaleid liner Prisma Illya 2wei! Herz!", "r": "6.0", "year": "2015", "votes": "210", "genre": "Action, Comedy", "desp": "The third season in the Fate/Kaleid liner Prisma Illya series, following from where the second season, Fate/Kaleid liner Prisma Illya 2wei finished."}, {"m_title": "Shimajiro", "r": "7.5", "year": "1993", "votes": "16", "genre": "Family", "desp": "Shimajiro the tiger boy goes on adventures with animal friends as they explore the world and learn new words in this 2D animated cartoon, there are live action scenes where Shimajiro mascots and his friend mascots meet people on the show."}, {"m_title": "Tokyo ESP", "r": "6.4", "year": "2014", "votes": "292", "genre": "Action, Comedy", "desp": "A serie of several teenagers who has super powers that can do amazing things , and the hero of this serie is the girl that find her self with a white hair when she use her super power ,. watch the dude, no spoilers in summary!"}, {"m_title": "BT X Neo", "r": "7.1", "year": "1997", "votes": "38", "genre": "Action, Adventure", "desp": "A continuation TV series where Bt X tries to rescue his brother."}, {"m_title": "Mirage of Blaze", "r": "7.3", "year": "2002", "votes": "160", "genre": "Drama, Fantasy", "desp": "Takaya, an ordinary high school boy, gets drawn into an ancient war of darkness and tragedy when he discovers that he is the reincarnation of a feudal lord from Japans Warring States Period."}, {"m_title": "Soul Hunter", "r": "6.9", "year": "1999", "votes": "115", "genre": "Action, Adventure", "desp": "A young master wizard gets sent to destroy the evil warlocks of his lands."}, {"m_title": "Hyouge Mono", "r": "7.7", "year": "2011", "votes": "88", "genre": "Comedy, History", "desp": "During the Age of Warring States, the upcoming warlord Sasuke Furuta is torn between his political ambition and his love of the tea ceremony."}, {"m_title": "Android Kikaider: The Animation", "r": "7.1", "year": "2000", "votes": "170", "genre": "Sci-Fi, Adventure", "desp": "The genius robotics professor, Dr. Komyoji has created Jiro (who has the ability to transform into Kikaider) Ã¢ÂEURÂ a humanoid robot tasked with the protection of Dr. KomyojiÃ¢ÂEURÂ(TM)s. watch the dude, no spoilers in summary!"}, {"m_title": "Arcadia of My Youth: Infinite Course SSX", "r": "7.6", "year": "1982", "votes": "281", "genre": "Action, Adventure", "desp": "When Aliens conquer earth and enslave the human race, all hope for freedom is abandoned. But one man will not give up. Captain Harlock, A brave space pilot, leads the resistance and vows to. watch the anime dude!"}, {"m_title": "Skate Leading Stars", "r": "6.4", "year": "2020", "votes": "88", "genre": "Sport", "desp": "Kensei Maeshima left the world of figure skating after his rival said, Youll never beat me. Years later, hes using his athleticism to help several high school teams, never focusing on. watch the dude, no spoilers in summary!"}, {"m_title": "Simoun", "r": "6.5", "year": "2006", "votes": "157", "genre": "Action, Adventure", "desp": "In the world of Daikuuriku, everyone is born female, and chooses which sex they wish to become at age 17. In this world, the peaceful theocracy of Simulacrum is guarded by magical flying. watch the dude, no spoilers in summary!"}, {"m_title": "Tamayomi: The Baseball Girls", "r": "6.5", "year": "2020", "votes": "63", "genre": "Sport", "desp": "Yomi Takeda was a pro pitcher of her schools team in junior high. However, she was the only ace in her club and other clubs players could never catch up with Yomi, hence Yomi quits. watch the dude, no spoilers in summary!"}, {"m_title": "Lagrange: The Flower of Rin-ne", "r": "6.0", "year": "2012", "votes": "148", "genre": "Action, Comedy", "desp": "Madoka an energetic school girl is recruited by a mysterious girl called Lan to pilot a robot in order to protect her city from enemies from space with an as of yet unknown goal."}, {"m_title": "Kenyuu densetsu Yaiba", "r": "7.2", "year": "1993", "votes": "121", "genre": "Action, Adventure", "desp": "Yaiba is a samurai child with a magic sword what fight against demons and devil bunnies to defeat evil."}, {"m_title": "Kimikiss: Pure Rouge", "r": "6.6", "year": "2007", "votes": "128", "genre": "Comedy, Drama", "desp": "Kouichi Sanada and Kazuki Aiharas childhood friend Mao Mizusawa has returned after living in France for a couple of years. Kouichi is surprised to learn that, since her parents are. watch the dude, no spoilers in summary!"}, {"m_title": "Armor Shop for Ladies and Gentlemen", "r": "5.2", "year": "2018", "votes": "51", "genre": "Comedy, Fantasy", "desp": "In a world of sword and sorcery a young villager named Kautsu comes upon a unique armor shop that only deals with a certain kind of equipment for female adventurers."}, {"m_title": "King of the Braves GaoGaiGar", "r": "8.0", "year": "1997", "votes": "114", "genre": "Action, Adventure", "desp": "A special organization using giant robots and aided by an alien boy battles the threat of a menace that inflicts havoc by turning humans into destructive mecha menaces."}, {"m_title": "Moonlight Mile", "r": "7.1", "year": "2007", "votes": "37", "genre": "Adventure, Drama", "desp": "Gorou and Lostman at the peak of Mount Everest see the ISA Space Station in the sky above and become determined to go into space. ISA begins a program to research and obtain a new energy source which has been discovered on the moon."}, {"m_title": "Getter Robo Ark", "r": "5.5", "year": "2021", "votes": "38", "genre": "Action, Adventure", "desp": "The story centers on the conflict against the Andromeda Country, the new enemy of humanity that seeks to destroy the source of the getter rays that has devastated its civilization. Hope. watch the dude, no spoilers in summary!"}, {"m_title": "Princess Resurrection", "r": "6.5", "year": "2007", "votes": "149", "genre": "Action, Comedy", "desp": "Hiro is killed saving a beautiful woman who is actually part of the demonic royal family and makes him her servant by resurrecting him."}, {"m_title": "Danchigai", "r": "5.8", "year": "2015", "votes": "138", "genre": "Comedy", "desp": "Danchigai presents the daily life of Haruki Nakano and his sisters, Mutsuki, Yayoi, Uzuki and Satsuki, who all share an apartment."}, {"m_title": "Serendipity the Pink Dragon", "r": "7.0", "year": "1983", "votes": "438", "genre": "Short, Adventure", "desp": "After a boy is castaway on an island a pink dinosaur hatches. This series follows the adventures of both of them on the island an there friends."}, {"m_title": "Ro-Kyu-Bu!", "r": "5.8", "year": "2011", "votes": "79", "genre": "Drama, Sport", "desp": "A high school student volunteers as the coach of the girls basketball team at an elementary school."}, {"m_title": "Boes", "r": "7.7", "year": "1987", "votes": "427", "genre": "Comedy", "desp": "The outrageous adventures of Olly the Ox and his animal friends as they keep a farm going by themselves."}, {"m_title": "Net Ghost PiPoPa", "r": "6.7", "year": "2008", "votes": "23", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Seiken Densetsu: Legend of Mana - The Teardrop Crystal", "r": "5.6", "year": "2022", "votes": "63", "genre": "Adventure, Fantasy", "desp": "Not far from the town of Domina lives Shiloh, a boy who keeps hearing a voice in his dreams about a mission he must fulfill. But what mission could he be needed for remains unknown. That. watch the dude, no spoilers in summary!"}, {"m_title": "Supa Miruku Chan", "r": "7.2", "year": "1998", "votes": "25", "genre": "Comedy, Sci-Fi", "desp": "Milk-chan and her friends are hired to solve a variety of problems."}, {"m_title": "Tokunana", "r": "6.6", "year": "2019", "votes": "106", "genre": "Action, Adventure", "desp": "A special police unit of superhumans is created to deal with a criminal group seeking mystic powers."}, {"m_title": "Mazinger Edition Z: The Impact!", "r": "7.3", "year": "2009", "votes": "258", "genre": "Sci-Fi", "desp": "A modern retelling of the endless battle between Koji Kabuto and Mazinger against Dr. Hell and his army of mechanic brutes."}, {"m_title": "The Galaxy Railways", "r": "7.0", "year": "2003", "votes": "129", "genre": "Adventure, Drama", "desp": "Interstellar travel has become possible, via Space Trains that run on the lines of the Galaxy Railways which travel throughout the universe, all of which protected by the SDF (Space Defense Force)."}, {"m_title": "Gundam Reconguista in G", "r": "5.1", "year": "2014", "votes": "154", "genre": "Action, Adventure", "desp": "The Universal Century has passed. Humanitys prosperity has ushered in a new era of global peace known as the Regild Century. Bellri Zenam, a pilot training with the Capital Guard, is. watch the dude, no spoilers in summary!"}, {"m_title": "Alice & Zouroku", "r": "6.8", "year": "2017", "votes": "210", "genre": "Adventure, Drama", "desp": "The story centers on a little girl called Sana, who is one of the children that holds the power of Alices Dream, an ability that enables her to materialize anything she imagines. After. watch the dude, no spoilers in summary!"}, {"m_title": "Brave Witches", "r": "6.1", "year": "2016", "votes": "125", "genre": "Animation", "desp": "The war on pants continues! Follow the newest group of witches with the 502nd Joint Fighter Wing, the Brave Witches."}, {"m_title": "The Thousand Musketeers", "r": "2.6", "year": "2018", "votes": "30", "genre": "Action, Fantasy", "desp": "Despair War is a battle between ancient guns and contemporary guns. Due to a nuclear war, the world was destroyed. Under the full governance of a world empire, people are living with their. watch the dude, no spoilers in summary!"}, {"m_title": "Natsuyuki Rendezvous", "r": "7.2", "year": "2012", "votes": "183", "genre": "Drama, Fantasy", "desp": "Hazuki is secretly in love with his florist boss, but his chances of a successful romance seem slim when he not only has to overcome shyness, but also the possessive ghost of her dead husband."}, {"m_title": "ID-0", "r": "6.1", "year": "2017", "votes": "338", "genre": "Action, Adventure", "desp": "In the midst of a field exercise operating I-Machines, Alliance Academy student Maya Mikuri is thrown into an entirely different kind of adventure."}, {"m_title": "Bamboo Blade", "r": "7.3", "year": "2007", "votes": "302", "genre": "Comedy, Drama", "desp": "A school kendo club is in need of talent and participation when the petite and talented Tamaki is drafted. With her understated sense of justice and ample strength, Tamaki becomes an. watch the dude, no spoilers in summary!"}, {"m_title": "Toradora SOS!", "r": "5.1", "year": "2009", "votes": "149", "genre": "Short, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Shin Megami Tensei Devil Children", "r": "7.2", "year": "2000", "votes": "26", "genre": "Action, Adventure", "desp": "A series based off of the Shin Megami Tensei: Devil Children franchise."}, {"m_title": "Yawara! A Fashionable Judo Girl", "r": "7.4", "year": "1989", "votes": "122", "genre": "Action, Comedy", "desp": "Follows the story of a Japanese girl named Yawara Inokuma over a four year period as she prepares for the Judo competition at the 1992 Barcelona Olympics."}, {"m_title": "Glass Mask", "r": "8.2", "year": "2005", "votes": "87", "genre": "Drama, Romance", "desp": "Two girls, one with formal training and one without, compete for the role of the Crimson Goddess throughout their developing years in this classic shoujo adaptation of Sizue Miuchis work of the same name."}, {"m_title": "Saber Marionette J to X", "r": "7.0", "year": "1998", "votes": "165", "genre": "Action, Adventure", "desp": "Taking place mere months after the Saber Marionette J OVA series, this newest installment in the Saber Marionette line follows the continuing adventures of Otaru and his clan of selfless,. watch the dude, no spoilers in summary!"}, {"m_title": "Haruka Nogizakas Secret", "r": "6.3", "year": "2008", "votes": "161", "genre": "Comedy, Romance", "desp": "In this anime series, Haruka is the popular new girl in school, but shes got a nerdy secret. This series follows her friendship with Yuto, a classmate whos got a crush on Haruka."}, {"m_title": "Afterschool Dice Club", "r": "6.6", "year": "2019", "votes": "183", "genre": "Comedy, Drama", "desp": "Aya moves and starts to attend an all-girls high school in Kyoto. Her first friend is her reserved classmate Miki. After school one day, the committee chairwoman Midori takes them both to. watch the dude, no spoilers in summary!"}, {"m_title": "Futsal Boys!!!!!", "r": "6.4", "year": "2022", "votes": "87", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Xenosaga: The Animation", "r": "6.1", "year": "2005", "votes": "184", "genre": "Action, Adventure", "desp": "4000 years have passed since humanity abandoned Earth. Forced to fight an alien form of life, Gnosis, mankind developed KOS-MOS - a battle android with amazing capabilities. When a ship. watch the dude, no spoilers in summary!"}, {"m_title": "Tales of the Abyss", "r": "6.7", "year": "2008", "votes": "208", "genre": "Adventure, Drama", "desp": "Pampered prince Luke is suddenly thrown from a life of luxury to an adventure that will lead him to the truth about his reality."}, {"m_title": "Dear Boys", "r": "6.5", "year": "2003", "votes": "69", "genre": "Drama, Sport", "desp": "A defunct high school basketball team finds a new student and things may start looking up for them."}, {"m_title": "Ai shite naito", "r": "6.4", "year": "1983", "votes": "313", "genre": "Comedy, Family", "desp": "Love triangle between a girl from a traditional family and two members of a rock band."}, {"m_title": "Fafner", "r": "7.0", "year": "2004", "votes": "213", "genre": "Action, Drama", "desp": "The youth of Tatsumiya Island believed that they had ordinary lives... until the advent of the Festum, extraterrestrial intelligent life forms intent on assimilating all humans. Abruptly,. watch the dude, no spoilers in summary!"}, {"m_title": "Kamichu!", "r": "7.1", "year": "2005", "votes": "189", "genre": "Adventure, Drama", "desp": "A middle school girl suddenly becomes a god. Now she must take on godly duties as well as attend school."}, {"m_title": "Young Black Jack", "r": "6.8", "year": "2015", "votes": "156", "genre": "Drama", "desp": "Young Black Jack chronicles the story of a man capable of performing unthinkable feats of medicine, and the journey that leads him to become the legend known as Black Jack."}, {"m_title": "The Gymnastics Samurai", "r": "7.1", "year": "2020", "votes": "152", "genre": "Drama, Sport", "desp": "Year 2002. Japans gymnastics world was once strong. Former Japanese representative Shoutarou Aragaki (29), who devoted his time to gymnastics, was unable to perform as expected. Even so,. watch the dude, no spoilers in summary!"}, {"m_title": "Eat-Man 98", "r": "6.7", "year": "1998", "votes": "87", "genre": "Action, Adventure", "desp": "Bolt Crank, the Worlds Greatest Mercenary, who has the ability to eat any object, mostly metal and weapons. When the time comes, he can reproduce those weapons and items in his right hand."}, {"m_title": "Mashiro-Iro Symphony: Love Is Pure White", "r": "6.7", "year": "2011", "votes": "124", "genre": "Romance", "desp": "When a private school for girls start taking in boys its not very popular with the female students. But slowly their opinion begin to change because of a male student, Shingo. Based on a video game"}, {"m_title": "BT X", "r": "6.8", "year": "1996", "votes": "133", "genre": "Action, Adventure", "desp": "The story follows Teppei, a Caretaker of a farm located in Kamui Island. He visits His genius brother who studies Robotics in Germany only to see him get abducted by a group of half human. watch the dude, no spoilers in summary!"}, {"m_title": "El-Hazard: The Wanderers", "r": "6.5", "year": "1995", "votes": "155", "genre": "Adventure, Comedy", "desp": "Thrown into another world, a group of high school friends look for a way home while helping the people they meet there."}, {"m_title": "ArtisWitch", "r": "7.8", "year": "2021", "votes": "28", "genre": "Animation", "desp": "A witch-in-training named Nina runs a magical shop where she grants customers secret wishes by looking deeper into their hearts, both their incredible dreams and darkest fantasies."}, {"m_title": "Demon Lord Dante", "r": "4.8", "year": "2002", "votes": "138", "genre": "Fantasy, Horror", "desp": "A boy becomes connected to a potentially world-ending giant monster, Demon Lord Dante."}, {"m_title": "Kurokami: The Animation", "r": "6.8", "year": "2009", "votes": "224", "genre": "Action, Adventure", "desp": "In this world, there are three identical looking people in existence who split between them an energy-like force called Terra. This energy can be attributed to the amount of Luck or Life. watch the dude, no spoilers in summary!"}, {"m_title": "Mojacko", "r": "9.1", "year": "1995", "votes": "21", "genre": "Comedy, Fantasy", "desp": "The life of an ordinary boy named Sarao completely changes after he meets and makes friends with two stranded aliens who find themselves lost on planet earth. Sarao helps the two aliens to. watch the dude, no spoilers in summary!"}, {"m_title": "Prince of Stride: Alternative", "r": "6.5", "year": "2016", "votes": "329", "genre": "Sport", "desp": "Stride is an extreme form of sport. It involves 6 players on a team that runs relay races in towns. The story takes place at Honan Academy where two first-year high-school students try to recruit members for their club."}, {"m_title": "Warlords of Sigrdrifa", "r": "6.1", "year": "2020", "votes": "74", "genre": "Action, Drama", "desp": "The Pillars suddenly appeared above Earth, threatening all life. A god who called himself Odin came to the aid of humanity as it was driven to near complete defeat. To counter the Pillars,. watch the dude, no spoilers in summary!"}, {"m_title": "Slow Loop", "r": "6.6", "year": "2022", "votes": "117", "genre": "Drama, Sport", "desp": "After teaching a girl named Koharu how to fish, Hiyori comes to a startling realization: Her new friend is about to become family."}, {"m_title": "Sakura Wars: The Animation", "r": "6.4", "year": "2020", "votes": "67", "genre": "Action, Sci-Fi", "desp": "An anime series followup to Sakura Wars (2019), members of the Imperial Combat Revue must protect a young girl named Klara, the sole survivor of her combat troupe and whom posses some mysterious powers."}, {"m_title": "Farewell, My Dear Cramer", "r": "6.4", "year": "2021", "votes": "96", "genre": "Sport", "desp": "A group of School children along with their legendary Female Pro football player coach work hard to make their team a success and highlight womans football as popularity continues to fall down."}, {"m_title": "Dual! Parallel Trouble Adventure", "r": "7.1", "year": "1999", "votes": "157", "genre": "Comedy, Sci-Fi", "desp": "A boy who sees visions of giant robot battles in modern Japan is accidentally transported to the parallel dimension in which the fighting is taking place."}, {"m_title": "Nagagutsu o haita neko no bôken", "r": "8.2", "year": "1992", "votes": "24", "genre": "Adventure, Fantasy", "desp": "Anime adaptation of Charles Perraults series Puss-in-Boots."}, {"m_title": "Cinderella Nine", "r": "5.5", "year": "2019", "votes": "44", "genre": "Drama, Sport", "desp": "When Arihara Tsubasa enters Rigahama Municipal High School and learns that it has no baseball club, she starts up the Girls Baseball Club on her own. Drawn to the club are girls who have. watch the dude, no spoilers in summary!"}, {"m_title": "Futari Wa Pretty Cure Max Heart", "r": "7.0", "year": "2005", "votes": "64", "genre": "Action, Adventure", "desp": "Shortly after the fall of the Wicked King, the Queen of the Garden of Light lost her memory and came to Earth, in the form of a 12-year-old girl named Hikari Kujou. Now, she with the help. watch the dude, no spoilers in summary!"}, {"m_title": "Coyote Ragtime Show", "r": "6.4", "year": "2006", "votes": "200", "genre": "Action, Adventure", "desp": "In an alternate galaxy, theres an outlaw named Mister whos 10 days from his prison release. He escapes from prison with the help of his comrades Bishop and Katana. As they seek out to. watch the dude, no spoilers in summary!"}, {"m_title": "Last Exile: Ginyoku no Fam", "r": "6.9", "year": "2011", "votes": "240", "genre": "Action, Adventure", "desp": "Aerial adventures continue in the sky above the world of Prester."}, {"m_title": "Tokyo Tribe 2", "r": "5.7", "year": "2006", "votes": "17", "genre": "Action, Crime", "desp": "This is the other side of Tokyo, a Tokyo of a different color. There, various tribes are living. The SARU from Japans Musashino, WU-RONZE of Bukuro, and HANDS of Shindyuku - Young men. watch the dude, no spoilers in summary!"}, {"m_title": "eX-Driver", "r": "6.6", "year": "2000", "votes": "114", "genre": "Action, Sci-Fi", "desp": "In a future world where all cars are controlled by artificial intelligence, which often breaks down, one elite group of kids has been selected to drive gasoline-powered non-AI controlled. watch the dude, no spoilers in summary!"}, {"m_title": "Mighty Cat Masked Niyander", "r": "9.0", "year": "2000", "votes": "33", "genre": "Comedy, Family", "desp": "Whenever anyone calls for help, Mighty Cat Masked Niyander is always there. A true superhero, who fights the bad guys and always saves his friends, while protecting his secret identity."}, {"m_title": "Knight Hunters: Weiß Kreuz", "r": "7.5", "year": "2001", "votes": "99", "genre": "Animation", "desp": "During the day, theyre four cute boys who run their flower shop. By night, they instantly turn from innocent to bad as they go out and execute the people who do illegal things. Assassins.. watch the dude, no spoilers in summary!"}, {"m_title": "BeyWheelz", "r": "6.0", "year": "2012", "votes": "56", "genre": "Action, Adventure", "desp": "BeyWheelz is set in the futuristic city of Destection where our hero, Sho, is champion of the BeyWheelz World Tournament! This peaceful city is home to exciting and challenging battles. watch the dude, no spoilers in summary!"}, {"m_title": "Nana maru san batsu", "r": "6.7", "year": "2017", "votes": "87", "genre": "Comedy", "desp": "Shiki Koshiyama enter his new high school, and there he introduced to the world of competitive quiz bowl. after his experience his first competition he decided to listen to his persistence. watch the dude, no spoilers in summary!"}, {"m_title": "Endro~!", "r": "6.5", "year": "2019", "votes": "122", "genre": "Adventure, Comedy", "desp": "A party of adventures attempt to defeat a demon lord. They fail, accidentally sending her back in time. The demon lord decides to go to the adventurer school where these four girls trained, and become their teacher."}, {"m_title": "Honey and Clover II", "r": "8.1", "year": "2006", "votes": "390", "genre": "Comedy, Drama", "desp": "Honey and Clover II continues with the life of Takemoto, Yamada and Morita with each of them taking different paths and discovering themselves, story is narrated in voice of Takemoto just. watch the dude, no spoilers in summary!"}, {"m_title": "Hanasakeru Seishônen", "r": "7.2", "year": "2009", "votes": "136", "genre": "Drama, Romance", "desp": "Kajika Louisa Kugami Burnsworth is the only daughter of Harry Burnsworth, an influential and respected industrialist who has the power to move the world. There was a threat on Kajikas life. watch the anime dude!"}, {"m_title": "Blassreiter", "r": "6.7", "year": "2008", "votes": "252", "genre": "Action, Sci-Fi", "desp": "The story is set in a fictional Germany and centers around the outbreak of bio-mechanical creatures named Demoniacs, who rise from corpses and attack people mindlessly. All the while, a. watch the dude, no spoilers in summary!"}, {"m_title": "Twilight Q", "r": "6.6", "year": "1987", "votes": "162", "genre": "Animation", "desp": "Diving into the sea, Maymui is surprised to find a camera containing images of herself with a man she has never seen before. On top of this, the camera turns out to be a model not yet for sale."}, {"m_title": "Heavens Design Team", "r": "6.9", "year": "2021", "votes": "210", "genre": "Comedy, Fantasy", "desp": "I made the light, the water, and the earth below. Now its time to make the life that dwells upon it. Actually - that sounds like a giant headache, so Ill contract it out."}, {"m_title": "Mushibugyo", "r": "6.6", "year": "2013", "votes": "136", "genre": "Action, Adventure", "desp": "Well for starters the plot is about a young Samurai called Jinbei Tsukishima who joins the insect magistrate office which is an organization created to fight against giant insects who have invaded the earth."}, {"m_title": "Moyashimon", "r": "7.2", "year": "2007", "votes": "195", "genre": "Comedy, Drama", "desp": "Follow the life of Tadayasu Sawaki as he enrolls in an Agricultural University and learn about bacteria and viruses that he alone can see with the naked eye."}, {"m_title": "Selector Infected WIXOSS", "r": "6.1", "year": "2014", "votes": "172", "genre": "Fantasy, Thriller", "desp": "Its hard being the new girl, and Ruuko Kominato knows this better than anyone. She has yet to make any new friends after moving to a new town. Her brother sees her struggling and gives her. watch the anime dude!"}, {"m_title": "Shonen onmyoji", "r": "7.1", "year": "2006", "votes": "132", "genre": "Action, Comedy", "desp": "Abe no Masahiro is the grandson of the greatest Onmyouji Abe no Seimei. His grandfather sealed his sixth sense (the ability to sense monsters) in order to protect him. He doesnt know about. watch the anime dude!"}, {"m_title": "Layton Mystery Tanteisha: Katori no Nazotoki File", "r": "6.8", "year": "2018", "votes": "36", "genre": "Animation", "desp": "Layton Mystery Tanteisha: Katori no Nazotoki File (Layton Mystery Detective Agency: Kats Mystery Solving Files) takes place during the events of Laytons Mystery Journey: Katrielle and the Millionaires Conspiracy."}, {"m_title": "Minami-ke: Okawari", "r": "6.5", "year": "2008", "votes": "154", "genre": "Comedy, Drama", "desp": "The second season of Minami-ke. It picks up where the first season left off, and just like the first season, its about the daily lives of the three Minami sisters, Haruka, Kana and Chiaki, and their school friends."}, {"m_title": "Selection Project", "r": "7.6", "year": "2021", "votes": "66", "genre": "Comedy, Drama", "desp": "Held every summer, the national show Selection Project is the biggest gateway for girls who strive to be idols, and the place where the legendary idol Akari Amazawa was born."}, {"m_title": "Stand My Heroes: Piece of Truth", "r": "7.0", "year": "2019", "votes": "65", "genre": "Animation", "desp": "Drugs have become the center of conflict in Japan and crime involving them has been increasing. The police department settled on creating a unit STAND that would include specialized experts and professionals to handle the issue."}, {"m_title": "The Mystic Archives of Dantalian", "r": "6.6", "year": "2011", "votes": "180", "genre": "Adventure, Fantasy", "desp": "Hugh Anthony Disward inherits an old mansion and a personal library from his grandfather. In the basement of the mansion, he meets a mysterious girl-Dalian. She is a Dantalian and a gateway. watch the anime dude!"}, {"m_title": "Gun Frontier", "r": "6.4", "year": "2002", "votes": "118", "genre": "Action, Comedy", "desp": "Sea Pirate Harlock and Tochiro try to find a lost clan of Japanese immigrants in this Westerns style version spin off."}, {"m_title": "Tenshi no 3P!", "r": "6.4", "year": "2017", "votes": "41", "genre": "Music", "desp": "Kyo Nukui is a teenager who refuses to go to school, and spends his days composing songs and publishing them online. One day, he is approached by a trio of orphaned little girls who ask for. watch the anime dude!"}, {"m_title": "Kirarin Revolution", "r": "7.1", "year": "2006", "votes": "65", "genre": "Comedy, Music", "desp": "A 14 year old girl wishes to become a successful artist in order to impress the boy of her dreams who is a professional singer."}, {"m_title": "Momokuri", "r": "6.2", "year": "2015", "votes": "122", "genre": "Comedy", "desp": "Momokuri starts of with a girl who sends a letter co fearing her love to Momo-kun. Momo says yes seeing that see is a beautiful girl. But what he doesnt realize is that he is dating a stalking/pervert."}, {"m_title": "Africa no Salaryman", "r": "6.5", "year": "2019", "votes": "76", "genre": "Comedy", "desp": "The comedy follows a lion, toucan, and lizard as they live the lives of office workers in a capitalist society in Japan, while also dealing with their unique situations as animals living beyond the savanna and the food chain."}, {"m_title": "Gundam Build Fighters Try", "r": "6.9", "year": "2014", "votes": "147", "genre": "Action, Comedy", "desp": "Seven years after its predecessor, Fumina Hoshino is the host of the Gunpla club in her high school, and, unfortunately, she is the only member and in danger of it being shut down. When a. watch the dude, no spoilers in summary!"}, {"m_title": "180 Byou de Kimi no Mimi wo Shiawase ni Dekiru ka?", "r": "5.1", "year": "2021", "votes": "49", "genre": "Animation", "desp": "This anime will let viewers experience ASMR in 180 seconds. In the story, the heroine challenges her classmates to share in her hobby of recording ASMR works."}, {"m_title": "Yumeiro Patissiere", "r": "7.8", "year": "2009", "votes": "82", "genre": "Comedy, Drama", "desp": "side from her deep passion for eating cakes, the clumsy Ichigo Amano has never been successful at anything. After an encounter with Henri Lucas, a famous patissier, Ichigos amazing tasting. watch the anime dude!"}, {"m_title": "Hiiro no Kakera", "r": "6.8", "year": "2012", "votes": "335", "genre": "Fantasy, Romance", "desp": "Believing she is to live with her grandmother because her parents are transferred abroad, Tamaki Kasuga travels to the town of Kifumura. There, she is told that she is actually to perform. watch the dude, no spoilers in summary!"}, {"m_title": "Shoot! Goal to the Future", "r": "5.8", "year": "2022", "votes": "69", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Grander Musashi RV", "r": "6.9", "year": "1998", "votes": "30", "genre": "Adventure, Sport", "desp": "sorry no summary not available!"}, {"m_title": "Petite Princess Yucie", "r": "7.4", "year": "2002", "votes": "87", "genre": "Comedy, Drama", "desp": "Petite Princess Yucie is the story of a 17 year old girl stuck in a 10 year body. She wants to become an adult so she enters in the platina princess competition. She will have to compete. watch the dude, no spoilers in summary!"}, {"m_title": "Kinmoza!", "r": "6.7", "year": "2013", "votes": "221", "genre": "Animation", "desp": "Shinobu Omiya is a Japanese high school girl who, five years ago, had a homestay in England with a girl named Alice Cartelet. One day, Shinobu receives a letter from Alice saying she is. watch the dude, no spoilers in summary!"}, {"m_title": "Silent Möbius", "r": "6.6", "year": "1998", "votes": "176", "genre": "Action, Crime", "desp": "A team of policewomen with supernatural powers try to protect the Earth from an alien invasion known as the Lucifer Hawks."}, {"m_title": "Hatsukoi rimitetto", "r": "7.0", "year": "2009", "votes": "129", "genre": "Comedy, Romance", "desp": "A series relating the intertwined stories about the first loves of several middle-schoolers and high-schoolers. Each episode tends to focus on a different character, however the. watch the dude, no spoilers in summary!"}, {"m_title": "Tropical-Rouge! Pretty Cure", "r": "8.1", "year": "2021", "votes": "41", "genre": "Action, Adventure", "desp": "It is the eighteenth anime series of the Pretty Cure franchise."}, {"m_title": "Antique Bakery", "r": "7.2", "year": "2008", "votes": "78", "genre": "Comedy, Romance", "desp": "The loves and lives of four workers at the Antique Bakery in Tokyo."}, {"m_title": "R.O.D the TV", "r": "7.5", "year": "2003", "votes": "486", "genre": "Action, Adventure", "desp": "Nenene, a famous female writer, hires the Paper Sisters, so named due to their ability to manipulate and control paper, as bodyguards."}, {"m_title": "Banner of the Stars II", "r": "7.2", "year": "2000", "votes": "407", "genre": "Action, Romance", "desp": "Now that three years has passed since their first adventure, Lafiel is now captain on a new ship and Jinto has just finished his training. They go out to complete a mission of defending the Laptic Gate."}, {"m_title": "Ojisan and Marshmallow", "r": "6.4", "year": "2016", "votes": "130", "genre": "Comedy, Romance", "desp": "The story focuses on Hige a middle-aged employee who loves marshmallows and wakabayashi a young girl who would like to get Higes attention and ideally a relationship with him."}, {"m_title": "Patlabor: The New Files", "r": "7.6", "year": "1990", "votes": "280", "genre": "Action, Comedy", "desp": "The new adventures of the Patlabor team, and the conclusion of the Griffin story-line, in a 16 episode release, following the successful TV series."}, {"m_title": "Angel Links", "r": "6.0", "year": "1999", "votes": "86", "genre": "Action, Adventure", "desp": "Meifon is quite an overachiever for a sixteen-year-old girl. Shes the head of a major corporation, and the captain of a state-of-the-art ship which provides a free pirate fighting service.. watch the anime dude!"}, {"m_title": "Schwarzesmarken", "r": "5.9", "year": "2016", "votes": "175", "genre": "Drama, Sci-Fi", "desp": "In 1983 the East German Army 666th TSF Squadron Swarzesmarken are fighting on the front lines against the alien invaders known as the BETA."}, {"m_title": "Mikagura School Suite", "r": "6.3", "year": "2015", "votes": "60", "genre": "Comedy, Fantasy", "desp": "An anime based on a novel based on a Vocaloid-based song. Catch all that? Mikagura High School is a boarding school where each student must join an activity club upon enrollment, and then. watch the dude, no spoilers in summary!"}, {"m_title": "Gravion Zwei", "r": "7.1", "year": "2004", "votes": "49", "genre": "Comedy, Sci-Fi", "desp": "After Gravion was able to use its ultimate attack, the Zeravire threat seemed to have temporary stopped. However when it rises back up all of a sudden, the Earth Gult and its trump card,. watch the dude, no spoilers in summary!"}, {"m_title": "Sexy Commando Side Story: Thats Amazing!! Mr. Masaru", "r": "6.8", "year": "1998", "votes": "23", "genre": "Comedy", "desp": "Okometsubu Fujiyama has recently transferred over to a new school, Wakame High School. His goal is to make 100 friends, until he meets the extremely weird Masaru Hananakajima. Masaru is a. watch the dude, no spoilers in summary!"}, {"m_title": "Toka Gettan: The Moonlight Lady Returns", "r": "5.6", "year": "2007", "votes": "19", "genre": "Comedy, Drama", "desp": "Toka Kamiazuma encounters a girl named Momoka Kawakabe who stays Kamiazuma clan which brings a chain of events to life."}, {"m_title": "Chô robot seimeitai Transformer: Micron densetsu", "r": "7.2", "year": "2003", "votes": "56", "genre": "Family", "desp": "Armadas universe is most distinguished by the presence and importance of Mini-Cons, a race of human-sized Transformers that, when powerlinxed to their larger brethren, would unleash a. watch the dude, no spoilers in summary!"}, {"m_title": "Shangri-La", "r": "6.4", "year": "2009", "votes": "85", "genre": "Action, Drama", "desp": "The story is set in a neo-futuristic age, in which Tokyo has turned into the worlds largest jungle-polis that absorbs carbon dioxide. A tower city called Atlas houses the rich and. watch the dude, no spoilers in summary!"}, {"m_title": "Cosmic Baton Girl Comet-san", "r": "7.1", "year": "2001", "votes": "37", "genre": "Animation", "desp": "The story revolves around Princess Comet, a twelve year old girl who is also the princess of the Harmonica Star country of the Triangle Nebula. She was meant to meet the prince of the ...                See full synopsis »"}, {"m_title": "Invaders of the Rokujyouma!?", "r": "6.5", "year": "2014", "votes": "154", "genre": "Comedy, Fantasy", "desp": "After his father is transferred away for work, Koutarou must find a cheap place to live by himself while attending high school. He thinks that hes found just the place: Room 106 at Corona. watch the dude, no spoilers in summary!"}, {"m_title": "Makiba no shôjo Katori", "r": "7.0", "year": "1984", "votes": "513", "genre": "Drama", "desp": "It has been three years since little Katris mother left for Germany leaving her daughter behind in Finland, where she now lives with her grandparents. With Finland still under Russian. watch the dude, no spoilers in summary!"}, {"m_title": "Neo Ranga", "r": "6.8", "year": "1998", "votes": "84", "genre": "Adventure, Horror", "desp": "Three sisters whose brother married into a remote island tribe inherit the islands guardian god, a giant monster."}, {"m_title": "Zoku sayonara zetsubô sensei", "r": "7.6", "year": "2008", "votes": "373", "genre": "Comedy", "desp": "The continuing adventures of the inconsolable teacher Nozomu Itoshiki and his class of insane students."}, {"m_title": "Young Disease Outburst Boy", "r": "6.0", "year": "2019", "votes": "50", "genre": "Comedy", "desp": "Mizuki Hijiri, is a high school girl who just transferred schools. At her new school, she has a fateful encounter with boys suffering from chuunibyou (adolescent delusions of grandeur).. watch the dude, no spoilers in summary!"}, {"m_title": "Phantasy Star Online 2: The Animation", "r": "6.3", "year": "2016", "votes": "83", "genre": "Action, Fantasy", "desp": "Itsuki Tachibana is a student of Seiga Academy, a co-ed boarding high school with excellent reputation, located in a suburb outside the city center. On a day of second semester, he was. watch the dude, no spoilers in summary!"}, {"m_title": "Roketto gâru", "r": "7.2", "year": "2007", "votes": "31", "genre": "Adventure, Sci-Fi", "desp": "Three schoolgirls are conscripted into the Solomon Space Agency to become the youngest ever astronauts of Japan."}, {"m_title": "Unlimited Fafnir", "r": "5.6", "year": "2015", "votes": "126", "genre": "Fantasy, Romance", "desp": "Midgar, all-girl academy, would have been notable just for the action of accepting its first and only male student, Yuu Mononobe. But Midgar stands out for much more than that: its a. watch the dude, no spoilers in summary!"}, {"m_title": "Super Dimension Century Orguss", "r": "7.3", "year": "1983", "votes": "69", "genre": "Action, Adventure", "desp": "A young dimension-hopping fighter pilot lands in a warring amalgamate world, the fate of which is in his hands."}, {"m_title": "Luck & Logic", "r": "5.3", "year": "2016", "votes": "116", "genre": "Action, Fantasy", "desp": "Logicalists are people who have acquired special power by entering in trance with Goddesses. They work under special police named ALCA under government and protect people against foreigners coming from another world."}, {"m_title": "Fighting Foodons", "r": "6.2", "year": "2001", "votes": "140", "genre": "Adventure, Comedy", "desp": "In the land of the Fighting Foodons, regular recipes turn into meal-time monsters when the art of culinary combat is concocted. Its all friendly competition, that is, until evil becomes. watch the dude, no spoilers in summary!"}, {"m_title": "Shônen Maid", "r": "7.1", "year": "2016", "votes": "131", "genre": "Comedy, Drama", "desp": "An elementary boy moves in with his uncle after his mom dies and begins working for him as a maid."}, {"m_title": "Ange Vierge", "r": "5.6", "year": "2016", "votes": "40", "genre": "Action, Comedy", "desp": "The story of the card game follows what happens when Hairou portals suddenly open, fusing five different worlds together. As a result, various mysterious Exceed powers are awakened in. watch the dude, no spoilers in summary!"}, {"m_title": "Elegant Yokai Apartment Life", "r": "7.1", "year": "2017", "votes": "43", "genre": "Mystery", "desp": "sorry no summary not available!"}, {"m_title": "Hacka Doll the Animation", "r": "5.5", "year": "2015", "votes": "41", "genre": "Short, Comedy", "desp": "The anime personifies the Hackadoll apps customization engine as three Hacka Dolls -- the personal entertainment AI and main navigator Hacka Doll #1, the anime expert Hacka Doll #2, and the knowledgeable otaku Hacka Doll #3."}, {"m_title": "Hero Tales", "r": "7.0", "year": "2007", "votes": "106", "genre": "Action, Adventure", "desp": "Lord Keiro, the Shogun of the Xian Empire, blazes a trail of terror while seeking the sacred sword that will make him a god. Standing in his way is Taito, a powerful star reborn in human. watch the dude, no spoilers in summary!"}, {"m_title": "Night Raid 1931", "r": "6.5", "year": "2010", "votes": "78", "genre": "Action, Drama", "desp": "In Shanghai, in 1931, the seeds of future devastating war are being planted by various factions. Sakurai Kikan, a secret agency whose agents have paranormal abilities, must uncover their plans and stop them before its too late."}, {"m_title": "Melody of Oblivion", "r": "6.1", "year": "2004", "votes": "59", "genre": "Horror, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Kaginado", "r": "6.9", "year": "2021", "votes": "39", "genre": "Short, Comedy", "desp": "Six worlds of dramatic content converge and collide, leading to comedic and dramatic encounters."}, {"m_title": "Yuri Kuma Arashi", "r": "6.7", "year": "2015", "votes": "266", "genre": "Drama, Fantasy", "desp": "In a world of war between bears and humans, a young schoolgirl is sent into an adventure quest to find the origin of this conflict."}, {"m_title": "Hiatari Ryôkô!", "r": "7.0", "year": "1987", "votes": "30", "genre": "Comedy, Drama", "desp": "Kasumi Kishimoto, a 15-year-old girl, is boarding at her aunts house, which is near the school she will be attending in April. Upon arriving, shes surprised to find out that four male students are already living there."}, {"m_title": "Phi Brain: Puzzle of God", "r": "6.2", "year": "2011", "votes": "127", "genre": "Action, Adventure", "desp": "Kaito Daimon and his friends must solve puzzles."}, {"m_title": "Star Twinkle PreCure", "r": "8.1", "year": "2019", "votes": "29", "genre": "Action, Adventure", "desp": "The Starry Sky World is the home to the twelve Star Princesses who are based on the Zodiac signs and maintain order in the universe. But when the Notraiders attack the Star Palace, the. watch the dude, no spoilers in summary!"}, {"m_title": "Finding Full Moon", "r": "7.7", "year": "2002", "votes": "249", "genre": "Comedy, Drama", "desp": "Twelve-year-old Mitsuki loves to sing and she dreams of becoming a famous singer to fulfill a promise she made to Eichi, a close friend who moved away to American two years previously.. watch the dude, no spoilers in summary!"}, {"m_title": "Vipers Creed", "r": "6.9", "year": "2009", "votes": "52", "genre": "Action, Sci-Fi", "desp": "By the mid 21st century the world has been devastated by environmental pollution and world war. Failed attempts to repair the environmental damage have made travel by air and sea impossible. watch the anime dude!"}, {"m_title": "Starmyu", "r": "6.9", "year": "2015", "votes": "56", "genre": "Comedy, Music", "desp": "Ayanagi Academy: home of musical dreams and insane competition. New student Yuta Hoshitani dreams of being in show business, but to get there hell have to gain acceptance from the Kao. watch the dude, no spoilers in summary!"}, {"m_title": "Ai Tenchi Muyo!", "r": "5.7", "year": "2014", "votes": "63", "genre": "Short, Comedy", "desp": "A reboot of the Tench Muyo anime In TV short form. The Series takes place in the city of Takahashi, the real-life sponsor of the show."}, {"m_title": "Yuyushiki", "r": "6.6", "year": "2013", "votes": "143", "genre": "Comedy", "desp": "The adventures of the Data Processing Club - Yuzuko, Yui and Yukari - as they live out their lives with fun and adventure."}, {"m_title": "Mobile Suit Gundam MS IGLOO: The Hidden One Year War", "r": "6.0", "year": "2004", "votes": "119", "genre": "Action, Sci-Fi", "desp": "Jotunheim, a Principality of Zeons space cargo ship, is tasked to test and evaluate various prototype weapons during the One Year War in the year of 0079 UC."}, {"m_title": "Blue Dragon: Trials of the Seven Shadows", "r": "7.2", "year": "2008", "votes": "47", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Kumamiko - Girl Meets Bear", "r": "5.9", "year": "2016", "votes": "82", "genre": "Comedy", "desp": "What does a bear know about life in the big city? For Machi Amayadoris sake, hopefully a lot. The young shrine maiden has spent her whole life in the rural mountains with Natsu, her. watch the dude, no spoilers in summary!"}, {"m_title": "SoltyRei", "r": "7.2", "year": "2005", "votes": "245", "genre": "Action, Drama", "desp": "On a wrecked, class-divided world, Solty, a robot with no memory, is adopted by an unlikely guardian: a bounty hunter looking for his missing daughter. Together, they discover secrets about their world that will change their lives forever."}, {"m_title": "Sister Princess", "r": "5.0", "year": "2001", "votes": "57", "genre": "Comedy, Romance", "desp": "An ordinary young man is made to live with twelve lovable little sisters, each with their own distinct quirks and personalities."}, {"m_title": "Haré+Guu", "r": "7.8", "year": "2001", "votes": "300", "genre": "Comedy, Fantasy", "desp": "A jungle boy named Hare becomes a companion of a strange girl named Guu who apparently has a strange world inside of her stomach."}, {"m_title": "Tenkû senki Shurato", "r": "7.3", "year": "1989", "votes": "260", "genre": "Animation", "desp": "During a tournament, friends Gai Kuroki and Shurato Hidaka are transported to an alternative world, Tenkukai, by a goddess. When they wake up, Shurato learns he is now the king of lions and that his former friend Gai turned into his enemy."}, {"m_title": "Gaikotsu Shotenin Honda-san", "r": "6.8", "year": "2018", "votes": "231", "genre": "Comedy", "desp": "The day-to-day happenings that take place at a bookstore where the love of manga is abundant."}, {"m_title": "Corrector Yui", "r": "7.1", "year": "1999", "votes": "62", "genre": "Adventure, Comedy", "desp": "Yui is an average schoolgirl who lives in a future where all computers are supported by a single global network known as COMNET. Yui is a computer-illiterate girl who after a computer-lab. watch the dude, no spoilers in summary!"}, {"m_title": "Lost Universe", "r": "6.9", "year": "1998", "votes": "197", "genre": "Adventure, Comedy", "desp": "Kain Blueriver, the hero of this sci-fi comedy, is a Trouble Contractor: he handles cases too difficult and/or dangerous for the Universal Guardians. He always wears a blue cape - even with. watch the anime dude!"}, {"m_title": "Kinnikuman nisei", "r": "6.9", "year": "2002", "votes": "123", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Kobato.", "r": "7.3", "year": "2009", "votes": "242", "genre": "Adventure, Comedy", "desp": "Follow the journey of a mysterious girl, as she must complete her given task to fulfill her wish. To obtain her wish she must pass the test within four seasons."}, {"m_title": "And Yet the Town Moves", "r": "7.2", "year": "2010", "votes": "123", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Overman King Gainer", "r": "7.1", "year": "2002", "votes": "63", "genre": "Adventure, Sci-Fi", "desp": "In the distant future, Earths population has to live in domed cities and professional gamer Sanga and mercenary Gain Bijou build an Overman to lead a exodus out of the city."}, {"m_title": "Phantom in the Twilight", "r": "5.5", "year": "2018", "votes": "128", "genre": "Adventure, Drama", "desp": "A girl in London encounters a cafe that opens only at night, which stands on the boundary of the worlds of humans and shadows."}, {"m_title": "Samurai Flamenco", "r": "6.5", "year": "2013", "votes": "311", "genre": "Action, Comedy", "desp": "A male model decides to become a superhero, and a police officer finds out his real identity. These two young men will come face to face with hardships of being crime-fighters while discovering what it truly means to be a hero of justice."}, {"m_title": "Fate/kaleid liner Prisma Illya 2wei", "r": "6.3", "year": "2014", "votes": "253", "genre": "Action, Comedy", "desp": "When things seem peaceful for Illya and Miyu, they are tasked by Rin and Luvia to settle a strange phenomenon in the city. But an even weirder thing happens when a girl who looks exactly like Illya appears."}, {"m_title": "Little Busters! Refrain", "r": "7.4", "year": "2013", "votes": "126", "genre": "Comedy, Drama", "desp": "Season 2 and the finally of Little Busters!"}, {"m_title": "Tonde Buurin", "r": "6.7", "year": "1994", "votes": "136", "genre": "Action, Adventure", "desp": "A young girl acquires the ability to become a super-powered alien pig. Not too pleased with being a pig, she has to become a super-powered girl by fighting crime and rescuing people."}, {"m_title": "Aria the Scarlet Ammo AA", "r": "5.6", "year": "2015", "votes": "130", "genre": "Action, Comedy", "desp": "Akari Mamiya, a first-year student from Tokyo Butei High, idolizes the S-rank Butei Aria Holmes Kanzaki and wishes to follow in her footsteps."}, {"m_title": "Hinako Note", "r": "7.1", "year": "2017", "votes": "106", "genre": "Comedy", "desp": "Hinako Sakuragi, a girl who just moved to Tokyo to attend Fujiyama High School as a freshman. She started living at Hitotose Bookstore which is also an apartment and a cafe along with her. watch the dude, no spoilers in summary!"}, {"m_title": "Warau Arsnotoria Sun!", "r": "6.4", "year": "2022", "votes": "57", "genre": "Action, Comedy", "desp": "This is the story of the fun-filled daily life of girls who strive to be true ladies, while being educated and trained in magic, and learning proper manners at the dorm-based magical academy city of Ashram."}, {"m_title": "Legends of the Dark King: A Fist of the North Star Story", "r": "6.6", "year": "2008", "votes": "206", "genre": "Action, Drama", "desp": "Focuses on the formation of an empire, specifically a hegemony, by an inheritor of the fist of the north star, Raoh. Accompanied by his two loyal comrades Souga and Reina, as well as a. watch the dude, no spoilers in summary!"}, {"m_title": "Star Ocean EX", "r": "6.8", "year": "2001", "votes": "94", "genre": "Action, Fantasy", "desp": "An anime television series based on Star Ocean: The Second Story."}, {"m_title": "Z/X: Ignition", "r": "6.0", "year": "2014", "votes": "46", "genre": "Adventure, Fantasy", "desp": "Five portals to different parallel worlds suddenly appear. Each must fight for survival with the key being the card device that can capture the invading Z/X creatures from the other worlds."}, {"m_title": "Myself; Yourself", "r": "6.6", "year": "2007", "votes": "353", "genre": "Drama, Romance", "desp": "For the sake of parents business plans, Hidaka Sana moved to Tokyo. Five years later, he has become a high school student, and returned to his hometown, Sakuranomori. He feels uneasy. watch the dude, no spoilers in summary!"}, {"m_title": "Otome youkai Zakuro", "r": "6.8", "year": "2010", "votes": "145", "genre": "Action, Comedy", "desp": "The magical-romantic adventures of a group of half-demon-girls aided by teen soldiers in their battles against demons in the Taisho period Japan."}, {"m_title": "Gorgeous Butterfly: Young Nobunaga", "r": "6.3", "year": "2019", "votes": "51", "genre": "Animation", "desp": "The series is a historical drama aesthetically and boldly adapting the life of Oda Nobunaga in his teens up to his time as a warlord against his brother, Nobuyuki."}, {"m_title": "Kâito jôkâ", "r": "7.4", "year": "2014", "votes": "50", "genre": "Adventure, Comedy", "desp": "Phantom thief Joker travels the world in search for new treasures too find and claim with his new Partner Hachi. Along the way they out smart police, team up with other Phantom thieves and fight evil forces."}, {"m_title": "Yuragi-sou no Yuuna-san OAD", "r": "6.7", "year": "2018", "votes": "7", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Norn9", "r": "5.8", "year": "2016", "votes": "160", "genre": "Drama, Romance", "desp": "A man is transported through time and space to a special starship, where he and 12 other people must work out what happened."}, {"m_title": "Major 2nd", "r": "7.6", "year": "2018", "votes": "59", "genre": "Sport", "desp": "Daigo, the son of renowned baseball player Gorou Shigeno, believes he has to live up to his fathers name by becoming an all-star player himself. With great dreams, as soon as he reaches. watch the dude, no spoilers in summary!"}, {"m_title": "Amanchu!", "r": "6.8", "year": "2016", "votes": "167", "genre": "Comedy, Drama", "desp": "Originally from a big city, Futaba Ooki spends most days cataloguing her life through pictures on her phone. Upon moving to a seaside town, she gazes out over the vast expanse of sea and. watch the dude, no spoilers in summary!"}, {"m_title": "Kill Me Baby", "r": "6.6", "year": "2012", "votes": "198", "genre": "Comedy, Family", "desp": "Yasuna Oribe, a student at school, is friends with a killer named Sonya. Yasuna is best friends with her, but Sonya tends to beat her up every time she falls for her pranks. Sonya is usually called the assassin in some episodes."}, {"m_title": "Vatican Miracle Examiner", "r": "6.5", "year": "2017", "votes": "87", "genre": "Mystery", "desp": "Two priests charged with traveling the world to validate miracles find themselves tangled in events filled with deceit. As members of the Assembly of Saints, Robert Nicholas and Kou Hiraga seek the truth behind the supposed works of God."}, {"m_title": "Shingeki no Kyojin: Picture Drama", "r": "5.8", "year": "2013", "votes": "189", "genre": "Short, Comedy", "desp": "Following the characters of Attack on Titan, this comedy Chibi show explores their days in training from a satirical angel."}, {"m_title": "Dream Eater Merry", "r": "6.3", "year": "2011", "votes": "163", "genre": "Action, Comedy", "desp": "Dream Eater Merry Episodes"}, {"m_title": "Whistle!: Dream Team", "r": "8.0", "year": "2002", "votes": "31", "genre": "Action, Drama", "desp": "Everybody has a dream, and for Shou Kazamatsuri, that dream is simple: he wants to be the best soccer player he can possibly be. Although he is not very good at the game and hes not very tall."}, {"m_title": "Shigofumi", "r": "7.1", "year": "2008", "votes": "186", "genre": "Drama, Fantasy", "desp": "A mail carrier girl named Fumiko and her talking staff named Kanaka deliver letters of final words from the recently deceased, known as shigofumi, to the living."}, {"m_title": "Extreme Hearts", "r": "6.5", "year": "2022", "votes": "43", "genre": "Music, Sport", "desp": "Hiyori Hayama is an aspiring singer who moved to Tokyo to make her debut. She has always loved singing and hopes to stand on stage like her favorite artist. Unfortunately, the entertainment. watch the anime dude!"}, {"m_title": "Yozakura karutetto", "r": "6.4", "year": "2008", "votes": "94", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Brigadoon: Marin to Melan", "r": "7.6", "year": "2000", "votes": "64", "genre": "Action, Adventure", "desp": "Set in Tokyo, 1969, a junior high school students life is changed forever when another world, filled with dangerous creatures, appears over planet earth."}, {"m_title": "Happiness Charge PreCure!", "r": "7.6", "year": "2014", "votes": "43", "genre": "Action, Adventure", "desp": "The ruler of the evil Phantom Empire, Queen Mirage, begins her invasion on Earth using an army of Choiarks and powerful monsters called Saiarks. All across the world, Pretty Cures are. watch the dude, no spoilers in summary!"}, {"m_title": "Neuro: Supernatural Detective", "r": "6.8", "year": "2007", "votes": "61", "genre": "Comedy, Mystery", "desp": "Yako Katsuragi is an acclaimed high school detective, who, despite her young age, has solved many cases. In fact, there seems to be no case that she cant solve, or any culprit that escapes. watch the anime dude!"}, {"m_title": "Night Head 2041", "r": "6.1", "year": "2021", "votes": "115", "genre": "Action, Crime", "desp": "The story follows the Kirihara brothers who from a young age were incarcerated in a secure scientific facility due to their supernatural powers, having escaped after the barrier that was preventing their malfunctions."}, {"m_title": "Binan Koukou Chikyuu Bouei-bu Love!", "r": "7.3", "year": "2015", "votes": "123", "genre": "Action, Adventure", "desp": "5 boys in high school become magical boys and fight evil with the power of love when they meet a pink wombat from outer space."}, {"m_title": "Vanguard OverDress", "r": "6.5", "year": "2021", "votes": "25", "genre": "Action", "desp": "sorry no summary not available!"}, {"m_title": "Kurau: Phantom Memory", "r": "7.3", "year": "2004", "votes": "162", "genre": "Action, Comedy", "desp": "Its the year 2100, and the moon is now home to a colony of scientists and pioneers. Its also host to dangerous experiments into strange new forms of energy, and it isnt long before one. watch the dude, no spoilers in summary!"}, {"m_title": "Azuki-chan", "r": "6.9", "year": "1995", "votes": "39", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Inari, Konkon, Koi Iroha", "r": "6.7", "year": "2014", "votes": "164", "genre": "Comedy, Fantasy", "desp": "Fushimi Inari is a shy, not-so-bright middle school girl living in Kyotos Fushimi ward. She has a crush on her classmate Tanbabashi, but cannot express her feelings. One day, as thanks for. watch the anime dude!"}, {"m_title": "Gegege no Kitarô", "r": "7.6", "year": "2007", "votes": "20", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Tenshi ni narumon", "r": "7.3", "year": "1999", "votes": "55", "genre": "Comedy, Fantasy", "desp": "A boy has his life turned upside-down when an extended family of underworld creatures moves into his house, among them a vampire, an invisible girl, and a hyper-active girl with a halo above her head named Noelle."}, {"m_title": "Wakakusa monogatari nan to Jô sensei", "r": "7.6", "year": "1993", "votes": "238", "genre": "Comedy, Drama", "desp": "The adventures and misadventures of Jo Bhaer and her pupils at their new Plumfield school-house."}, {"m_title": "Buddy Complex", "r": "7.0", "year": "2014", "votes": "140", "genre": "Action, Sci-Fi", "desp": "On the first day back after summer break, high-school student Aoba is attacked by a giant robot that appears out of the sky. As hes pursued through the city, his classmate Hina appears in. watch the dude, no spoilers in summary!"}, {"m_title": "Prima Doll", "r": "6.8", "year": "2022", "votes": "47", "genre": "Drama, Sci-Fi", "desp": "Café Kuronekotei boasts an unusual roster of employees. A group of automata, or autonomous mechanical dolls, serve their patrons with a smile, but they werent always so suited to domestic life."}, {"m_title": "Transformers: Beast Wars Metals", "r": "6.6", "year": "1999", "votes": "90", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Ef: A Tale of Melodies.", "r": "7.5", "year": "2008", "votes": "476", "genre": "Drama, Mystery", "desp": "In a story set years in the past, Himura Yuu is a studious and diligent young man intent solely on maintaining his top academic position at Otowa Academy. One day, he meets a mysterious. watch the dude, no spoilers in summary!"}, {"m_title": "To Heart", "r": "6.9", "year": "1999", "votes": "77", "genre": "Comedy, Drama", "desp": "Adapted from a dating sim game of the same name. Fujita Hiroyuki is an always sleepy and difficult to motivate high school boy, but he has a kind heart, and the gift to make friends with. watch the dude, no spoilers in summary!"}, {"m_title": "The Girl Who Leapt Through Space", "r": "7.0", "year": "2009", "votes": "43", "genre": "Adventure, Comedy", "desp": "Set far in the future, humans now live in space colonies orbiting far from Earth."}, {"m_title": "Leviathan: The Last Defense", "r": "6.0", "year": "2013", "votes": "37", "genre": "Adventure, Comedy", "desp": "The story is set in Aquafall, a fantasy world abound with water and greenery, and populated by dragons and fairies. Meteorites suddenly bring forth evil creatures that threaten all living. watch the dude, no spoilers in summary!"}, {"m_title": "Sorcerer Hunters OAV", "r": "6.3", "year": "1996", "votes": "97", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "RideBack", "r": "6.8", "year": "2009", "votes": "187", "genre": "Action, Drama", "desp": "Rin Yugata, a dancer, attends college after quitting ballet after an injury prevents her from performing in a play."}, {"m_title": "Rabu Oru Pure", "r": "6.9", "year": "2022", "votes": "87", "genre": "Sport", "desp": "The story follows Ryō Mizushima, who joined his middle schools badminton team full of motivation and he strives to become a top athlete and take his high school team to the inter-high tournament."}, {"m_title": "Interlude", "r": "6.5", "year": "2003", "votes": "49", "genre": "Adventure, Mystery", "desp": "A students typical teenage life is disrupted by disturbing dreams and bizarre occurrences. After meeting a strange girl that resides in an alternate reality, they set out together seeking. watch the dude, no spoilers in summary!"}, {"m_title": "Softenni!", "r": "7.7", "year": "2011", "votes": "11", "genre": "Comedy, Sport", "desp": "Asuna is a girl who loves tennis to the point where she practices her swings with a pitchfork while working on the family farm. At school she is a member of the Soft Tennis Club along with. watch the dude, no spoilers in summary!"}, {"m_title": "Kagewani", "r": "6.1", "year": "2015", "votes": "101", "genre": "Short, Horror", "desp": "sorry no summary not available!"}, {"m_title": "Majikano", "r": "6.2", "year": "2006", "votes": "88", "genre": "Animation", "desp": "Haruo Yoshikawas life goes from average to unbelievable when he learns his familys secret - theyre all witches!! Things get crazier with the arrival of Ayumi - an enchanting young girl. watch the dude, no spoilers in summary!"}, {"m_title": "Divergence Eve: Misaki Chronicles", "r": "6.6", "year": "2004", "votes": "27", "genre": "Drama, Horror", "desp": "A sequel to Divergence Eve (2003), it starts 2315 when Misaki is in the military academy so she can know more about her late dad who was a soldier/pilot."}, {"m_title": "Aooni the Blue Monster", "r": "5.3", "year": "2016", "votes": "55", "genre": "Comedy, Horror", "desp": "Rumors spread about a mansion and ao oni plays little tricks on his guess."}, {"m_title": "Shakugan no Shana III: Final", "r": "7.1", "year": "2011", "votes": "351", "genre": "Action", "desp": "Yuji disappeared the fateful night he was supposed to choose between a life combating evil by Shanas side or as a normal teenager. He returns from near-death to lead the Crimson Denizens. watch the dude, no spoilers in summary!"}, {"m_title": "Go! Princess PreCure", "r": "8.3", "year": "2015", "votes": "66", "genre": "Action, Adventure", "desp": "A long time ago, a young girl named Haruka Haruno dreams on becoming a Princess like the ones in fairy tales. However, she is always being bullied by her classmates in kindergarten due to. watch the dude, no spoilers in summary!"}, {"m_title": "Intrigue in the Bakumatsu: Irohanihoheto", "r": "6.9", "year": "2006", "votes": "117", "genre": "Action, Drama", "desp": "The Shogunate is in its final years, and war is fast approaching. When Akizuki Yojiro, a dark and mysterious mercenary, nears something supernatural with some kind of importance to him, the. watch the anime dude!"}, {"m_title": "Boys be...", "r": "6.1", "year": "2000", "votes": "181", "genre": "Comedy, Romance", "desp": "The lives and loves of a gang of high school friends, tracked over the school year of 1999."}, {"m_title": "Battle Athletes Victory", "r": "5.7", "year": "1997", "votes": "289", "genre": "Action, Adventure", "desp": "Akari Kanzaki has joined a girls academy to try to achieve her dream of going to the University Satellite and her dream is to win the Cosmo Beauty an she will need to work hard to do her best to win this."}, {"m_title": "Mashin Hero Wataru", "r": "7.4", "year": "1988", "votes": "134", "genre": "Adventure, Comedy", "desp": "Wataru Ikusabe gets his clay robot model transformed into a mighty Mecha to help save a Mystical World. Anime plays a lot like an adventure video game."}, {"m_title": "Neppû Kairiku Bushi Road", "r": "7.1", "year": "2013", "votes": "93", "genre": "Action, Fantasy", "desp": "An asteroid collision brought with it a deadly menace known as Shinbo lead humanity to retreat to land reclaimed from the sea. The one ray of hope is a device that can fight the Shinbo called The Mechanical Divine General."}, {"m_title": "Sei jûshi Bisumaruku", "r": "6.4", "year": "1984", "votes": "43", "genre": "Action, Adventure", "desp": "In the age of space colonies, four ace pilots and sharp shooters from different nations and backgrounds join forces to fend off the alien threat of Deathcula. Together they pilot the powerful giant robot Bismarck."}, {"m_title": "Luna Varga", "r": "7.2", "year": "1991", "votes": "16", "genre": "Adventure, Comedy", "desp": "A princess finds a ancient power and it allows her to summon a dinosaur demon that will obey her. However, she gets attached to its head and she is his brain and she must save her captured sister."}, {"m_title": "Spiral: The Bonds of Reasoning", "r": "7.1", "year": "2002", "votes": "269", "genre": "Crime, Drama", "desp": "A teen amateur detective must use his amazing powers of logical deduction to learn about the mysterious Blade Children and stop them."}, {"m_title": "Persia, the Magic Fairy", "r": "7.3", "year": "1984", "votes": "57", "genre": "Family, Comedy", "desp": "Little girl Pelsia can transform into a pretty teenager and tries to bring happiness to the world."}, {"m_title": "Cestvs: The Roman Fighter", "r": "6.7", "year": "2021", "votes": "71", "genre": "Action, Drama", "desp": "54 AD. Cestus, a young boy orphaned by the Roman empire and made a slave, is placed into a training school for pugilists. It is here that he begins his journey to defy fate and fight for his own freedom."}, {"m_title": "Shadow Skill - Eigi", "r": "7.2", "year": "1998", "votes": "101", "genre": "Action, Adventure", "desp": "In the land of Kuruda, warriors with magical powers and incredible fighting skills battle for the ultimate prize: the title of Sevaar, the strongest warrior in the land. Elle Ragu,. watch the dude, no spoilers in summary!"}, {"m_title": "Glass Fleet: La Legende du Vent de lUnivers", "r": "6.9", "year": "2006", "votes": "79", "genre": "Adventure, Sci-Fi", "desp": "A tyrannical Emperor has his grasp over the galaxy. The rebels outnumbered and outgunned try to locate the mythical Glass Fleet that once served the true ruler. They do find one of the royal descendants, now a space pirate."}, {"m_title": "Ah! My Goddess: Flights of Fancy", "r": "7.0", "year": "2006", "votes": "160", "genre": "Animation", "desp": "Set one year after Aa. Megami-sama!. The song may say thank heaven for little girls, but whom is college student Morisato Keiichi supposed to thank when three honest to goodness Goddesses. watch the dude, no spoilers in summary!"}, {"m_title": "Zone of the Enders: Dolores, i", "r": "7.5", "year": "2001", "votes": "103", "genre": "Action, Adventure", "desp": "James Links is a 49-year-old space trucker anxious to return to Earth and reunite with his estranged children Leon and Noel. While transporting some goods to Earth, he stumbles upon. watch the dude, no spoilers in summary!"}, {"m_title": "Scar on the Praeter", "r": "5.9", "year": "2021", "votes": "56", "genre": "Action, Sci-Fi", "desp": "In the wake of a rebellion in the Akatsuki Special Ward of Tokyo, agents known as the Scard rise up to become the protectors of peace."}, {"m_title": "Sukaruman", "r": "6.7", "year": "2007", "votes": "81", "genre": "Action, Horror", "desp": "A young man named Hayato, revisits his old hometown and got involved into the mysteries around masked murderer."}, {"m_title": "Tales of Phantasia: The Animation", "r": "6.2", "year": "2004", "votes": "26", "genre": "Short, Adventure", "desp": "If there is evil in this world, it lurks in the hearts of men. In the land of Midgard a bloodthirsty conqueror breaks the seal holding Dhaos, the Demon King whose goal is to eradicate all. watch the dude, no spoilers in summary!"}, {"m_title": "Yôkai ningen Bem", "r": "8.0", "year": "2006", "votes": "10", "genre": "Action, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Galerians: Rion", "r": "5.6", "year": "2002", "votes": "165", "genre": "Action, Drama", "desp": "A boy awakes, imprisoned in a hospital without a name or memory. He is immediately attacked and discovers to his surprise that he has incredible telekinetic powers. A girl, her psychic. watch the dude, no spoilers in summary!"}, {"m_title": "Denki-Gai no Honya-san", "r": "6.6", "year": "2014", "votes": "139", "genre": "Comedy", "desp": "A young and shy boy named Umio, started to work for a comic book store. All his other friends already work there, and they get into some new trouble every week."}, {"m_title": "Sacks & Guns!!", "r": "6.0", "year": "2021", "votes": "158", "genre": "Sci-Fi", "desp": "In distant future, long after the Western calendar is no longer used, humans live shoulder-to-shoulder, cramped in the Labyrinth, an underground world deep below the surface. The place is. watch the anime dude!"}, {"m_title": "Argentosoma", "r": "6.6", "year": "2000", "votes": "160", "genre": "Action, Adventure", "desp": "Undergoing an identity shift following a horrible laboratory accident involving an alien creature known as Frank, a young man joins an anti-alien task force to exact his revenge."}, {"m_title": "Magical Project S", "r": "7.6", "year": "1996", "votes": "74", "genre": "Comedy, Fantasy", "desp": "In the magical kingdom Juraihelm, Tsunami is in line to be crowned as queen but must complete a test first. She must choose someone to make the world a better place."}, {"m_title": "Shinkansen Henkei Robo Shinkalion the Animation", "r": "7.1", "year": "2018", "votes": "17", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Machine Robo: Revenge of Cronos", "r": "6.9", "year": "1986", "votes": "25", "genre": "Action, Adventure", "desp": "Taking place in the middle of the Wolf Sword Legend feature, this story centers on Leina Haruka, whose plane has been hijacked by a group of thugs led by the supernatural wizard Zarik."}, {"m_title": "Ray the Animation", "r": "6.9", "year": "2006", "votes": "33", "genre": "Drama", "desp": "A female surgeon has the eyes to see inside a patient to find out what is wrong with them."}, {"m_title": "Ginga Kikoutai: Majestic Prince", "r": "7.0", "year": "2013", "votes": "98", "genre": "Animation", "desp": "In the half of 21st century, humans leave the Earth and live in space. In order to adapt to the environment and deal with the hostile aliens, genetically engineered children are artificially raised and trained to be pilots of armed robots."}, {"m_title": "Honobono Log", "r": "6.7", "year": "2016", "votes": "146", "genre": "Short, Romance", "desp": "Very short but beautifully drawn episodes that depict mild crises and heartwarming moments from the lives of happy couples and families, this series was supposedly made to teach the otaku audience better interpersonal skills."}, {"m_title": "Wave!!: Surfing Yappe!!", "r": "6.6", "year": "2021", "votes": "70", "genre": "Animation", "desp": "The projects story is set in the city of Oarai in Ibaraki prefecture, and centers on Masaki Hinaoka, who befriends transfer student Sho Akitsuki before the summer break and ends up getting. watch the anime dude!"}, {"m_title": "Dodge Danpei", "r": "7.2", "year": "1991", "votes": "462", "genre": "Adventure, Comedy", "desp": "A 10 years old red-haired kid want be like his father and become in the best dodge ball player of all the times."}, {"m_title": "RD: Sennô chôsashitsu", "r": "6.6", "year": "2008", "votes": "84", "genre": "Action, Crime", "desp": "In the near future, social media, security, administrative and communication systems have been replaced with Meta Real Network or Metal for short, a VR network. Cyber Divers, a special police unit, deal with those who abuse this network."}, {"m_title": "Patapata hikousen no bouken", "r": "8.5", "year": "2002", "votes": "82", "genre": "Adventure, Drama", "desp": "The plot is about a young girl who is looking for her older brother. The girl goes on an adventure in a world verry diffrent from our own. The series is inspired by some of Jules Vernes books."}, {"m_title": "Earl and Fairy", "r": "6.7", "year": "2008", "votes": "309", "genre": "Comedy, Drama", "desp": "While studying fairies in Victorian England, young beautiful Lydia meets a mysterious Earl named Edgar."}, {"m_title": "Miimu iro iro yume no tabi", "r": "8.2", "year": "1983", "votes": "135", "genre": "Adventure, Family", "desp": "A little pink genie from a computer, helps two kids learn about important people and events from history by taking them to those times."}, {"m_title": "President, Its Time for Battle!", "r": "6.0", "year": "2020", "votes": "77", "genre": "Adventure, Fantasy", "desp": "Set in an alternative world where the player acts as the president to recruit and train adventurers. The adventurers are then dispatched into dungeons to fight monsters and recover loot."}, {"m_title": "The Urara of Labyrinth Town", "r": "6.7", "year": "2017", "votes": "122", "genre": "Comedy, Fantasy", "desp": "Chiya enters the town of Meirocho to begin her training as a fortune tellers apprentice, and, hopefully, become the next Urara!"}, {"m_title": "Angel Heart", "r": "7.4", "year": "2005", "votes": "97", "genre": "Action, Comedy", "desp": "A young Taiwanese assassin code named Glass Heart committed suicide by jumping off a building, and her heart was pierced by metal fence. Miraculously, her life was saved by heart. watch the dude, no spoilers in summary!"}, {"m_title": "Tetsujin 28", "r": "7.4", "year": "2004", "votes": "53", "genre": "Action, Adventure", "desp": "In Post-WWII Japan, boy detective Shotaro overcomes adversaries with his giant robot Tetsujin 28-go, uncovering remnants from the War that had been bubbling under the surface of society."}, {"m_title": "Hyper Police", "r": "7.6", "year": "1997", "votes": "117", "genre": "Action, Comedy", "desp": "It is the year 22 H. C. (Holy Century), Shinjuku is populated by monsters and goblins. Humans are endangered and the town is rife with crime."}, {"m_title": "Indian Summer", "r": "6.2", "year": "2007", "votes": "21", "genre": "Comedy, Romance", "desp": "When Murase Takaya, an average and somewhat perverted boy, decides to buy a robot maid to keep him company, it is the robot that chooses him. However, when the maid is delivered, her. watch the dude, no spoilers in summary!"}, {"m_title": "Mamotte! Lollipop", "r": "6.8", "year": "2006", "votes": "53", "genre": "Animation", "desp": "A normal, average teen girl accidently swallows something called a crystal pearl, which many sorcerers are after! As she is in danger, 2 boys come down and vow to protect her. She is no longer an ordinary girl!"}, {"m_title": "Chimpui", "r": "7.9", "year": "1989", "votes": "24", "genre": "Comedy, Sci-Fi", "desp": "Chimpui tells the story of two cute aliens that come to Earth to find the girl who will get married with their prince of planet Mahl."}, {"m_title": "Aikatsu Stars!", "r": "8.2", "year": "2016", "votes": "169", "genre": "Adventure, Comedy", "desp": "Yume Nijino aims to become a top idol and enrolls in the Yotsusei Gakuen. This academy has a special group called the S4, who are the top four active idols in the school. Yume and the other. watch the anime dude!"}, {"m_title": "Stellvia", "r": "6.5", "year": "2003", "votes": "141", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Yu Yu Hakusho: Eizou Hakusho", "r": "7.8", "year": "1995", "votes": "211", "genre": "Short, Comedy", "desp": "Various clips of Yu Yu Hakusho stories featuring Yusuke, Kurama, Hiei, and Kuwabara, including new interviews with the Japanese voice actors."}, {"m_title": "Chosoku supina", "r": "7.0", "year": "1998", "votes": "32", "genre": "Action, Sport", "desp": "A 5th grade boy enters a Yo-Yo tournament."}, {"m_title": "Ultra Maniac", "r": "7.6", "year": "2002", "votes": "84", "genre": "Comedy, Fantasy", "desp": "Finding the perfect relationship isnt easy for anyone. Mr. Mikami tries to make a move on Ms. Tamura with a new vest while Kaji finds himself stalked by Mayas depressing friend. Not only. watch the dude, no spoilers in summary!"}, {"m_title": "Looking Up at the Half-Moon", "r": "6.9", "year": "2006", "votes": "191", "genre": "Comedy, Drama", "desp": "The story of Hantsuki focuses on the budding relationship between the seventeen year old Yuichi Ezaki and Rika Akiba. They are hospitalized in Yuichis home town for their conditions.. watch the dude, no spoilers in summary!"}, {"m_title": "Lensman: Galactic Patrol", "r": "6.2", "year": "1984", "votes": "44", "genre": "Sci-Fi, Action", "desp": "sorry no summary not available!"}, {"m_title": "Legendary Idol Eriko", "r": "7.5", "year": "1989", "votes": "63", "genre": "Drama, Music", "desp": "Erika is the daughter of a record and media company. Things turn a rye and an on off rivalry begins as Erika tries to deal with becoming an idol herself and learning how to deal with other. watch the dude, no spoilers in summary!"}, {"m_title": "Tokyo Underground", "r": "6.4", "year": "2002", "votes": "63", "genre": "Action, Adventure", "desp": "Under the capital city of Tokyo, Japan, there exists a large, vast, and unknown world known as Underground. There, people known as Elemental Users exist; people who have the ability to. watch the dude, no spoilers in summary!"}, {"m_title": "Wake Up, Girls!", "r": "6.5", "year": "2014", "votes": "92", "genre": "Comedy, Drama", "desp": "Green Leaves Entertainment is a tiny production company on the verge of going out of business. The president Tange hatches an idea of producing an idol group. On the brash presidents. watch the dude, no spoilers in summary!"}, {"m_title": "Megane na Kanojo", "r": "6.5", "year": "2010", "votes": "18", "genre": "Comedy, Romance", "desp": "Four unrelated short stories regarding four bespectacled girls and the love of their lives."}, {"m_title": "Student Councils Discretion", "r": "6.3", "year": "2009", "votes": "79", "genre": "Comedy, Drama", "desp": "The student council of Hekiyou Gakuen is chosen by popular vote, so of course all the members are cute girls. All except Ken Sugisaki, that is. Ken managed to get a spot by being the top. watch the dude, no spoilers in summary!"}, {"m_title": "Slow Start", "r": "6.2", "year": "2018", "votes": "114", "genre": "Comedy, Drama", "desp": "Hana Ichinose, a 17 years old girl starting her high school life in a new school. She is a Rounin, a student who had a gap year who was forced to miss her high school entrance exam due to. watch the anime dude!"}, {"m_title": "Infini-T Force", "r": "7.4", "year": "2017", "votes": "45", "genre": "Action, Sci-Fi", "desp": "A lonely girl comes across a magic pencil that brings heroes from another dimension to her world."}, {"m_title": "Kamisama Dolls", "r": "6.7", "year": "2011", "votes": "131", "genre": "Action, Drama", "desp": "Kyohei Kuga moves to Tokyo looking for a new life. However, the discovery of a bloody corpse is linked to the life he was trying to escape. Kyohei is a seki with the ability to. watch the dude, no spoilers in summary!"}, {"m_title": "H2O: Footprints in the Sand", "r": "6.1", "year": "2008", "votes": "147", "genre": "Comedy, Drama", "desp": "Hirose is a blind young male high school student, with mystery cause. In order to heal his medical condition, he is sent to live in a village with his uncle. There, he meets several girls that will change his life."}, {"m_title": "Hayate, the Combat Butler 2", "r": "7.3", "year": "2009", "votes": "150", "genre": "Action, Comedy", "desp": "Hayate is still working as the Sanzenin Butler and this season features a lot more Hinagiku, as scenarios that include herself and Hayate allude to potentially mutual feelings between them.. watch the anime dude!"}, {"m_title": "Saiyuki Reload Blast", "r": "7.7", "year": "2017", "votes": "109", "genre": "Action, Adventure", "desp": "The Sanzo Party has finally reached the West of Tougenkyou and encounter new enemies. It also reveals some truth about their past life."}, {"m_title": "Kenkô zenrakei suieibu umishô", "r": "6.9", "year": "2007", "votes": "103", "genre": "Comedy, Romance", "desp": "A student that fears water due to a childhood drowning incident is manager of the high school swim team. A carefree girl that is a swimming prodigy transfers to the school and after her houseboat is destroyed she begins living with him."}, {"m_title": "RIN-NE", "r": "6.6", "year": "2015", "votes": "164", "genre": "Action, Comedy", "desp": "As a child Sakura Mamiya mysteriously disappeared in the woods. She returned safe and sound, but since then she has had the power to see ghosts. At school, her always-absent classmate shows up, and hes far more than what he seems."}, {"m_title": "Libra of Nil Admirari", "r": "7.2", "year": "2018", "votes": "52", "genre": "Fantasy", "desp": "In an alternate timeline, Japans Taisho era lasted until 1936 instead of ending in 1926. During this time, Tsugumi Kuze agrees to an arranged marriage to save her familys honor, but. watch the dude, no spoilers in summary!"}, {"m_title": "Knyacki", "r": "8.2", "year": "1995", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sunday Without God", "r": "6.4", "year": "2013", "votes": "291", "genre": "Fantasy, Mystery", "desp": "Since 15 years ago, no new lives are born, and no one can die. And then, gravekeepers appeared in the world, with the ability to give rest to the living dead. Therefore the people said,. watch the dude, no spoilers in summary!"}, {"m_title": "Magatsu Wahrheit: Zuerst", "r": "7.1", "year": "2020", "votes": "98", "genre": "Action, Fantasy", "desp": "Two young people living in the Wahrheit Empire are planning to relocate to the capital city: the shy Inumael who works as a transporter and left his beloved sister behind at his parents. watch the dude, no spoilers in summary!"}, {"m_title": "Rentaru magika", "r": "6.7", "year": "2007", "votes": "71", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Zan sayonara zetsubô sensei", "r": "7.7", "year": "2009", "votes": "261", "genre": "Comedy", "desp": "The continuing adventures of the despairing teacher Nozomu Itoshiki and his class of deranged students."}, {"m_title": "Ghost Slayers Ayashi", "r": "6.5", "year": "2006", "votes": "75", "genre": "Fantasy, History", "desp": "Feudal Japan is attacked by beasts called Youi. Ryuudou Yukiatsu, a floater and an employee at a bath house has the power of Ayagami (The power to find the words behind things) which. watch the dude, no spoilers in summary!"}, {"m_title": "Locodol", "r": "8.2", "year": "2014", "votes": "87", "genre": "Comedy", "desp": "In the town of Nagarekawa, Nanako Usami, an ordinary high school girl, is approached by her uncle to become a local idol or Locodol."}, {"m_title": "Devil Survivor 2: The Animation", "r": "6.2", "year": "2013", "votes": "251", "genre": "Action, Fantasy", "desp": "A catastrophic event announces the existence of a powerful alien being that wants to destroy the world. The event also gives a group of children the ability to summon demons and use them to fight the alien force."}, {"m_title": "Beet the Vandel Buster", "r": "6.6", "year": "2004", "votes": "25", "genre": "Action, Adventure", "desp": "The world is terrorized by demon lord Vandel and its brethren. A team of demon hunters known as Busters are worlds only hope. They lose, but a young boy gets their weapons and trains to lead a new team of Busters against Vandel."}, {"m_title": "Video Senshi Laserion", "r": "8.2", "year": "1984", "votes": "103", "genre": "Action, Adventure", "desp": "Takashi Katori is a big fan of online games, and so is his American friend Sarah. However, one of their RPGs goes too far, as their virtual world and the Lezarion robot that Takashi created. watch the anime dude!"}, {"m_title": "Fist of the Blue Sky: Re: Genesis", "r": "4.3", "year": "2018", "votes": "67", "genre": "Action", "desp": "Long before Kenshiro said, Youre already dead, a different Kenshiro walked the violent streets of Shanghai. The city is in constant crisis as two mafia families battle one another for. watch the dude, no spoilers in summary!"}, {"m_title": "Lady Jewelpet", "r": "7.7", "year": "2014", "votes": "17", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "A Thirty-Year Olds Health and Physical Education", "r": "5.8", "year": "2011", "votes": "94", "genre": "Comedy, Fantasy", "desp": "A 30 years old man and a 30 years old woman are so desperate that Gods of sexual love appears in front of them in order to help them to get graduated from their virginity."}, {"m_title": "Tongari boshi no Memoru", "r": "6.9", "year": "1984", "votes": "196", "genre": "Family, Fantasy", "desp": "A group of 245 inhabitants from a planet called Riruru make a crash landing onto Earth from their spaceship. This is due to a failure in the spaceship. Aboard the spaceship is a young tiny girl named Memole which the series is centered on."}, {"m_title": "Winter Sonata", "r": "6.7", "year": "2009", "votes": "32", "genre": "Drama, Romance", "desp": "Yoo Jin falls in love with Joon Sang as a young girl. After losing him in an accident, she decides to marry her childhood friend. However, Yoo Jin meets someone looking exactly like her lost love, putting her in a bind."}, {"m_title": "Ultimate Girls", "r": "7.0", "year": "2005", "votes": "28", "genre": "Action, Comedy", "desp": "During a monster attack on the city Silk and her two friends are accidentally crushed to death by the hero UFO-man, he uses his abilities to resurrect them as super-heroines powered by embarrassment."}, {"m_title": "Kemono no Souja Erin", "r": "7.6", "year": "2009", "votes": "219", "genre": "Adventure, Drama", "desp": "In the land of Ryoza, the neighboring provinces of Shin-Ou and Tai-Kou have been at peace. Queen Shinou is the ruler of Ryoza and her greatest general, Grand Duke Taikou, defends the. watch the dude, no spoilers in summary!"}, {"m_title": "Yoroshiku meka dokku", "r": "6.9", "year": "1984", "votes": "14", "genre": "Action, Comedy", "desp": "Three young mechanics open a sports car preparation workshop and experience numerous motoring adventures. One of them is also a pilot and will make his way into the world of Japanese racing. watch the anime dude!"}, {"m_title": "Magical Emi, the Magic Star", "r": "6.3", "year": "1985", "votes": "144", "genre": "Family, Comedy", "desp": "Mai can transform into a famous magician star Magical Emi..."}, {"m_title": "RPG Fudousan", "r": "7.1", "year": "2022", "votes": "79", "genre": "Comedy, Fantasy", "desp": "An estate agency in an RPG environment aids in helping various fantasy creatures find a suitable home."}, {"m_title": "Osomatsu-kun", "r": "8.0", "year": "1988", "votes": "29", "genre": "Comedy", "desp": "The adventures of young Osomatsu Matsuno and his five twin brothers."}, {"m_title": "High School! Kimengumi", "r": "7.4", "year": "1985", "votes": "98", "genre": "Comedy, Romance", "desp": "College anime that follows a group of friends who solve the strange situations in which they get into with a lot of humor. All amongst very stereotyped characters."}, {"m_title": "Estab Life: Great Escape", "r": "6.7", "year": "2022", "votes": "42", "genre": "Action, Sci-Fi", "desp": "The distant future. After reaching its peak, Earths population fell into decline. AI was developed to help preserve the species and manage the ecosystem; a grand experiment in human. watch the dude, no spoilers in summary!"}, {"m_title": "Graveyard Kitaro", "r": "7.0", "year": "2008", "votes": "67", "genre": "Horror", "desp": "sorry no summary not available!"}, {"m_title": "Battle Girl High School", "r": "5.1", "year": "2017", "votes": "26", "genre": "Action, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "LBX Girls", "r": "7.1", "year": "2021", "votes": "32", "genre": "Action, Sci-Fi", "desp": "In an unplanned field trip, Riko is transported to an alternate Japan where metal-based life forms known as Mimesis ravage the world. Only girls equipped with LBX armored weaponry can stand up to this scourge."}, {"m_title": "Es Otherwise", "r": "6.2", "year": "2003", "votes": "81", "genre": "Action, Fantasy", "desp": "Based on the super-popular manga, Es Otherwise is the story of a latent psychic brought into the fold of a shadow-y crime-fighting corporation. Kai Kudou is a superhuman with unique. watch the dude, no spoilers in summary!"}, {"m_title": "Day Break Illusion", "r": "6.0", "year": "2013", "votes": "35", "genre": "Fantasy, Thriller", "desp": "12 year old Akari is a fortune teller whose fortunes are always spot on. She uses Tarot cards, which are cards she received after she passed away. After the anniversary of the passing of. watch the dude, no spoilers in summary!"}, {"m_title": "Kaiketsu zorori", "r": "7.8", "year": "2004", "votes": "50", "genre": "Adventure, Comedy", "desp": "To fulfill his mothers last wish, a journeying cunning fox desperately tries to find his own castle and wife."}, {"m_title": "Tytania", "r": "6.9", "year": "2008", "votes": "78", "genre": "Adventure, Drama", "desp": "In a future where mankind is scattered across the stars, the Tytania clan fights a war with the city-state of Euriya over a new piece of technology."}, {"m_title": "Baribari Densetsu", "r": "7.1", "year": "1986", "votes": "23", "genre": "Action, Sport", "desp": "The film is about 4 high school seniors, who race on the mountain roads at night in order to refine their motorcycle skills build up their skill in an attempt to become professional motorcycle racers."}, {"m_title": "H2", "r": "7.4", "year": "1995", "votes": "10", "genre": "Comedy, Romance", "desp": "Hiro has two loves: baseball and porn, but due to an elbow injury, gives up baseball, choosing a school with no baseball team. His childhood friend Hikari attends a different school with. watch the dude, no spoilers in summary!"}, {"m_title": "Shattered Angels", "r": "6.4", "year": "2007", "votes": "80", "genre": "Action, Drama", "desp": "Shattered Angels revolves around the life of Kuu Shiratori, a seemingly normal highschool girl who enjoys her school life in the giant city Academica and dreams of finding an handsome prince."}, {"m_title": "Before Green Gables", "r": "8.2", "year": "2009", "votes": "86", "genre": "Drama", "desp": "The story of what happened to Anne Shirley as orphan during her childhood."}, {"m_title": "Mirai shônen Konan 2 - Taiga daibôken", "r": "7.3", "year": "1999", "votes": "38", "genre": "Adventure, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Haruka: Beyond the Stream of Time - A Tale of the Eight Guardians", "r": "6.8", "year": "2004", "votes": "43", "genre": "Drama, Fantasy", "desp": "Akane and her friends are dragged by a demon to the Heian era, where she becomes the Priestess of the Dragon God. There, she must stop demons from taking over the land with the eight guardians sworn to protect her."}, {"m_title": "Hell Girl: Two Mirrors", "r": "7.9", "year": "2006", "votes": "469", "genre": "Drama, Fantasy", "desp": "Ai Enma once again returns as the Jigoku Shoujo, who uses Jigoku Tsuushin (Hell Hotline) to allow people to post their grudges on their tormentors, sending them straight to Hell. In. watch the dude, no spoilers in summary!"}, {"m_title": "Gensou Sangokushi: Tengen Reishinki", "r": "5.0", "year": "2021", "votes": "34", "genre": "Fantasy", "desp": "In spite of their defeat, the remnant factions of the Taipingtao Revolution continue to resist, plunging China into a seemingly never-ending war. Moreover, the negative emotions left behind. watch the anime dude!"}, {"m_title": "Suzumiya Haruhi chan no yûutsu", "r": "6.6", "year": "2009", "votes": "438", "genre": "Comedy", "desp": "A parody of the original show. Based on the parody manga series."}, {"m_title": "Tegami Bachi Reverse", "r": "7.3", "year": "2010", "votes": "73", "genre": "Adventure, Drama", "desp": "A young boy with silver hair is shocked and frightened when a group of mysterious men burst into his home and kidnap his mother. That same day, a handsome young mail man with similar silver. watch the anime dude!"}, {"m_title": "Dai-Guard", "r": "7.4", "year": "1999", "votes": "78", "genre": "Action, Adventure", "desp": "Thirteen years after their sudden disappearance, an alien race known as the Heterodyne resurface without warning. To combat the Heterodyne, three office workers from the 21st Century. watch the dude, no spoilers in summary!"}, {"m_title": "Toei Robot Girls Z", "r": "6.0", "year": "2014", "votes": "46", "genre": "Action, Comedy", "desp": "At the Nerima Ward Ooizumi Academy Photonic District, the Robot Girls are selling Photonic Energy, a new type of cheap and green power. The labs research has been going nowhere and. watch the dude, no spoilers in summary!"}, {"m_title": "Samurai Girl: Real Bout High School", "r": "5.6", "year": "2001", "votes": "74", "genre": "Action, Comedy", "desp": "Teens at a high school settle their problems by dueling with each other."}, {"m_title": "Bubu Chacha", "r": "7.2", "year": "1996", "votes": "35", "genre": "Adventure, Family", "desp": "A young boys dog is run over, but is reincarnated into a toy car."}, {"m_title": "Monster Strike", "r": "4.8", "year": "2015", "votes": "50", "genre": "Action", "desp": "Ren Homura, a middle school student who moves back to his hometown with his mother and sister, but he apparently cannot remember much of his life there. After having to get his cellphone. watch the dude, no spoilers in summary!"}, {"m_title": "Double Decker! Doug & Kirill", "r": "6.7", "year": "2018", "votes": "119", "genre": "Animation", "desp": "Doug Billingham, a seasoned investigator, is partnered with rookie cop Kirill Vrubel to take the unpredictable synthetic drug, Anthem, off the streets of Lisvalletta with the help of the SEVEN-O Special Crime Investigation Unit."}, {"m_title": "Sabans Adventures of the Little Mermaid", "r": "6.7", "year": "1991", "votes": "127", "genre": "Family, Fantasy", "desp": "In this Japanese/French anime series, we follow the many adventures of Marina, who first wanted to be human under Hedwig the sea hags spell. After her spell breaks, due to a snotty. watch the dude, no spoilers in summary!"}, {"m_title": "Yugo the Negotiator", "r": "6.7", "year": "2004", "votes": "37", "genre": "Action, Drama", "desp": "A college student finds a book in his schools garden. Little does he know, this book is the key to control and manipulate death itself. Yugo the Negotiator follows the story of his rise and demise."}, {"m_title": "Space Battleship Tiramisu", "r": "6.4", "year": "2018", "votes": "64", "genre": "Comedy, Sci-Fi", "desp": "In preparation for his vital wartime missions, ace pilot Subaru Ichinose launches an all-out food-based assault on his own face."}, {"m_title": "Ring ni kakero", "r": "7.0", "year": "2003", "votes": "27", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Captain Earth", "r": "6.0", "year": "2014", "votes": "110", "genre": "Action, Sci-Fi", "desp": "High-school student Daichi Manatsu works for the Globe organization to pilot a giant robot called the Earth Engine Impacter to protect the Earth from the invading alien force known as the Kill-T-Gang, from the planet Uranus."}, {"m_title": "Angel Tales", "r": "7.0", "year": "2001", "votes": "31", "genre": "Comedy", "desp": "A down on his luck guy is told by a fortune teller that his luck will change. Soon after three girls appear claiming to be his guardian angels."}, {"m_title": "Tattoon Master", "r": "6.5", "year": "1996", "votes": "24", "genre": "Adventure, Comedy", "desp": "A anthropologist runs into a tribe and after the tribe leader sees a photo of the anthropologists son, Hibio, the tribe leader wants to become the wife to him."}, {"m_title": "Princess Nine", "r": "7.7", "year": "1998", "votes": "116", "genre": "Drama, Family", "desp": "The story of a baseball team formed at an all-girls high school competing to enter and win the national championship."}, {"m_title": "Kasumin", "r": "8.0", "year": "2001", "votes": "9", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Tales of Eternia: The Animation", "r": "5.9", "year": "2001", "votes": "38", "genre": "Action, Adventure", "desp": "In order to prevent the catastrophe, the Grand Fall, Rid and his companions Farah, Keel and Meredy have obtained the Greater Spirits. A series of adventures forces them to remain on the islands, where a more immediate threat to Inferia."}, {"m_title": "Allison & Lillia", "r": "7.6", "year": "2008", "votes": "85", "genre": "Action, Adventure", "desp": "Follows a young woman then, later, her daughter as they get caught up in political intrigue, assassination attempts, war ending adventures and their own romantic involvements."}, {"m_title": "Rozen Maiden: Zurückspulen", "r": "6.9", "year": "2013", "votes": "133", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Anima Yell!", "r": "7.8", "year": "2018", "votes": "99", "genre": "Comedy, Sport", "desp": "Kohane Hatoya, a girl who is constantly helping others, takes an interest in cheerleading. She decides to start her own Cheer Club upon entering high school, recruiting her best friend Uki Sawatari and veteran cheerleader Hizume Arima."}, {"m_title": "Regalia: The Three Sacred Stars", "r": "7.5", "year": "2016", "votes": "19", "genre": "Action, Sci-Fi", "desp": "When a large mecha attacks the Enastria Empire, sisters Yui and Rena are pulled from their peaceful life and into avortex of destiny!"}, {"m_title": "RobiHachi", "r": "5.9", "year": "2019", "votes": "121", "genre": "Comedy, Sci-Fi", "desp": "Two losers set out in search of a planet of happiness."}, {"m_title": "Twin Princess of Wonder Planet", "r": "5.4", "year": "2005", "votes": "41", "genre": "Action, Adventure", "desp": "In the hollow Mysterious Planet, seven kingdoms co-exist, all of them lit by the Suns Blessing from the Kingdom of the Sun, in the middle of the planet. The light of the core sun is. watch the dude, no spoilers in summary!"}, {"m_title": "Disgaea", "r": "6.1", "year": "2006", "votes": "114", "genre": "Action, Comedy", "desp": "Prince Laharl is awakened after 2 years of sleep by an Angel, Flonne, who was ordered to assassinate him but fails. Both discover Laharls father passed away. Theyre later joined by Etna,. watch the dude, no spoilers in summary!"}, {"m_title": "Classroom Crisis", "r": "6.7", "year": "2015", "votes": "101", "genre": "Drama, Sci-Fi", "desp": "With aims to one day reach out into the stars, humanity has colonized all the planets of the solar system. This story of young high school hopefuls is set in Fourth Tokyo in a Japanese. watch the dude, no spoilers in summary!"}, {"m_title": "Starship Operators", "r": "7.1", "year": "2005", "votes": "114", "genre": "Drama, Sci-Fi", "desp": "Starship Operators of the Planet Kibi are returning home after the maiden voyage of the Amaterasu, they find that their home planet has been taken over by Henrietta Alliance of Planetary Nations."}, {"m_title": "The Familiar of Zero: Rondo of Princesses", "r": "7.1", "year": "2008", "votes": "786", "genre": "Action, Adventure", "desp": "Following Saitos heroic actions in the war with Albion in the second season, Saito and Louise return to Tristain, only to have the runes which bind Saito and Louise disappear. Saito and. watch the dude, no spoilers in summary!"}, {"m_title": "Kosoku Denjin Albegas", "r": "7.8", "year": "1983", "votes": "59", "genre": "Action, Sci-Fi", "desp": "The Gerinya Empire attacks the Earth after hundreds of years of peace. The last hope seems to be Dr. Mizukis last creation, the trio of Arbegas robots, able to combine and form one mighty metal giant."}, {"m_title": "Aokana: Four Rhythm Across the Blue", "r": "6.3", "year": "2016", "votes": "126", "genre": "Drama, Sci-Fi", "desp": "Set in a world where people can take to the skies, soaring above land and sea, an area called the Four Islands Archipelago has a technology called Grav-Shoes, which allow the wearer to fly.. watch the anime dude!"}, {"m_title": "Moeyo Ken TV", "r": "6.5", "year": "2005", "votes": "42", "genre": "Action, Comedy", "desp": "Set in an alternate universe Meiji Era, the lead male character of the story, Ryunosuke Sakamoto, is travelling from Shanghai towards Kyoto with his newly-licensed y?kai Nekomaru."}, {"m_title": "Active Raid", "r": "5.4", "year": "2016", "votes": "79", "genre": "Action, Crime", "desp": "From Wikipedia: Active Raid takes place in the near future with advanced technology, particularly a system known as ACTIVE (Armored Combined Tactical Intelligence Vanguard Elements). Due to. watch the anime dude!"}, {"m_title": "Blue Drop", "r": "6.2", "year": "2007", "votes": "153", "genre": "Fantasy, Mystery", "desp": "All the blue drop stories revolve around a war between humanity and the Arume, an alien race consisting soley of women. Both manga series are set after the war while the anime is set before. watch the anime dude!"}, {"m_title": "Listeners", "r": "5.1", "year": "2020", "votes": "135", "genre": "Adventure, Music", "desp": "In a world without music, a young female cyborg uploads music into her body and brings about a revolution."}, {"m_title": "Girls Beyond the Wasteland", "r": "5.5", "year": "2016", "votes": "76", "genre": "Comedy, Romance", "desp": "Buntarou doesnt know what he wants to do in the future. He does not have any kind of dream that he wants to pursue, so currently he just spends his days hanging around with his friends.. watch the dude, no spoilers in summary!"}, {"m_title": "Tokyo Pig", "r": "7.9", "year": "1997", "votes": "58", "genre": "Action, Comedy", "desp": "Discover the magical world of Spencer. Hes a super-intelligent third-grader who writes fantastically impossible things in his diary...and they come to life! Spencer learns how pigs really. watch the dude, no spoilers in summary!"}, {"m_title": "Rokuhoudou Yotsuio Biyori", "r": "7.9", "year": "2018", "votes": "78", "genre": "Comedy, Drama", "desp": "Welcome to the Rokuhodo teahouse, run by four talented and handsome men who try to help out troubled customers."}, {"m_title": "Hatena Illusion", "r": "6.1", "year": "2020", "votes": "67", "genre": "Animation", "desp": "Makoto não pode esperar para começar o seu aprendizado mágico e se reconectar com a filha do seu mentor, Hatena, na sua mansão assombrada."}, {"m_title": "Wakaba Girl", "r": "6.3", "year": "2015", "votes": "43", "genre": "Comedy", "desp": "Wakaba Kohashi, a sheltered rich girl, dreams of becoming a fashionable and trendy gyaru because she admires their outgoing and carefree nature. With this goal in mind, Wakaba begins the. watch the dude, no spoilers in summary!"}, {"m_title": "Tokimeki Tonight", "r": "7.9", "year": "1982", "votes": "84", "genre": "Comedy, Fantasy", "desp": "The trials and travails of a supernatural girl and her family."}, {"m_title": "Onmyô taisenki", "r": "7.7", "year": "2004", "votes": "21", "genre": "Action, Adventure", "desp": "All his life, Riku Tachibana has been raised by his grandfather. For some reason, the old man has always been fond of strange hand gestures, and theyve rubbed off on Riku, who performs. watch the dude, no spoilers in summary!"}, {"m_title": "Majin Bone", "r": "7.5", "year": "2014", "votes": "20", "genre": "Action, Adventure", "desp": "Now, in the modern age, the creator of the universe, Majin, has revived suddenly. At the same time, visitors from the darkness have descended to Earth. As danger nears the Earth, three men. watch the dude, no spoilers in summary!"}, {"m_title": "Onegai! Samia don", "r": "7.1", "year": "1985", "votes": "72", "genre": "Adventure, Fantasy", "desp": "adaptation of Edith Nesbits book Five Children and It: while exploring an abandoned lime quarry three siblings and their neighbor meet Psameed/Samia-don, the last sand fairy."}, {"m_title": "Hello Kittys Paradise", "r": "7.1", "year": "1993", "votes": "23", "genre": "Family", "desp": "sorry no summary not available!"}, {"m_title": "Kore ga watashi no go shujin sama", "r": "5.8", "year": "2005", "votes": "174", "genre": "Comedy, Romance", "desp": "Two sisters are employed as maids to attend to an orphaned millionaire... and an obnoxious pervert!"}, {"m_title": "Thermae Romae", "r": "6.0", "year": "2012", "votes": "118", "genre": "Animation", "desp": "Lucius, a Roman architect, is transported through time to modern-day Japan, where he learns about Japanese bathhouses and uses this to his gain back home."}, {"m_title": "Inugami-san to Nekoyama-san", "r": "6.0", "year": "2014", "votes": "148", "genre": "Short, Comedy", "desp": "Yachiyo Inugami is a dog-like girl who loves cats, whilst Suzu Nekoyama is a cat-like girl who loves dogs. When the two meet, an instant attraction is formed between the two."}, {"m_title": "Nobunaga the Fool", "r": "6.2", "year": "2014", "votes": "116", "genre": "Action, Fantasy", "desp": "Jeanne Kaguya DArc was prophesied to be the Savior of the World at her birth. On a different planet, Nobunaga the Fool longs for the power to change the world. When these two people meet, the culture of both worlds will be transformed."}, {"m_title": "Sonic Soldier Borgman", "r": "8.0", "year": "1988", "votes": "15", "genre": "Action, Drama", "desp": "A cyborg team of teens fight off a group of extra-dimensional creatures called the Youmas."}, {"m_title": "Akazukin Chacha", "r": "7.9", "year": "1994", "votes": "78", "genre": "Comedy, Fantasy", "desp": "Akazukin Chacha, or Red Hood Chacha, by Ayahana Min, is loosely (extremely loosely) based on Little Red Riding Hood. The main characters are: Chacha, a little red-hooded girl who is an. watch the dude, no spoilers in summary!"}, {"m_title": "Mayoi neko overrun!", "r": "6.1", "year": "2010", "votes": "105", "genre": "Comedy, Fantasy", "desp": "Having lived in the same orphanage as children, Takumi and Fumino grew up together and remain fast friends. Now living and working at a bakery, Takumi does his best to prevent catastrophe,. watch the dude, no spoilers in summary!"}, {"m_title": "Komori-san Cant Decline", "r": "5.4", "year": "2015", "votes": "109", "genre": "Short, Comedy", "desp": "Shuri Komori is a girl who is so nice that she can not turn down peoples requests. She gains amazing abilities while she fulfills peoples requests."}, {"m_title": "Madô kingu Granzort", "r": "8.2", "year": "1989", "votes": "128", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Shin Koihime musô", "r": "6.8", "year": "2009", "votes": "39", "genre": "Action, Adventure", "desp": "Kanu and Choihi rescue a mysterious girl and discover shes the real Ryubi - the one they had known previously was an imposter."}, {"m_title": "Tentai Senshi Sunred", "r": "8.2", "year": "2008", "votes": "74", "genre": "Animation", "desp": "Tentai Senshi Sunred is a powerful, battle tested hero of many years. He has sold his super bike, which is seen in the opening credits. Now he smokes a lot, is effectively unemployed, and. watch the dude, no spoilers in summary!"}, {"m_title": "Cue!", "r": "7.6", "year": "2022", "votes": "66", "genre": "Music", "desp": "16 aspiring voice actresses aim to achieve their dreams to become seiyuu stars."}, {"m_title": "Mai otome zwei", "r": "7.0", "year": "2006", "votes": "70", "genre": "Comedy, Fantasy", "desp": "Picking up one year after the events of Mai-Otome, Arika is now officially Windblooms Otome. In Ares an asteroid is detected in the sky on a collision course with Earl. After a successful. watch the dude, no spoilers in summary!"}, {"m_title": "Lapis Re:Lights", "r": "6.7", "year": "2020", "votes": "53", "genre": "Fantasy, Music", "desp": "Based on a mixed media project that blends fantasy, magical girl and idol elements, Lapis Re:LiGHTs will follow a group of students as they train to become idols. Together, theyll use the. watch the dude, no spoilers in summary!"}, {"m_title": "Gundam Evolve", "r": "6.1", "year": "2001", "votes": "51", "genre": "Short, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Magical Angel Sweet Mint", "r": "7.5", "year": "1990", "votes": "18", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Blue Blink", "r": "7.4", "year": "1989", "votes": "106", "genre": "Adventure, Family", "desp": "sorry no summary not available!"}, {"m_title": "Empty Can!", "r": "5.4", "year": "2009", "votes": "164", "genre": "Action, Comedy", "desp": "Hobbies are often a great way of meeting new people, but how could Kakeru Diachi, who collects rare juice cans, have ever suspected that hed meet a fascinating new girl when he attempted. watch the dude, no spoilers in summary!"}, {"m_title": "Mahô shôjo tai Arusu", "r": "7.2", "year": "2004", "votes": "86", "genre": "Adventure, Family", "desp": "Ordinary schoolgirl Arusu believes in magic and is transported to a world in which witches exist."}, {"m_title": "Moetan", "r": "6.4", "year": "2007", "votes": "54", "genre": "Comedy, Fantasy", "desp": "Moetan is the story of Ink a high school girl that is very short who meets a talking duck and becomes a Magical Girl that teaches her crush Nao English in disguise."}, {"m_title": "Somedays Dreamers", "r": "6.7", "year": "2003", "votes": "133", "genre": "Adventure, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Great Dangaioh", "r": "5.6", "year": "2001", "votes": "16", "genre": "Action, Adventure", "desp": "In Japan, a bright light blasted up from the ground, people argued what it was, a little girl named Miya is the only one who knows the truth about the light. Now, ten years later, robots. watch the dude, no spoilers in summary!"}, {"m_title": "Chronicles of the Going Home Club", "r": "7.0", "year": "2013", "votes": "47", "genre": "Comedy", "desp": "Its the beginning of a new year of high school, and school clubs are diligently recruiting members to take part in their activities. However, Natsuki Ando has decided to join the Going. watch the dude, no spoilers in summary!"}, {"m_title": "Overlord: Pure Pure Pleiades", "r": "6.6", "year": "2015", "votes": "74", "genre": "Comedy, Fantasy", "desp": "Lord Ainz embarks on a series of comedic mishaps when he tries to deal with his maid subordinates the Pleiades."}, {"m_title": "Sands of Destruction", "r": "6.9", "year": "2008", "votes": "94", "genre": "Adventure, Fantasy", "desp": "SEGA and FUNimation are joining forces to bring you a thrilling new anime based on the epic adventure RPG of the same name. Available exclusively on Nintendo DS. Morte is a lonely beauty. watch the dude, no spoilers in summary!"}, {"m_title": "Cleopatra D.C.", "r": "6.0", "year": "1989", "votes": "47", "genre": "Action, Mystery", "desp": "Cleopatra DC, the largest company in America, is run by the smart and beautiful Cleopatra Corns... a fun-loving, reckless playgirl."}, {"m_title": "Recorder to Randoseru", "r": "6.4", "year": "2012", "votes": "67", "genre": "Comedy", "desp": "Recorder and Randsell is the story of the siblings Atsushi, a young school boy who looks like a grown man, and his sister Atsumi in high school who looks like a small kid. It follows their comical and absurd life."}, {"m_title": "Gosenzosama Banbanzai!", "r": "8.0", "year": "1989", "votes": "65", "genre": "Comedy, Drama", "desp": "A family is thrown into irreparable turmoil by the arrival of a descendant from the future."}, {"m_title": "Girlish Number", "r": "6.5", "year": "2016", "votes": "82", "genre": "Comedy, Drama", "desp": "College student Chitose Karasuma is determined not to do boring things as she enters the adult world. To this end, this bad-mannered beauty barges into a facility that trains would-be voice. watch the anime dude!"}, {"m_title": "Hikari no densetsu", "r": "7.1", "year": "1986", "votes": "77", "genre": "Drama, Music", "desp": "Hikari is a young teen girl who wants to be a great gymnast."}, {"m_title": "SD Gundam World Heroes", "r": "5.5", "year": "2021", "votes": "12", "genre": "Action, Comedy", "desp": "The balance of the worlds is maintained by heroes. Suddenly a red-hot meteor falls upon one of these worlds, called Neo World. An amnesiac youth named Wukong Impulse Gundam appears at the. watch the dude, no spoilers in summary!"}, {"m_title": "Tico & Friends", "r": "7.7", "year": "1994", "votes": "156", "genre": "Adventure", "desp": "An eleven year old girls befriends a orca whale."}, {"m_title": "Gunjou no Fanfare", "r": "5.8", "year": "2022", "votes": "25", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Tenshi na konamaiki", "r": "7.1", "year": "2002", "votes": "117", "genre": "Comedy, Drama", "desp": "A pretty schoolgirl hides a secret: she is actually a boy who got turned into a girl and is looking for a way to change herself back."}, {"m_title": "A Destructive God Sits Next to Me", "r": "6.5", "year": "2020", "votes": "86", "genre": "Comedy", "desp": "Koyuki Seri is just trying to have a normal school life, but instead he ends up acting as the straight man to his odd classmate, Hanatori Kabuto. Kabuto is is a delusional high school. watch the dude, no spoilers in summary!"}, {"m_title": "Yoru no Kuni", "r": "7.8", "year": "2021", "votes": "53", "genre": "Fantasy, Music", "desp": "A short anime series following a group of troubled individuals meeting with a mysterious being named Yoru in a dream-like land called the Night World."}, {"m_title": "Fireball Charming", "r": "6.6", "year": "2011", "votes": "26", "genre": "Short, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Eria 88", "r": "6.9", "year": "2004", "votes": "134", "genre": "Action, Drama", "desp": "In the early 1980s, a war is raging in Arslan, a fictional Middle Eastern kingdom. Mercenary fighter pilots from all over the world are hired or shanghaied to fight. They live in a secret desert airbase called Area 88."}, {"m_title": "Blue Comet SPT Layzner", "r": "7.2", "year": "1985", "votes": "48", "genre": "Action, Adventure", "desp": "In an alternate 1996, the US and USSR take the Cold War to Mars. A group of teenage cadets, the Cosmic Culture Club, are sent as a message of goodwill to both sides by being assigned to the. watch the anime dude!"}, {"m_title": "Ninku", "r": "7.8", "year": "1994", "votes": "39", "genre": "Action, Adventure", "desp": "Once upon a time, there was a war between the Imperial army and Ninku, an elite unit of 12 zodiac warriors led by their master. Just when the Ninku were about to win the war, their master. watch the dude, no spoilers in summary!"}, {"m_title": "Digimon Universe: App Monsters", "r": "6.7", "year": "2016", "votes": "104", "genre": "Action, Adventure", "desp": "Haru Shinkai, a Junior High Student discovers an Appmon, which reveals himself to be Gatchmon and they become partners. Together, with other members, they plan to stop an artificial intelligence, Leviathan, from taking over humanitys."}, {"m_title": "Garo: Guren no tsuki", "r": "5.7", "year": "2015", "votes": "54", "genre": "Action, Fantasy", "desp": "In the capital city of an elegant, feudal society, a powerful spiritual barrier protects people from demonic monsters known as ���horrors�۝-at least, those. watch the anime dude!"}, {"m_title": "Virus Buster Serge", "r": "5.7", "year": "1997", "votes": "83", "genre": "Action, Adventure", "desp": "Set in 2097, in Neo Hong Kong where a virus made by genetic engineering. In order to stop it, S.T.A.N.D is a police force coming to defeat the infected machines and humans. Serge (main. watch the dude, no spoilers in summary!"}, {"m_title": "Shingu: Secret of the Stellar Wars", "r": "7.5", "year": "2001", "votes": "44", "genre": "Sci-Fi", "desp": "A small Japanese town is in fact the home of Shingu, a legendary protector of the Earth, controlled by the student council who go through lifes trials and tribulations."}, {"m_title": "Monsuta Hai Kowa ike Garuzu", "r": "6.9", "year": "2014", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Metal Fighter Miku", "r": "7.2", "year": "1994", "votes": "32", "genre": "Comedy, Sci-Fi", "desp": "A female pro-wrestler competes with her team in a futuristic wrestling circuit."}, {"m_title": "A Piece of Phantasmagoria", "r": "7.2", "year": "1995", "votes": "54", "genre": "Animation", "desp": "Based on Phantasmagoria, a picture book, this is a reflective anime set in a fantasy and minimalist world. The series Consists of fifteen episodes, each five minutes in length."}, {"m_title": "Minami Kamakura High School Girls Cycling Club", "r": "6.7", "year": "2017", "votes": "48", "genre": "Drama, Sport", "desp": "Maiharu Hiromi has moved to Kamakura Nagasaki, and rides a bicycle to school everyday. Then she meets Akizuki Tomoe, the leader of the girls cycling club. She therefore joins the club and her life gradually begins to change."}, {"m_title": "Nyanko Days", "r": "6.6", "year": "2017", "votes": "97", "genre": "Animation", "desp": "A series stories about a shy girl Yuko Konagai and her cats daily lives."}, {"m_title": "Aiura", "r": "6.1", "year": "2013", "votes": "157", "genre": "Short", "desp": "sorry no summary not available!"}, {"m_title": "Bonjour Sweet Love Pâtisserie", "r": "5.4", "year": "2014", "votes": "88", "genre": "Romance", "desp": "sorry no summary not available!"}, {"m_title": "Beyraiderz Shogun", "r": "7.4", "year": "2013", "votes": "16", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Hime-chan no Ribbon", "r": "6.5", "year": "1992", "votes": "225", "genre": "Comedy, Romance", "desp": "A series about a tomboyish junior high school student that is given a magic ribbon that she is to keep for one year, allowing her to temporarily transform into anyone she wishes."}, {"m_title": "Healin Good PreCure", "r": "7.7", "year": "2020", "votes": "38", "genre": "Action, Adventure", "desp": "A 13-year-old Nodoka Hanadera recently moved to the town of Sukoyaka (Sukoyaka-shi, Sukoyaka City) with her family in hopes that it can give her a fresh start from her life in the city.. watch the dude, no spoilers in summary!"}, {"m_title": "Shoukoushi Sedi", "r": "7.6", "year": "1988", "votes": "63", "genre": "Family, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Betterman", "r": "6.8", "year": "1999", "votes": "99", "genre": "Action, Adventure", "desp": "The enigmatic Akamatsu Industries, disguised as a heavy machine factory in Tokyo, is a clandestine organization that uses neural enhanced weapons known as Awakeners to battle a mysterious virus."}, {"m_title": "Galilei Donna", "r": "6.2", "year": "2013", "votes": "72", "genre": "Adventure, Sci-Fi", "desp": "Three sisters, Hozuki, Kazuki, and Hazuki, are descendants of Galileo and have completely different personalities and tastes; they never meet eye-to-eye. One day, the girls are suddenly. watch the dude, no spoilers in summary!"}, {"m_title": "Sengoku musou", "r": "6.7", "year": "2014", "votes": "81", "genre": "Action, History", "desp": "Based on the hit video game, this dramatic re-imagining of one of Japans most revered historical periods drops audiences into the middle of an epic battle royal. Great warriors and dashing. watch the anime dude!"}, {"m_title": "Aikatsu Friends!", "r": "7.9", "year": "2018", "votes": "122", "genre": "Adventure, Comedy", "desp": "Lets go, together. Head for the goal, Diamond Friends!! The new anime will have a theme of Aikatsu with your friends! and will center on a new protagonist named Aine Yuuki, who is good. watch the anime dude!"}, {"m_title": "Nobunagun", "r": "6.7", "year": "2014", "votes": "112", "genre": "Action", "desp": "During a field trip, Shio is revealed to be an E-Gene Holders, a person who wield weapons infused with the spirits of historical figures. The soul of Oda Nobunaga awakens after she tries to rescue a friend."}, {"m_title": "Transformers: Galaxy Force", "r": "7.4", "year": "2005", "votes": "86", "genre": "Action, Fantasy", "desp": "Japanese reboot show that was abridged and reedited to become Transformers: Cybertron in the US. A black hole threatens Cybertron, so evacuation begins. Autobots and Decepticons end up on the present day Earth and battle again."}, {"m_title": "Grimms Notes The Animation", "r": "5.4", "year": "2019", "votes": "57", "genre": "Action, Adventure", "desp": "The game takes place in a world created by a beings known as Story Tellers. At birth, all inhabitants of this world are bestowed with a book of fate where all their lives are written in. watch the anime dude!"}, {"m_title": "Da Capo", "r": "6.2", "year": "2003", "votes": "112", "genre": "Comedy, Drama", "desp": "Junichi Asakura lives with his adoptive sister Nemu in the crescent-shaped island of Hatsune Jima, a place where cherry blossoms bloom throughout the year. In this island, people have. watch the dude, no spoilers in summary!"}, {"m_title": "Himitsu: Top Secret - The Revelation", "r": "6.7", "year": "2008", "votes": "41", "genre": "Crime, Drama", "desp": "In the near future, it is possible to extract memories from a persons brain up until 48 hours after their death. A special police unit uses this method to solve crimes, but this raises many ethical dilemmas and privacy concerns."}, {"m_title": "Hakuouki: Sekkaroku OVA", "r": "8.2", "year": "2011", "votes": "54", "genre": "Action, Adventure", "desp": "After learning rumors regarding an attack, the Shinsengumi send Chizuru Yukimura to Shimabara on a top secret spy mission to find out more."}, {"m_title": "Gu-Gu Ganmo", "r": "7.2", "year": "1984", "votes": "7", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Tonagura!", "r": "6.7", "year": "2006", "votes": "100", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Pita-Ten", "r": "7.3", "year": "2002", "votes": "56", "genre": "Comedy, Drama", "desp": "About a girl who has wings and is cute she loves to hang out with her human friend who loves to play all day and laugh all night they are all fine and they love to play he give me but by. watch the dude, no spoilers in summary!"}, {"m_title": "Arc the Lad", "r": "5.5", "year": "1999", "votes": "172", "genre": "Action, Adventure", "desp": "Join Elk, the youngest member of the Hunters Guild, who finds himself propelled into an epic odyssey across a mysterious world in search of the enigmatic outlaw known as Arc The Lad.. watch the dude, no spoilers in summary!"}, {"m_title": "Sougen no chiisana tenshi busshu beibii", "r": "7.4", "year": "1992", "votes": "55", "genre": "Animation", "desp": "This japanese anime is about a 12 years old girl, who moved from Britain to Kenya since childhood and her adventures in the Savannas."}, {"m_title": "Sacred Seven", "r": "6.1", "year": "2011", "votes": "93", "genre": "Action, Sci-Fi", "desp": "Alma Tandouji lives a lonely and solitary life in a certain port city within the Kantou region. One day, he is approached by Aiba Ruri who asks for his help, for the power of the Sacred Seven which resides within Alma."}, {"m_title": "Pri Para", "r": "8.3", "year": "2014", "votes": "38", "genre": "Comedy, Drama", "desp": "After finding a lost bag belonging to a girl named Mirei Minami, elementary school student Laala Manaka gets a chance to perform as an idol singer in an idol theme park."}, {"m_title": "Chidori RSC", "r": "5.7", "year": "2019", "votes": "49", "genre": "Comedy, Sport", "desp": "Kokura Hikari is girl who loves rifle shooting. She entered Chidori High School because it had a shooting club, but finds that the club had been disbanded. She had only one day to find enough members to reform the club."}, {"m_title": "Kemurikusa.", "r": "6.9", "year": "2019", "votes": "96", "genre": "Action, Sci-Fi", "desp": "Mysterious sisters and a boy go on a journey around the world of red fog to find a place for a peaceful life with water while they fight against weird lives called Akamushi."}, {"m_title": "Mitsuboshi Colors", "r": "6.8", "year": "2018", "votes": "129", "genre": "Comedy, Family", "desp": "Three little girls have fun around Tokyos Ueno Park pretending that theyre protecting the peace of their town."}, {"m_title": "Digimon Savers", "r": "6.2", "year": "2006", "votes": "568", "genre": "Action, Adventure", "desp": "On a certain day, before a fight that Masaru had been training for, a strange dinosaur-like creature appears. This strange creature, Agumon, is a Digital Monster from another world, the. watch the anime dude!"}, {"m_title": "Comet Lucifer", "r": "5.2", "year": "2015", "votes": "97", "genre": "Adventure, Fantasy", "desp": "A magical adventure from Studio 8-Bit full of magic, crystals, and more. Sougo Amagi is a young orphan in a mining town on Gift, a world where enchanting blue crystals lay deep in the earth. watch the anime dude!"}, {"m_title": "Ai shôjo Porianna monogatari", "r": "7.3", "year": "1986", "votes": "160", "genre": "Drama, History", "desp": "Based on the two Glad Books by Eleanor H. Porter, Pollyanna is a newly orphaned girl sent to live with her strict Aunt Polly in the town of Beldingsville. With her love for gladness,. watch the dude, no spoilers in summary!"}, {"m_title": "Armored Fleet Dairugger XV", "r": "5.2", "year": "1982", "votes": "61", "genre": "Action, Adventure", "desp": "An space exploration expedition is protected by an unique armored combat group that can combine into one giant robot when necessary."}, {"m_title": "Nijûmensô no musume", "r": "7.1", "year": "2008", "votes": "63", "genre": "Animation", "desp": "Chiko is the daughter of the wealthy Mikamo family who has to live with her aunt and uncle after her parents passed away. Because her aunt wants to inherit the Mikamo familys wealth, she. watch the dude, no spoilers in summary!"}, {"m_title": "Tokimeki Memorial: Only Love", "r": "6.8", "year": "2006", "votes": "41", "genre": "Comedy, Drama", "desp": "The story revolves around a second year high school student, Riku Aoba, who has just recently transferred to Holy Cross High School, where he notices, upon joining, several unique and funny. watch the anime dude!"}, {"m_title": "Konpeki no Kantai", "r": "5.2", "year": "1993", "votes": "7", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Futari wa Pretty Cure Splash Star", "r": "8.3", "year": "2006", "votes": "105", "genre": "Action, Adventure", "desp": "During the summer festival five years ago, two girls met at a mysterious tree and saw two glowing spheres. Now, these two girls Saki Hyuuga, ace pitcher on the school softball team; and Mai. watch the anime dude!"}, {"m_title": "Onipan!", "r": "6.0", "year": "2022", "votes": "23", "genre": "Comedy", "desp": "With the power of onipan, three oni-girls become human in order to help fix the relationship between humans and oni. They transfer into a normal Tokyo high school and begin to revitalize everything around them."}, {"m_title": "Actors: Songs Connection", "r": "7.1", "year": "2019", "votes": "30", "genre": "Animation", "desp": "The story follows the everyday lives of students and teachers at the Private Amaka Academy, a school established by the the large university town group Slive Central."}, {"m_title": "Laughing Under the Clouds Gaiden", "r": "8.1", "year": "2018", "votes": "19", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Snack World", "r": "5.5", "year": "2017", "votes": "14", "genre": "Animation", "desp": "Welcome to SNACK WORLD! This is a nutty universe filled with dreams and adventures. Here, you can find everything from dragons to smart devices to Chup, the hyperactive hero of this story."}, {"m_title": "A Dark Rabbit Has Seven Lives", "r": "6.0", "year": "2011", "votes": "12", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Saiyuki Reload", "r": "7.3", "year": "2003", "votes": "51", "genre": "Action, Comedy", "desp": "Priest Genjo Sanzo and companions Cho Hakkai, Sha Gojyo, and Son Goku maintain their westward journey to stop the resurrection of the demon Gyoumao. As the reputation of the Sanzo Ikkou. watch the dude, no spoilers in summary!"}, {"m_title": "Cheer Boys!!", "r": "6.3", "year": "2016", "votes": "109", "genre": "Drama, Sport", "desp": "Haruki Bandou trades Judo for cheerleading when his childhood friend Kazuma Hashimoto invites him to join the BREAKERS."}, {"m_title": "Gun-dou Musashi", "r": "5.2", "year": "2006", "votes": "24", "genre": "Action, Fantasy", "desp": "Monkey Punch recreates Miyamoto Musashi in his own version. This Musashi uses both a sword and a pair of pistols to fight off supernatural enemies."}, {"m_title": "Calimero", "r": "7.2", "year": "1992", "votes": "55", "genre": "Family", "desp": "sorry no summary not available!"}, {"m_title": "Hanada Shonen-shi", "r": "7.6", "year": "2002", "votes": "59", "genre": "Comedy, Drama", "desp": "After being hit by a truck and getting 9 stitches in the back of his head, Hanada Ichiro, the local troublemaker in a small village, becomes able to see ghosts, all of whom want him to fulfill their unfinished business."}, {"m_title": "Ristorante Paradiso", "r": "7.3", "year": "2009", "votes": "80", "genre": "Drama, Romance", "desp": "Nicoletta travels to Italy from Japan to meet her estranged mother who she has not seen since she was a young child. Once there, Nicoletta becomes immersed working in her stepfathers. watch the dude, no spoilers in summary!"}, {"m_title": "Encouragement of Climb: Next Summit", "r": "7.5", "year": "2022", "votes": "30", "genre": "Adventure, Comedy", "desp": "While reminiscing about her journey so far, Aoi Yukimura finds herself eager to go mountaineering once more. A likely opportunity arises when Koharu Senjuin invites Aoi to check out the. watch the dude, no spoilers in summary!"}, {"m_title": "12-Sai. Chiccha na Mune no Tokimeki", "r": "6.0", "year": "2016", "votes": "50", "genre": "Comedy, Romance", "desp": "Hanabi Ayase and her best friend, Yui Aoi, are 6th graders in school. When Hanabi is chosen to work with Yuuto Takao for an upcoming test, she begins to fall in love with him and they both. watch the dude, no spoilers in summary!"}, {"m_title": "Trouble Chocolate", "r": "7.1", "year": "1999", "votes": "31", "genre": "Action, Comedy", "desp": "Cacao, a young student, finds that he might be a powerful magic user."}, {"m_title": "Babel II: Beyond Infinity", "r": "5.5", "year": "2001", "votes": "20", "genre": "Action, Adventure", "desp": "Koichi Kamiya is a normal student, but one day, a mysterious voice wakes him up from his dream, and asks him to fight against evil in the name of Babel II. The next day, he meets Juju, Wong. watch the anime dude!"}, {"m_title": "Hell Girl: The Fourth Twilight", "r": "6.4", "year": "2017", "votes": "102", "genre": "Animation", "desp": "Ai and her enigmatic companions continue to carry out their work, ferrying soul after suffering soul into the depths of Hell. However, as of late, a mysterious girl has been following them.. watch the anime dude!"}, {"m_title": "Fantastic Children", "r": "7.4", "year": "2004", "votes": "225", "genre": "Adventure, Fantasy", "desp": "A girl named Helga is an orphan with her only friend Cheeto by her side. She has this place in her head that she continues to draw dreaming she will get there someday. Meanwhile the children of befort are looking for her."}, {"m_title": "Edens Bowy", "r": "6.8", "year": "1999", "votes": "44", "genre": "Action, Adventure", "desp": "Yorn gazes up at two massive cities floating in the sky and dreams of discovering their mysteries. Without warning, those mysteries come looking for him. Edenic assassins murder his father. watch the dude, no spoilers in summary!"}, {"m_title": "Red Baron", "r": "6.1", "year": "1994", "votes": "61", "genre": "Action, Sci-Fi", "desp": "Ken Kurenai, the pilot of a giant robot named Red Baron, must take on various opponents in Metal Fight games, the popular televised fighting robot tournament."}, {"m_title": "Puzzle & Dragons X", "r": "7.1", "year": "2016", "votes": "14", "genre": "Adventure, Fantasy", "desp": "Winners reap rewards in the puzzle wars. But what if you lose? Only the brave can take on these monstrous challenges!"}, {"m_title": "JINSEI Life Consulting", "r": "5.8", "year": "2014", "votes": "38", "genre": "Comedy, Drama", "desp": "When bespectacled Yuki Akamatsu joins his schools 2nd Newspaper Club, he finds himself caught in the crossfire between three pretty panelists assigned to the Life Advice column."}, {"m_title": "Stratos 4", "r": "7.0", "year": "2003", "votes": "70", "genre": "Action, Sci-Fi", "desp": "Fifty years ago, scientists discover a group of comets that are on a collision course with Earth. To combat certain annihilation, the United Nations builds a base on Shimoji Island where we. watch the anime dude!"}, {"m_title": "Hey! Bumbo", "r": "6.2", "year": "1985", "votes": "141", "genre": "Adventure, Drama", "desp": "A series following the adventures of a young boy and Boumbo, a talking yellow car who hatched from an egg, as they travel the world in search for Boumbos mother. A car salesman is after to catch Boumbo."}, {"m_title": "Soukyuu no Fafner Dead Agressor: The Beyond", "r": "7.2", "year": "2019", "votes": "9", "genre": "Action, Drama", "desp": "Soukyuu no Fafner The Beyond is the fifth installment in the Soukyuu no Fafner series. It takes place after Soukyuu no Fafner Exodus."}, {"m_title": "A-Channel", "r": "6.5", "year": "2011", "votes": "177", "genre": "Comedy, Drama", "desp": "As Run, Nagi and Yuuko begin their sophomore year while Tooru begins her freshman year, their exciting high school life filled with drama, humor and adventure begins for the four friends during their everyday lives."}, {"m_title": "Toaru Majutsu no Index-tan", "r": "6.9", "year": "2009", "votes": "25", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Saki", "r": "6.9", "year": "2009", "votes": "120", "genre": "Animation", "desp": "A tenth-former wore out an old acquaintance by name Saki Miyanaga in the school club of mahjong only because that knew rules, and for a game the fourth was not enough. Certainly, Saki. watch the dude, no spoilers in summary!"}, {"m_title": "Eternal Boys", "r": "6.0", "year": "2022", "votes": "8", "genre": "Music", "desp": "The story follows six middle-aged men who have experienced success and failures in their lives. One day, they decide to become an idol group and overcome such barriers as their age and their physical condition."}, {"m_title": "Umi no Yami, Tsuki no Kage", "r": "6.1", "year": "1989", "votes": "11", "genre": "Drama, Horror", "desp": "Ruka and Rumi Kobayakawa are twin sisters in love with the same man, athletic upperclassman Katsuyuki Touma. At first Rumi is supportive of Rukas fledgling romance with the handsome. watch the dude, no spoilers in summary!"}, {"m_title": "Clamp School", "r": "7.4", "year": "1997", "votes": "69", "genre": "Action, Comedy", "desp": "CLAMP School has all the size and resources of a small city. Over 10,000 people - students and their families - live, work, and study within its self-sufficient campus. Any student with. watch the dude, no spoilers in summary!"}, {"m_title": "KJ File", "r": "3.3", "year": "2022", "votes": "6", "genre": "Action, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Otogi Zoshi", "r": "7.3", "year": "2004", "votes": "109", "genre": "Action", "desp": "It is in the middle of the Heian era -- By famine and the epidemic, the capital was totally ruined. However, in spite of that situation, the samurai and the Yin Yang fortune-teller were. watch the dude, no spoilers in summary!"}, {"m_title": "Zaion: I Wish You Were Here", "r": "5.8", "year": "2001", "votes": "58", "genre": "Drama, Mystery", "desp": "In the future, a meteor crashes into Earth bringing with it a virus known as M34 and attacks humans, turning them into vicious monsters. The fate of the world rests in the hands of 16-year-old girl with the power to defeat the virus."}, {"m_title": "Don Dracula", "r": "6.2", "year": "1982", "votes": "92", "genre": "Comedy, Horror", "desp": "After living in Transylvania for several years, Earl Dracula has moved to Japan. In the Nerima Ward of Tokyo, he and his daughter, Chocola, and faithful servant Igor. While Chocola. watch the dude, no spoilers in summary!"}, {"m_title": "Ano Natsu de Matteru Special", "r": "7.1", "year": "2014", "votes": "68", "genre": "Animation", "desp": "To combat the approaching boredom of summer, friends decide to make a film together documenting their memories together."}, {"m_title": "The Rolling Girls", "r": "6.3", "year": "2015", "votes": "100", "genre": "Adventure, Comedy", "desp": "Ten years after the Great Tokyo War, each prefecture in Japan has split up into independent nations. Each nation is ruled by a prophet called Mosa and an army called Mob as they start. watch the dude, no spoilers in summary!"}, {"m_title": "Hime-sama Goyojin", "r": "4.2", "year": "2006", "votes": "7", "genre": "Comedy", "desp": "A young schoolgirls life turns chaotic when she gets a magical crown stuck to her forehead."}, {"m_title": "Atom the Beginning", "r": "6.6", "year": "2017", "votes": "73", "genre": "Sci-Fi", "desp": "The story of two friends, Umataro Tenma and Hiroshi Ochanomizu, who are genius graduate students studying robotics. The pair have recently completed a robot called A106 that features their experimental Bewusstsein AI."}, {"m_title": "Renkin Sankyû Magical? Pokân", "r": "7.5", "year": "2006", "votes": "51", "genre": "Comedy, Fantasy", "desp": "The misadventures of four princesses from the Netherworld: Uma the Witch, Pachira the Vampire, Liru the Werewolf, and Aiko the Android."}, {"m_title": "Sengoku Collection", "r": "7.4", "year": "2012", "votes": "26", "genre": "Comedy, Fantasy", "desp": "In a time line different from our own, the generals of the Warring States period, vie for power. But they are the same generals we know in name only: their appearance, personality, and. watch the dude, no spoilers in summary!"}, {"m_title": "Machine Robo: Winner Battle Hackers", "r": "7.8", "year": "1987", "votes": "10", "genre": "Adventure, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Aikatsu on Parade!", "r": "6.3", "year": "2019", "votes": "10", "genre": "Adventure, Comedy", "desp": "Raki Kiseki is a second-year middle school student who transfers to Star Harmony Academy to become an idol. However, when she uses an Aikatsu Pass she received from her elder sister Saya,. watch the dude, no spoilers in summary!"}, {"m_title": "WWW.Working!!", "r": "6.8", "year": "2016", "votes": "187", "genre": "Comedy, Drama", "desp": "A boy named Daisuke Higashida a serious first class student going to Higashizaka High school and lives a quite life. That is not because of his family which he hates its just a character. watch the dude, no spoilers in summary!"}, {"m_title": "Super Dimensional Cavalry: Southern Cross", "r": "6.8", "year": "1984", "votes": "155", "genre": "Sci-Fi", "desp": "In the far future, a human colony planet fights a desperate war against a formidable alien invasion force with its own tensions."}, {"m_title": "Digimon Xros Wars: The Young Hunters Who Leapt Through Time", "r": "7.0", "year": "2011", "votes": "72", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Amaenaideyo!! Katsu!!", "r": "6.5", "year": "2006", "votes": "99", "genre": "Animation", "desp": "Ikkos trials as a monk in training continue, as the story behind his mystical holy powers begins to unfold. To protect Ikko and their decency the nuns try in vain to stop him from using his powers"}, {"m_title": "Detective Opera Milky Holmes", "r": "7.6", "year": "2010", "votes": "13", "genre": "Adventure, Comedy", "desp": "Toys are powers bestowed upon those chosen to wield them, each having qualities that are unique to their users. Some of these individuals are born pure, using their powers to become. watch the dude, no spoilers in summary!"}, {"m_title": "Inukami!", "r": "6.8", "year": "2006", "votes": "115", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Battery", "r": "5.5", "year": "2016", "votes": "116", "genre": "Drama, Sport", "desp": "Seiha wants to play baseball while Takumi says he cant., but Seiha and Go are enjoying a game of catch. Takumi gets angry and throws the ball in the woods behind the shrine, and Seiha. watch the dude, no spoilers in summary!"}, {"m_title": "Moldiver", "r": "6.1", "year": "1993", "votes": "130", "genre": "Action, Adventure", "desp": "A naïve young girl accidentally modifies her brothers secret superhero uniform and becomes the unwilling superheroine Moldiver."}, {"m_title": "Reideen: The Superior", "r": "6.3", "year": "1996", "votes": "23", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Hayate the Combat Butler: Cant Take My Eyes Off You", "r": "7.1", "year": "2012", "votes": "63", "genre": "Comedy, Romance", "desp": "A month has passed and living at the Sanzenin Mansion once again, Nagi returns to her old ways of life, until she receives word from American authorities informing her that she has a week. watch the dude, no spoilers in summary!"}, {"m_title": "Our Homes Fox Deity", "r": "6.6", "year": "2008", "votes": "68", "genre": "Adventure, Fantasy", "desp": "The Takagami family welcomes their new visitor, a fox deity, Tengo Kugen to ward off evil spirits into their daily life."}, {"m_title": "Battle Athletess Daiundoukai Restart!", "r": "5.2", "year": "2021", "votes": "21", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Digimon Xros Wars: The Evil Death Generals and the Seven Kingdoms", "r": "7.0", "year": "2011", "votes": "33", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Three Leaves, Three Colors", "r": "7.4", "year": "2016", "votes": "100", "genre": "Comedy", "desp": "Futaba, Teru, and Youko prove that it can take even the smallest connection to form a friendship. But that doesnt mean itll be easy!"}, {"m_title": "Ebiten: Koritsu Ebisugawa Koko Tenmonbu", "r": "5.0", "year": "2012", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Oroshitate Musical Nerima Daikon Brothers", "r": "7.4", "year": "2006", "votes": "80", "genre": "Comedy, Music", "desp": "A musical series about two brothers and their cousin working on a daikon farm and trying to raise enough money to build their own concert dome, all while getting involved in absurd hijinks."}, {"m_title": "Tekken Chinmi", "r": "7.5", "year": "1988", "votes": "12", "genre": "Adventure, Comedy", "desp": "A teenage boy who tries to complete kung fu at a temple in China."}, {"m_title": "Mami the Esper", "r": "7.5", "year": "1987", "votes": "24", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Minami-ke: Tadaima", "r": "7.5", "year": "2013", "votes": "52", "genre": "Comedy, Drama", "desp": "The everyday lives of the Minami sisters continue. Chiaki, the youngest, continues to call people idiots while worshipping her eldest sister. Kana, the middle, still cant figure out that. watch the dude, no spoilers in summary!"}, {"m_title": "Purinsesu purinsesu", "r": "6.6", "year": "2006", "votes": "92", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Cinderella Boy", "r": "6.2", "year": "2003", "votes": "44", "genre": "Action, Comedy", "desp": "Ranma and Rella are both detectives seeking mysteries and crimes to solve. But one night their lives change when encountering an accident theyll both be effected by the strike of Midnight."}, {"m_title": "SD Gundam World Sangoku Soketsuden", "r": "5.7", "year": "2019", "votes": "11", "genre": "Action, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Hayate the Combat Butler: Cuties", "r": "6.9", "year": "2013", "votes": "36", "genre": "Comedy, Romance", "desp": "After the events of Hayate the Combat Butler: Cant Take My Eyes Off You, Hayate, Nagi and Maria return to their daily lives at the Sanzenin Mansion and the Violet Mansion."}, {"m_title": "Aqua Age", "r": "6.9", "year": "1996", "votes": "18", "genre": "Romance", "desp": "Yuuko, a shy girl in Middle School, has feelings for her friend who lives next door. But then she finds out her best friend also likes him too."}, {"m_title": "Final Approach", "r": "5.3", "year": "2004", "votes": "29", "genre": "Comedy, Drama", "desp": "Ever since their parents died a few years ago, Ryo and his sister Akane have been living alone together. Despite their difficult situation, they are still living reasonably happy and normal. watch the anime dude!"}, {"m_title": "Gegege no Kitarô", "r": "7.6", "year": "1996", "votes": "19", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Warau Salesman NEW", "r": "6.8", "year": "2017", "votes": "20", "genre": "Fantasy, Horror", "desp": "Each episode follows Fukuzou Moguro, a traveling salesman, and his current customer. Moguro deals in things that give his customers their hearts desire, and once his deals are made and. watch the dude, no spoilers in summary!"}, {"m_title": "Reporter Blues", "r": "8.0", "year": "1991", "votes": "34", "genre": "Adventure, Crime", "desp": "sorry no summary not available!"}, {"m_title": "Bomberman Jetters", "r": "7.5", "year": "2002", "votes": "36", "genre": "Animation", "desp": "The Jetters are a highly trained intergalactic police force for keeping unique items safe from the Hige-Hige bandits. Mighty, an expert Bomberman and the leader of the Jetters, disappears. watch the dude, no spoilers in summary!"}, {"m_title": "Magic Kyun! Renaissance", "r": "7.3", "year": "2016", "votes": "75", "genre": "Music, Romance", "desp": "A world where art becomes magic. In this world, people who can inspire passion with their Magic Arts are called Artistas, and are employed in show business. In Hoshinomori Private Magical. watch the dude, no spoilers in summary!"}, {"m_title": "Fighting Beauty Wulong", "r": "5.8", "year": "2005", "votes": "9", "genre": "Action, Comedy", "desp": "After winning the Prime Mat martial arts tournament, Mao Lan is determined to live as a normal high school girl; but as always, trials and tribulations stand in her way. Shes tasked with. watch the dude, no spoilers in summary!"}, {"m_title": "Dan Doh!!", "r": "7.3", "year": "2004", "votes": "14", "genre": "Sport", "desp": "Dandoh is a teen boy who gets entered into a golf tournament."}, {"m_title": "Super Doll Lika-chan", "r": "6.2", "year": "1998", "votes": "25", "genre": "Adventure, Comedy", "desp": "Licca is in grave danger once she finds out that she is the princess of the Doll Kingdom. Now, the evil Dr. Scarecrow is after her and her throne. So Liccas grandmother gives her a set of. watch the dude, no spoilers in summary!"}, {"m_title": "Metal Armor Dragonar", "r": "6.8", "year": "1987", "votes": "22", "genre": "Action, Sci-Fi", "desp": "The Earth Alliance fights against the Lunar Empire of Giganos with the help of an elite trio of super robots."}, {"m_title": "Huckleberry Finn Monogatari", "r": "7.1", "year": "1994", "votes": "17", "genre": "Family", "desp": "Based on Mark Twains famous book about the adventures of young orphan Huckleberry Finn."}, {"m_title": "Vampire Holmes", "r": "2.9", "year": "2015", "votes": "38", "genre": "Comedy, Mystery", "desp": "sorry no summary not available!"}, {"m_title": "Yume no crayon oukoku", "r": "7.6", "year": "1997", "votes": "24", "genre": "Adventure, Comedy", "desp": "Princess Silver of the crayon kingdom goes on a journey looking for a mysterious boy who turned her parents into stone in hopes of turning them back to normal."}, {"m_title": "Ochikobore Fruit Tart", "r": "6.4", "year": "2020", "votes": "54", "genre": "Comedy, Drama", "desp": "An aspiring idol recruits three former dropouts to form a musical group."}, {"m_title": "The Galaxy Railways: Eternal Divergence", "r": "6.8", "year": "2006", "votes": "12", "genre": "Adventure, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Gunslinger Stratos: The Animation", "r": "5.7", "year": "2015", "votes": "87", "genre": "Action, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Fortune Arterial: Red Promise", "r": "6.9", "year": "2010", "votes": "105", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Zo Zo Zombie", "r": "7.8", "year": "2020", "votes": "7", "genre": "Animation", "desp": "Zo Zo Zombie is our undead hero who, with his best human friend Isamu, is just trying to do some good in this world but somehow always ends up in pieces. Literally."}, {"m_title": "Alice in Deadly School", "r": "5.2", "year": "2021", "votes": "25", "genre": "Animation", "desp": "The story begins with the peaceful daily life of schoolgirls-which suddenly breaks down when former schoolmates become zombies. The story follows the actions taken by the girls left behind on the school roof."}, {"m_title": "Stop!! Hibari-kun!", "r": "7.9", "year": "1983", "votes": "27", "genre": "Comedy, Romance", "desp": "Kousaku Sakamoto is a high school student who has to live with Yakuza boss Ibari Oozora and his four(?) daughters as the last wish after the death of his mother. However Hibari turns out to be a trans girl for the surprise of Kousaku."}, {"m_title": "Uchi no Shishou wa Shippo ga Nai", "r": "5.9", "year": "2022", "votes": "62", "genre": "Comedy, Fantasy", "desp": "Mameda is a shape shifting Tanuki girl who dreams of becoming human someday. One day, she turns into a black-haired beauty and heads out to Osaka, but is soon discovered. One beautiful. watch the dude, no spoilers in summary!"}, {"m_title": "Cube of Evil Spirits", "r": "7.1", "year": "2008", "votes": "104", "genre": "Mystery, Thriller", "desp": "Police detective Kiba finds himself investigating a very strange case involving a girl hit by a train, her actress sister, and a sinister hospital shaped like a box. As the girls friends. watch the dude, no spoilers in summary!"}, {"m_title": "Burn-Up Excess", "r": "6.6", "year": "1997", "votes": "162", "genre": "Action, Adventure", "desp": "Theyre back! Rio, Nanval, Lilica, Maya and Yuji are back on the force and ready to go beyond the call of duty in their greatest adventure yet in this continuation of the Burn Up Warrior. watch the dude, no spoilers in summary!"}, {"m_title": "Girl Friend BETA", "r": "4.0", "year": "2014", "votes": "33", "genre": "Comedy, Romance", "desp": "A TV anime adaptation of the Girlfriend (Kari) a mobile game where players can have girlfriends through playing the game."}, {"m_title": "Future Police Urashiman", "r": "8.2", "year": "1983", "votes": "62", "genre": "Action, Comedy", "desp": "A teenage boy with a cat slipped into Neo-Tokio of A.D. 2050 from Tokyo of A.D. 1983, and he lost his memory. Captain Gondo of a brunch police station Magnapolice-38 gives him a name as Ryû. watch the anime dude!"}, {"m_title": "Phantasy Star Online 2: Episode Oracle", "r": "6.7", "year": "2019", "votes": "21", "genre": "Action, Fantasy", "desp": "An animated adaptation of episodes 1-3 (out of 6) of the popular Japanese MMO, Phantasy Star Online 2."}, {"m_title": "Nogizaka Haruka no Himitsu: Purezza", "r": "6.6", "year": "2009", "votes": "82", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Brighter Than the Dawning Blue", "r": "7.0", "year": "2006", "votes": "58", "genre": "Comedy, Drama", "desp": "The moon and the earth are linked by a single contact point, Mitsuru ga Sakichau Ouren Rakuniushi where Tatsuya lives under the supervision of the Sphere Kingdom (Moon Kingdom)s Princess. watch the anime dude!"}, {"m_title": "Shadowverse", "r": "6.3", "year": "2020", "votes": "41", "genre": "Fantasy", "desp": "The story centers on Hiiro Ryuugasaki, a student at Tensei Academy. Through a strange incident, Hiiro obtains a mysterious smartphone."}, {"m_title": "Element Hunters", "r": "7.7", "year": "2009", "votes": "20", "genre": "Adventure, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Shirogane no Ishi: Argevollen", "r": "6.1", "year": "2014", "votes": "73", "genre": "Action, Sci-Fi", "desp": "In a time of war two countries fight for their own ideals using any means necessary. Susumu Tokimune is a new recruit who in his first battle encounters an engineer with a new machine called Argevollen, which will hopefully bring peace."}, {"m_title": "Genshiken Nidaime", "r": "6.8", "year": "2013", "votes": "88", "genre": "Animation", "desp": "After reflecting upon last years low freshman turnout, Chika Ogiue, the new president of Shiiou Universitys Society for the Study of Modern Visual Culture (aka Genshiken) seeks out new. watch the dude, no spoilers in summary!"}, {"m_title": "Rumble Garanndoll", "r": "5.7", "year": "2021", "votes": "37", "genre": "Action, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "You0 DECO", "r": "5.8", "year": "2022", "votes": "52", "genre": "Mystery, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Mahô tsukai Sarî", "r": "6.9", "year": "1989", "votes": "23", "genre": "Comedy, Fantasy", "desp": "Sally, the princess of the land of magic, comes down to earth against her fathers will."}, {"m_title": "The Highschool Life of a Fudanshi", "r": "6.3", "year": "2016", "votes": "165", "genre": "Comedy", "desp": "A comical look into the life of a boy, who just wants to read his BL- manga in peace."}, {"m_title": "Koufuku Graffiti", "r": "6.7", "year": "2015", "votes": "80", "genre": "Comedy", "desp": "Ryo Machiko is a second-year middle school girl who has been living by herself since her grandmother died. Despite having exceptional skill in cooking, Ryo has felt her cooking hasnt been. watch the dude, no spoilers in summary!"}, {"m_title": "Legend of the Mystical Ninja", "r": "4.0", "year": "1997", "votes": "8", "genre": "Action, Comedy", "desp": "Makuamuge, an evil overlord wants to take over the real world and video game world so ninja Goemon and his friends must stop him."}, {"m_title": "Gunparade March", "r": "6.9", "year": "2003", "votes": "104", "genre": "Action, Sci-Fi", "desp": "In 1945, the Pacific War ended only to see the beginning of an alien invasion. For half a century, Earth has united to fight them off. In 1999, a group of Japanese high school students. watch the dude, no spoilers in summary!"}, {"m_title": "Circlet Princess", "r": "7.0", "year": "2019", "votes": "11", "genre": "Action, Sci-Fi", "desp": "Virtual Reality and the arrival of Circlet Technology has changed peoples lives. A new competitive electronic sport called Circlet Bout emerges, and schools duel to determine the strongest. watch the anime dude!"}, {"m_title": "Ôkami kakushi", "r": "6.0", "year": "2010", "votes": "93", "genre": "Horror, Mystery", "desp": "sorry no summary not available!"}, {"m_title": "Cosmo Warrior Zero", "r": "7.0", "year": "2001", "votes": "111", "genre": "Action, Adventure", "desp": "Captain Zero, commanding a spaceship crew of human and aliens, is assigned a near-impossible mission: capture the space pirate Captain Harlock. Will he succeed?"}, {"m_title": "Gal & Dino", "r": "6.8", "year": "2020", "votes": "46", "genre": "Comedy", "desp": "Wasted from the night before, Kaede wakes up to find herself at home with a dinosaur she had brought along with her. The two then start living together as an odd pair"}, {"m_title": "Battle Programmer Shirase", "r": "6.4", "year": "2003", "votes": "81", "genre": "Comedy, Sci-Fi", "desp": "A young computer hacker who doesnt work for money, steals from big bank corporations sucking every last cent out of them, all while battling the King of America."}, {"m_title": "Genji monogatari sennenki: Genji", "r": "6.2", "year": "2009", "votes": "60", "genre": "Drama, History", "desp": "The life of a handsome Japanese prince, and his various love affairs with several different women."}, {"m_title": "The Girl in Twilight", "r": "6.4", "year": "2018", "votes": "45", "genre": "Action, Mystery", "desp": "October, 2018. Girls who live in a provincial city perform a certain ritual. The girls are in the Crystal Radio Club. It is an interest circle that Asuka Tsuchimiya, a girl known for her. watch the dude, no spoilers in summary!"}, {"m_title": "Hanaukyô meido-tai", "r": "6.4", "year": "2001", "votes": "105", "genre": "Comedy, Romance", "desp": "After his mother dies, a 14-year-old boy named Taro is invited by his grandfather to the Hanaukyo mansion. Soon after he arrives, his grandfather disappears, and Taro learns that not only. watch the dude, no spoilers in summary!"}, {"m_title": "Abarenbou Kishi! Matsutarou", "r": "5.9", "year": "2014", "votes": "45", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Dokkoida!?", "r": "7.6", "year": "2003", "votes": "29", "genre": "Comedy, Sci-Fi", "desp": "A comedic Japanese novel series about a boy, Suzuo Sakurazaki, who is hired by a preteen space alien girl, Tanpopo, to try out an experimental new suit developed by the intergalactic toy. watch the dude, no spoilers in summary!"}, {"m_title": "Mahoraba: Heartful Days", "r": "7.3", "year": "2005", "votes": "57", "genre": "Comedy, Drama", "desp": "Shiratori Ryuushi is a young artist-in-training, about to start studying at a vocational school. Arriving at his new lodgings, Narutakisou, he finds himself the focus of attention for the collection of weirdos that inhabit the place."}, {"m_title": "Uta no prince-sama - maji love 2000%", "r": "6.3", "year": "2013", "votes": "131", "genre": "Music", "desp": "STARISH are back on the Master Course at Saotome Academy and are out making work for themselves as idols. Join STARISH on their latest adventures as they fight to become the best idol group in Japan."}, {"m_title": "Genki bakuhatsu Ganbaruger", "r": "7.5", "year": "1992", "votes": "14", "genre": "Adventure, Comedy", "desp": "Three boys, Kotarou, a ninja in training, Rikiya, a training baseball player and Yousuke, a studying scholar were living normal lives in Blue Sky City. One day, Kotarou along with his ...                See full synopsis »"}, {"m_title": "Ambassador Magma", "r": "6.3", "year": "1993", "votes": "28", "genre": "Short, Action", "desp": "The alien invader Goa plots to conquer the Earth. He first warns the Murakami family (father Atsushi, mother Tomoko, and son Mamoru) of their invasion, and demonstrates his powers by. watch the dude, no spoilers in summary!"}, {"m_title": "Spaceship Sagittarius", "r": "8.7", "year": "1986", "votes": "158", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Room Mate", "r": "4.9", "year": "2017", "votes": "33", "genre": "Short", "desp": "sorry no summary not available!"}, {"m_title": "Tetsuko no Tabi", "r": "5.0", "year": "2007", "votes": "10", "genre": "Comedy", "desp": "An anime based upon the real life hobby of Hirohiko Yokomi, a travel writer in Japan who has visited every train station in Japan over the past 15 or so years."}, {"m_title": "Gatchaman Crowds Insight", "r": "7.4", "year": "2015", "votes": "64", "genre": "Action, Adventure", "desp": "One day, a huge UFO spins over Japan and crashes into a rural farmland outside Nagaoka, disrupting Tsubasa Misudachis ordinary life. As the Gatchaman, the legendary defenders of Tachikawa. watch the dude, no spoilers in summary!"}, {"m_title": "Kiratto Pri Chan", "r": "7.5", "year": "2018", "votes": "7", "genre": "Action, Comedy", "desp": "First-year middle school girls Mirai Momoyama and Emo Moegi are two aspiring idols who decide to use the Pri Chan System, a system used by famous people and companies to broadcast content. watch the anime dude!"}, {"m_title": "Bubuki buranki", "r": "6.3", "year": "2016", "votes": "98", "genre": "Action, Drama", "desp": "Away from home for 10 years, Azuma Kazuki attacked by dozens of heavily armed people and taken as their prisoner. Fortunately, Azumas stay in captivity is short. Wielding a strange. watch the dude, no spoilers in summary!"}, {"m_title": "Spider Riders: Yomigaeru taiyou", "r": "6.9", "year": "2007", "votes": "17", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Stella Womens Academy, High School Division Class C³", "r": "6.6", "year": "2013", "votes": "49", "genre": "Comedy, Sport", "desp": "When Yura Yamatos roommate Sonora asks her to join her club, it quickly becomes clear that the girls of Stella Womens Academy are very different. The club is known as C3, whose main after school activity is playing survival games."}, {"m_title": "Try Knights", "r": "6.0", "year": "2019", "votes": "11", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "My Little Sister is an Osaka Momma", "r": "4.8", "year": "2013", "votes": "25", "genre": "Comedy", "desp": "Kyousukes younger sister Namika has been living apart from him for ten years. One day, she returns and begins to live with him again, but she has turned into an energetic Osaka girl."}, {"m_title": "Cyber Team in Akihabara", "r": "6.2", "year": "1998", "votes": "52", "genre": "Action, Adventure", "desp": "A young girl dreams of having her own Patapi mascot (similar to a Tamagotchi), that can be customized with parts."}, {"m_title": "Gegege no Kitarô", "r": "7.8", "year": "1985", "votes": "24", "genre": "Action, Comedy", "desp": "An animated series based on the popular Japanese series of comics by Mizuki Shigeru. Kitaro is a young ghost, the last of his family line of ghosts. With the help of his fathers eye (the. watch the dude, no spoilers in summary!"}, {"m_title": "Kurenai", "r": "6.8", "year": "2008", "votes": "179", "genre": "Comedy, Drama", "desp": "Kurenai works as a mediator, settling disputes between people. He asks for a bigger job and gets more than he bargained for when hes hired to guard and care for the runaway child of a powerful family, Murasaki Kuhoin."}, {"m_title": "Majikaru Tarurûto-kun", "r": "7.0", "year": "1990", "votes": "11", "genre": "Adventure, Comedy", "desp": "Fifth grader Edojou Honmaru is one of the most trodden-upon losers in his class, until he meets Magical Taruruuto-kun. Taruruutos powers help him deal with all his hardships, such as girls, bullies, and numerous other challenges."}, {"m_title": "Zashiki-Warashi no Tatami-chan", "r": "5.2", "year": "2020", "votes": "20", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Time Trouble Tondekeman", "r": "8.2", "year": "1989", "votes": "62", "genre": "Adventure, Comedy", "desp": "Two high school teens find a time travel machine and go back to Baghdad. Abdullah the villain, wants this machine."}, {"m_title": "Kaze no naka no shoujo: kinpatsu no Jeni", "r": "7.3", "year": "1992", "votes": "25", "genre": "Adventure, Drama", "desp": "Inspired by the American Popular Song of the same title, this series revolves around the growing relationship between Jeanie MacDowell and her future husband/aspiring songwriter, Stephen. watch the dude, no spoilers in summary!"}, {"m_title": "Love Love?", "r": "5.2", "year": "2004", "votes": "23", "genre": "Comedy, Romance", "desp": "The self-conscious high school student Ooizumi Naoto is the writer of the CosPrayers TV show, a secret known only to the producer of the show, Ikuta Mitsuki. He has a crush on Yagami Natsumi, one of the actresses in the show."}, {"m_title": "Yozakura Quartet: Hoshi No Umi", "r": "6.8", "year": "2010", "votes": "29", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Ai Mai Mi", "r": "5.4", "year": "2013", "votes": "70", "genre": "Short", "desp": "The story of anime follows girls in a manga club--Ai, Mai, Mi, and Ponoka-senpai--who might be fighting evil invaders threatening Earth, facing off against rivals in tournaments, and. watch the dude, no spoilers in summary!"}, {"m_title": "Majimoji Rurumo", "r": "6.7", "year": "2014", "votes": "82", "genre": "Comedy, Fantasy", "desp": "A sex-crazed high-schooler summons a witch who grants a wish in exchange for the soul of the boy. Refusing to take his soul, the witch gets punished and demoted an apprentice witch. To. watch the dude, no spoilers in summary!"}, {"m_title": "Yes! Precure 5", "r": "7.6", "year": "2007", "votes": "39", "genre": "Action, Adventure", "desp": "Nozomi Yumehara, a regular middle school student, finds a magical book called the Dream Collet in the library and meets Coco, a creature from the Palmier Kingdom. Nozomi decides to help. watch the dude, no spoilers in summary!"}, {"m_title": "Toppo Jijo", "r": "7.2", "year": "1988", "votes": "33", "genre": "Family", "desp": "sorry no summary not available!"}, {"m_title": "OZ", "r": "6.3", "year": "1992", "votes": "18", "genre": "Sci-Fi", "desp": "The story is set in 2021, 31 years after World War III. Filicia Epstein, a bionics genius, came to Sabano and Sergent Mutou was ordered to protect her. Searching for her only brother, Lyon,. watch the anime dude!"}, {"m_title": "Himitsu no Akko-chan", "r": "6.4", "year": "1998", "votes": "17", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Amazing Sarutobi", "r": "6.4", "year": "1982", "votes": "11", "genre": "Action, Adventure", "desp": "Comedy ninja anime set in Japanese college environment."}, {"m_title": "Chô Soku Henkei Gyrozetter", "r": "6.0", "year": "2012", "votes": "19", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Wild ARMs: Twilight Venom", "r": "6.3", "year": "1999", "votes": "66", "genre": "Action, Adventure", "desp": "A deadly gunslinger named Sheyenne Rainstorm who is trapped in the body of a ten year old boy, he and his friends embark on a journey to find his lost past and find out who switched his body."}, {"m_title": "Merc StoriA: The Apathetic Boy and the Girl in a Bottle", "r": "6.6", "year": "2018", "votes": "24", "genre": "Fantasy", "desp": "Yuu, a young healer with the power to heal the hearts of monsters, receives a mysterious bottle from his father as a gift. When Yuu touches the bottle, Merc, a girl made of liquid, appears. watch the dude, no spoilers in summary!"}, {"m_title": "Hina Logic: From Luck & Logic", "r": "5.6", "year": "2017", "votes": "24", "genre": "Animation", "desp": "Princess Liones is trading in her crown for a school uniform at ALCAs Logicalist school to join other talented students!"}, {"m_title": "Sadamitsu the Destroyer", "r": "6.5", "year": "2001", "votes": "30", "genre": "Action, Sci-Fi", "desp": "A gang leader and a sentient alien helmet team up, literally, to become the Destroyer, the ultimate warrior against alien warriors!"}, {"m_title": "Sketchbook: Full Colors", "r": "7.7", "year": "2007", "votes": "60", "genre": "Comedy, Drama", "desp": "Having recently joined her schools art club out of love for drawing in her sketchbook, a shy introverted girl begins to view her perspective of the world through the lens of being an artist."}, {"m_title": "Minami-ke: Okaeri", "r": "7.3", "year": "2009", "votes": "97", "genre": "Comedy", "desp": "A year has passed since Minami-ke: Okawari and the three sisters have grown up. Their likings and moods are almost the same. Haruka, the older sister, is a love-giving mother to the. watch the dude, no spoilers in summary!"}, {"m_title": "Seton dôbutsuki", "r": "8.0", "year": "1989", "votes": "30", "genre": "Animation", "desp": "Documentary in which the achievement of Ernest Black Wolf Thompson is settled in different Episodes. Every single episode is a documentary in which one sort of Animal is described."}, {"m_title": "SaiKano: Another Love Song", "r": "7.0", "year": "2005", "votes": "9", "genre": "Action, Drama", "desp": "Every weapon must have a prototype. For the ultimate weapon, career soldier Lt. Mizuki is that prototype. As she comes to the limit of her powers as a weapon, she must watch the unfolding of Chises growth into the final weapon."}, {"m_title": "Dino Adventure Jurassic Tripper", "r": "7.8", "year": "1995", "votes": "19", "genre": "Adventure, Sci-Fi", "desp": "Several children and teenagers get sent to another world where dinosaurs still roam."}, {"m_title": "Legendz: Tale of the Dragon Kings", "r": "8.3", "year": "2004", "votes": "14", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Matoi the Sacred Slayer", "r": "6.7", "year": "2016", "votes": "38", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "The Third: Aoi hitomi no shoujo", "r": "6.7", "year": "2006", "votes": "115", "genre": "Action, Adventure", "desp": "Honoka, a desert-wandering jack-of-all-trades, must deal with the technology-oppressing group known as The Third, as well as her own inner feelings and struggles."}, {"m_title": "Steel Angel Kurumi 2", "r": "6.8", "year": "2001", "votes": "33", "genre": "Action, Comedy", "desp": "Taking place 75 years after the first series, Steel Angel Kurumi 2 brings the Steel Angels in a new mis-adventure. High school student and aspiring cellist Nako Kagura accidentally. watch the dude, no spoilers in summary!"}, {"m_title": "Project A-Ko Versus", "r": "6.4", "year": "1990", "votes": "222", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "God Mazinger", "r": "7.5", "year": "1984", "votes": "42", "genre": "Action, Fantasy", "desp": "Yamato Hino is transported to a magical world to defend it. Unbeknownst to him, hes the only person that can bring the lands stone God Mazinger, to life. He does, and becomes the lands only hope against the evil ruler Dorado."}, {"m_title": "Planet With", "r": "6.7", "year": "2018", "votes": "130", "genre": "Sci-Fi", "desp": "Kuroi Souya is a high school-er living a peaceful life despite having no memories of his past. One day, however, his town is attacked by one of the mysterious Nebula Weapons. Together with. watch the dude, no spoilers in summary!"}, {"m_title": "Three Little Ghosts", "r": "6.9", "year": "1991", "votes": "15", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Zettai Shonen", "r": "7.3", "year": "2005", "votes": "49", "genre": "Drama, Fantasy", "desp": "Oftentimes, people are most vulnerable when they are lonely and unable to get along with those around them. Isolation is felt more keenly amongst a crowd, so some will try to find solace in. watch the anime dude!"}, {"m_title": "Umi Monogatari", "r": "6.8", "year": "2009", "votes": "36", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Kemono Friends 2", "r": "6.6", "year": "2019", "votes": "94", "genre": "Adventure, Comedy", "desp": "Different cast and crew were hired for this brand new adaptation of Kemono Friends after original director Tatsuki was fired by Kadokawa."}, {"m_title": "D.C.S.S: Da Capo Second Season", "r": "6.4", "year": "2005", "votes": "52", "genre": "Comedy, Drama", "desp": "Hatsune-Jima is an island formed from a dormant volcano and is known for its cherry trees that bloom out of season. Some people believe that one of the trees is capable of granting any wish. watch the anime dude!"}, {"m_title": "Gunslinger Girl: Il Teatrino", "r": "6.8", "year": "2008", "votes": "521", "genre": "Action, Drama", "desp": "In the heart of Italy, the Social Welfare Agency rescues young girls from hospital beds and gives them a second chance at life using the latest in cybernetic advancements. With their. watch the dude, no spoilers in summary!"}, {"m_title": "Cockroach Girls", "r": "6.1", "year": "2012", "votes": "13", "genre": "Comedy", "desp": "A heartwarming story about a cockroach girl who tries to be liked by humans while dodging the insecticide."}, {"m_title": "Magical Girl Lyrical Nanoha As", "r": "7.6", "year": "2005", "votes": "96", "genre": "Action, Fantasy", "desp": "Taking place after the first Nanoha series, Nanoha and Fate with their friends Yuuno, Arf, Chrono and the Space/Time Administration Bureau must team up again to stop rogue mages."}, {"m_title": "The Young Innkeeper Is a Grade Schooler!", "r": "6.7", "year": "2018", "votes": "38", "genre": "Fantasy", "desp": "The parents of Oriko Okko Seki are killed in a car accident and she moves in with her grandmother Mineko, who runs the Hananoyu Inn. Mineko then makes her the junior innkeeper. At first. watch the anime dude!"}, {"m_title": "Fractale", "r": "6.7", "year": "2011", "votes": "252", "genre": "Adventure, Comedy", "desp": "In a futuristic world, most people have come to be totally dependent on avatars they can use in the real world due to the Fractale System. The problems of this Utopian world are becoming. watch the dude, no spoilers in summary!"}, {"m_title": "Jewelpet Twinkle", "r": "8.7", "year": "2010", "votes": "31", "genre": "Fantasy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Soar High! Isami", "r": "7.7", "year": "1995", "votes": "35", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Endride", "r": "6.5", "year": "2016", "votes": "106", "genre": "Adventure, Fantasy", "desp": "When a crystal transports Shun to Endra, only Prince Emilio can help him back home. But first, he must help Emilio get back his throne."}, {"m_title": "Domo", "r": "7.2", "year": "2008", "votes": "33", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sensual Phrase", "r": "6.8", "year": "1999", "votes": "41", "genre": "Drama, Music", "desp": "An anime about a rock band whos lead vocalist falls in love with a high school girl."}, {"m_title": "Windy Tales", "r": "7.1", "year": "2004", "votes": "59", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Pluster World", "r": "8.1", "year": "2003", "votes": "10", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "If I See You in My Dreams", "r": "6.7", "year": "1998", "votes": "58", "genre": "Comedy, Drama", "desp": "The story of Masuo Fuguno, an unlucky salesman who constantly juggles between two women: his friend Miho and his love Nagisa."}, {"m_title": "Giant Gorg", "r": "7.3", "year": "1984", "votes": "38", "genre": "Action, War", "desp": "A Japanese teenager and his friend investigate a mysterious 2000 km island that appears out of nowhere. They are pursued by an evil organisation, GAIL, and join forces with the islands defender, Giant Gorg."}, {"m_title": "Kabukibu!", "r": "7.4", "year": "2017", "votes": "14", "genre": "Comedy", "desp": "Kurogo Kurusu is a high school student who loves all things Kabuki - the classical dance-dramas that have inspired Japans theater fans since the early 17th century. When he finds out that. watch the dude, no spoilers in summary!"}, {"m_title": "Magical Girl Lyrical Nanoha StrikerS", "r": "7.3", "year": "2007", "votes": "71", "genre": "Action, Comedy", "desp": "Subaru Nakajima is a young girl who gets trapped in an airport fire. The mage, Nanoha Takamichi, rescues her and becomes Subarus role model as she aspires to become a mage who can save. watch the dude, no spoilers in summary!"}, {"m_title": "Bakuen Campus Guardress", "r": "7.7", "year": "1994", "votes": "39", "genre": "Action, Comedy", "desp": "Within a high school is hidden the gate to the netherworld. Its students and teachers must fight demons who want the gate opened."}, {"m_title": "Boku no Patrasche", "r": "8.6", "year": "1992", "votes": "18", "genre": "Drama, History", "desp": "A boy is raised by his grandfather. After finding a dog, they try to use it to sell milk"}, {"m_title": "Kami Kuzu Idol", "r": "6.8", "year": "2022", "votes": "24", "genre": "Comedy, Music", "desp": "Kami Kuzu Idol is based upon the manga Phantom of the Idol. Music promoters have paired two good looking boys to make the pop idol duo ZINGS. They have some problems though. The nice. watch the dude, no spoilers in summary!"}, {"m_title": "elDLIVE", "r": "6.2", "year": "2017", "votes": "59", "genre": "Action, Crime", "desp": "Chuuta Kokonose has been hearing voices in his head since childhood. One day, he gets taken and transported to a space police station, filled with mysterious creatures and given a test, to check his ability to become a member of eLDLIVE."}, {"m_title": "Kaidan Restaurant", "r": "8.0", "year": "2009", "votes": "13", "genre": "Fantasy, Horror", "desp": "Shou, Ako and Reiko are three friends who live in a spooky town filled with ghost stories and haunted places, such as the abandoned Kaidan Restaurant. From an encounter with a bullied. watch the dude, no spoilers in summary!"}, {"m_title": "Night Head Genesis", "r": "6.0", "year": "2006", "votes": "133", "genre": "Drama, Horror", "desp": "Due to their paranormal abilities, two young brothers (Naoto and Naoya) are cast out by their parents and given into the custody of a research center. They escape fifteen years later, and. watch the dude, no spoilers in summary!"}, {"m_title": "Ramune", "r": "6.5", "year": "2005", "votes": "41", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Seraphim Call", "r": "7.8", "year": "1999", "votes": "18", "genre": "Drama, Romance", "desp": "An anime anthology series."}, {"m_title": "Uchû kôkyôshi Mater: Ginga tetsudô Three-Nine gaiden", "r": "7.4", "year": "2004", "votes": "38", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Long Riders!", "r": "7.2", "year": "2016", "votes": "49", "genre": "Sport", "desp": "After seeing someone ride a bike, a clumsy college student buys her own bike and soon discovers her passion in bike racing."}, {"m_title": "Reideen", "r": "7.2", "year": "2007", "votes": "39", "genre": "Action, Sci-Fi", "desp": "Saiga is a normal high school student with a gift in mathematics. His daily routine is disrupted when his family gets news that his Fathers remains have been discovered-a noted. watch the dude, no spoilers in summary!"}, {"m_title": "Ani Tore! EX", "r": "4.6", "year": "2015", "votes": "56", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Project Blue Chikyû SOS", "r": "5.9", "year": "2006", "votes": "46", "genre": "Action, Sci-Fi", "desp": "While deranged alien forces prepare a fleet of flying saucers for the invasion of all invasions, teenage geniuses Penny Carter and Billy Kimura join forces with a clandestine secret. watch the dude, no spoilers in summary!"}, {"m_title": "Chibi Devi!", "r": "7.5", "year": "2011", "votes": "26", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Bosco daiboken", "r": "8.5", "year": "1986", "votes": "158", "genre": "Action, Adventure", "desp": "The story is about a young elvish Princess Apricot whose mission is to return to her home - Fountain land, occupied by evil forces of a monster called Scorpion, before the total eclipse of. watch the dude, no spoilers in summary!"}, {"m_title": "The Brave Fighter of Legend Da-Garn", "r": "7.4", "year": "1992", "votes": "22", "genre": "Action, Sci-Fi", "desp": "Seiji Takasugi gets a jewel called the Aurin Witch that is Earths alter ego which chooses him to be the commander of eight brave robots. The robots are sleeping in various parts of the. watch the dude, no spoilers in summary!"}, {"m_title": "Ive Had Enough of Being a Magical Girl", "r": "5.7", "year": "2016", "votes": "34", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Yuki no Jo-Oh", "r": "6.9", "year": "2005", "votes": "21", "genre": "Fantasy", "desp": "Kai and Gerda are very dear friends who have been living together since young. However, one wintry day, the Snow Queen appears stirring up snow and wind, and a sliver of mirror gets lodged. watch the dude, no spoilers in summary!"}, {"m_title": "Celestial Method", "r": "6.4", "year": "2014", "votes": "74", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Demonbane", "r": "6.5", "year": "2006", "votes": "66", "genre": "Comedy, Romance", "desp": "Kuro Daijuuji, a poor detective living in Arkham City, was requested by Ruri Hado of Hado Financial Group to search for a magic book (grimoire). As Kuro searches for the book, he. watch the dude, no spoilers in summary!"}, {"m_title": "The Pilots Love Song", "r": "7.1", "year": "2014", "votes": "119", "genre": "Action, Adventure", "desp": "This story follows a former prince who lost everything to a revolution, and a girl who became the symbol for the revolution. The orphaned prince, now renamed Karueru, embarks on a one-way journey to discover the ends of the heavens."}, {"m_title": "Aquarian Age: Sign for Evolution", "r": "6.0", "year": "2002", "votes": "57", "genre": "Action, Romance", "desp": "Five supernatural factions have been fighting against each other for who knows how many centuries, with the beginning of Aquarian Age always in mind. Kyouta, soon begins to see visions of. watch the dude, no spoilers in summary!"}, {"m_title": "D_Cide Traumerei the Animation", "r": "5.6", "year": "2021", "votes": "46", "genre": "Action, Drama", "desp": "The stage is Shibuya. When Ryuuhei Oda was in elementary school, he admired his kind older brother. Ryuuhei, who witnessed the mysterious death of his brother, is now a cheerful high. watch the dude, no spoilers in summary!"}, {"m_title": "Seven of Seven", "r": "7.4", "year": "2002", "votes": "21", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Bakusô Kyôdai Lets & Go MAX", "r": "6.8", "year": "1998", "votes": "24", "genre": "Sport", "desp": "The story follows the life of Retsuya and Gouki, two brothers with the same objective as Go and Retsu, to race and win championships."}, {"m_title": "Middle and high school consistent! !! Kimetsu Gakuen Story", "r": "6.6", "year": "2019", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Anne Happy", "r": "6.9", "year": "2016", "votes": "98", "genre": "Comedy, Drama", "desp": "The story centers around class 1-7 of Tennomifune Academy, where all the students with bad karma or misfortune seem to have been gathered. Hibari, a student in this class, meets the. watch the dude, no spoilers in summary!"}, {"m_title": "Robby the Rascal", "r": "6.5", "year": "1982", "votes": "20", "genre": "Family, Sci-Fi", "desp": "In this Japanese anime series, we follow a cute (if rather dysfunctional and hyperactive) robot and his many adventures with his robot and children pals. Meanwhile, the diabolical. watch the dude, no spoilers in summary!"}, {"m_title": "The Daichis: Earths Defense Family", "r": "7.8", "year": "2001", "votes": "23", "genre": "Comedy, Sci-Fi", "desp": "The Daichis are a typical Japanese family on the verge of collapse. The mother wants a divorce and its every person for him or herself. One day they are recruited by mysterious powers to. watch the dude, no spoilers in summary!"}, {"m_title": "Natsu no arashi!", "r": "7.0", "year": "2009", "votes": "69", "genre": "Comedy, Romance", "desp": "Thirteen-year-old Yasaka is a boy staying at his grandfathers house during his summer vacation. One day he entered a store and met Arashi, a beautiful sixteen-year-old girl working there.. watch the dude, no spoilers in summary!"}, {"m_title": "Aquarion Logos", "r": "5.9", "year": "2015", "votes": "34", "genre": "Sci-Fi", "desp": "Celebrating its 10th anniversary, Aquarion returns with a new series, new characters and a brand new adventure! Inspired by the classic mecha series of the 70s and 80s, Aquarion Logos is. watch the dude, no spoilers in summary!"}, {"m_title": "Extra Olympia Kyklos", "r": "7.1", "year": "2020", "votes": "51", "genre": "Comedy", "desp": "Demetrios, a vase painter in Ancient Greece, has been transported to Tokyo, Japan, during the 1964 Summer Olympics. Introducing Ryo Fuji, a renowned director of Japanese commercials and trailers."}, {"m_title": "Wacky TV Na Na Na", "r": "6.6", "year": "2018", "votes": "10", "genre": "Short, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Mars Daybreak", "r": "7.3", "year": "2004", "votes": "60", "genre": "Sci-Fi", "desp": "Mars is now almost covered by ocean. Many have chosen the pirates life, but the crew of the Aurora catches the eye of the ruling Earth Government. Animation by studio BONES. Music by Kaoru Wada."}, {"m_title": "Beast Fighter - The Apocalypse", "r": "5.7", "year": "2003", "votes": "11", "genre": "Action, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Woodpecker Detectives Office", "r": "7.0", "year": "2020", "votes": "63", "genre": "History, Mystery", "desp": "A ghost was seen on the 12th floor of the Asakusa Juunikai building. A newspaper article featuring the sighting brought together Takuboku Ishikawa, who runs a private detective agency and his assistant Kyosuke Kindaichi to solve the case."}, {"m_title": "F-Zero: Farukon densetsu", "r": "7.2", "year": "2003", "votes": "17", "genre": "Sport", "desp": "The year is 2201 and across the universe rabid fans flock to F-Zero tracks everywhere. The prizes are enormous and the stakes grow even higher when the outlaws compete to win - at any cost.. watch the anime dude!"}, {"m_title": "Armored Trooper VOTOMS: Pailsen Files", "r": "6.5", "year": "2007", "votes": "19", "genre": "Sci-Fi, War", "desp": "A few weeks after the Gilgamesh Confederations victory over the Balarant Union at Sunsa, the Melkian Space Force High Command arrests Red Shoulder commander Col Yoran Pailsen. He undergoes a court-martial over his management of the unit."}, {"m_title": "Miyuki", "r": "7.5", "year": "1983", "votes": "45", "genre": "Family, Romance", "desp": "Masato Wakamatsu, 16 years old, is working at the beach one summer. Having made a bad impression on his classmate/crush Miyuki Kashima, he makes a pass at another pretty girl. To Masatos. watch the dude, no spoilers in summary!"}, {"m_title": "Tactics", "r": "7.1", "year": "2004", "votes": "106", "genre": "Action, Adventure", "desp": "Kantarou has had the 6th sense to see demons for as long as he can remember, and has become friends with them, but also has the job of yokai busting. With his tengu and his kitsune, they. watch the dude, no spoilers in summary!"}, {"m_title": "Kinnikuman: Kinniku-sei ôi sôdatsu hen", "r": "7.0", "year": "1991", "votes": "30", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Over Drive Girl 1/6 - Amazing Stranger", "r": "5.6", "year": "2019", "votes": "18", "genre": "Comedy, Romance", "desp": "Haruto Bouida is an otaku who is only interested in 2D girls and doesnt care about real women. When a figurine of one of his favorite female anime characters, Nona, starts moving and talking, a strange relationship ensues."}, {"m_title": "World Witches Take Off!", "r": "5.2", "year": "2021", "votes": "28", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Jungle King Taa-chan", "r": "7.4", "year": "1993", "votes": "17", "genre": "Action, Comedy", "desp": "A parody of the legend of Tarzan."}, {"m_title": "Irodorimidori", "r": "5.7", "year": "2022", "votes": "15", "genre": "Short, Music", "desp": "At Maigahara Music College Affiliate School Maigahara Senior High, theres a rumor circulating among students: If you give an amazing performance at the school festival, you get special. watch the dude, no spoilers in summary!"}, {"m_title": "El-Hazard 2: The Magnificent World", "r": "6.7", "year": "1997", "votes": "132", "genre": "Short, Fantasy", "desp": "While Makoto and his friends search for a missing friend, his enemies find a new super-weapon who is not what she seems to be."}, {"m_title": "B-Project", "r": "6.5", "year": "2016", "votes": "33", "genre": "Drama, Music", "desp": "sorry no summary not available!"}, {"m_title": "Maho Tsukai PreCure!", "r": "7.8", "year": "2016", "votes": "30", "genre": "Action, Adventure", "desp": "One day, Mirai was in the park when a witch, Riko falls off her broom and together with Mofurun- Mirais talking teddy bear, they became Precure to fight stop the villains/ monsters called Yokubal while collecting remaining Linkle Stones."}, {"m_title": "Luminous Witches", "r": "6.4", "year": "2022", "votes": "27", "genre": "Music, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Getsumento Heiki Mina", "r": "6.6", "year": "2007", "votes": "20", "genre": "Animation", "desp": "Actor Itoh Atsushi plays Yamada, whose main love in life is the Mina anime, a cutesy busty heroine with floppy rabbit ears and carrots sticking out her rear. (Source: AniDB)"}, {"m_title": "Angels Feather", "r": "4.5", "year": "2006", "votes": "36", "genre": "Drama, Romance", "desp": "Shou gets transferred to a boys only private school, and thinks he meets his lost brother, Kai who denies it. He re-encounters his best friend Naoto, then a mysterious earthquake happens. watch the dude, no spoilers in summary!"}, {"m_title": "GEAR senshi Dendô", "r": "7.2", "year": "2000", "votes": "23", "genre": "Action, Adventure", "desp": "The story takes place in the future where war machines from evil mechanical alien empire Garufa finally reaches Earth. In order to protect earth, an Earth defense organization called GEAR (. watch the anime dude!"}, {"m_title": "Shikizakura", "r": "5.5", "year": "2021", "votes": "22", "genre": "Drama, Fantasy", "desp": "Shikizakura: where cherry blossoms are in full bloom while trees are colored with autumn leaves. In this special place, where the transient world meets the spirit world, a ritual to save. watch the dude, no spoilers in summary!"}, {"m_title": "Natsuiro kiseki", "r": "7.2", "year": "2012", "votes": "46", "genre": "Animation", "desp": "The four girls, Natsumi, Saki, Yuka, and Ringo have always been good friends and in the past shared all their experiences. Now however, friction has started to develop between Natsumi and. watch the dude, no spoilers in summary!"}, {"m_title": "La croisée dans un labyrinthe étranger", "r": "7.6", "year": "2011", "votes": "81", "genre": "Drama", "desp": "In the late 19th century, a young Japanese girl comes to France for the first time."}, {"m_title": "Karasu Tengu Kabuto", "r": "7.2", "year": "1990", "votes": "28", "genre": "Action, Adventure, Animation", "desp": "In the Tensho era, a man named Kabuto who has a talking sword that is hungry for blood and the Gods of the North, South East and West, has to follow his destiny of combating the Black Night Demon who has returned from Hell."}, {"m_title": "As Miss Beelzebub Likes It", "r": "6.4", "year": "2018", "votes": "60", "genre": "Comedy, Fantasy", "desp": "In the hell, there lives their ruler, Beelzebub who loves fluffy things, her Assistant, Mullin whos a Tsundere and other demons. See their day to day life in the Heaven that looks like Hell."}, {"m_title": "Saiko ama Gobarian", "r": "7.7", "year": "1983", "votes": "11", "genre": "Action, Sci-Fi", "desp": "An alien metaphysician/mech designer and defector of the Garadian Empire trains multiple psychic youths to use their talents against the oncoming Garadian invasion of Earth."}, {"m_title": "Hana no mahoutsukai Mary Bell", "r": "7.1", "year": "1992", "votes": "30", "genre": "Adventure, Comedy", "desp": "Mary Bell, a magical girl who looks around five years old, comes to the Human World from a magical world called the Flower Magic World. Using her Flower Magic she helps and encourages ...                See full synopsis »"}, {"m_title": "Kono Aozora Ni Yakusoku Wo", "r": "4.4", "year": "2007", "votes": "18", "genre": "Romance", "desp": "sorry no summary not available!"}, {"m_title": "Ochame kami monogatari koro koro Pollon", "r": "7.4", "year": "1982", "votes": "249", "genre": "Adventure, Comedy", "desp": "Pollon, a sweet little girl who is the daughter of the God of the Sun, Apollo. Pollons goal in life is to grow up to become a beautiful and powerful goddess. She attempts to do good deeds. watch the dude, no spoilers in summary!"}, {"m_title": "Ninja Nonsense", "r": "7.3", "year": "2004", "votes": "148", "genre": "Comedy", "desp": "The story of the friendship between Kaede, an ordinary schoolgirl, and Shinobu, an enthusiastic but incompetent trainee female ninja."}, {"m_title": "Venus Versus Virus", "r": "5.9", "year": "2007", "votes": "81", "genre": "Action, Drama", "desp": "Pricked by a special broach, Sumire finds herself able to see evil spirits (viruses). Lucia exterminates viruses and mistakenly shoots Sumire with an antiviral bullet, which transforms her. watch the dude, no spoilers in summary!"}, {"m_title": "Pia Carrot e Yokoso! 2 DX", "r": "5.4", "year": "1999", "votes": "11", "genre": "Short, Comedy", "desp": "Kouji wants to work part-time, so, when he saw a cute girl with the Pia Carrot uniform, he tried to get a job there. When hes on the way, he accidentally bumped Azusa, and with some. watch the dude, no spoilers in summary!"}, {"m_title": "Licensed by Royalty", "r": "6.4", "year": "2003", "votes": "47", "genre": "Action, Adventure", "desp": "Two hit men are assigned a case to protect the Royal family of Ishtar and their assets which includes their daughter, Noelle."}, {"m_title": "Kanamemo", "r": "6.5", "year": "2009", "votes": "93", "genre": "Comedy", "desp": "Kana Nakamichi is a homeless orphan who finds refuge, a new home and new friends at a newspaper delivery office."}, {"m_title": "Kujibiki anbaransu", "r": "5.7", "year": "2004", "votes": "61", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Burn-Up Scramble", "r": "6.3", "year": "2004", "votes": "25", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Wish Upon the Pleiades", "r": "6.2", "year": "2015", "votes": "15", "genre": "Fantasy, Sci-Fi", "desp": "Subaru is a junior high student that loves the stars. One day, after school, she meets aliens from Pleiades in outer space. The Pleiadesians said they summoned a helper from Earth to. watch the dude, no spoilers in summary!"}, {"m_title": "Code-E", "r": "6.5", "year": "2007", "votes": "30", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Tayutama: Kiss on my deity", "r": "6.5", "year": "2009", "votes": "62", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Hataraki Man", "r": "7.1", "year": "2006", "votes": "80", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Steam Detectives", "r": "6.4", "year": "1998", "votes": "23", "genre": "Action, Adventure", "desp": "Steam City is a place where the only fuel source is coal, and the only means to produce energy is the steam engine. As the only source of energy, the steam engine has been the focus of. watch the dude, no spoilers in summary!"}, {"m_title": "Shin Tennis no Ouji-sama: Hyoutei vs Rikkai - Game of Future", "r": "7.4", "year": "2021", "votes": "9", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "To Heart 2", "r": "6.3", "year": "2005", "votes": "22", "genre": "Comedy, Drama", "desp": "Following her graduation from middle school, Konomi Yuzuhara enters the same high school as Takaaki Kono, her childhood friend."}, {"m_title": "Ground Defense Force Mao-chan", "r": "7.0", "year": "2002", "votes": "31", "genre": "Family, Sci-Fi", "desp": "The Japanese military selects three 8-year-old girls to defend Japan against cute aliens."}, {"m_title": "Adventures of the Mini Goddesses", "r": "6.8", "year": "1998", "votes": "50", "genre": "Animation", "desp": "A random collection of adventures of the Goddesses featured multiple versions of Ah My Goddess. Parodies of other works, and a large number of jokes pervade this series of shorts in which. watch the dude, no spoilers in summary!"}, {"m_title": "Moshi koukou yakyuu no joshi manêjâ ga Dorakkâ no Manejimento wo yondara", "r": "6.3", "year": "2011", "votes": "31", "genre": "Drama, Sport", "desp": "Minami joins her High School baseball team as a team manager after finding out that her best friend Yuuki is in the hospital and cant be a team manager any more. In order to try to fill in. watch the anime dude!"}, {"m_title": "Sola", "r": "6.8", "year": "2007", "votes": "192", "genre": "Action, Drama", "desp": "High school boy falls for a mysterious girl he meets on the street. Turns out shes a powerful and almost immortal creature damned to walk the Earth alone forever. She is being hunted, so the boy invites her to hide at his place."}, {"m_title": "Sentimental Journey", "r": "7.6", "year": "1998", "votes": "6", "genre": "Drama, Romance", "desp": "Sentimental Journey is a collection of twelve short stories about twelve different girls. Though unrelated, there is one common theme that binds the episodes together: each girls bittersweet experience of first love."}, {"m_title": "Flag", "r": "6.5", "year": "2006", "votes": "133", "genre": "Action, Sci-Fi", "desp": "Saeko Shirasu is a 25-year-old war frontline photo-journalist who became a celebrity after taking a picture of civilians raising a makeshift UN flag in war-torn Uddiyana. The image then. watch the dude, no spoilers in summary!"}, {"m_title": "Strange Dawn", "r": "5.6", "year": "2000", "votes": "25", "genre": "Action, Adventure", "desp": "Two teenage girls (Yuko and Eri) are transported to a magical planet which is dominated by small semi-humans. To the girls surprise they are treated as Gods by the little creatures."}, {"m_title": "Tesla Note", "r": "5.7", "year": "2021", "votes": "38", "genre": "Action", "desp": "Mission T is a secret operation to save the world from destruction. Trained as a ninja from a young age, Botan Negoro, raised to become the ultimate spy, teams up with another excellent spy, Kuruma."}, {"m_title": "Onegai mai merodi: Kuru kuru shaffuru!", "r": "7.2", "year": "2006", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "One Stormy Night: Secret Friends", "r": "5.1", "year": "2012", "votes": "10", "genre": "Family", "desp": "A wolf and a goat embark on adventures as secret friends after meeting one stormy night."}, {"m_title": "Yattâman", "r": "6.8", "year": "2008", "votes": "31", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Namuamida Butsu! Rendai Utena", "r": "5.5", "year": "2019", "votes": "20", "genre": "Animation", "desp": "The Thirteen Buddhas along with Taishakuten and Bonten, must confront the evil Mara. Mara seeks revenge against Shaka Nyorai, who had previously saved the world from suffering and conflict by achieving enlightenment."}, {"m_title": "Shin Jeeg", "r": "7.4", "year": "2007", "votes": "36", "genre": "Action, Sci-Fi", "desp": "The story of Kotetsushin Jeeg takes place fifty years after the original and features a new cast of characters - primarily the new main character Kenji Kusanagi, a high school student and ...                See full synopsis »"}, {"m_title": "Uchi Tama?! Uchi no Tama Shirimasen ka?", "r": "6.7", "year": "2020", "votes": "30", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Tsukipro the Animation", "r": "3.8", "year": "2017", "votes": "15", "genre": "Music", "desp": "The music anime will follow the daily lives and associated drama of four of TsukiPros very unique groups - SolidS, Quell, SOARA, and Growth."}, {"m_title": "Deji Meets Girl", "r": "5.5", "year": "2021", "votes": "66", "genre": "Animation", "desp": "One summer Maise Higa, a listless first-year high school girl who works part-time at the front desk of her familys hotel in Okinawa, meets an Ichiro Suzuki, a mysterious youth who came alone from Tokyo as a hotel guest."}, {"m_title": "DD Fist of the North Star", "r": "6.9", "year": "2013", "votes": "37", "genre": "Action, Comedy", "desp": "A parody of the post-apocolyptic anime, Fist of the North Star, wherin the nuclear holocaust never happens, and so Kenshiro and his brothers have to settle into a regular society and find jobs!"}, {"m_title": "Busou Shinki", "r": "5.6", "year": "2012", "votes": "41", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Omishi mahou gekijou Risky Safety", "r": "7.3", "year": "1999", "votes": "26", "genre": "Animation", "desp": "An apprentice shinigami (Japanese Death God), Risky, is determined to take Moes life. However, Risky is magically joined with an apprentice angel, Safety, so Risky turns into Safety. watch the dude, no spoilers in summary!"}, {"m_title": "Nurse Witch Komugi-chan R", "r": "5.8", "year": "2016", "votes": "25", "genre": "Comedy, Fantasy", "desp": "Komugi Yoshida, a second year middle school student who is also an idol, encounters a creature named Usa-P. Usa-P gives her magical powers to fight off evil beings."}, {"m_title": "Mission Outer Space Srungle", "r": "7.4", "year": "1983", "votes": "14", "genre": "Action, Sci-Fi", "desp": "In the year 2545, an experiment gone wrong puts test pilot David Jance through the center of the galaxy and into another universe, which ends up in the middle of life and death struggle against the armies of GRIP, led by Dark Star."}, {"m_title": "Megaton-kyuu Musashi", "r": "6.5", "year": "2021", "votes": "26", "genre": "Animation", "desp": "99.9% of the human race has been wiped out by a Dracters,.To save Humanity chosen ones, have become pilots of Rogue and fighting against aliens."}, {"m_title": "Wild Knights Gulkeeva", "r": "8.8", "year": "1995", "votes": "6", "genre": "Action, Adventure", "desp": "Touya has to protect his world with the help of the three Animanoids on his side."}, {"m_title": "Hanayamata", "r": "7.2", "year": "2014", "votes": "87", "genre": "Animation", "desp": "Hana N. Fountainstand is a transfer student who loves Japanese culture. She decides to make a YosakoI group with Naru Sekiya, Yaya Sasame, Tami Nishimikado, and Machi Tokiwa."}, {"m_title": "Polyphonica", "r": "6.5", "year": "2007", "votes": "28", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "The World of Narue", "r": "7.3", "year": "2003", "votes": "106", "genre": "Comedy, Romance", "desp": "Kazuto is a fourteen year old boy who meets a strange girl named Narue who says that she is an alien."}, {"m_title": "Kanon", "r": "6.6", "year": "2002", "votes": "155", "genre": "Comedy, Drama", "desp": "Changing schools is something Yuuchi is accustomed to, due to his parents profession, and with their recent venture to Africa, he is returning to his aunts home after 7 years. The time. watch the dude, no spoilers in summary!"}, {"m_title": "Tsubasa: Shunraiki Zenpen", "r": "7.4", "year": "2009", "votes": "137", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Dotto! Koni-chan", "r": "7.0", "year": "2000", "votes": "53", "genre": "Comedy, Fantasy", "desp": "The bizarre, funny and inexplicable adventures of a really lucky guy named Koni and his really crazy friends."}, {"m_title": "Oishinbo", "r": "7.8", "year": "1988", "votes": "21", "genre": "Comedy, Drama", "desp": "Oishinbo is a drama about newspaper reporters. The main character is a cynical food critic named Yamaoka."}, {"m_title": "Uta kata", "r": "6.6", "year": "2004", "votes": "55", "genre": "Drama, Fantasy", "desp": "One day a girl named Ichika was cleaning out this old room in the school for summer break clean up. When she forgets her phone she goes back to get it when suddenly a girl comes out of the. watch the dude, no spoilers in summary!"}, {"m_title": "Guardian Ninja Mamoru!", "r": "6.3", "year": "2006", "votes": "44", "genre": "Comedy, Romance", "desp": "Nerdy boy Mamoru is secretly a highly competent Ninja on a mission to protect his clumsy neighbor Yuuna."}, {"m_title": "Baby Felix", "r": "6.4", "year": "2000", "votes": "35", "genre": "Family", "desp": "sorry no summary not available!"}, {"m_title": "Goku sayonara zetsubô sensei", "r": "8.0", "year": "2008", "votes": "109", "genre": "Comedy, Drama", "desp": "The new adventures of the negative-minded teacher Nozomu Itoshiki and his class of nice but nutty students."}, {"m_title": "Shin Koihime musô: Otome tairan", "r": "7.5", "year": "2010", "votes": "30", "genre": "Adventure, Comedy", "desp": "When theres trouble afoot in ancient China, its up to the Sisterhood of Steel to set things right once again. So, when a spell is cast on Kashin that begins to slowly turn her into a cat,. watch the anime dude!"}, {"m_title": "Gaiking: Legend of Daikû Maryû", "r": "6.7", "year": "2005", "votes": "17", "genre": "Adventure, Sci-Fi", "desp": "Former punk Sakuragi Hanamichi, joins his high school basketball team attempting to impress his dream girl Haruko. Naturally athletic, yet lacking any experience in the sport, will Sakuragi. watch the anime dude!"}, {"m_title": "Two Car", "r": "5.3", "year": "2017", "votes": "37", "genre": "Action, Comedy", "desp": "Yuri Miyata and Megumi Meguro battle six other rival pairs in the sport of motorcycle sidecar racing."}, {"m_title": "Magical Girl Lyrical Nanoha ViVid", "r": "6.5", "year": "2015", "votes": "34", "genre": "Action", "desp": "Set four years after StrikerS. Nanohas adopted daughter Vivio is given the Sacred Heart device and gains the ability to transform into an adult. She then befriends another girl with the same ability."}, {"m_title": "Wake Up, Girls! New Chapter", "r": "8.0", "year": "2017", "votes": "6", "genre": "Comedy, Drama", "desp": "After overcoming countless challenges and finally emerging victorious over the nations idol group I-1 Club at the Idol Festival, Wake Up, Girls - under the management of tiny Sendai. watch the dude, no spoilers in summary!"}, {"m_title": "Shin Atashinchi", "r": "7.6", "year": "2015", "votes": "23", "genre": "Comedy", "desp": "Outrageous misadventures of an almost normal family with a housewife, her husband, and their two kids Yuusuke and Mikan. Wacky humor about this weird familys daily life."}, {"m_title": "Jewelpet Sunshine", "r": "8.4", "year": "2011", "votes": "11", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Tsukiuta", "r": "7.0", "year": "2016", "votes": "15", "genre": "Animation", "desp": "Six Gravity and Procellarum are two rival male idol groups who happen to live in the same dorm. Six Gravity is made up of young men who represent the months of December to May, while. watch the dude, no spoilers in summary!"}, {"m_title": "Jingai-san no Yome", "r": "5.2", "year": "2018", "votes": "90", "genre": "Animation", "desp": "Tomari Hinowa is a normal high school boy, until one day hes told that he has to become the wife of a mysterious creature called Kanenogi. This is the start of their newly married life."}, {"m_title": "The Human Crazy university", "r": "5.3", "year": "2022", "votes": "13", "genre": "Animation", "desp": "Satake Hirofumi is a prisoner on death row for murdering his fiancée. For this, he has earned the interest of a research institution called Human Crazy University."}, {"m_title": "Nyanpire: The Animation", "r": "6.4", "year": "2011", "votes": "54", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Tokusô kihei Dorvack", "r": "7.9", "year": "1983", "votes": "19", "genre": "Sci-Fi", "desp": "When aliens invade Earth, Triple Command, led by fearless Masato Mugen, is placed in charge of defense. Masato is aided by Frenchman Pierre Bonaparte and Japanese-American girl Loui Oberon."}, {"m_title": "Magic Users Club! TV", "r": "6.5", "year": "1999", "votes": "32", "genre": "Comedy, Romance", "desp": "The aliens have been defeated, and the Bell has been transformed into a gigantic cherry tree. Now, the brave Magic Users Club must reconvene to deal with a new problem. Apparently, some. watch the dude, no spoilers in summary!"}, {"m_title": "Wz", "r": "6.4", "year": "2019", "votes": "13", "genre": "Animation", "desp": "Yukiya, spends his time DJ-ing alone. One day, he sees a live broadcast from that world. Yukiya believes he cant do anything alone, but that he could accomplish something if he were doing it together with someone else."}, {"m_title": "Hidamari Sketch × Honeycomb", "r": "7.3", "year": "2012", "votes": "32", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Kiddy Girl-and", "r": "7.5", "year": "2009", "votes": "30", "genre": "Action, Adventure", "desp": "Ascoeur and Q-feuille work as waitresses while they train to become members of the galaxys shadow law enforcement agency, the ES Force."}, {"m_title": "Show by Rock!! Mashumairesh!!", "r": "8.4", "year": "2020", "votes": "56", "genre": "Comedy, Music", "desp": "Howan, an aspiring musician from the far North, travels to the colorful and musical Midi City where she finds her calling along and her best friends!."}, {"m_title": "Shuffle! Memories", "r": "5.9", "year": "2007", "votes": "38", "genre": "Comedy, Drama", "desp": "A recap of the Shuffle world, set ten years in the future. Gods, Demons, and Humans freely visit the other worlds as if traveling overseas, and romance is in the air. Demons, Gods, And. watch the dude, no spoilers in summary!"}, {"m_title": "Geisters: Fractions of the Earth", "r": "6.2", "year": "2001", "votes": "11", "genre": "Action, Drama", "desp": "The planet Earth has been turned into an apocalypse and humanity has been split in two different ways. The Dobias went to space and Shioru turned to the underground. Now centuries later,. watch the dude, no spoilers in summary!"}, {"m_title": "Dragon League", "r": "7.2", "year": "1993", "votes": "30", "genre": "Adventure, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Onsen yôsei Hakone-chan", "r": "5.4", "year": "2015", "votes": "30", "genre": "Animation", "desp": "After many years of dormant rest, a hot springs fairy awakens in modern-day Japan. However, while she slept, she took on the appearance of a young girl. She decides to cooperate with the locals whilst trying to regain her powers."}, {"m_title": "Genji Tsûshin Agedama", "r": "8.4", "year": "1991", "votes": "17", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Ramen Fighter Miki", "r": "8.2", "year": "2006", "votes": "50", "genre": "Action, Comedy", "desp": "The comical adventures of a ramen delivery girl who loves fighting so much she screws up her job."}, {"m_title": "Blessing of the Campanella", "r": "5.5", "year": "2010", "votes": "25", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Bottle Fairy", "r": "6.7", "year": "2003", "votes": "47", "genre": "Animation", "desp": "A TV show about 4 fairies: Kururu, Chiriri, Sarara and Hororo. They have come to the human world in small bottles. They wish to become human. A young man, Sensei-san, teaches them about the human world."}, {"m_title": "Kin iro no koruda: Primo passo", "r": "7.0", "year": "2006", "votes": "121", "genre": "Comedy, Drama", "desp": "Hino Kahoko is a student whose school is divided into two branches-the Normal Branch, in which students wear gray uniforms, and the Music Branch, in which the students wear white. The. watch the dude, no spoilers in summary!"}, {"m_title": "Mahoromatic: Im Home!", "r": "7.0", "year": "2009", "votes": "32", "genre": "Action, Comedy", "desp": "While Suguru and his classmates prepare a maid cafe to celebrate their towns anniversary the gynoid combat maids Minawa and Mahoro are attacked."}, {"m_title": "Canvas2: Niji iro no sukecchi", "r": "6.7", "year": "2005", "votes": "37", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Gôru FH", "r": "6.7", "year": "1994", "votes": "29", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Binchô tan", "r": "7.5", "year": "2006", "votes": "30", "genre": "Animation", "desp": "Bincho-tan, an orphan lives alone in the forest, she always has a piece of coal bonded on her head. Occasionally, she ventures into the town situated near the mountain, in search of work. watch the dude, no spoilers in summary!"}, {"m_title": "Mahou Shoujo? Naria Girls", "r": "1.4", "year": "2016", "votes": "10", "genre": "Comedy, Fantasy", "desp": "After the Ice Queen brings eternal winter to the land of Nariadia, the only hope to restore balance is to gather human warriors and give them the power of Naria crystals. For this reason, Animaru has chosen the middle schoolers warriors."}, {"m_title": "Butlers: Chitose Momotose Monogatari", "r": "5.0", "year": "2018", "votes": "7", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Magical Circle Guru Guru", "r": "7.7", "year": "1994", "votes": "93", "genre": "Comedy, Family", "desp": "A fantasy comedy about children destined to save the world from an ancient evil while constantly parodying aspects of console role playing games."}, {"m_title": "Tamayura: Hitotose", "r": "6.8", "year": "2011", "votes": "53", "genre": "Comedy, Drama", "desp": "As a little girl, Fu Sawataris father taught her to love photography. They took pictures everywhere they went. But after he passed away, seeing those photographs only served as a reminder. watch the dude, no spoilers in summary!"}, {"m_title": "Panzer World Galient", "r": "7.8", "year": "1984", "votes": "19", "genre": "Sci-Fi", "desp": "In the medieval world of Arst, Prince Jordy Volder and his allies must use the might of a powerful giant robot, the ancient Panzer Galient, to liberate their world from a tyrant."}, {"m_title": "Sci-Fi Harry", "r": "5.8", "year": "2000", "votes": "28", "genre": "Drama, Horror", "desp": "A bullied shy awkward teenage slowly gains mysterious psychic powers leading him into a dark conspiracy involving a clandestine organization with lethal and frightening consequences for all."}, {"m_title": "Gad Guard", "r": "7.3", "year": "2003", "votes": "76", "genre": "Sci-Fi", "desp": "Hajikis struggling to make ends meet on the hard-luck streets of Night Town, where if youve got a Gad, youve got the power of a robotic Techode at your disposal, and youve got a chance.. watch the anime dude!"}, {"m_title": "Genma taisen - Shinwa zenya no shou", "r": "3.3", "year": "2002", "votes": "51", "genre": "Adventure, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Wixoss Diva(A)Live", "r": "5.9", "year": "2021", "votes": "18", "genre": "Sci-Fi", "desp": "Wixoss Diva(A)Live is an original anime series, the fifth anime series in the WIXOSS franchise. It is scheduled to begin airing in January 2021."}, {"m_title": "Kimi no Iru Machi", "r": "5.9", "year": "2013", "votes": "13", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Gokujô!! Mecha mote iinchô", "r": "7.1", "year": "2009", "votes": "11", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Folktales from Japan", "r": "7.4", "year": "2012", "votes": "34", "genre": "Animation", "desp": "Anime television series that adapts various traditional stories from Japan."}, {"m_title": "801 T.T.S. Airbats", "r": "6.3", "year": "1994", "votes": "111", "genre": "Short, Action", "desp": "Takuya Isurugi who is an aviation mechanic and Anime fan, gets transferred to the 801st Tactical Training Squadro but gets more than he thinks where he meets an all female aerial acrobatic team."}, {"m_title": "Himote House", "r": "7.5", "year": "2018", "votes": "84", "genre": "Comedy, Fantasy", "desp": "From Kotaro Ishidate, creator of some popular ad-lib animes. The new anime centers on five girls and one cat all living as housemates in Nakano: the three Himote sisters (Tokiyo, Kinami,. watch the dude, no spoilers in summary!"}, {"m_title": "Papuwa", "r": "7.5", "year": "2003", "votes": "13", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Night Wizard the Animation", "r": "5.9", "year": "2007", "votes": "25", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Panda Zetto: The Robonimation", "r": "6.8", "year": "2004", "votes": "33", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Neo anjerîku: Abisu", "r": "6.6", "year": "2008", "votes": "72", "genre": "Adventure, Comedy", "desp": "A young girl is chosen to be savior of the universe, who, alongside her fellow companions, must unite their power to save the queen and the cosmos."}, {"m_title": "Senki Zessho Symphogear GX", "r": "7.3", "year": "2015", "votes": "66", "genre": "Action, Music", "desp": "With their new friends Shirabe, Kirika, and Maria, Hibiki, Tsubasa, and Chris just work to save the world under the organization S.O.N.G. However, even though the Noise no longer are a threat to the world, a new villain emerges."}, {"m_title": "Ninja Collection", "r": "5.5", "year": "2020", "votes": "45", "genre": "Fantasy, Horror", "desp": "Set in modern-day Tokyo where dreams and desires are afloat, a group of ninjas called Toukeshuu is set to stop the spread of darkness that prevails in the world using ancient techniques passed down through generations"}, {"m_title": "Seiyus Life!", "r": "6.2", "year": "2015", "votes": "27", "genre": "Comedy", "desp": "Based on the Dōjinshi (self-published works) created by accomplished voice actress Masumi Asano and Hayate the Combat Butler creator Kenjiro Hata, Seiyus Life! gives a true insiders. watch the anime dude!"}, {"m_title": "Mahôjin guru guru", "r": "7.5", "year": "2017", "votes": "49", "genre": "Action, Adventure", "desp": "The seal that for hundreds of years had confined the demon king Giri has worn off. A young boy named Nike is chosen (against his will) as the hero. Together with Kukuri, a girl who is the. watch the dude, no spoilers in summary!"}, {"m_title": "Human Crossing", "r": "7.8", "year": "2003", "votes": "18", "genre": "Drama", "desp": "Though seemingly unconnected, the characters in HUMAN CROSSING all have something in common. They are dealing with the sometimes joyous, but often horrific realities of everyday life. Set. watch the dude, no spoilers in summary!"}, {"m_title": "Maesetsu!", "r": "6.1", "year": "2020", "votes": "19", "genre": "Comedy", "desp": "Four girls work hard to become great comedians."}, {"m_title": "Human Bug Daigaku", "r": "6.4", "year": "2022", "votes": "10", "genre": "Animation", "desp": "Follows Satake Hirofumi, a prisoner on death row for killing his fiancée and an Undeadman who survived a lot desperate experiences. He has gained the interest of a research institution and soon, the truth enlightens a bigger conspiracy."}, {"m_title": "Hidamari Sketch x 365", "r": "8.0", "year": "2008", "votes": "94", "genre": "Comedy, Drama", "desp": "After passing her entrance exam, Yuno enrolls at Yamabuki High School as part of an art degree. She stays at the Hidamari Apartments along with best friend Miyako, and seniors Hiro and Sae."}, {"m_title": "Gambler Densetsu Tetsuya", "r": "6.9", "year": "2000", "votes": "19", "genre": "Mystery", "desp": "In the year 1947, the people of Shinjuku are down on their luck. With little money to buy food or necessities, some resort to gambling in order to survive. Traveling Tetsuya chooses to. watch the dude, no spoilers in summary!"}, {"m_title": "Natsunagu!", "r": "5.3", "year": "2020", "votes": "44", "genre": "Adventure, Drama", "desp": "A young girl visits Kumamoto Prefecture of Japan in the aftermath of its 2016 earthquakes. Despite the tragedy, people there are optimistic and inspiring."}, {"m_title": "Meiken rasshî", "r": "7.6", "year": "1996", "votes": "14", "genre": "Drama, Family", "desp": "A Japanese animated version of the famous American TV series about a family of farmers and their dog, Lassie. In each episode, someone in the family manages to get into some kind of trouble. watch the anime dude!"}, {"m_title": "Nurse Witch Komugi", "r": "6.7", "year": "2002", "votes": "26", "genre": "Short, Comedy", "desp": "Maya the Goddess of Vaccine World has to send Mugimaru to Earth to find someone who will accept the Vaccine World and turn into Magical Nurse. He finds Komugi Nakahara to fight Ungrar."}, {"m_title": "Somedays Dreamers II: Sora", "r": "7.0", "year": "2008", "votes": "41", "genre": "Drama, Romance", "desp": "Get ready for a second magical journey to the world of Somedays Dreamers, where spell-casting is a profession that requires both the proper training AND a license. Its to get that license. watch the anime dude!"}, {"m_title": "IDOLM@STER: Xenoglossia", "r": "4.9", "year": "2007", "votes": "57", "genre": "Action, Comedy", "desp": "The girls of the Idolmaster series are re-imagined as the pilots of giant robots defending the Earth."}, {"m_title": "Jûsenki Erugaimu", "r": "6.0", "year": "1984", "votes": "78", "genre": "Action, Sci-Fi", "desp": "Daba Myroad, the lone survivor of the Yaman Clan, enlists the help of his friends, and the giant mecha L-Gaim, to retake the Pentagona star system from the immortal ruler Oldna Poseidal."}, {"m_title": "Round Vernian Vifam", "r": "7.0", "year": "1983", "votes": "36", "genre": "Action, Adventure", "desp": "A ragtag bunch of children and teenagers must work together, using a squadron of giant robots and a training ship, to travel back to Earth after their colony is wiped out in an alien attack."}, {"m_title": "Comic Party: Revolution", "r": "6.6", "year": "2005", "votes": "42", "genre": "Comedy, Drama", "desp": "Comic Party Revolution is the sequel to Comic Party (2001).Kazuki, Taishi, and all their crazy friends are back and getting into all kinds of misadventures. Manga and madness abound as these college students navigate otaku subculture."}, {"m_title": "Dream Fighter Wingman", "r": "7.1", "year": "1984", "votes": "26", "genre": "Animation", "desp": "A big superhero fan, Kenta created his own character: Wingman. However, his pranks tend to irritate his teachers as well as his junior high school sweetheart, the lovely Miku. The. watch the dude, no spoilers in summary!"}, {"m_title": "Gingitsune", "r": "7.4", "year": "2013", "votes": "106", "genre": "Comedy, Drama", "desp": "Makoto is the fifteenth generation heiress of a small shrine to the god Inari. Her father is still the shrines priest, but Inaris divine messenger, Gintarou, has appeared to Makoto,. watch the dude, no spoilers in summary!"}, {"m_title": "HisCoool! Seha Girls", "r": "6.2", "year": "2014", "votes": "31", "genre": "Comedy", "desp": "The project focuses on various anthropomorphism Sega consoles, known as Sega Hard Girls or SeHa Girls for short, each with their own unique personalities. The anime series follows three. watch the anime dude!"}, {"m_title": "The Miyakawa Familys Hunger", "r": "6.1", "year": "2013", "votes": "57", "genre": "Short, Comedy", "desp": "A spin-off of Lucky Star. Centers around big sister Hinata Miyakawa and little sister Hikage Miyakawas impoverished daily life at home (due to Hinatas wasteful habits) and Hikages life at elementary school."}, {"m_title": "Wagamama High Spec", "r": "4.0", "year": "2016", "votes": "37", "genre": "Comedy, Drama", "desp": "Even though hes just a student, the protagonist in Wagamama High Spec also happens to be the author of a manga published in a weekly magazine. Since its mainly an ecchi and romcom type of. watch the anime dude!"}, {"m_title": "Tsukumogami Kashimasu", "r": "7.0", "year": "2018", "votes": "29", "genre": "Fantasy", "desp": "The lives of two siblings who trade in tsukumogami, objects that gain sentience after living a hundred years."}, {"m_title": "Lime-iro Senkitan", "r": "5.8", "year": "2003", "votes": "25", "genre": "Romance", "desp": "sorry no summary not available!"}, {"m_title": "Rumiko Takahashi Anthology", "r": "7.4", "year": "2003", "votes": "30", "genre": "Drama", "desp": "Animated adaption of an anthology of 13 short manga stories created by Rumiko Takahashi."}, {"m_title": "Symphogear G", "r": "7.4", "year": "2013", "votes": "82", "genre": "Action, Adventure", "desp": "After the events of Senki Zessho Symphogear, the incoming threat of the fall of the moon gets Tachibana HIbiki, Kazanari Tsubasa, and Yukine Chris on another mission to save the world once again."}, {"m_title": "Misutâ ajikko", "r": "7.0", "year": "1987", "votes": "59", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Futakoi", "r": "5.9", "year": "2004", "votes": "47", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "At the Mercy of the Sky", "r": "6.9", "year": "2009", "votes": "67", "genre": "Comedy, Drama", "desp": "Studio Comet brings us this visually striking anime about a bookworm-ish high school student who returns to his hometown and joins the local astronomy club. Saku Ooyagi is back, and both he. watch the anime dude!"}, {"m_title": "Aria the Natural", "r": "8.1", "year": "2006", "votes": "248", "genre": "Comedy, Drama", "desp": "In the continuation of the last series, Akari Mizunashi continues her job as a gondolier along with her friends from the other two rival companies, Aika and Alice. As they continue their. watch the dude, no spoilers in summary!"}, {"m_title": "Kyatto Ninden Teyandee", "r": "7.2", "year": "1990", "votes": "29", "genre": "Action, Adventure", "desp": "The series is set in Little Tokyo, a mechanical city which fuses feudal Japanese culture with contemporary culture, and is populated by cybernetic anthropomorphic animals. The city is. watch the dude, no spoilers in summary!"}, {"m_title": "Mirakuru gâruzu", "r": "7.4", "year": "1993", "votes": "56", "genre": "Drama, Family", "desp": "sorry no summary not available!"}, {"m_title": "Hakuouki", "r": "6.5", "year": "2021", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Arad Senki: Slap Up Party", "r": "6.3", "year": "2009", "votes": "11", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Shainingu tiâzu kurosu windo", "r": "5.6", "year": "2007", "votes": "58", "genre": "Adventure, Fantasy", "desp": "Gates develop between two worlds, causing residents to be displace and fight monsters."}, {"m_title": "Gekidol", "r": "5.7", "year": "2021", "votes": "42", "genre": "Drama, Music", "desp": "A group of girls fascinated by the Theatrical Material System pursue their dreams of being on stage after the disappearance of an entire city. How will their fates entwine?"}, {"m_title": "Sky Girls", "r": "6.8", "year": "2007", "votes": "39", "genre": "Action, Sci-Fi", "desp": "In the future Earth is at war with an alien race which has decimated most of the population, the young girls of the Sonic Drivers mecha unit fight against the threat."}, {"m_title": "Papillon Rose", "r": "4.8", "year": "2006", "votes": "22", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Strike Witches: 501 Butai Hasshin Shimasu!", "r": "5.5", "year": "2019", "votes": "29", "genre": "Animation", "desp": "As newcomer Yoshika gradually discovers, however, lots of shenanigans go on at the base during downtime, and this is an account of such nonsense."}, {"m_title": "Innocent Venus", "r": "5.7", "year": "2006", "votes": "67", "genre": "Adventure, Drama", "desp": "The year is 2010 AD and Hyper Hurricanes are being born all over the world have cause very bad damage. Five billion people have died which has decreased the population to only 3 billion.. watch the dude, no spoilers in summary!"}, {"m_title": "Kidou Senshi Gundam AGE: Memory of Eden", "r": "5.6", "year": "2013", "votes": "18", "genre": "Action, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Poyopoyo Kansatsu Nikki", "r": "8.8", "year": "2012", "votes": "7", "genre": "Short, Comedy", "desp": "The anime adaptation of Ru Tatsukis manga of the same name centers around a mysterious round cat Poyo and people in his life."}, {"m_title": "Nekojiru gekijô - jirujiru Original", "r": "6.9", "year": "1999", "votes": "59", "genre": "Animation", "desp": "A story of Nyaka and Nyatta based on the comics by the late Nekojiru. Contains 23 animated shorts aired TV Asahi Networks show Bokusho Mondai Boss Chara Ou. and 4 unaired episodes."}, {"m_title": "Zenonzard: The Animation", "r": "7.8", "year": "2019", "votes": "25", "genre": "Animation", "desp": "The year 20XX. Humans called Concodes and ultra-high-performance AI called Codeman team up as buddies to compete in the card battle game Zenozard. People are going wild with excitement at the world championships, The Zenon."}, {"m_title": "G-on Riders", "r": "6.6", "year": "2002", "votes": "19", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Saiyuki Reload Gunlock", "r": "7.5", "year": "2004", "votes": "36", "genre": "Action, Adventure", "desp": "Genjo Sanzo, Cho Hakkai, Sha Gojyo, and Son Goku fight their way to their goal, their path is fraught with internal strife. When they encounter a formidable pair of adversaries from the west."}, {"m_title": "Taisho Baseball Girls", "r": "7.0", "year": "2009", "votes": "56", "genre": "Comedy, Drama", "desp": "In Taisho period Japan where women are seen as only good for housework, nine students at an all-girls high school learn how to play baseball to prove that they can compete with the boys."}, {"m_title": "Tsuyokiss: Cool×Sweet", "r": "5.9", "year": "2006", "votes": "41", "genre": "Comedy, Romance", "desp": "Sunao Konoe starts her first day at a new school and is surprised to find that there is no drama club. So, she decides to make one of her own. But between a childhood friend showing up, the. watch the anime dude!"}, {"m_title": "Transformers: Mystery of Convoy", "r": "6.1", "year": "2015", "votes": "11", "genre": "Animation", "desp": "A series of comedic meetings centered on discussing Transformers and its three-decade history."}, {"m_title": "Chocotto Sister", "r": "6.4", "year": "2006", "votes": "32", "genre": "Comedy, Drama", "desp": "After his mother suffers a miscarriage a boy makes a Christmas wish for a little sister. Many years later a woman claiming to be Santa Claus delivers on the wish giving him a young sister."}, {"m_title": "Argonavis from Bang Dream! Animation", "r": "6.6", "year": "2020", "votes": "17", "genre": "Drama, Music", "desp": "Ren Nanahoshi is a lonely college student who isnt good at communicating with others. He remembers the thrill of watching a live band perform when he was young, and spent most of his days. watch the dude, no spoilers in summary!"}, {"m_title": "Gargoyle of Yoshinaga House", "r": "7.9", "year": "2006", "votes": "20", "genre": "Comedy, Fantasy", "desp": "When Yoshinaga Futaba wins the first prize in a lottery, the prize turns out to be a stone gargoyle. More surprising yet, the gargoyle turns out to be alive."}, {"m_title": "Teppen-!!!!!!!!!!!!!!!", "r": "5.7", "year": "2022", "votes": "33", "genre": "Comedy", "desp": "Yayoi Sakamoto, a die-hard fan of comedians and comedy acts, reunites with a childhood friend. They put together a routine at a park, like they did before, in order to enter a local shopping areas contest."}, {"m_title": "Itsuka Tenma no Kuro Usagi", "r": "6.6", "year": "2011", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Nanaka 6/17", "r": "6.5", "year": "2003", "votes": "27", "genre": "Comedy, Romance", "desp": "The story focuses on a cold and distant 17-year-old girl, Nanaka Kirisato, who only cares about studying to get good grades in school and being accepted into a good college. Nenji, her. watch the dude, no spoilers in summary!"}, {"m_title": "Etotama", "r": "7.1", "year": "2015", "votes": "48", "genre": "Action, Comedy", "desp": "The animes story revolves around Nya-tan, the cat of the Chinese astrology who desperately wants to become a member of the Chinese zodiac. Nya-tan is simple, and she tries various foolish. watch the dude, no spoilers in summary!"}, {"m_title": "Meganebu!", "r": "5.6", "year": "2013", "votes": "27", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Kyoro-chan", "r": "7.9", "year": "1999", "votes": "34", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Wandaba Style", "r": "6.2", "year": "2003", "votes": "17", "genre": "Comedy, Sci-Fi", "desp": "Michael Hanagata, manager of Mix Juice, an Idol group trying to make it big, strikes a deal with Teen Genius Susumu Tsukumo. Together they plan to land on the moon with eco-friendly experiments, using Mix Juice as his pilots."}, {"m_title": "Wind: A Breath of Heart", "r": "5.1", "year": "2004", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Amatsuki", "r": "6.3", "year": "2008", "votes": "122", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Rilu Rilu Fairilu: Yosei no Door", "r": "5.7", "year": "2016", "votes": "8", "genre": "Family, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Hapinesu!", "r": "5.7", "year": "2006", "votes": "37", "genre": "Comedy, Romance", "desp": "There is a school for regular students and another for students who can use magic. There will be a problem in the magic school and all the students will go to the normal students school."}, {"m_title": "Doki Doki School Hours", "r": "6.8", "year": "2004", "votes": "38", "genre": "Comedy", "desp": "A very short 27 year old teacher has an interesting time trying to teach her class. She has to deal with the eccentricities of her students as they present themselves over the school year.. watch the dude, no spoilers in summary!"}, {"m_title": "Im Home! Chibi Godzilla", "r": "7.0", "year": "2020", "votes": "14", "genre": "Animation", "desp": "Chibi Godzilla lives with a woman named Satomi."}, {"m_title": "Muka Muka Paradise", "r": "6.6", "year": "1993", "votes": "95", "genre": "Adventure, Comedy", "desp": "Uiba Shikatani is the daughter of a pet shop owner but gets sad when she finds out some reptile eggs dont hatch. Too try to make her happy her father brings her a giant striped egg which. watch the dude, no spoilers in summary!"}, {"m_title": "Android Ana Maico 2010", "r": "6.5", "year": "1998", "votes": "16", "genre": "Comedy, Sci-Fi", "desp": "A radio station attempts to boost its ratings by hiring an android radio host."}, {"m_title": "My Wife Is a Magical Girl: Bewitched Agnes", "r": "6.8", "year": "2005", "votes": "8", "genre": "Comedy, Fantasy", "desp": "Ureshiko Asaba is a twenty-something housewife that few people know is also the magical girl named Agnes Bell."}, {"m_title": "Nanatsuiro Drops", "r": "6.4", "year": "2007", "votes": "26", "genre": "Romance", "desp": "Tsuwabuki is a normal student, though not very social. One day he meets a new transfer student, named Sumomo Akihime, and another girl, both the only members of the gardening club.. watch the dude, no spoilers in summary!"}, {"m_title": "Meine Liebe", "r": "6.0", "year": "2004", "votes": "24", "genre": "Fantasy", "desp": "In Kuchen, an island off the edge of Europe, there exists an elite boarding school: The Rosenstolz Academy. Meine Liebe explores the lives of five students who are Strahl (or scholars who aid in protecting the kingdom) candidates."}, {"m_title": "Armored Trooper Votoms: Brilliantly Shining Heresy", "r": "7.1", "year": "1994", "votes": "13", "genre": "Action, Drama", "desp": "Chirico awakens from a cold sleep and gets separated from Fyana. He tries to find her, but also finds the new Pope is being nominated and Chirico considered to be The Untouchable, one of. watch the dude, no spoilers in summary!"}, {"m_title": "Puchimas!: Petit Idolm@ster", "r": "5.0", "year": "2013", "votes": "43", "genre": "Animation", "desp": "Hyperactive hijinks are sure to follow when the idol girls of 765 Productions discover chibis with adorable abilities."}, {"m_title": "Panyo Panyo Di Gi Charat", "r": "7.5", "year": "2002", "votes": "16", "genre": "Comedy", "desp": "Princess Dejiko goes around helping people with the goal to make everyone happy while trying to hide shes a princess from her friends."}, {"m_title": "Miss Monochrome", "r": "6.1", "year": "2013", "votes": "69", "genre": "Comedy, Music", "desp": "An android, Miss Monochrome, goes on a quest to become Japans newest pop idol."}, {"m_title": "Starry Sky", "r": "5.3", "year": "2008", "votes": "7", "genre": "Romance", "desp": "sorry no summary not available!"}, {"m_title": "Rejendo obu duo", "r": "4.2", "year": "2004", "votes": "53", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Violinist of Hameln", "r": "7.6", "year": "1996", "votes": "20", "genre": "Adventure, Drama", "desp": "Hameln, a musician that plays a chello like magical violin is the friend of a girl called Flute. They live in a village called Stacato which is a peacefull village. At the time of the next. watch the dude, no spoilers in summary!"}, {"m_title": "GJ-bu", "r": "6.8", "year": "2013", "votes": "99", "genre": "Drama, Romance", "desp": "Shinomiya Kyouya is forced to become a new member of the GJ, an unidentified club that duels in a room of the former building of a certain school. Here he meets the club leader, Mao, a. watch the dude, no spoilers in summary!"}, {"m_title": "Clockwork Fighters Hiwous War", "r": "6.6", "year": "2000", "votes": "18", "genre": "Action, Adventure", "desp": "During the 19th Century in Japan, clockwork dolls are very common for festivals and entertainment. But when the Western world comes to introduce its invention of steam engines, clockwork dolls will be more than handcrafted machines."}, {"m_title": "Kamen No Maid Guy", "r": "6.5", "year": "2008", "votes": "68", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Bucky: The Incredible Kid", "r": "7.1", "year": "1999", "votes": "34", "genre": "Adventure, Comedy", "desp": "Bucky, the protagonist of the history, is a normal boy that lives in the first world. He lives with a single and humble ambition: to dominate the world (in the sense of the whole planet).. watch the dude, no spoilers in summary!"}, {"m_title": "Hidamari Sketch X Hoshimittsu", "r": "6.4", "year": "2010", "votes": "80", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Yobarete tobidete! Akubi-chan", "r": "7.7", "year": "2001", "votes": "12", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Kagihime monogatari: Eikyû arisu rondo", "r": "6.1", "year": "2006", "votes": "14", "genre": "Action, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Kokoro Library", "r": "6.4", "year": "2001", "votes": "6", "genre": "Comedy, Drama", "desp": "Deep in the mountains lies a library with the same name as one of its caretakers, Kokoro. With her sisters Inna and Aruto, she strives to uphold the best characteristic of a librarian -- to. watch the anime dude!"}, {"m_title": "Dash! Yonkuro", "r": "7.0", "year": "1989", "votes": "21", "genre": "Action, Sport", "desp": "sorry no summary not available!"}, {"m_title": "Kogepan", "r": "6.9", "year": "2001", "votes": "42", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Lovedol: Lovely Idol", "r": "5.1", "year": "2006", "votes": "12", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Yakushiji Ryouko no Kaiki Jikenbo", "r": "7.2", "year": "2008", "votes": "17", "genre": "Animation", "desp": "The story revolves around Ryouko Yakushiji, a 27-year-old multilingual investigator that graduated from Tokyo Universitys elite Department of Law, and the bizarre cases she faces while. watch the dude, no spoilers in summary!"}, {"m_title": "My-Otome 0: S.ifr", "r": "7.1", "year": "2008", "votes": "52", "genre": "Fantasy, Sci-Fi", "desp": "This is the prequel of My-Otome, which predates the events before the My-Otome Series. In this series, it focuses upon the origins of Lena Sayers, the mother of Arika Yumemiya and former. watch the dude, no spoilers in summary!"}, {"m_title": "Hareluya II BØY", "r": "6.3", "year": "1997", "votes": "13", "genre": "Action, Comedy", "desp": "Hibino Hareluya is a delinquent teenager who is tough and loves to pack a punch. His father is none other than the almighty God who does not approve very well of his sons rebellious acts .. watch the anime dude!"}, {"m_title": "King of the Braves GaoGaiGar Final: Grand Glorious Gathering", "r": "7.3", "year": "2005", "votes": "9", "genre": "Action, Sci-Fi", "desp": "Following Gutsy Galaxy Guards victory over the Zonder Empire and the 31 Primevals, a new threat makes their appearance on Earth. GGG - with Guy Shishio and the newly-constructed GaoFighGar. watch the anime dude!"}, {"m_title": "To Heart: Remember My Memories", "r": "4.9", "year": "2004", "votes": "13", "genre": "Comedy, Drama", "desp": "Time flies by; Hiroyuki and Akari are 12th-graders now. Both of them and their friends have grown up gradually in someway, physically or mentally, but one of their best friends is missing:. watch the dude, no spoilers in summary!"}, {"m_title": "Pastel Memories", "r": "4.5", "year": "2019", "votes": "20", "genre": "Adventure", "desp": "In a future Akihabara where otaku culture has declined. Players play as a shopkeeper of an otaku shop. Together with the heroines who are in fact otaku girls, the shopkeeper start helping people claim their lost otaku memories back."}, {"m_title": "Super Robot Wars: Original Generation: The Animation", "r": "5.7", "year": "2005", "votes": "14", "genre": "Action, Adventure", "desp": "Dr. Jurgen of DC developed a global defense system to combat potential alien threats. This system has two elements. First is the VTX-001 Vartoul unmanned anti alien PT. Governing these. watch the dude, no spoilers in summary!"}, {"m_title": "Fafner Exodus", "r": "7.2", "year": "2015", "votes": "37", "genre": "Action, Drama", "desp": "New Fafner team under Tatsumiya Island and Perseus Company led by General Narain have to take two girls Miwa and Emery as communicators to a place where the girls can contact with a new Mir. watch the anime dude!"}, {"m_title": "Chance Pop Session", "r": "6.6", "year": "2001", "votes": "14", "genre": "Comedy, Music", "desp": "Three girls strive to become pop idols."}, {"m_title": "Shanguri ra", "r": "6.2", "year": "2009", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Nana the Supergirl", "r": "6.3", "year": "1983", "votes": "39", "genre": "Comedy, Family", "desp": "sorry no summary not available!"}, {"m_title": "Monokurômu fakutâ", "r": "6.7", "year": "2008", "votes": "65", "genre": "Action, Mystery", "desp": "The story revolves around the high school student Akira Nikaido, a typical slacker living a normal life. That is, until he meets the mysterious Shirogane, a man who suddenly appears and tells him that they have a destiny together."}, {"m_title": "Gurazeni", "r": "6.9", "year": "2018", "votes": "16", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Futakoi Alternative", "r": "6.3", "year": "2005", "votes": "50", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Nangoku shônen Papuwa-kun", "r": "6.5", "year": "1992", "votes": "22", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Kamisama kazoku", "r": "6.6", "year": "2006", "votes": "55", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Tsuki wa higashi ni hi wa nishi ni: Operation Sanctuary", "r": "5.2", "year": "2004", "votes": "19", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Fushigi boshi no futago hime: Gyu!", "r": "8.1", "year": "2006", "votes": "26", "genre": "Action, Adventure", "desp": "Second season of twin princess of wonder planet"}, {"m_title": "Mikan Enikki", "r": "7.6", "year": "1992", "votes": "14", "genre": "Comedy, Romance", "desp": "Mikan Enikki features an cat whose name is Mikan, in Japanese means tangerine and color orange. He can talk the human language, read, write and do several tasks like a human. The story. watch the dude, no spoilers in summary!"}, {"m_title": "Marginal #4 Kiss Kara Tsukuru Big Bang", "r": "5.1", "year": "2017", "votes": "12", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Action Heroine Cheer Fruits", "r": "6.8", "year": "2017", "votes": "15", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Xabungle", "r": "7.3", "year": "1982", "votes": "44", "genre": "Sci-Fi", "desp": "Jiron Amos, a member of the oppressed Civilian sect on the planet Zora, joins a group of rebels called the Sandrats as they use the stolen giant robot Xabungle to fight their oppressors, the elite Innocents."}, {"m_title": "Dennou Boukenki Webdiver", "r": "7.0", "year": "2001", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Miracle Giants Dome-kun", "r": "7.7", "year": "1989", "votes": "49", "genre": "Comedy, Drama", "desp": "A boy named Dome (Giants Stadium) inherited from his father the love for baseball and his great sense for the sport, his father, a professional player, before dying, had time to teach him a. watch the anime dude!"}, {"m_title": "Gallery Fake", "r": "7.9", "year": "2005", "votes": "28", "genre": "Action, Mystery", "desp": "Fujita Reiji is the owner of an art gallery, Gallery Fake. He deals with fake items, and is also familiar with the black market. However, he has a special talent for art, and used to work. watch the dude, no spoilers in summary!"}, {"m_title": "Magical Kanan", "r": "7.0", "year": "2005", "votes": "7", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Yûsha Exkaiser", "r": "6.7", "year": "1990", "votes": "21", "genre": "Action, Sci-Fi", "desp": "When aliens plan to invade earth, the young Kouta along with a giant robot from outspace named Exkaiser must team up to save the world and his friends."}, {"m_title": "Dojin Work", "r": "6.9", "year": "2007", "votes": "51", "genre": "Comedy", "desp": "Dojin Work follows a girl named Najimi Osana and her exposure to the amateur manga world."}, {"m_title": "Legend of Basara", "r": "7.2", "year": "1998", "votes": "17", "genre": "Adventure, Drama", "desp": "In small village, twins are born: boy, Tatara and girl, Sarasa. The prophet Nagi declares that Tatara is chosen one to free Japan from the until ruthless Red King slays Tatara, Sarasa took her brothers name to embark on a quests."}, {"m_title": "Pokonyan", "r": "5.8", "year": "1993", "votes": "99", "genre": "Animation", "desp": "Rocky Rackat (Pokonyan) is a funny little creature thats a bit of a cross between a cat and a raccoon.Amy discovers Rocky in her backpack among her camping gear.Rocky uses his powers to. watch the dude, no spoilers in summary!"}, {"m_title": "Kisudamu", "r": "6.5", "year": "2007", "votes": "24", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Katsudou dai shashin: Mankatsu", "r": "7.0", "year": "2004", "votes": "14", "genre": "Action, Comedy", "desp": "A collection of Monkey Punch related stories, including Lupin the 3rd and other previous works."}, {"m_title": "Polyphonica Crimson S", "r": "7.0", "year": "2009", "votes": "19", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Venger Robo", "r": "7.2", "year": "1991", "votes": "28", "genre": "Action, Adventure", "desp": "It has been 15 years since the fateful battle with The Empire of 100 Demons. Humanity is under attack by the scientist Dr. Lando with their mechanical monsters empire. Getter Team has begun. watch the anime dude!"}, {"m_title": "Bad Boys", "r": "6.0", "year": "1993", "votes": "6", "genre": "Short", "desp": "sorry no summary not available!"}, {"m_title": "Starship Girl Yamamoto Yohko", "r": "6.2", "year": "1999", "votes": "13", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Lance N Masques", "r": "5.0", "year": "2015", "votes": "36", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Bakumatsu Rock", "r": "5.9", "year": "2014", "votes": "26", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Granbelm", "r": "6.5", "year": "2019", "votes": "62", "genre": "Animation", "desp": "The anime takes place in a world that long ago featured the existence of magic, but has long since lost that ability. The story begins when the very normal high school student Mangetsu. watch the dude, no spoilers in summary!"}, {"m_title": "I-Chu: Halfway Through the Idol", "r": "6.6", "year": "2021", "votes": "25", "genre": "Animation", "desp": "Youths aiming to become full-fledged idols enroll in the Étoile Vio School where the students are called IChu. But soon after they begin, Kuma Kocho gets them off to a rocky start with a single phrase."}, {"m_title": "Gdgd Fairies", "r": "7.0", "year": "2011", "votes": "11", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "W Wish", "r": "3.5", "year": "2004", "votes": "26", "genre": "Drama, Fantasy", "desp": "Juuna Tono is a boy without a past. Orphaned by a tragic accident that claimed the lives of his parents and his memory, Juuna and his younger twin sister Senna have only survived by their. watch the dude, no spoilers in summary!"}, {"m_title": "Shugokyara!! Doki", "r": "7.4", "year": "2008", "votes": "45", "genre": "Animation", "desp": "An elementary school girl has created a mask to hide her true self at school but one night when she wishes to be able to express her true self three guardian eggs appear."}, {"m_title": "Heisei Inu Monogatari Bow", "r": "7.0", "year": "1993", "votes": "10", "genre": "Adventure, Comedy", "desp": "Bow is a homeless dog, who enjoys his carefree life. His dumb innocence often leads to disaster. But somehow people are charmed by him. School children, a lonesome cartoonist, and a gangster family all play an important role in Bows life."}, {"m_title": "Moeru! Onisan", "r": "7.3", "year": "1988", "votes": "13", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Pan De Peace!", "r": "5.3", "year": "2016", "votes": "55", "genre": "Animation", "desp": "Highschool friends get together to eat, talk and get excited for different kinds of bread."}, {"m_title": "Potemayo", "r": "7.9", "year": "2007", "votes": "37", "genre": "Comedy, Drama", "desp": "Moriyama finds a cute creature that he names Potemayo. Potemayo starts following him everywhere often riding on top of his head."}, {"m_title": "Sasami mahô shôjo kurabu", "r": "5.8", "year": "2006", "votes": "43", "genre": "Fantasy", "desp": "Under the guise of the Cooking Club, Sasami and her friends will learn to harness their magical abilities!"}, {"m_title": "Ôedo roketto", "r": "7.5", "year": "2007", "votes": "43", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Fantasista Doll", "r": "5.1", "year": "2013", "votes": "28", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Lostorage incited Wixoss", "r": "5.9", "year": "2016", "votes": "45", "genre": "Drama, Fantasy", "desp": "High school student Homura Suzuko has returned to Ikebukuro, the town where she grew up. Were friends forever! She treasures her memories of her old friend from when she was little,. watch the dude, no spoilers in summary!"}, {"m_title": "Pugyuru", "r": "4.9", "year": "2004", "votes": "10", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Bakumatsu Gijinden Roman", "r": "5.0", "year": "2013", "votes": "26", "genre": "Fantasy, History", "desp": "The story takes place in Kyoto in the late 19th century. Manjiro has two faces-during the day, he works as a helper for people in Kyoto, but he also acts in secret to return peoples property that has been stolen from corrupt men in power."}, {"m_title": "Sora o miageru shôjo no hitomi ni utsuru sekai", "r": "5.9", "year": "2009", "votes": "29", "genre": "Action, Fantasy", "desp": "To save both the Magical Kingdom and the Heavens and restore the flow of akuto, the flow of energy of everything, the Magical King Munto must follow a vision and find the girl Yumemi in the. watch the anime dude!"}, {"m_title": "Cinderella Girls Gekijou", "r": "8.0", "year": "2017", "votes": "7", "genre": "Short, Comedy", "desp": "Our idol friends are back again today. Get a rare look at how idols spend their days off, as well as their comical interactions with each other in this hyper-fun-filled short anime."}, {"m_title": "Sisters of Wellber", "r": "5.9", "year": "2007", "votes": "42", "genre": "Adventure, Comedy", "desp": "Princess Rita stabbed her groom to be and ran away. Infuriated King Ranbahnhof threatens to wage war unless Rita is captured and publicly executed."}, {"m_title": "Obotchamakun", "r": "8.2", "year": "1989", "votes": "26", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Psychic Academy", "r": "6.6", "year": "2002", "votes": "27", "genre": "Comedy, Romance", "desp": "Ai Shiomi has just joined a elite academy for gifted students that have psychic powers like his older brother before him. He also has a choice: Choose a girl from his past or his new soulmate?"}, {"m_title": "Kyomu Senshi Miroku", "r": "6.6", "year": "1989", "votes": "21", "genre": "Fantasy, Horror", "desp": "A psychic ninja fantasy suggesting that the assassins didt die out during the Sengoku era but burrowed under Japan, where they fought a new war against the alien occupants of a crashed spaceship."}, {"m_title": "The Three-eyed One", "r": "7.0", "year": "1990", "votes": "24", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Atelier Escha & Logy: Alchemists of the Dusk Sky", "r": "6.3", "year": "2014", "votes": "70", "genre": "Animation", "desp": "Atelier Escha and Logy: Alchemists of the Dusk Sky is a Japanese role-playing video game developed by Gust Co. Ltd. Given the project code A15, it is the 15th game in the official Atelier. watch the anime dude!"}, {"m_title": "Porufi no nagai tabi", "r": "6.4", "year": "2008", "votes": "21", "genre": "War", "desp": "The story is of a Greek boy, Porphyras and his sister Mina, who have been orphaned after a devastating earthquake. Rather than staying in a foster home, they decide to travel across all of Europe, and thus their adventures begin."}, {"m_title": "One Off", "r": "6.5", "year": "2012", "votes": "24", "genre": "Animation", "desp": "Haruno Shiozaki is a young high school student living in a small town. After sixteen years stuck there, she wants to leave her hometown, but is uneasy about the future. She then meets. watch the dude, no spoilers in summary!"}, {"m_title": "Koisuru tenshi Angelique: Kokoro no mezameru toki", "r": "6.8", "year": "2006", "votes": "17", "genre": "Drama, Fantasy", "desp": "A young girl named Ange is summoned to another world where she has been chosen to become the legendary Etoile whose mission is to save the newly-born cosmos named the Cosmos of the Holy. watch the dude, no spoilers in summary!"}, {"m_title": "Gift: Eternal Rainbow", "r": "5.6", "year": "2006", "votes": "25", "genre": "Comedy, Drama", "desp": "The seaside village of Narasakicho is special in a number of strange ways. Crowned by an eternal rainbow, the village is rumored to grant all of its residents a once-in-a-life-time magic. watch the dude, no spoilers in summary!"}, {"m_title": "Hidamari no ki", "r": "8.0", "year": "2000", "votes": "17", "genre": "Drama, History", "desp": "sorry no summary not available!"}, {"m_title": "Izumo: Flash of a Brave Sword", "r": "5.6", "year": "2005", "votes": "12", "genre": "Fantasy", "desp": "Based on the H-RPG by Studio ego. Yagi Takeru is a student who goes to the Izumo Academy. He lost his parents when he was little, but he got adopted by Touma Rokusuke, who is like a. watch the dude, no spoilers in summary!"}, {"m_title": "Glass Maiden", "r": "6.0", "year": "2008", "votes": "29", "genre": "Action, Mystery", "desp": "In a city full of scum, called Rags Town, a detective named Shun decides to help a woman who has lost her memory and tries to help her solve her mystery but doesnt know that this will change his life."}, {"m_title": "Over Drive", "r": "7.1", "year": "2007", "votes": "48", "genre": "Comedy, Romance", "desp": "Over drive is the story of how Mikoto Shinozaki becomes interested in cycling and all the ups and downs he must live to get his dream of becoming Tour de Frances champion come true."}, {"m_title": "Kôchû ôja mushikingu: Mori no tami no densetsu", "r": "8.0", "year": "2005", "votes": "20", "genre": "Adventure, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Masou Kishin Cybuster", "r": "5.8", "year": "1999", "votes": "17", "genre": "Adventure, Sci-Fi", "desp": "In an environmentally wrecked future Tokyo, a young man named Ken Ando joins an organization called DC. Little does he realize that his fate and that of Earth will be linked to a mysterious mecha from another world."}, {"m_title": "Wooser no Sono Higurashi", "r": "5.4", "year": "2012", "votes": "25", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Kojika monogatari", "r": "7.4", "year": "1983", "votes": "58", "genre": "Drama", "desp": "Animated series of the classic novel. Anyone who grew up in the 80s grew up with this cartoon."}, {"m_title": "Yadamon", "r": "5.8", "year": "1992", "votes": "7", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Invincible King Tri-Zenon", "r": "6.8", "year": "2000", "votes": "9", "genre": "Sci-Fi", "desp": "During the Edo Period, 3 mysterious space capsules were sent down to earth. Fear that it might bring chaos to the earth, the humans at that time buried these 3 objects down beneath the. watch the dude, no spoilers in summary!"}, {"m_title": "Poor Sisters Story", "r": "7.0", "year": "2006", "votes": "24", "genre": "Comedy, Drama", "desp": "A junior high school student named Kyô and an elementary school student Asu, who live alone. Their mother passed away years ago. They live happily together to conquer each difficulties and receive help from the kind people around them."}, {"m_title": "Mangirl!", "r": "4.8", "year": "2013", "votes": "57", "genre": "Short, Comedy", "desp": "A team of girls with zero experience in manga editing are off and running toward their dream of creating the biggest manga magazine in Japan, They seem to do nothing but run into problems. watch the dude, no spoilers in summary!"}, {"m_title": "Zan sayonara zetsubô sensei bangai-chi", "r": "8.0", "year": "2009", "votes": "86", "genre": "Animation", "desp": "The continuing adventures of Nozomu Itoshiki as despair and his class of eccentric students continues to stalk him."}, {"m_title": "Saint October", "r": "7.6", "year": "2007", "votes": "14", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Saki Achiga-hen episode of side-A", "r": "7.1", "year": "2012", "votes": "34", "genre": "Comedy", "desp": "Shizuno Takakamo and the rest of the girls in the mahjong club of the all-girls school of Achiga."}, {"m_title": "Zegapain", "r": "7.4", "year": "2006", "votes": "132", "genre": "Action, Romance", "desp": "Sogoru Kyo is a normal high school student with great passion for the things he holds dear. In fact, his love for swimming is so great he is attempting to single-handedly keep the team. watch the dude, no spoilers in summary!"}, {"m_title": "Whats Michael", "r": "6.8", "year": "1988", "votes": "16", "genre": "Comedy", "desp": "A fat orange cat gets into trouble."}, {"m_title": "Nishi no yoki majo: Astraea testament", "r": "6.0", "year": "2006", "votes": "43", "genre": "Drama, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Galaxy Angel Rune", "r": "6.6", "year": "2006", "votes": "24", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Smash Hit!", "r": "6.4", "year": "2004", "votes": "14", "genre": "Comedy", "desp": "Mitsuki Ikuta a TV producer and 25 year old virgin has a dream of working on serious crime dramas but is appointed to work on the Cosprayers show."}, {"m_title": "Today in Class 5-2", "r": "7.1", "year": "2008", "votes": "39", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Nyorôn Churuya san", "r": "5.6", "year": "2009", "votes": "146", "genre": "Comedy", "desp": "In this series of original net animations, Churuya-san faces constant bullying from the SOS Brigade, the jealousy of Asakura, and a disheartening shortage of smoked cheese. Nyoron!"}, {"m_title": "Baromu wan", "r": "6.1", "year": "2002", "votes": "11", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Mewkledreamy", "r": "6.8", "year": "2020", "votes": "14", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Gakuen Utopia Manabi Straight!", "r": "6.4", "year": "2007", "votes": "19", "genre": "Comedy", "desp": "A group of young high school girls living in the year 2035 while they attend the all-girl Seioh Private High School. Since the birth rate has dropped dramatically, schools are being closed. watch the dude, no spoilers in summary!"}, {"m_title": "Ask Dr. Rin!", "r": "6.8", "year": "2001", "votes": "16", "genre": "Fantasy", "desp": "Meirin Kanzaki is a girl in junior high school who happens to be a feng-shui master. Although she uses her powers to give love advice, shes sadly pining after her cynical childhood friend,. watch the anime dude!"}, {"m_title": "Military!", "r": "5.0", "year": "2015", "votes": "66", "genre": "Comedy", "desp": "When his father is mistakenly drafted into the military and hes targeted by the enemy spies, High School student Yank Sohei must live with two young, sometimes crazy soldiers sent to protect him."}, {"m_title": "Free Kick Toward Tomorrow", "r": "6.7", "year": "1992", "votes": "54", "genre": "Sport", "desp": "The lives and soccer games of the young members of Godai Team."}, {"m_title": "Waccha PriMagi!", "r": "5.5", "year": "2021", "votes": "7", "genre": "Fantasy, Music", "desp": "Middle-school first-year student Hibino Matsuri loves festivals, and she dreams of one day being able to star in PriMagi, a magic-like form of stage entertainment borne from song, dance. watch the dude, no spoilers in summary!"}, {"m_title": "Wish Upon the Pleiades", "r": "5.8", "year": "2011", "votes": "43", "genre": "Comedy, Fantasy", "desp": "Subaru is a young girl who discovers that her best friend Aoi is part of magical group trying to recover alien engine fragments so that a little being from the Pleiades star system can. watch the dude, no spoilers in summary!"}, {"m_title": "Da Capo II", "r": "6.6", "year": "2007", "votes": "44", "genre": "Comedy, Drama", "desp": "The undying Sakura Trees return. The story once again takes place on Hatsunejima, but is set 53 years after the events of D.C. and D.C.S.S.. Although some characters will probably look a. watch the dude, no spoilers in summary!"}, {"m_title": "Happy World!", "r": "6.1", "year": "2002", "votes": "19", "genre": "Short, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Lemon Angel Project", "r": "5.6", "year": "2006", "votes": "19", "genre": "Drama, Music", "desp": "Lemon Angel was once the greatest girl band of Japan, but then it broke up and most of its members vanished. When a music producer decides to put together a new Lemon Angel, average high. watch the dude, no spoilers in summary!"}, {"m_title": "Futari wa Milky Holmes", "r": "6.6", "year": "2013", "votes": "15", "genre": "Comedy, Mystery", "desp": "Two years have passed since the event of Tantei Opera Milky Holmes, a new group of phantom thieves called Color the Phantom have appeared at Yokohama to bring a revolution. Kazumi Tokiwa. watch the dude, no spoilers in summary!"}, {"m_title": "Shônen Ashibe GO! GO! Goma-chan", "r": "7.2", "year": "2016", "votes": "10", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Schoolgirl Strikers: Animation Channel", "r": "6.4", "year": "2017", "votes": "20", "genre": "Action, Sci-Fi", "desp": "The newly established private high school for girls, Goryoukan Academy, has a hidden face that people on the outside dont know about... Mysterious apparitions called Oburi have appeared. watch the dude, no spoilers in summary!"}, {"m_title": "Cluster Edge", "r": "7.1", "year": "2005", "votes": "34", "genre": "Action, Sci-Fi", "desp": "Agate Fluorite is the newest member of Cluster E.A., an elite school where sons of prestigious familys from around the world study together. But soon Agate will learn he has a secret power, to create miracles."}, {"m_title": "Turning Girls", "r": "5.2", "year": "2013", "votes": "11", "genre": "Comedy", "desp": "The story is about girls who face a battle against the turning point of their lives."}, {"m_title": "Aah! Harimanada", "r": "5.2", "year": "1992", "votes": "10", "genre": "Comedy, Sport", "desp": "When Harimanada makes his grand entrance into the world of Sumo, he causes complete uproar throughout the entire arena. The novelty headmask is clearly not a good idea but not content with. watch the dude, no spoilers in summary!"}, {"m_title": "Fairy Musketeers", "r": "5.5", "year": "2006", "votes": "20", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Sabiiro no Armor: Reimei", "r": "4.0", "year": "2022", "votes": "23", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Miracle Train: Oedo-sen E Youkoso", "r": "6.9", "year": "2009", "votes": "19", "genre": "Comedy, Drama", "desp": "Its a common urban legend. Are you familiar with this rumor? In Tokyos subways, there runs a legendary train that appears only to lost ladies. It is known as the Miracle Train. Will you fall in love with a train station?"}, {"m_title": "6-tsugo to ouma no monogatari", "r": "7.0", "year": "2016", "votes": "7", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Majime ni fumajime kaiketsu zorori", "r": "6.8", "year": "2005", "votes": "11", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Chô mashin eiyûden Wataru", "r": "8.0", "year": "1997", "votes": "32", "genre": "Adventure, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Chaika: The Coffin Princess Avenging Battle", "r": "7.1", "year": "2014", "votes": "74", "genre": "Action, Adventure", "desp": "The search for the remains of Emperor Gaz continues. Chaika still in search of knowing who she really is and what her purpose is. Similarly, the Red Chaika continues the search for the rest. watch the anime dude!"}, {"m_title": "Mamotte Shugogetten!", "r": "6.2", "year": "1998", "votes": "10", "genre": "Comedy, Fantasy", "desp": "A lonely young boy is besieged with a bevy of female guardian spirits sworn to do his every whim."}, {"m_title": "Uchû kaizoku Mito no daibôken", "r": "6.3", "year": "1999", "votes": "10", "genre": "Action, Comedy", "desp": "Mito isnt just another space pirate, shes a small, child-like alien with enough guts to outshine a supernova. Known as the galaxys most dangerous pirate, all she really wants is to be called Mom."}, {"m_title": "Anisava", "r": "4.4", "year": "2013", "votes": "11", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Taido rain burû", "r": "6.8", "year": "2005", "votes": "45", "genre": "Action, Adventure", "desp": "In the future, 90 percent of Earth is under water and the new UN are slowly trying to rebuild the word. A military general decides that only a military regime could pull that off and attempts a coup. Only one brave teen can stop him."}, {"m_title": "Idol Angel Yokoso Yoko", "r": "7.3", "year": "1990", "votes": "10", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Shippu! Iron Leaguer", "r": "7.5", "year": "1993", "votes": "12", "genre": "Action, Fantasy", "desp": "In a world where robots battle each other in violent sport games known as the Iron League, a team of robots use fair play and sportsmanship to beat their rivals and aim for the top."}, {"m_title": "Yuuki Yuuna wa Yuusha de Aru: Dai Mankai no Shou", "r": "7.3", "year": "2021", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Rizerumain", "r": "6.2", "year": "2002", "votes": "69", "genre": "Comedy, Romance", "desp": "A teenage boy with an interest in older women is forced by the government to marry a genetic experiment with the body of a 12 year old girl."}, {"m_title": "Goldfish Warning!", "r": "7.6", "year": "1991", "votes": "15", "genre": "Comedy", "desp": "A spoiled child is transferred to a rural public school, where cows, chickens and other farm animals also attend to classes, together with her pink flying kingyo (goldfish)."}, {"m_title": "Ayatsuri Sakon", "r": "7.1", "year": "1999", "votes": "41", "genre": "Mystery, Thriller", "desp": "The series depicts the journey of Sakon who is a ventriloquist, and his puppet Ukon; their ability to solve mysteries, and the unusual bond between them."}, {"m_title": "Million Doll", "r": "7.8", "year": "2015", "votes": "69", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Good Day for Gag Manga", "r": "7.7", "year": "2005", "votes": "16", "genre": "Comedy", "desp": "Gyagu Manga Biyori is hyperactive, random, and nonsensical, revolving around various plots and characters throughout the series. It makes absolutely no sense."}, {"m_title": "Rakugo tennyo Oyui", "r": "6.3", "year": "2006", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Wonder Beat S", "r": "8.0", "year": "1986", "votes": "32", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Usakame", "r": "5.4", "year": "2016", "votes": "39", "genre": "Comedy, Sport", "desp": "Each of the four girls of Usakame High School tennis club has their own dreams and goals to pursue. Their daily lives at the tennis club are always exciting and never boring."}, {"m_title": "Shin Chuuka Ichiban!", "r": "6.8", "year": "2019", "votes": "21", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Time Travel Shoujo: Mari Waka to 8-nin no Kagakusha-tachi", "r": "6.9", "year": "2016", "votes": "12", "genre": "Sci-Fi", "desp": "Mari, her best friend, and her best friends brother use a special compass to travel through time."}, {"m_title": "Magic of Stella", "r": "7.3", "year": "2016", "votes": "50", "genre": "Comedy", "desp": "Upon enrolling in high school, Tamaki Honda joins a club for making doujin games known as the SNS Club. Joined by programmer Shiina, writer Ayame, and composer Kayo, Tamaki begins working as an illustrator for the clubs next game."}, {"m_title": "Mushiuta", "r": "5.9", "year": "2007", "votes": "25", "genre": "Action, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Faiabôru", "r": "6.8", "year": "2008", "votes": "42", "genre": "Comedy, Sci-Fi", "desp": "In the distant future, gynoid duchess Drossel von Flügel lives in a manor with her arachnoid servant Gedächtnis."}, {"m_title": "Nobunaga No Shinobi", "r": "7.5", "year": "2016", "votes": "36", "genre": "Comedy", "desp": "This animation as based on Oda Nobunaga and what he mainly went through in his life. A girl and boy Shinobi named Chidori and Sukezo join Nobunagas perilous journey to unify Japan with a. watch the dude, no spoilers in summary!"}, {"m_title": "Hitohira", "r": "6.6", "year": "2007", "votes": "36", "genre": "Drama, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Shin shirayuki hime densetsu Prétear", "r": "7.9", "year": "2001", "votes": "109", "genre": "Adventure, Comedy", "desp": "Pretear is about a 16 year old girl named Himeno who soon finds out she can become Pretear with the help of the Leafe Knights because of the power that lies within her. Her father, Kaoru,. watch the anime dude!"}, {"m_title": "Modern Magic Made Simple", "r": "5.9", "year": "2009", "votes": "46", "genre": "Action, Comedy", "desp": "A clumsy high school girl is looking to change herself. She finds a brochure for a school of magic with a powerful head magician that specializes in magic related to computers."}, {"m_title": "Hâi sureppu Jun", "r": "7.2", "year": "1985", "votes": "36", "genre": "Action, Romance", "desp": "Jun Nonomiya is an extremely intelligent girl, who spends most of her time building new inventions: her two best friends, Floppy (Kichinosuke) and Flappy (Yukinojo), are just two robots created by her."}, {"m_title": "Scorching Ping Pong Girls", "r": "6.5", "year": "2016", "votes": "48", "genre": "Sport", "desp": "Tsumujikaze Koyori, a girl who loves ping pong, has just transferred to Suzumegahara Middle School. Along with her friends who strive to be the national champions, can Koyori bring the winds of change to the school?!"}, {"m_title": "Animaru yokochô", "r": "6.5", "year": "2005", "votes": "16", "genre": "Comedy, Fantasy", "desp": "A 5 year old girl discovers three animals who lead her to another world."}, {"m_title": "Bakegyamon", "r": "6.9", "year": "2006", "votes": "17", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Bus Gamer", "r": "5.2", "year": "2008", "votes": "121", "genre": "Action", "desp": "sorry no summary not available!"}, {"m_title": "The World of Golden Eggs", "r": "7.6", "year": "2005", "votes": "26", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "La Corda dOro: Primo Passo", "r": "7.3", "year": "2005", "votes": "19", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Ongaku shoujo", "r": "5.7", "year": "2018", "votes": "18", "genre": "Music", "desp": "There are eleven girls that consist the C-class Ongaku Shoujo idol unit under Pine Records. Despite generating red sales marks and not being very popular, Producer Ikebashi and the members. watch the dude, no spoilers in summary!"}, {"m_title": "Mokke", "r": "6.8", "year": "2007", "votes": "45", "genre": "Comedy, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Popolocrois monogatari", "r": "7.3", "year": "1998", "votes": "18", "genre": "Adventure, Fantasy", "desp": "Prince Pietro is the half-human, half-dragon prince of the kingdom of Popolocrois. After the evil GamiGami tries to take over the Kingdom, a mysterious girl named Hyuu appears, which is the beginning of an adventure for Pietro."}, {"m_title": "Living for the Day After Tomorrow", "r": "7.0", "year": "2006", "votes": "53", "genre": "Animation", "desp": "Young Karada makes a wish to become an adult, while older Shoko longs to relive her youth. The wish comes true, and both girls must learn how to live under the very unusual circumstances."}, {"m_title": "Haunted Junction", "r": "5.8", "year": "1997", "votes": "207", "genre": "Comedy, Fantasy", "desp": "Three students, the son of a Catholic priest, the son of a Buddhist monk and the daughter of a Shinto priest are recruited into a haunted high schools Holy Student Council and given the. watch the dude, no spoilers in summary!"}, {"m_title": "Kiniro No Corda: Blue Sky", "r": "6.5", "year": "2014", "votes": "33", "genre": "Animation", "desp": "Soon after transferring to Seisou Gakuin, Kanade finds the orchestra club in the middle of preparing for the National concur. Teaming up with the other members, she will improve her skills,. watch the anime dude!"}, {"m_title": "Iczer Reborn", "r": "6.1", "year": "1990", "votes": "44", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Ashita tenki ni naare", "r": "6.9", "year": "1984", "votes": "22", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Pupipo!", "r": "6.6", "year": "2013", "votes": "35", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Kishin Taisen Gigantic Formula", "r": "7.2", "year": "2007", "votes": "11", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Buzâ bîtâ", "r": "6.1", "year": "2007", "votes": "26", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Ookiku furikabutte: Natsu no taikai hen", "r": "8.0", "year": "2010", "votes": "75", "genre": "Comedy", "desp": "Ren Mihashi was the ace of his middle schools baseball team, but due to his poor pitching, they could never win. Constant losses eventually lead to his teammates bullying him and reached. watch the dude, no spoilers in summary!"}, {"m_title": "VazzRock the Animation", "r": "4.4", "year": "2022", "votes": "10", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Rail Romanesque", "r": "5.6", "year": "2020", "votes": "55", "genre": "Animation", "desp": "Set in Hinomoto, a fictional version of Japan, where for a long time railway travel served as the most important form of transport."}, {"m_title": "I Am Takamiya Nasuno!", "r": "5.5", "year": "2015", "votes": "14", "genre": "Comedy", "desp": "A spin-off from Teekyuu which centers on the daily life of Nasuno Takamiya, the exceedingly airheaded school tennis club member in Teekyu (2012), and her friends."}, {"m_title": "Yuuki Yuuna wa Yuusha de Aru Churutto!", "r": "6.5", "year": "2021", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Meruhen Medohen", "r": "5.4", "year": "2018", "votes": "99", "genre": "Fantasy", "desp": "When the two girls meet, magic begins. Hazuki Kagimura loves stories, an orthodox girl who is overly imaginative. Because her relationship with her new family does not go well, the. watch the dude, no spoilers in summary!"}, {"m_title": "Acrobunch", "r": "6.9", "year": "1982", "votes": "10", "genre": "Animation", "desp": "Archeologist Tetsuya Randou leads his family on an adventure around the world to find out the legend of Quaschika, which supposedly hides an unfounded treasure. At the same time the Goblin. watch the dude, no spoilers in summary!"}, {"m_title": "Submarine Super 99", "r": "6.2", "year": "2003", "votes": "23", "genre": "Action, Adventure", "desp": "In the year 20XX AD, mysterious shipwrecks have been continuing to occur near Japan. Oki Juzo and Itsuro disappears to investigate these incidents. Just before they left, they came by a key. watch the anime dude!"}, {"m_title": "Yumetsukai", "r": "6.6", "year": "2006", "votes": "20", "genre": "Fantasy", "desp": "Mishima Touko and Mishima Rinko are Dream Users, people who have power of the world of dreams. Their job is to solve mysteries and fight evil in this dream world (created by the human spirit)."}, {"m_title": "Amazing Nuts!", "r": "5.9", "year": "2006", "votes": "19", "genre": "Adventure, Crime", "desp": "sorry no summary not available!"}, {"m_title": "Dancouga Nova", "r": "7.2", "year": "2007", "votes": "9", "genre": "Action", "desp": "The series takes place in 2107,[1] where small wars take place all over the globe. A team of four people; Aoi Hidaka, Kurara Tachibana, Sakuya Kamon, Johnny Bernet, are chosen to be the. watch the dude, no spoilers in summary!"}, {"m_title": "Honto ni Atta! Reibai-Sensei", "r": "2.0", "year": "2011", "votes": "23", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Magical Somera-chan", "r": "5.4", "year": "2015", "votes": "23", "genre": "Comedy", "desp": "The story follows the everyday life of Somera Nonomoto who can use the strongest kenpo, Nonomoto Mahou-ken, which is inherited from her mother with her younger sister Kukuru. (Source: Crunchyroll)"}, {"m_title": "Shounen Hollywood", "r": "7.0", "year": "2014", "votes": "13", "genre": "Comedy, Music", "desp": "Five boys band together to form the next generation of Shonen Hollywood, a legendary idol group from their childhoods."}, {"m_title": "Patalliro Saiyuki", "r": "6.4", "year": "2005", "votes": "19", "genre": "Adventure, Comedy", "desp": "An adaptation of the Chinese folk tale Journey to the West, starring the mischievous imp Patalliro as the Monkey King and his friends as his companions."}, {"m_title": "Leina: Wolf Sword Legend", "r": "3.4", "year": "1988", "votes": "34", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Fresh PreCure!", "r": "8.9", "year": "2009", "votes": "18", "genre": "Action, Adventure", "desp": "Love Momozono is a 14-year-old student at Yotsuba Junior High School who at her own expense, one day, visits a show by the well-known Trinity dance company and decides to become a dancer. A. watch the anime dude!"}, {"m_title": "Cute Executive Officer", "r": "6.9", "year": "2021", "votes": "17", "genre": "Comedy", "desp": "The president of Mujina Company, Najimu Mujina, isnt originally an adult, nor does she possess a genius intellect: shes a very ordinary 5-year-old girl. Followed by her indulgent and. watch the dude, no spoilers in summary!"}, {"m_title": "Hatenkô yûgi", "r": "6.6", "year": "2008", "votes": "48", "genre": "Action, Adventure", "desp": "With these words, Rahzel, the daughter of a rich family, is kicked out of her house and sent on her journey. Along the way she meets up with Heat and Alzeid, two men with very different. watch the dude, no spoilers in summary!"}, {"m_title": "Purizumu âku", "r": "6.1", "year": "2007", "votes": "26", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Marginal Prince", "r": "5.0", "year": "2006", "votes": "26", "genre": "Musical", "desp": "sorry no summary not available!"}, {"m_title": "Dream Festival", "r": "7.1", "year": "2016", "votes": "20", "genre": "Music", "desp": "The Dream Festival is the stage that all idols dream of singing on."}, {"m_title": "Gifuu Doudou!!: Kanetsugu to Keiji", "r": "7.1", "year": "2013", "votes": "18", "genre": "Action, History", "desp": "Two heros of ancient Japan, Naoe Kanetsugu and Maeda Keiji, recount the events from their past which brought them to where they are now."}, {"m_title": "Yomigaeru sora: Rescue Wings", "r": "6.8", "year": "2006", "votes": "63", "genre": "Drama, Romance", "desp": "Kazuhiro Uchida is transferred to a rescue centre located in a small town while training to become a fighter pilot. Initially, Kazuhiro thinks negatively about his new occupation, due to. watch the dude, no spoilers in summary!"}, {"m_title": "Muteking the Dancing Hero", "r": "5.6", "year": "2021", "votes": "10", "genre": "Comedy, Musical", "desp": "An alien sheriff comes to Earth to stop four alien criminals from laying waste to it."}, {"m_title": "Tribe Cool Crew", "r": "7.1", "year": "2014", "votes": "10", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "The Everyday Tales of a Cat God", "r": "7.2", "year": "2011", "votes": "9", "genre": "Comedy, Fantasy", "desp": "Komiya Yuzu is running an antique shop. Mayu, a cat god (nekogami), is living off Yuzu and leads an idle life playing games. Lots of other gods visit Mayu and enjoy merrymaking."}, {"m_title": "Prince Mackaroo", "r": "6.8", "year": "1998", "votes": "23", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Ippatsu Kiki Musume", "r": "6.5", "year": "1999", "votes": "36", "genre": "Action, Comedy", "desp": "A young Chinese woman named Kunyan finds herself subjected to a variety of bizarre, life-threatening situations. She must act quickly if she hopes to escape her plight in one piece."}, {"m_title": "Bird Island", "r": "7.2", "year": "2003", "votes": "9", "genre": "History, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Rainy Cocoa", "r": "3.8", "year": "2015", "votes": "68", "genre": "Comedy", "desp": "When Aoi falls into a job at the café he feels like things are finally looking up- until Keiichi Iwase shows up. Did this day just get worse, or better?"}, {"m_title": "Kuro Majo-san ga Tooru!!", "r": "4.7", "year": "2012", "votes": "11", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sûpâ bikkuri man", "r": "7.0", "year": "1992", "votes": "9", "genre": "Fantasy", "desp": "After of the big war between angels and demons, the Earth is almost destroyed and his cities devastated. Wanted to restore the peace and the harmony, four young child decides to fight. watch the dude, no spoilers in summary!"}, {"m_title": "Di Gi Charat: Winter Garden", "r": "6.1", "year": "2006", "votes": "9", "genre": "Comedy, Drama", "desp": "This is sweet and bitter love story of Dejiko who is a grown up. The story begins on the Christmas day. Dejiko or Di Gi Carat is working part-time at a bakery shop. On her way from the shop. watch the anime dude!"}, {"m_title": "Chiis Sweet Home: Atarashii Ouchi", "r": "6.3", "year": "2009", "votes": "9", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Tesagure! Bukatsu-mono", "r": "7.4", "year": "2013", "votes": "18", "genre": "Comedy", "desp": "The ultimate fourth wall breaking ad-lib meta anime. Four high school girls spend time chatting about extracurricular activities and gradually the line between the voice actress and characters are not so clear."}, {"m_title": "Re-Kan!", "r": "4.8", "year": "2015", "votes": "67", "genre": "Comedy, Fantasy", "desp": "High school girl Hibiki Amami has a sixth sense for spirits which kept her lonely as a child. Will it be different now that shes transferred to a new school?"}, {"m_title": "Shions King", "r": "7.3", "year": "2007", "votes": "46", "genre": "Drama, Mystery", "desp": "As a child, Shion Ishiwataris parents were murdered in their own home. Shion escaped unharmed, yet the trauma caused her to lose her voice. The only evidence police found was a shougi. watch the dude, no spoilers in summary!"}, {"m_title": "The woodcutters Treasure", "r": "8.8", "year": "2018", "votes": "61", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Damekko dôbutsu", "r": "7.1", "year": "2005", "votes": "35", "genre": "Comedy", "desp": "One day, Uruno, the useless Wolf, is looking for a new home. He finds a forest full of animals that dont quite fit their parts. The Bunny is ill tempered, the Cheetah is clumsy, etc. He finds other useless people to be friends with."}, {"m_title": "Fantastic Detective Labyrinth", "r": "5.8", "year": "2007", "votes": "10", "genre": "Fantasy, Mystery", "desp": "sorry no summary not available!"}, {"m_title": "Saishû shiken kujira", "r": "4.2", "year": "2007", "votes": "10", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Hulaing Babies", "r": "5.5", "year": "2019", "votes": "14", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Tetsujin 28-gou FX", "r": "6.2", "year": "1992", "votes": "8", "genre": "Action, Sci-Fi", "desp": "By 2030, Shotaros son was in charge of Tetsujin-28. An Interpol agent like his father, he used the aged robot to combat the Pink Mafia and the Franken organization."}, {"m_title": "Eikoku ikka, Nippon o taberu", "r": "6.6", "year": "2015", "votes": "14", "genre": "Animation", "desp": "The anime is based on the essay Sushi and Beyond by British travel and food journalist Michael Booth. The book is about the authors 100-day trip in Japan with his family to sample Japanese cuisine from north to south."}, {"m_title": "Gin-iro no Olynssis", "r": "5.8", "year": "2006", "votes": "7", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Yat! Budget! Space Tours", "r": "6.8", "year": "1996", "votes": "12", "genre": "Adventure, Comedy", "desp": "Goro Hoshiwata tries to find his long lost father whos only possession he has is an old photograph."}, {"m_title": "Mitchiri Neko", "r": "7.8", "year": "2018", "votes": "6", "genre": "Animation", "desp": "The cute and mysterious cat-like creatures called Mitchiri Neko that seem to gather around each other if you leave them alone."}, {"m_title": "Super Robot Wars OG: The Inspector", "r": "6.9", "year": "2010", "votes": "17", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Naked Wolves", "r": "7.8", "year": "2009", "votes": "8", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Ozuma", "r": "6.2", "year": "2012", "votes": "48", "genre": "Action, Adventure", "desp": "In a desert world, the pirate crew Baldanos searches for the sand whale Ozuma."}, {"m_title": "Inu to Neko Docchi mo Katteru to Mainichi Tanoshii", "r": "7.0", "year": "2020", "votes": "53", "genre": "Animation", "desp": "A fun comedy about a young woman named Matsumoto and her day-to-day life with her adorably gullible dog and sweetly malicious cat."}, {"m_title": "Banner of the Stars III", "r": "7.3", "year": "2005", "votes": "52", "genre": "Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Pâman", "r": "8.8", "year": "1983", "votes": "49", "genre": "Adventure, Comedy", "desp": "Mitsuo encountered Birdman, an alien who is a part of a federation who tries to maintain peace in the galaxy."}, {"m_title": "Fireball Humorous", "r": "7.2", "year": "2017", "votes": "7", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Kamigami no Ki", "r": "6.0", "year": "2016", "votes": "7", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Chitose Get You!!", "r": "5.5", "year": "2012", "votes": "40", "genre": "Short, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Misutenaide Daisy", "r": "6.5", "year": "1997", "votes": "18", "genre": "Comedy, Romance", "desp": "A deranged madman becomes obsessed with controlling an innocent young schoolgirl. He will threaten her with death, humiliation and the loss of her own will and identity. Terrors biological, chemical and nuclear will come into play."}, {"m_title": "Telepathy Shojo Ran", "r": "6.5", "year": "2008", "votes": "15", "genre": "Fantasy, Mystery", "desp": "sorry no summary not available!"}, {"m_title": "Ayumayu gekijô", "r": "4.6", "year": "2006", "votes": "16", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Spirit Hero Wataru 2", "r": "7.7", "year": "1990", "votes": "29", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Kurayami Santa", "r": "6.5", "year": "2015", "votes": "12", "genre": "Adventure, Horror", "desp": "In 1960s Japan, a ghoul is tasked with maintaining the boundaries between the human world and the underworld."}, {"m_title": "Kyoshin to Hyouka no Shiro", "r": "5.4", "year": "2020", "votes": "5", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Shinya! Tensai Bakabon", "r": "6.4", "year": "2018", "votes": "15", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Armored Police Metal Jack", "r": "6.2", "year": "1991", "votes": "7", "genre": "Action, Sci-Fi", "desp": "in the year 2015, murder and crime have gotten very high in Tokyo and the police cant handle it all and three men are wounded while protecting a small boy, so they are brought back to life reincarnated as cyborgs."}, {"m_title": "Gebaude-Baude", "r": "5.6", "year": "2020", "votes": "5", "genre": "Comedy, Sci-Fi", "desp": "A cute little robot that likes to throw her hips around is regarded as a princess by a large spider-ish caretaker robot. She awakens each day to a short adventure which culminates quickly into a dreamlike resolution."}, {"m_title": "Kantoku Fuyuki Todoki", "r": "5.4", "year": "2014", "votes": "22", "genre": "Biography, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Fairy Dick", "r": "7.4", "year": "1992", "votes": "9", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Êsu wo nerae! 2", "r": "6.0", "year": "1988", "votes": "8", "genre": "Drama, Sport", "desp": "sorry no summary not available!"}, {"m_title": "Ginyuu mokushiroku: Meine liebe wieder", "r": "5.9", "year": "2006", "votes": "14", "genre": "Drama, History", "desp": "sorry no summary not available!"}, {"m_title": "Re: Stage! Dream Days", "r": "6.1", "year": "2019", "votes": "16", "genre": "Drama, Music", "desp": "The story centers around the first-year student Mana Shikimiya as she pursue her dreams of winning at the Prism Stage, the competition that determines the top middle-school idol."}, {"m_title": "Hyakko", "r": "7.1", "year": "2008", "votes": "70", "genre": "Comedy", "desp": "While attending an elite private high school, Torako, Ayumi, Tatsuki, and Suzume overcome their diverse backgrounds and develop an everlasting friendship. Despite their differences, the. watch the dude, no spoilers in summary!"}, {"m_title": "GA: Geijutsuka âto dezain kurasu", "r": "6.4", "year": "2009", "votes": "23", "genre": "Comedy", "desp": "The Ayanoi High School features the Geijutsuka Art Design Class (GA) that focuses on the arts. Five close friends - the energetic hime-prankster Noda Miki; the level-headed, cynical. watch the dude, no spoilers in summary!"}, {"m_title": "Yasai no yôsei N.Y. salad", "r": "8.6", "year": "2007", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sôya monogatari", "r": "6.7", "year": "1984", "votes": "164", "genre": "Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Urarochi Diamond", "r": "4.6", "year": "2001", "votes": "10", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Uta no purinsusama: Maji Love reboryûshonzu", "r": "4.3", "year": "2015", "votes": "31", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Morita-san is Taciturn", "r": "6.3", "year": "2011", "votes": "12", "genre": "Comedy", "desp": "Morita Mayu, a high school student who is kinder than any others, but thinks too much, and leads herself to a speechless high school student. Facing with many problems and even make a fool. watch the dude, no spoilers in summary!"}, {"m_title": "Chiruran 1/2", "r": "3.8", "year": "2017", "votes": "8", "genre": "Action, Comedy", "desp": "Unlike the historical Shinsengumi, whose daily lives were filled with blade, blood, and betrayal, lives of Shinsengumi members here are always funny and laid-back."}, {"m_title": "D4 Princess", "r": "6.0", "year": "1999", "votes": "11", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Lord of Lords Ryu Knight", "r": "7.6", "year": "1994", "votes": "12", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Mock & Sweet", "r": "7.7", "year": "1986", "votes": "24", "genre": "Adventure, Fantasy", "desp": "Mock and his sister, Sweet, are curious about the fuss on the earth and eventually decide to come out of the ground. They save the poor, the innocent and the brave whenever and wherever these people need help."}, {"m_title": "Kimagure Robot", "r": "5.4", "year": "2004", "votes": "8", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Sora to umi no aida", "r": "5.0", "year": "2018", "votes": "24", "genre": "Sci-Fi", "desp": "Set in Onomichi City, six girls dream of becoming Space Fishermen. They aim to capture god-eating monsters that leave fish vulnerable, as well as enemies which they encounter."}, {"m_title": "Call of the Wild", "r": "7.6", "year": "1982", "votes": "9", "genre": "Adventure", "desp": "sorry no summary not available!"}, {"m_title": "Haô Taikei Ryû Knight: Adeus Legend", "r": "8.2", "year": "1994", "votes": "7", "genre": "Action, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Happy Lesson: Advance", "r": "6.4", "year": "2003", "votes": "14", "genre": "Comedy, Romance", "desp": "Chitose, a high school student, shares his house with five women he calls his mothers, who also happen to be his school teachers. One might think the continued attention of this many. watch the dude, no spoilers in summary!"}, {"m_title": "Sparrows Hotel", "r": "5.0", "year": "2013", "votes": "19", "genre": "Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Saru Getchu: On Air", "r": "5.6", "year": "2006", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "JK meshi!", "r": "6.7", "year": "2015", "votes": "82", "genre": "Short, Comedy", "desp": "A series of anime shorts that feature three high school girls that enjoy cooking B-class dishes."}, {"m_title": "Super Seisyun Brothers", "r": "5.3", "year": "2013", "votes": "34", "genre": "Animation", "desp": "The story involves two pairs of teenage elder sister and younger brother, the Shinmotos who are a bit narcissistic and the Saitous who look a bit mysterious. It follows their everyday life in school and at home."}, {"m_title": "Magica Wars", "r": "5.6", "year": "2014", "votes": "17", "genre": "Action, Fantasy", "desp": "Mahou Shoujo Taisen is a project created from a variety show for the Otaku 2.5 Chigen TV (2.5 Dimension TV). The characters are the mahou shoujo that represent each local town in Japan.. watch the dude, no spoilers in summary!"}, {"m_title": "Haitai Nanafa", "r": "5.7", "year": "2012", "votes": "17", "genre": "Animation", "desp": "Haitai Nanafa is a slice-of-life anime set in Okinawa. It follows an energetic middle school girl named Nanafa Kyan. She and her sisters, Nao and Kokona, live with their grandmother in a. watch the dude, no spoilers in summary!"}, {"m_title": "Sararîman Kintarô", "r": "6.0", "year": "2002", "votes": "19", "genre": "Drama", "desp": "sorry no summary not available!"}, {"m_title": "Futatsu no supika", "r": "7.1", "year": "2003", "votes": "29", "genre": "Drama", "desp": "sorry no summary not available!"}, {"m_title": "Twin Hawks", "r": "6.2", "year": "1984", "votes": "30", "genre": "Sport", "desp": "The story of two competing motorcycle racers, who both bear the title of Hawk, as they strive to become the best racer in the world."}, {"m_title": "Mission-E", "r": "6.4", "year": "2008", "votes": "10", "genre": "Action, Comedy", "desp": "sorry no summary not available!"}, {"m_title": "Nyanbo!", "r": "5.1", "year": "2016", "votes": "14", "genre": "Family", "desp": "The Nyanbo are creatures with square bodies with ears and tails that came to Earth from outer space."}, {"m_title": "Mori no tonto tachi", "r": "7.0", "year": "1984", "votes": "47", "genre": "Adventure, Fantasy", "desp": "The touching adventures of the Santa Claus granddaughter."}, {"m_title": "Gunparade Orchestra", "r": "6.8", "year": "2005", "votes": "11", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sore dake ga Neck", "r": "5.9", "year": "2020", "votes": "17", "genre": "Animation", "desp": "Through the appearance of a convenience store clerk who does not talk properly, while raising problems with the way of modern communication, we will approach the secret of the mysterious hero Mutou."}, {"m_title": "Donyatsu", "r": "5.9", "year": "2013", "votes": "23", "genre": "Comedy, Sci-Fi", "desp": "Donut shaped cats inhabits a future desecrated earth, living their lives and having fun with their friends."}, {"m_title": "Wrestler gundan seisenshi Robin Jr.", "r": "7.3", "year": "1989", "votes": "10", "genre": "Adventure, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "AWOL: Absent Without Leave", "r": "5.0", "year": "1998", "votes": "7", "genre": "Action, Sci-Fi", "desp": "Cyress is a peaceful place. People are going about their lives. Two lovers embrace, a family throws a birthday party for their child, a group of people party at a local restaurant. But. watch the dude, no spoilers in summary!"}, {"m_title": "Guraburu!", "r": "6.2", "year": "2020", "votes": "17", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Romantica Clock", "r": "3.9", "year": "2014", "votes": "18", "genre": "Animation", "desp": "Akane: Shes beautiful and popular, lively and full of vitality; Aoi: Looks like a small animal but is actually very clever. Who knew that these two very different people were actually - twin siblings?."}, {"m_title": "Miyanishi Tatsuya gekijou Omae umasoudana", "r": "7.0", "year": "2010", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Usaru-san", "r": "3.6", "year": "2009", "votes": "6", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Saiyuki Reload: Burial", "r": "7.4", "year": "2007", "votes": "34", "genre": "Animation", "desp": "Taking place in the past it offers a view at the Saiyuki quartet before their formation and their journey to the west."}, {"m_title": "Kocchi muite Miiko", "r": "8.5", "year": "1998", "votes": "23", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Ishida & Asakura", "r": "5.3", "year": "2013", "votes": "26", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Aguu: Tensai Ningyou", "r": "5.4", "year": "2018", "votes": "17", "genre": "Action, Drama", "desp": "sorry no summary not available!"}, {"m_title": "Buzâ bîtâ", "r": "6.0", "year": "2005", "votes": "33", "genre": "Sport", "desp": "sorry no summary not available!"}, {"m_title": "Double-J", "r": "5.7", "year": "2011", "votes": "12", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Crane Game Girls", "r": "5.5", "year": "2016", "votes": "100", "genre": "Comedy, Sci-Fi", "desp": "sorry no summary not available!"}, {"m_title": "Super Robot Taisen OG: Divine Wars", "r": "6.9", "year": "2006", "votes": "19", "genre": "Action, Adventure", "desp": "Human civilization has been expanding into space. Secret technology has been uncovered, apparently belonging to alien aggressors, heading to Earth, to invade the planet...or worse, to claim. watch the anime dude!"}, {"m_title": "Ryusei Sentai Musumet", "r": "6.8", "year": "2004", "votes": "8", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Uta no purinsusama: Maji Love rejendo sutâ", "r": "3.1", "year": "2016", "votes": "21", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Happy Lesson OVA Series", "r": "6.5", "year": "2001", "votes": "15", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Kobayashi ga kawaisugite tsurai!! Gêmudemo kyunmoe ga tomaranai", "r": "5.7", "year": "2015", "votes": "8", "genre": "Romance", "desp": "Bundled with the limited edition of third volume of Kobayashi ga Kawai Sugite Tsurai!. shoujo manga with 2 episode."}, {"m_title": "Gdgd Mens Party", "r": "6.2", "year": "2018", "votes": "7", "genre": "Comedy, Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Hâi Akko desu", "r": "7.2", "year": "1988", "votes": "8", "genre": "Comedy", "desp": "Akko-chan, who is a new young wife, experiences her daily life."}, {"m_title": "Solo Leveling", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Adventure", "desp": "Anime adaptation of the South Korean web novel Solo Leveling"}, {"m_title": "Tomb Raider", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Adventure", "desp": "Follows heroine Lara Croft as she explores new territory."}, {"m_title": "Sacrificial Princess & the King of Beasts", "r": "0", "year": "upcoming", "votes": "0", "genre": "Fantasy, Romance", "desp": "The king of the beasts and demons receives female human sacrifices to assert the dominance over the human race. However, the 99th sacrifice intrigues him and lets her stay as his consort."}, {"m_title": "Rurouni Kenshin", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Adventure", "desp": "During the violent era known as the Bakumatsu, there was an Ishin assassin known only as the Hitokiri Battousai, whose skill and brutality became near legendary in the age of Restoration he. watch the anime dude!"}, {"m_title": "Yakitori: Soldiers of Misfortune", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Adventure", "desp": "A young mans rebel against authorities on a future version of Earth, after he has been recruited into an interplanetary military unit that suffers a 70% casualty rate."}, {"m_title": "My Happy Marriage", "r": "0", "year": "upcoming", "votes": "0", "genre": "Drama, Fantasy", "desp": "An unhappy young woman from an abusive family is married off to a fearsome and chilly army commander. But the two learn more about each other, love may have a chance."}, {"m_title": "Zom 100: Bucket List of the Dead", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Comedy", "desp": "In a trash-filled apartment, 24-year-old Akira Tendou watches a zombie movie with lifeless, envious eyes. After spending three hard years at an exploitative corporation in Japan, his spirit. watch the anime dude!"}, {"m_title": "Akuma-kun", "r": "0", "year": "upcoming", "votes": "0", "genre": "Adventure, Fantasy", "desp": "A boy trying create a society where humanity realizes its true potential and lives happily, with the help of some unlikely allies."}, {"m_title": "Kusuriya no Hitorigoto", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "It follows Maomao as she is kidnapped and sold into service in the palace. She has a quick wit, a sharp mind, and knowledge of medicine, and she encounters a someone at least as perceptive as she is: the head eunuch, Jinshi."}, {"m_title": "Onmyoji", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Drama", "desp": "The tale of Abe no Seimei, a legendary spell-caster who lived in ancient Japan."}, {"m_title": "Ooku: The Inner Chambers", "r": "0", "year": "upcoming", "votes": "0", "genre": "Drama, Fantasy", "desp": "In Edo period Japan, a strange new disease called the Red Pox has begun to prey on the countrys men. Within eighty years of the first outbreak, the male population has fallen by. watch the dude, no spoilers in summary!"}, {"m_title": "Undead Girl Murder Farce", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Sousou no Frieren", "r": "0", "year": "upcoming", "votes": "0", "genre": "Adventure, Fantasy", "desp": "An elf and her friends defeat a demon king in a great war. But the war is over, and the elf must search for a new way of life."}, {"m_title": "Shadowverse Flame", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Level 1 Demon Lord and One Room Hero", "r": "0", "year": "upcoming", "votes": "0", "genre": "Comedy, Fantasy", "desp": "A hero defeats a demon in a great battle. Years later, the demon revives as a child, and finds the hero is now a loser. Now both must work together to revive their former glory."}, {"m_title": "Hyakushô Kizoku", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Atelier Ryza: Ever Darkness & the Secret Hideout", "r": "0", "year": "upcoming", "votes": "0", "genre": "Action, Adventure", "desp": "sorry no summary not available!"}, {"m_title": "TenPuru: No One Can Live on Loneliness", "r": "0", "year": "upcoming", "votes": "0", "genre": "Comedy, Romance", "desp": "Akemitsu Akegami was always told by his father that no one can live alone...but hes sure determined. After all, his father sure wasnt saying it with the best intentions, and Akemitsu. watch the dude, no spoilers in summary!"}, {"m_title": "Genjitsu No Yohane -Sunshine in the Mirror-", "r": "0", "year": "upcoming", "votes": "0", "genre": "Fantasy", "desp": "sorry no summary not available!"}, {"m_title": "Helck", "r": "0", "year": "upcoming", "votes": "0", "genre": "Adventure, Fantasy", "desp": "A human enters a tournament of demons to become the king of demons."}, {"m_title": "BanG Dream! Its MyGO!!!!!", "r": "0", "year": "upcoming", "votes": "0", "genre": "Music", "desp": "sorry no summary not available!"}, {"m_title": "Dosanko gal wa namara menkoi", "r": "0", "year": "upcoming", "votes": "0", "genre": "Comedy, Romance", "desp": "sorry no summary not available!"}, {"m_title": "Unnamed Memory", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Story of a Small Senior in My Company", "r": "0", "year": "upcoming", "votes": "0", "genre": "Comedy, Romance", "desp": "Takuma is an office worker who is taken care of and coddled by his senior team member Shiori, who is a gorgeous, profoundly kind, loving, and diminutive woman. Takuma certainly hopes that. watch the dude, no spoilers in summary!"}, {"m_title": "THE iDOLM@STER MILLION LIVE!", "r": "0", "year": "upcoming", "votes": "0", "genre": "Music, Musical", "desp": "Spun off from The Idolm@ster (2011), this series follows a new group of idols working alongside the thirty-nine idols of the newly-formed 765PRO Live Theater base at 765 Productions and a producer at the 765 Theater Agency."}, {"m_title": "Mame kichi ma meko nito no nichijo", "r": "0", "year": "upcoming", "votes": "0", "genre": "Animation", "desp": "sorry no summary not available!"}, {"m_title": "Overtake!", "r": "0", "year": "upcoming", "votes": "0", "genre": "Sport", "desp": "Freelance photographer Kôya Madoka finds himself in a slump due to a certain reason. He goes to Fuji International Speedway to work on a story, and he meets the highschool F4 racer Haruka. watch the dude, no spoilers in summary!"}]

In [29]:
df = pd.DataFrame(data)

# genres = df['genre'].str.split(', ', expand=True)
# df = pd.concat([df, genres], axis=1)
# df.drop('genre', axis=1, inplace=True)

# Save the DataFrame to a CSV file
df.to_csv('anime_data.csv', index=True)

In [54]:
data = pd.read_csv("anime_data.csv")
data[1:100]

,Unnamed: 0,m_title,r,year,votes,genre,desp
1,1,Attack on Titan,9.1,2013,"408,892","Action, Adventure",After his hometown is destroyed and his mother...
2,2,Vinland Saga,8.8,2019,"49,162","Action, Adventure",Thorfinn pursues a journey with his fathers ki...
3,3,One Piece,8.9,1999,"138,438","Action, Adventure",Follows the adventures of Monkey D. Luffy and ...
4,4,Hells Paradise: Jigokuraku,8.5,2023,"2,007","Action, Adventure",A squad of prisoners and their guards are sent...
5,5,Oshi No Ko,8.9,2023,"2,406","Drama, Fantasy",A doctor and his recently-deceased patient are...
...,...,...,...,...,...,...,...
95,95,My Dress-Up Darling,8.0,2022,"10,435","Comedy, Romance",Traumatized by a childhood incident with a fri...
96,96,Darling in the Franxx,7.3,2018,"11,787","Action, Drama",In a future world where humanity has been driv...
97,97,The Rising of the Shield Hero,7.9,2019,"15,022","Action, Adventure",A gamer is magically summoned into a parallel ...
98,98,Trigun,8.2,1998,"22,872","Action, Adventure",Vash the Stampede is the most infamous outlaw ...


In [31]:
data['Unnamed: 0']

0          0
1          1
2          2
3          3
4          4
        ... 
2853    2853
2854    2854
2855    2855
2856    2856
2857    2857
Name: Unnamed: 0, Length: 2858, dtype: int64

In [32]:
data.shape

(2858, 7)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2858 entries, 0 to 2857
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2858 non-null   int64  
 1   m_title     2858 non-null   object 
 2   r           2858 non-null   float64
 3   year        2858 non-null   object 
 4   votes       2858 non-null   object 
 5   genre       2858 non-null   object 
 6   desp        2858 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 156.4+ KB


In [63]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load your dataset (assuming you have a DataFrame)
# Replace 'data.csv' with your dataset file or source
df = pd.read_csv('anime_data.csv')

# Text-based features: movie_description
# User-based features: user_ratings (r) and popularity_rank (Unnamed: 0)
# Metadata-based features: genre
# Target variable: m_title (movie_name) for recommendation

# Content-Based Filtering
# TF-IDF Vectorization for movie descriptions
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df['desp'])

# Combine user features
df['user_features'] = df['r'].astype(str) + ' ' + df['Unnamed: 0'].astype(str)

# Calculate cosine similarities between user features and descriptions
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to recommend movies based on a user-provided movie name
def recommend_movies_by_name(movie_name, n=10):
    # Get the index of the provided movie name
    movie_index = df[df['m_title'] == movie_name].index[0]

    # Calculate cosine similarities between user features and descriptions
    similar_m = list(enumerate(cosine_similarities[movie_index]))
    similar_m = sorted(similar_m, key=lambda x: x[1], reverse=True)
    similar_m = similar_m[1:n+1]

    # Extract movie indices
    movie_indices = [i[0] for i in similar_m]
    recommended_movies = df['m_title'].iloc[movie_indices]

    # Get additional information for recommended movies
    recommended_movie_info = df.loc[df['m_title'].isin(recommended_movies), ['m_title', 'r', 'desp', 'genre']]

    return recommended_movie_info

# Example: Recommend movies similar to a provided movie name
user_provided_movie = "Attack on Titan"
recommended_movies_info = recommend_movies_by_name(user_provided_movie)
recommended_movies_info


,m_title,r,desp,genre
318,Attack on Titan: Junior High,7.1,After his lunch is eaten by a Titan on the fir...,"Action, Adventure"
516,Girls Last Tour,7.7,Two girls wander a world destroyed by war.,"Adventure, Drama"
538,"Now and Then, Here and There",7.9,A Japanese boy named Shu tries to save a stran...,"Action, Adventure"
641,Monsuno,4.9,Monsuno hurtled through space eons before man ...,"Action, Adventure"
746,Mobile Suit Zeta Gundam,8.1,"Eight years after the One Year War, the Earth ...","Action, Drama"
1090,Muv-Luv Alternative,6.0,"In 1973, the invasion of an extra terrestrial ...","Action, Drama"
1390,Eureka Seven: Ao,6.7,"In a technologically advanced future, a young ...","Action, Drama"
1533,Pilot Candidate,5.7,The planet Zion is mankinds last hope for surv...,"Action, Drama"
2007,Monster Strike,4.8,"Ren Homura, a middle school student who moves ...",Action
2741,Sûpâ bikkuri man,7.0,After of the big war between angels and demons...,Fantasy
